In [39]:
import math
import statistics 
import random
import itertools
import heapq
import string
import queue
# list(itertools.product([100, 110, 130], [50, 75, 125]))

# Data Structures

#### trees and graphs!

##### binary search tree

In [165]:
from bintrees.bintree import BinaryTree
from bintrees.avltree import AVLTree

class TreeNode(object):

    """A Binary Search Tree Node."""
    
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.key = key
        self.value = value
        self.left = left
        self.right = right
        self.parent = parent
        

    @property
    def depth(self):
        """Return True if it's the left children."""
        d = 0
        n = self
        while n.parent:
            d += 1
            n = n.parent
        return d
    
    @property
    def height(self):
        """Return True if it's the left children."""
        def __height(n):
            if n is None: return -1
            return max(__height(n.left), __height(n.right)) + 1

        return __height(self)

    @property
    def adjacent(self):
        """Return list of children."""
        adjacent = []
        if self.left: adjacent += [self.left]
        if self.right: adjacent += [self.right]
        return adjacent

    @property
    def isleft(self):
        """Return True if it's the left children."""
        if self.parent: return self.parent.left == self
    
    @property
    def isright(self):
        """Return True if it's the right children."""
        if self.parent: return self.parent.right == self
    
    @property
    def isroot(self):
        """Return True if it's root node."""
        return self.parent is None
    
    @property
    def isleaf(self):
        """Return True if it's a leaf."""
        return self.left is None and self.right is None
    
    @property
    def hasboth(self):
        """Retrun True if has both children."""
        return self.left is not None and self.right is not None
    
    @property
    def hasany(self):
        """Retrun True if has any children."""
        return self.right or self.left
  
    @property
    def min(self):
        """Find minimum key in tree."""
        current = self
        while current.left: current = current.left
        return current

    @property
    def max(self):
        """Find minimum key in tree."""
        current = self
        while current.right: current = current.right
        return current   

    @property
    def successor(self):
        successor = None
        if self.right:
            successor = self.right.min
        else:
            if self.parent:
                if self.isleft:
                    successor = self.parent
                else:
                    self.parent.right = None
                    successor = self.parent.successor
                    self.parent.right = self
        return successor
    
    @property
    def predecessor(self):
        predecessor = None
        if self.left:
            predecessor = self.left.max
        else:
            if self.parent:
                if self.isright:
                    predecessor = self.parent
                else:
                    self.parent.left = None
                    predecessor = self.parent.predecessor
                    self.parent.right = self
        return predecessor    

    def remove(self, key):
        """
        Remove key and return successor.
        
        * Deleting a node with no children: simply remove the node.
        * Deleting a node with one child: remove the node and replace it with
          its child.
        * Deleting a node with two children: call the node to be deleted N.
          Do not delete N. Instead, choose either its in-order successor node
          or its in-order predecessor node, R. Copy the value of R to N, then
          recursively call delete on R until reaching one of the first two
          cases. If you choose in-order successor of a node, as right sub tree
          is not NIL (Our present case is node has 2 children), then its
          in-order successor is node with least value in its right sub tree,
          which will have at a maximum of 1 sub tree, so deleting it would fall
          in one of first 2 cases.
        """
        if key < self.key:
            self.left.remove(key)

        elif key > self.key:
            self.right.remove(key)

        else: 
            # if both children are present.
            if self.hasboth: 
                successor = self.right.min
                self.key = successor.key
                self.value = successor.value
                successor.remove(successor.key)
                return self
            
            # if the node has only a *left* child
            elif self.left:
                self.__delete_from_parent(self.left)
                return self.left
                
            # if the node has only a *right* child
            elif self.right:
                self.__delete_from_parent(self.right)
                return self.right

            # this node has no children
            else: 
                self.__delete_from_parent(None)
                return None

    def __delete_from_parent(self, child=None):
        """Delete current node from parent, update child accordingly."""
        if child: child.parent = self.parent
        if self.parent:
            if self.isleft:
                self.parent.left = child
            else:
                self.parent.right = child

class BinarySearchTree(object):
    
    """A Binary Search Tree implementation in python."""
    
    def __init__(self):
        self.root = None
        self.size = 0

    def __len__(self):
        return self.size
    
    def __iter__(self):
        return self.root.__iter__()
    
    def __setitem__(self, key, value):
        self.put(key, value)

    def __getitem__(self, key):
        return self.get(key)

    def __contains__(self, key):
        if self._get(key, self.root): return True
        return False
    
    def length(self):
        """Return tree's size."""
        return self.size

    def put(self, key, value):
        """Add a new node."""
        if self.root:
            self._put(key, value, self.root)
        else:
            self.size += 1
            self.root = TreeNode(key, value)

    def get(self, key):
        """Get return value for a given key."""
        node = self._get(key, self.root)
        if node: return node.value
        raise KeyError

    def delete(self, key):
        """Delete key from tree."""
        node = self._get(key, self.root)
        if not node: raise KeyError
        successor = node.remove(key)
        if node.isroot: self.root = successor
        self.size -= 1      

    def _put(self, key, value, current):
        """Recursively find the correct position for the new node."""
        if key == current.key:
            current.value = value
        elif key < current.key:
            if current.left:
                self._put(key, value, current.left)
            else:
                self.size += 1
                current.left = TreeNode(key, value, parent=current)
        else:
            if current.right:
                self._put(key, value, current.right)
            else:
                self.size += 1
                current.right = TreeNode(key, value, parent=current)

    def _get(self, key, current):
        """Recursively find key, or raise KeyError."""
        if not current: return None
        if current.key == key: return current
        if key < current.key: return self._get(key, current.left)
        return self._get(key, current.right)
    
    def _dfs(self, func, order=0):
        """
        Perform Depth First Search.
        
        Arguments:
            func (obj): function to a apply to the node.
            order(int): inorder = 0, preorder = -1, postorder = 1.
        """
        def _traverse(node):
            if order == -1: func(node)
            if node.left: _traverse(node.left)
            if order == 0: func(node)
            if node.right: _traverse(node.right)
            if order == 1: func(node)
        _traverse(self.root)

    def _bfs(self, func):
        """
        Perform Breadth First Search.
        
        Arguments:
            func (obj): function to a apply to the node.
        """
        q = queue.Queue()
        visited = {}
        func(self.root)
        visited[self.root.key] = None
        q.put(self.root)
        
        while not q.empty():
            r = q.get()
            for n in r.adjacent:
                if n.key in visited: continue
                func(n)
                visited[n.key] = None
                q.put(n)

    def _bfs_level(self, func):
        """
        Perform Breadth First Search.
        
        Arguments:
            func (obj): function to a apply to the node.
        """
        thislevel = [self.root]
        visited = {}
        while thislevel:
            nextlevel = list()
            for n in thislevel:
                if n.key in visited: continue
                visited[n.key] = None
                func(n)
                nextlevel += n.adjacent
            print("\n")
            thislevel = nextlevel
            

##### cracking 4.3

In [159]:
def balancedtree(tree, keys, data):
    """Given sorted keys and data, build a tree with minimal height."""
    n = len(keys)
    if n == 0: return
    mid = n // 2
    tree[keys[mid]] = data[keys[mid]]
    balancedtree(tree, keys[mid + 1:], data)
    balancedtree(tree, keys[:mid], data)

keys = list(range(20))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)
print("height", bst.root.height, "depth", bst.root.depth)
bst._bfs_level(lambda x: print(x.key))

nbst = BinarySearchTree()
for k in keys: nbst[k] = data[k]
# nbst._bfs_level(lambda x: print(x.key))

height 4 depth 0
10


5
15


2
8
13
18


1
4
7
9
12
14
17
19


0
3
6
11
16




##### cracking: 4.1

In [136]:
def isbalanced(root):
    thislevel = [root]
    visited = {}
    levels = []
    ret = 0
    i = 0
    while thislevel:
        levels.append(len(thislevel))
        expected = 2 ** i
        obtained = levels[i]
        if expected != obtained: ret += 1
        if ret > 1: break
        nextlevel = list()
        for n in thislevel:
            if n.key in visited: continue
            visited[n.key] = None
            nextlevel += n.adjacent
        thislevel = nextlevel
        i += 1
            
    return ret <= 1

keys = list(range(10))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)
print(isbalanced(bst.root))

nbst = BinarySearchTree()
for k in keys: nbst[k] = data[k]
print(isbalanced(nbst.root))
print(nbst.root.height)

True
False
9


##### cracking: 4.5

In [128]:
keys = list(range(30))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)

bt = TreeNode(key=keys[1], value=data[keys[1]])
bt.right = TreeNode(key=keys[2], value=data[keys[2]])
bt.left = TreeNode(key=keys[0], value=data[keys[0]])

# if this line is commented, bt becomes a BST.
bt.right.left = TreeNode(key=keys[3], value=data[keys[3]])

def isbst(node):
    
    def _traverse(n):
        left, right = True, True
        if n.left: left = n.key > n.left.key and _traverse(n.left)
        if n.right: right = n.key < n.right.key and _traverse(n.right)
        return left and right
    
    return _traverse(node)

print(isbst(bst.root))
print(isbst(bt))

True
False


##### cracking: 4.7

In [152]:
keys = list(range(20))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)

bt = TreeNode(key=keys[1], value=data[keys[1]])
bt.right = TreeNode(key=keys[2], value=data[keys[2]], parent=bt)
bt.left = TreeNode(key=keys[0], value=data[keys[0]], parent=bt)

bt.right.left = TreeNode(key=keys[3], value=data[keys[3]], parent=bt.right)
bt.left.right = TreeNode(key=keys[4], value=data[keys[4]], parent=bt.left)

n1 = bt.right.left
n2 = bt.left.right

def common_parent(n1, n2):
    if n1.isroot or n2.isroot: return None
    d1, d2 = n1.depth, n2.depth

    while d1 != d2:
        if d1 > d2:
            d1 -= 1
            n1 = n1.parent
        else:
            d2 -= 1
            n2 = n2.parent

    while n1.parent.key != n2.parent.key:
        n1, n2 = n1.parent, n2.parent

    return n1.parent.key
    
print(common_parent(n1, n2))
print(common_parent(bst._get(0, bst.root), bst._get(6, bst.root)))

1
5


##### cracking: 4.6

In [170]:
#### see TreeNode.successor
keys = list(range(20))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)
print(bst._get(4, bst.root).successor.key == 5)
print(bst._get(19, bst.root).successor == None)
print(bst._get(6, bst.root).predecessor.key == 5)
print(bst._get(0, bst.root).predecessor ==  None)

True
True
True
True


##### cracking 4.4

In [177]:
def lists(root):
    keys = lambda x: [e.key for e in x]
    thislevel = [root]
    lists = [keys(thislevel)]
    i = 0
    while thislevel:
        nextlevel = list()
        for n in thislevel:
            nextlevel += n.adjacent
        if nextlevel: lists.append(lists[i] + keys(nextlevel))
        thislevel = nextlevel
        i += 1
            
    return lists

keys = list(range(20))
data = {i: True for i in keys}
bst = BinarySearchTree()
balancedtree(bst, keys, data)
lists(bst.root)

[[10],
 [10, 5, 15],
 [10, 5, 15, 2, 8, 13, 18],
 [10, 5, 15, 2, 8, 13, 18, 1, 4, 7, 9, 12, 14, 17, 19],
 [10, 5, 15, 2, 8, 13, 18, 1, 4, 7, 9, 12, 14, 17, 19, 0, 3, 6, 11, 16]]

#### heaps

This module provides an implementation of the heap queue algorithm, also known as the priority queue algorithm.

Heaps are binary trees for which every parent node has a value less than or equal to any of its children. This implementation uses arrays for which heap[k] <= heap[2*k+1] and heap[k] <= heap[2*k+2] for all k, counting elements from zero. For the sake of comparison, non-existing elements are considered to be infinite. The interesting property of a heap is that its smallest element is always the root, heap[0].

The API below differs from textbook heap algorithms in two aspects: (a) We use zero-based indexing. This makes the relationship between the index for a node and the indexes for its children slightly less obvious, but is more suitable since Python uses zero-based indexing. (b) Our pop method returns the smallest item, not the largest (called a “min heap” in textbooks; a “max heap” is more common in texts because of its suitability for in-place sorting).

These two make it possible to view the heap as a regular Python list without surprises: heap[0] is the smallest item, and heap.sort() maintains the heap invariant!

To create a heap, use a list initialized to [], or you can transform a populated list into a heap via function heapify().

In [ ]:
import heapq

h = []
heapq.heappush(h, 1)
heapq.heappush(h, -11)
heapq.heappush(h, 30)
heapq.heappush(h, -2)
print(h)
print(heapq.heappop(h))
print(h)

#### priority queue

Python doc's implementation:

In [4]:
import itertools
from heapq import heappush, heappop


class PriorityQueue(object):
    
    def __init__(self):
        self.pq = []                         # list of entries arranged in a heap
        self.entry_finder = {}               # mapping of tasks to entries
        self.REMOVED = '<removed-task>'      # placeholder for a removed task
        self.counter = itertools.count()     # unique sequence count

    def add_task(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove_task(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def remove_task(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED

    def pop_task(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            priority, count, task = heappop(self.pq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')

        
if True:
    h = PriorityQueue()
    h.add_task("call santi")
    print(h.entry_finder)
    h.add_task("call santi")
    print(h.entry_finder)
    print(h.pop_task())
    print(h.entry_finder)

{'call santi': [0, 0, 'call santi']}
{'call santi': [0, 1, 'call santi']}
call santi
{}


A better implementation:

In [11]:
import heapq

class PriorityQueue(object):

    """
    Combined priority queue and set data structure.

    Acts like a priority queue, except that its items are guaranteed to be
    unique. Provides O(1) membership test, O(log N) insertion and O(log N)
    removal of the smallest item.

    Important: the items of this data structure must be both comparable and
    hashable (i.e. must implement __cmp__ and __hash__). This is true of
    Python's built-in objects, but you should implement those methods if you
    want to use the data structure for custom objects.
    """

    def __init__(self, items=[]):
        """
        Create a new PriorityQueueSet.

        Arguments:
            items (list): An initial item list - it can be unsorted and
                non-unique. The data structure will be created in O(N).
        """
        self.set = dict((item, True) for item in items)
        self.heap = list(self.set.keys())
        heapq.heapify(self.heap)

    def has_item(self, item):
        """Check if ``item`` exists in the queue."""
        return item in self.set

    def pop(self):
        """Remove and return the smallest item from the queue."""
        smallest = heapq.heappop(self.heap)
        del self.set[smallest]
        return smallest

    def add(self, item):
        """Add ``item`` to the queue if doesn't already exist."""
        if item not in self.set:
            self.set[item] = True
            heapq.heappush(self.heap, item)
            
if True:
    h = PriorityQueue()
    h.add("call santi")
    h.add("call santi")
    h.add("call susi")
    h.add("call adri")
    print(h.pop())
    print(h.pop())
    print(h.pop())
    
    h = PriorityQueue()
    h.add((4, "call santi"))
    h.add((3, "call pablo"))
    h.add((2, "call adri"))
    h.add((1, "call susi"))
    print(h.pop())
    print(h.pop())
    print(h.pop())
    print(h.pop())

call adri
call santi
call susi
(1, 'call susi')
(2, 'call adri')
(3, 'call pablo')
(4, 'call santi')


#### hash table

If there are too many inputs, the load factor (number of inputs divided by table size) becomes too large, too many inputs will be inserted into one index and the table will become slow, that is, the time complexity will increase. We aim for a  O(1)  time complexity for a table.

Load factor  α=n/k  should be low.  n  is the number of inputs and  k  is the table size.

If the number of inputs exceeds a particular threshold, we use techniques like table doubling to reduce the load factor.

An instance of HashMap has two parameters that affect its performance: initial capacity and load factor. The capacity is the number of buckets in the hash table, and the initial capacity is simply the capacity at the time the hash table is created. The load factor is a measure of how full the hash table is allowed to get before its capacity is automatically increased. When the number of entries in the hash table exceeds the product of the load factor and the current capacity, the hash table is rehashed (that is, internal data structures are rebuilt) so that the hash table has approximately twice the number of buckets.

As a general rule, the default load factor (.75) offers a good tradeoff between time and space costs. Higher values decrease the space overhead but increase the lookup cost (reflected in most of the operations of the HashMap class, including get and put). The expected number of entries in the map and its load factor should be taken into account when setting its initial capacity, so as to minimize the number of rehash operations. If the initial capacity is greater than the maximum number of entries divided by the load factor, no rehash operations will ever occur.

In [ ]:
class Table(object):
    
    """A simple implementation of a hash table."""
    
    def __init__(self, buckets=10):
        """Set table initial capacity."""
        self.buckets = buckets
        self.table = [[] for i in range(self.buckets)]
       
    @property
    def load_factor(self):
        """Compute current load_factor"""
        return self.entries / self.buckets
    
    @property
    def entries(self):
        """Compute current load_factor"""
        return sum([len(bucket) for bucket in self.table])
    
    
    def __str__(self):
        """Return a representation of the table."""
        return self.__repr__()
            
    def __repr__(self):
        string = ""
        for bucket in self.table:
            string += "".join(["'%s': %s, " % p for p in bucket])
        return "{%s}" % string
            
    def get_bucket(self, key):
        """get bucket for a given key."""
        return hash(key) % self.buckets
        
    def generate(self):
        """"Generate table given """
        old = self.table
        self.buckets = 2 * self.buckets
        self.table = [[] for i in range(self.buckets)]
        for bucket in old:
            for key, value in bucket:
                self.insert(key, value)
        
    def insert(self, key, value):
        """Insert a new key-value pair."""
        bucket = self.get_bucket(key)
        self.table[bucket].append((key, value))
        if self.load_factor > 0.75: self.generate()
        
    def get(self, key):
        """Get value for a given key."""
        bucket = self.get_bucket(key)
        for key, value in self.table[bucket]:
            if key == key:
                return value

    def pop(self, key):
        """Remove key value pair."""
        bucket = self.get_bucket(key)
        for i, pair in enumerate(self.table[bucket]):
            if key == pair[0]:
                self.table[bucket].pop(i)
                self.entries -= 1
                
if True:
    table = Table()
    for i in range(100):
        table.insert(i, i)
    print(table)

# Algorithms

#### heapsort

In [ ]:
import heapq
def heapsort(iterable):
    h = []
    for value in iterable:
        heapq.heappush(h, value)
    return [heapq.heappop(h) for i in range(len(h))]
heapsort([1, 3, 5, 7, 9, 2, 4, 6, 8, 0])

#### quicksort

In [ ]:
def qsort(array):
    """Sort array using quick sort."""
    less, equal, greater = [], [], []

    if len(array) > 1:
        pivot = array[0]
        for x in array:
            if x < pivot:
                less.append(x)
            if x == pivot:
                equal.append(x)
            if x > pivot:
                greater.append(x)

        # Don't forget to return something!
        # Note that you want equal, not pivot
        return qsort(less) + equal + qsort(greater)
    
    # when you only have one element in your array
    # just return the array.
    else:
        return array
if True:
    array = [54,26,93,17,77,31,44,55,20]
    print(qsort(array))

#### mergesort

In [ ]:
def msort(array):
    """Sort array using merge sort."""
    # print message to learn
    if len(array) > 1: print("split", array)

    # if len of array is 1 or 0, return array.    
    if len(array) < 2: return array
    
    result, mid = [], int(len(array) / 2)
    left, right = msort(array[:mid]), msort(array[mid:])
    l, r = 0, 0
    while l < len(left) and r < len(right):
        if left[l] > right[r]:
            result.append(right[r])
            r += 1
        else:
            result.append(left[l])
            l += 1

    # print message to learn.
    if len(result) > 2: print("merging", l, r)
    return result + left[l:] + right[r:]
if True:
    l = [54,26,93,17,77,31,44,55,2,34,7,464]
    print(msort(l))

# Math

#### lcm & gdc

In [ ]:
from functools import reduce
import math

def gcd(*integers):
    """Return the greatest common divisor of integers."""
    def gcd(a, b):
        for i in range(min(a, b), 0, -1):
            if a % i == 0 and b % i == 0: return i
    return reduce(gcd, integers)

def lcm(*numbers):
    """Return the lowest common multiple of integers."""
    def lcm(a, b):
        return (a * b) // gcd(a, b)
    return reduce(lcm, numbers, 1)

#### primes

In [ ]:
def primes(n):
    """Get primes between 0 and n using sieve of Erathostenes."""
    res = []
    composite = [False for i in range(n + 1)]
    for p in range(2, n + 1):
        if not composite[p]:
            for i in range(p, n + 1, p):
                composite[i] = True
            res.append(p)
    return res
 
def get_exponent(x, p):
    """Get the number of times x can be evenly devided by p."""
    r = 0
    while x % p == 0:
        r += 1
        x /= p
    return r

def pfactorize(n):
    """Return n's prime factorization."""
    r = []
    for p in primes(n):
        e = get_exponent(n, p)
        if e: r.append((p, e))
    return r

#### combinatronics

In [ ]:
def choose_iter(elements, length):
    for i in range(len(elements)):
        if length == 1:
            yield (elements[i],)
        else:
            for e in choose_iter(elements[i+1:len(elements)], length-1):
                yield (elements[i],) + e

def choose(l, k):
    return list(choose_iter(l, k))
if True:
    list(choose_iter([2, 3, 4], 2))

#### primesoccer: https://community.topcoder.com/stat?c=problem_statement&pm=10033&rd=13513&rm=&cr=22736218

Please use this Google doc to code during your interview. To free your hands for coding, we recommend that you use a headset or a phone with speaker option.


 Problem Statement for PrimeSoccer


Problem Statement
    	You are watching a soccer match, and you wonder what the probability is that at least one of the two teams will score a prime number of goals. The game lasts 90 minutes, and to simplify the analysis, we will split the match into five-minute intervals. The first interval is the first five minutes, the second interval is the next five minutes, and so on. During each interval, there is a skillOfTeamA percent probability that team A will score a goal, and a skillOfTeamB percent probability that teamB will score a goal. Assume that each team will score at most one goal within each interval. Return the probability that at least one team will have a prime number as its final score.
 
Definition
    	
Class:	PrimeSoccer
Method:	getProbability
Parameters:	int, int
Returns:	double
Method signature:	double getProbability(int skillOfTeamA, int skillOfTeamB)
(be sure your method is public)
    
 
Notes
-	The returned value must be accurate to within a relative or absolute value of 1E-9.
-	A prime number is a number that has exactly two divisors, 1 and itself. Note that 0 and 1 are not prime.
 
Constraints
-	skillOfTeamA will be between 0 and 100, inclusive.
-	skillOfTeamB will be between 0 and 100, inclusive.
 
Examples
0)	
    	
50
50
Returns: 0.5265618908306351
1)	
    	
100
100
Returns: 0.0
Both teams will score a goal in each interval, so the final result will be 18 to 18.
2)	
    	
12
89
Returns: 0.6772047168840167
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 422 Round 1 - Division I, Level One 
       Single Round Match 422 Round 1 - Division II, Level Two


In [ ]:
class PrimeSoccer(object):

    def getProbability(self, a, b):
        """
        Given skills of team A and B, get the probability of a prime in result.

        Parameters:
            a(int): integer between 0 and 100
        b(int): integer between 0 and 100
        Return:
        float: probability of a prime in result.
        """
        f = math.factorial
        a, b = a / 100, b / 100
        pa, pb = 0, 0 
        for k in [2, 3, 5, 7, 11, 13, 17]:
            nck = f(18) / (f(k) * f(18 - k))
            pa += nck * (a ** k) * ((1 - a) ** (18 - k))
            pb += nck * (b ** k) * ((1 - b) ** (18 - k))
        return pa + pb - pa * pb
if True:
    f = PrimeSoccer().getProbability
    print(f(50, 50))
    print(f(12, 89))

#### catandrat: https://community.topcoder.com/stat?c=problem_statement&pm=12932


Problem Statement
    	We have a circular tube. At one point the tube has an entrance. 



At time 0, a rat enters the tube via the entrance. The rat can move inside the tube in both directions (clockwise and counterclockwise). The rat can change direction in an instant. Its maximum speed is Vrat. 



At time T, the cat will enter the tube via the same entrance. The cat can also move in both directions and change its direction instantly. Its maximum speed is Vcat. 



For the purpose of this problem, you should imagine the tube as a circle with radius R, and the rat and the cat as points on said circle. 



The cat is trying to catch the rat as quickly as possible. The rat is trying not to be caught, and if that is impossible, to be caught as late as possible. At any time (starting at time 0) the cat and the rat know each other's position. Already at time 0 the rat knows the time T. Assume that both the cat and the rat behave optimally. 



You are given the ints R, T, Vrat, and Vcat. If the cat will catch the rat successfully, return how much time it'll take the cat to catch the rat. Otherwise, return -1.0.
 
Definition
    	
Class:	CatAndRat
Method:	getTime
Parameters:	int, int, int, int
Returns:	double
Method signature:	double getTime(int R, int T, int Vrat, int Vcat)
(be sure your method is public)
    
 
Notes
-	The cat and rat cannot leave the tube.
-	Your return value must have a relative or absolute error less than 1e-9.
 
Constraints
-	R, T, Vrat, and Vcat will each be between 1 and 1000, inclusive.
 
Examples
0)	
    	
10
1
1
1
Returns: -1.0
The rat can survive. During the first unit of time it has to run away from the entrance. Then, once the cat enters the tube, the rat can always run in the same direction and with the same speed as the cat.
1)	
    	
10
1
1
2
Returns: 1.0
The cat is now faster than the rat. The best strategy for the rat is to enter the tube and to start running in either direction at its maximum speed. The cat will then enter the tube, run in the same direction as the rat, and catch it in exactly 1 unit of time.
2)	
    	
10
1
2
1
Returns: -1.0
The rat is now faster than the cat. It can survive, for example by using the strategy described in Example 0.
3)	
    	
1000
1000
1
1000
Returns: 1.001001001001001
4)	
    	
1
1000
1
2
Returns: 3.141592653589793
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 623 Round 1 - Division II, Level Two

In [ ]:
import math
class CatAndRat(object):
    
    def getTime(self, R, T, Vr, Vc):
        """Return time required to catch the rat."""
        if Vr >= Vc: return -1.0
        S = math.pi * R
        Dr = min(Vr * T, S)
        return Dr / (Vc - Vr)
    
if True:
    f = CatAndRat().getTime
    print(f(10, 1, 1, 1))
    print(f(10, 1, 1, 2))
    print(f(10, 1, 2, 1))
    print(f(1000, 1000, 1, 1000))
    print(f(1, 1000, 1, 2))

#### missinglcm: https://community.topcoder.com/stat?c=problem_statement&pm=13766

 Problem Statement for MissingLCM


Problem Statement
    	
The least common multiple (denoted "lcm") of a non-empty sequence of positive integers is the smallest positive integer that is divisible by each of them. For example, lcm(2)=2, lcm(4,6)=12, and lcm(1,2,3,4,5)=60.

Alice had a positive integer N. Then she chose some positive integer M that was strictly greater than N. Afterwards, she computed two values: the value A = lcm(N+1, N+2, ..., M) and the value B = lcm(1, 2, ..., M). She was surprised when she saw that A = B.

You are given the int N. Find and return the smallest M Alice could have chosen. (Such an M will always exist.)

 
Definition
    	
Class:	MissingLCM
Method:	getMin
Parameters:	int
Returns:	int
Method signature:	int getMin(int N)
(be sure your method is public)
    
 
Constraints
-	N will be between 1 and 1,000,000, inclusive.
 
Examples
0)	
    	
1
Returns: 2
Alice needs to choose an M > 1 such that lcm(2,...,M) = lcm(1,...,M). We can see M=2 is the minimum value that works, since lcm(1,2) = lcm(2) = 2.
1)	
    	
2
Returns: 4
2)	
    	
3
Returns: 6
We have lcm(4,5,6) = lcm(1,2,3,4,5,6) = 60.
3)	
    	
4
Returns: 8
4)	
    	
5
Returns: 10
5)	
    	
42
Returns: 82
Oh... that doesn't fit the pattern.
6)	
    	
999999
Returns: 1999966


In [ ]:
class MissingLCM(object):
    
    def getmin_improved(self, N):
        "return the smallest M Alice could have chosen."
        res = 0

        # For each prime number <= N
        for p in primes(N):
            max_exp = 0
            i = p;
            
            # first find the maximum exponent of p  among numbers <= N
            # (in the explanation , this max_exp is c)
            while i <= N:
                max_exp = max(max_exp, get_exponent(i, p))
                i += p

            # seek the minimum i such that get_exponent(i, p) >= max_exp:
            while get_exponent(i, p) < max_exp:
                i += p;

            # the maximum for all ps is the result:
            res = max(res, i)
        
        return res
    
    def getmin(self, N):
        "return the smallest M Alice could have chosen."
        # We know that M is strictly greater than N
        # Therefore it starts as M = N + 1.
        M = N + 1
        
        # SequenceM [sm] are all the integers from 1 to M inclusive
        sm = list(range(1, M + 1))
        
        # SequenceN[sn] are all the integers from N to the end of sequenceM.
        sn = sm[N:]
        while True:
            # If lcm for both sequence is the same, return break
            if lcm(*sm) == lcm(*sn): break
                
            # else, increase M and append it to [sm], then update [sn].
            M += 1
            sm.append(M)
            sn = sm[N:]
        return M

if True:
    f = MissingLCM().getmin_improved
    print([f(i) for i in [1, 2, 3, 4, 5, 42, 999999]])

#### fibonaccipositioning: https://community.topcoder.com/stat?c=problem_statement&pm=6160





 Problem Statement for FibonacciPositioning


Problem Statement
    	The fibonacci sequence is a sequence of integers in which each number is equal to the sum of the two preceding numbers. The first two integers in the sequence are both 1. Formally:
F1 = 1
F2 = 1
Fi = Fi-1 + Fi-2 for each i > 2
The beginning of this sequence is 1,1,2,3,5,8,13,21.

We'll define the fibonacci position of an integer greater than or equal to 1 as follows:
The fibonacci position of 1 is 2 (since F2 = 1)
The fibonacci position of any integer n > 1 such that Fi = n is i
The fibonacci position of any integer n > 1 such that it is strictly between Fi and Fi+1 is i+(n-Fi)/(Fi+1-Fi) (informally, this means it is linearly distributed between Fi and Fi+1)
As examples, if FP(n) is the fibonacci position of n,

FP(1)=2 (first rule) 

FP(5)=5 (second rule F5 = 5) 

FP(4)=4.5 (third rule, is right in the middle of F4 = 3 and F5 = 5)

Given an integer n, return its fibonacci position as a double.
 
Definition
    	
Class:	FibonacciPositioning
Method:	getFPosition
Parameters:	int
Returns:	double
Method signature:	double getFPosition(int n)
(be sure your method is public)
    
 
Notes
-	The returned value must be accurate to 1e-9 relative or absolute.
 
Constraints
-	n will be between 1 and 100000000 (108), inclusive.
 
Examples
0)	
    	
1
Returns: 2.0
1)	
    	
5
Returns: 5.0
2)	
    	
4
Returns: 4.5
Examples from the problem statement.
3)	
    	
100
Returns: 11.2
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 298 Round 1 - Division I, Level One 
       Single Round Match 298 Round 1 - Division II, Level Two


In [ ]:
class FibonacciPositioning(object):

    def getFPosition(self, n):
        """Return fibonacci position of n."""
        if n == 1: return 2.0
        fib = [0, 1, 1, 2]
        i, f = 2, 1
        while f <= n:
            f = fib[i + 1] + fib[i]
            fib.append(f)
            i += 1
        return i + (n - fib[i])/(fib[i + 1] - fib[i])
if True:
    f = FibonacciPositioning().getFPosition
    print(f(1) == 2.0)
    print(f(2) == 3.0)
    print(f(3) == 4.0)
    print(f(5) == 5.0)
    print(f(4) == 4.5)
    print(f(6) == 5.333333333333333)
    print(f(21) == 8.0)
    print(f(22) == 8.076923076923077)
    print(f(23) == 8.153846153846153)
    print(f(100000000) == 39.94028487238683)
    print(f(99999999) == 39.94028484680364)
    print(f(99999998) == 39.94028482122046)
    print(f(90000009) == 39.68445321652186)
    print(f(10) == 6.4)
    print(f(100) == 11.2)
    print(f(1000) == 16.021311475409835)
    print(f(10000) == 20.77373834011002)
    print(f(100000) == 25.538625776397517)
    print(f(1000000) == 30.32662490835795)
    print(f(10000000) == 35.13546384489119)
    print(f(34) == 9.0)
    print(f(55) == 10.0)
    print(f(144) == 12.0)
    print(f(233) == 13.0)
    print(f(377) == 14.0)
    print(f(610) == 15.0)
    print(f(987) == 16.0)
    print(f(63245986) == 39.0)
    print(f(39088169) == 38.0)
    print(f(24157817) == 37.0)
    print(f(14930352) == 36.0)
    print(f(9227465) == 35.0)
    print(f(5702887) == 34.0)
    print(f(3524578) == 33.0)
    print(f(3524579) == 33.00000045907169)
    print(f(3524577) == 32.9999992572064)
    print(f(63245987) == 39.00000002558319)
    print(f(63245985) == 38.99999995860553)
    print(f(31622993) == 37.5)
    print(f(31622994) == 37.500000066977655)
    print(f(31622992) == 37.499999933022345)
    print(f(9) == 6.2)
    print(f(99) == 11.181818181818182)
    print(f(999) == 16.01967213114754)
    print(f(9999) == 20.773499162879695)
    print(f(99999) == 25.53860420979986)
    print(f(9999999) == 35.13546366954141)
    print(f(999999) == 30.32662296369905)
    print(f(11) == 6.6)
    print(f(111) == 11.4)
    print(f(1111) == 16.20327868852459)
    print(f(11111) == 21.024390243902438)
    print(f(111111) == 25.778252242926158)
    print(f(1111111) == 30.542697903074313)
    print(f(7) == 5.666666666666667)
    print(f(8) == 6.0)
    print(f(9) == 6.2)
    print(f(10) == 6.4)
    print(f(12) == 6.8)
    print(f(13) == 7.0)
    print(f(14) == 7.125)
    print(f(15) == 7.25)
    print(f(16) == 7.375)
    print(f(17) == 7.5)
    print(f(18) == 7.625)
    print(f(19) == 7.75)
    print(f(20) == 7.875)
    print(f(53532) == 24.249991276127997)
    print(f(60696) == 24.499982552255993)
    print(f(67860) == 24.74997382838399)
    print(f(86617) == 25.25)
    print(f(98209) == 25.5)
    print(f(109801) == 25.75)
    print(f(140149) == 26.249996667777406)
    print(f(158905) == 26.499993335554816)
    print(f(177661) == 26.749990003332222)
    print(f(226766) == 27.24999794057318)
    print(f(257114) == 27.499995881146358)
    print(f(287462) == 27.74999382171954)
    print(f(366915) == 28.249997454408454)
    print(f(416020) == 28.5)
    print(f(465124) == 28.749997454408454)
    print(f(593681) == 29.249997640106855)
    print(f(673134) == 29.499998426737903)
    print(f(752587) == 29.74999921336895)
    print(f(960597) == 30.249999513835277)
    print(f(1089154) == 30.49999902767055)
    print(f(1217711) == 30.74999854150583)
    print(f(1554279) == 31.25)
    print(f(1762289) == 31.5)
    print(f(1970299) == 31.75)
    print(f(2514876) == 32.2499998143016)
    print(f(2851443) == 32.4999996286032)
    print(f(3188010) == 32.7499994429048)
    print(f(4069155) == 33.24999988523208)
    print(f(4613732) == 33.49999977046415)
    print(f(5158309) == 33.74999965569623)
    print(f(6584031) == 34.24999985813905)
    print(f(7465176) == 34.5)
    print(f(8346320) == 34.74999985813905)
    print(f(10653186) == 35.24999986848766)
    print(f(12078908) == 35.49999991232511)
    print(f(13504630) == 35.749999956162554)
    print(f(17237218) == 36.24999997290697)
    print(f(19544084) == 36.49999994581394)
    print(f(21850950) == 36.7499999187209)
    print(f(27890405) == 37.25)
    print(f(31622993) == 37.5)
    print(f(35355581) == 37.75)
    print(f(45127623) == 38.249999989651386)
    print(f(51167077) == 38.499999979302764)
    print(f(57206531) == 38.74999996895415)
    print(f(100000000) == 39.94028487238683)
    print(f(100) == 11.2)
    print(f(1) == 2.0)
    print(f(2) == 3.0)
    print(f(10000000) == 35.13546384489119)
    print(f(21345341) == 36.695205996446475)
    print(f(13) == 7.0)
    print(f(2584) == 18.0)
    print(f(3) == 4.0)
    print(f(4) == 4.5)
    print(f(21) == 8.0)

#### pipecuts: https://community.topcoder.com/stat?c=problem_statement&pm=3994

 Problem Statement for PipeCuts


Problem Statement
    	A 100 meter long pipe must be cut in two places. It can only be cut at certain places, where it was originally welded from smaller pipes. If the two cut locations are chosen at random (each potential location has equal probability of being chosen), find the probability of a resulting pipe being longer than L meters. 

Create a method named probability that accepts a int[] weldLocations and int L as parameters. It should calculate the probability of one or more resulting pipes being strictly longer than L if the two cut locations are chosen at random from weldLocations. Each element in weldLocations represents the number of meters from the left end of the pipe.
 
Definition
    	
Class:	PipeCuts
Method:	probability
Parameters:	int[], int
Returns:	double
Method signature:	double probability(int[] weldLocations, int L)
(be sure your method is public)
    
 
Notes
-	Your return value must have a relative or absolute error less than 1e-9.
 
Constraints
-	weldLocations will have between 2 and 50 elements, inclusive.
-	Each element in weldLocations will be between 1 and 99, inclusive.
-	weldLocations will not contain duplicate elements.
-	L will be between 1 and 100, inclusive.
 
Examples
0)	
    	
{25, 50, 75}
25
Returns: 1.0
Any random set of cuts results in a pipe being longer than 25 meters.
1)	
    	
{25, 50, 75}
50
Returns: 0.0
This time, it is impossible to cut the pipe such that some resulting pipe is longer than 50 meters.
2)	
    	
{25, 50, 75}
24
Returns: 1.0
3)	
    	
{99, 88, 77, 66, 55, 44, 33, 22, 11}
50
Returns: 0.7222222222222222
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 233 Round 1 - Division I, Level One 
       Single Round Match 233 Round 1 - Division II, Level T



In [ ]:
class PipeCuts(object):

    def probability(self, l, L):
        """
        Return the probability of having a cut longer than L.

        Parameters:
            l(list): Weld locations starting from left of pipe.
            L(int): minimum length.
        Return:
            int: Probability of having a cut longer than L.
        """
        l.sort()
        n, ret = len(l), 0
        tot = n * (n - 1) / 2
        for i in range(n - 1):
            for j in range(i + 1, n):
                if l[i] > L or l[j] - l[i] > L or 100 - l[j] > L:
                    ret += 1
        return ret / tot
if True:
    f = PipeCuts().probability
    print(f([25, 50, 75], 25) == 1.0)
    print(f([25, 50, 75], 50) == 0.0)
    print(f([25, 50, 75], 24) == 1.0)
    print(f([99, 88, 77, 66, 55, 44, 33, 22, 11], 50) == 0.7222222222222222)
    print(f([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 34) == 0.9974489795918368)
    print(f([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 95) == 0.002551020408163265)
    print(f([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 60) == 0.4362244897959184)
    print(f([50, 51], 50) == 0.0)
    print(f([50, 51], 49) == 1.0)
    print(f([67, 21, 90, 77, 19, 38, 85, 91, 23, 29, 42], 71) == 0.18181818181818182)
    print(f([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 34) == 0.9974489795918368)
    print(f([31, 55, 22, 99, 1, 49, 21, 7, 3, 88, 89, 36, 41], 77) == 0.2948717948717949)
    print(f([2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 55) == 0.5892857142857143)
    print(f([99, 97, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 73, 71, 69, 67, 65, 63, 61, 59, 57, 55, 53, 51, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 51) == 0.9208163265306123)
    print(f([99, 97, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 73, 71, 69, 67, 65, 63, 61, 59, 57, 55, 53, 51, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98], 60) == 0.6048979591836735)
    print(f([54, 69, 86, 50, 21, 66, 35, 62, 55, 44, 25, 2, 36, 37, 53, 82, 14, 19, 92, 60, 68, 16, 87, 78, 29, 31, 34, 77, 33, 74, 51, 97, 46, 58, 39, 71, 72, 32, 88, 42], 74) == 0.07307692307692308)
    print(f([5, 92, 62, 69, 42, 11, 91, 24, 7, 78, 46, 44, 15, 90, 82, 71, 20, 39, 13, 96, 14, 19, 93, 38, 47, 12, 75, 45, 9, 2, 81, 73, 17, 64, 60, 26, 22, 84, 32, 67, 56, 30, 85, 66, 36], 86) == 0.04141414141414142)
    print(f([8, 92, 80, 13, 95, 86, 4, 76, 85, 94, 41, 91, 57, 47], 75) == 0.5274725274725275)
    print(f([12, 30, 87, 77, 22, 46, 98, 64, 20, 71, 44, 45, 36, 33, 92, 68, 97], 61) == 0.45588235294117646)
    print(f([74, 5, 72, 20, 29, 91, 80, 14, 30, 39, 46, 12, 95, 15, 93, 82, 79, 42, 25, 1, 84, 37, 7, 19, 9, 50, 35, 54, 44, 65, 68, 6, 28, 66, 99, 21], 69) == 0.353968253968254)
    print(f([18, 21, 20, 13, 92, 73, 80, 97, 27, 77, 16, 89, 7, 4, 65, 26, 1, 14, 25, 54, 28, 40, 34, 55, 86, 29, 98, 41, 2, 75, 94], 50) == 0.8451612903225807)
    print(f([12, 43, 73, 82, 3, 85, 7, 67, 66, 6, 21, 71, 96, 28, 14, 1, 36, 60, 62, 61, 94, 9, 79, 84, 50, 65, 80, 76, 44, 59, 87, 88, 32], 48) == 0.8428030303030303)
    print(f([51, 5, 19, 50, 55, 4, 33, 95, 57, 32, 13, 75, 21, 16, 42, 44, 14, 94, 96, 1, 53, 30, 88, 63, 25, 64, 83, 59, 20, 34, 99, 29, 54], 92) == 0.028409090909090908)
    print(f([36, 53, 89, 55, 63, 17, 85, 70, 60, 24, 58, 78, 14, 97, 52, 91, 64, 50, 44, 15, 4, 95, 46, 40, 82, 71, 88, 56, 77, 12, 41, 34, 75, 67, 57, 87, 2, 65, 69, 96], 44) == 0.8769230769230769)
    print(f([27, 40, 67, 21, 57, 75, 42, 93, 46, 78, 54, 29, 53, 74, 22, 44, 13, 97, 95, 39, 63, 70, 65, 81, 14, 82], 82) == 0.018461538461538463)
    print(f([33, 96, 73, 72, 50, 76, 49, 41, 22, 39, 34, 6, 30, 65, 14, 40, 7, 43, 27, 99, 8, 78, 87, 36, 32, 15, 45, 23, 77, 20, 56, 35, 53, 31, 54, 66], 36) == 0.9761904761904762)
    print(f([87, 49, 75, 41, 34, 9, 77, 1, 89, 66, 18, 58, 72, 31, 54, 43, 27, 60, 40, 38, 84, 86, 63, 70, 95, 69, 79, 26, 8, 55, 97, 17], 43) == 0.8508064516129032)
    print(f([85, 16, 97, 8, 52], 71) == 0.5)
    print(f([92, 17, 21, 10, 4, 90, 52, 66, 58, 95, 53, 22, 99, 1, 63], 61) == 0.5428571428571428)
    print(f([25, 50, 75], 50) == 0.0)
    print(f([30, 60, 90], 50) == 0.6666666666666666)
    print(f([99, 88, 77, 66, 55, 44, 33, 22, 11], 50) == 0.7222222222222222)
    print(f([50, 75], 49) == 1.0)
    print(f([1, 2], 50) == 1.0)
    print(f([99, 1], 98) == 0.0)
    print(f([80, 90, 10, 20], 50) == 1.0)
    print(f([24, 49], 50) == 1.0)
    print(f([20, 60, 70], 45) == 0.6666666666666666)
    print(f([15, 45, 56], 50) == 0.3333333333333333)
    print(f([25, 75], 70) == 0.0)
    print(f([25, 50, 60], 39) == 1.0)
    print(f([10, 30, 59, 1], 59) == 0.5)
    print(f([25, 75], 30) == 1.0)
    print(f([75, 76], 74) == 1.0)
    print(f([25, 26], 25) == 1.0)
    print(f([26, 40], 25) == 1.0)

#### buildbridge: https://community.topcoder.com/stat?c=problem_statement&pm=6122

In [ ]:
class BuildingBridge(object):

    def howManyCards(self, D, L):
        """Return minimun number of cards required to build bridge."""
        ret, d = 0, 0
        while True:
            ret += 1
            d += L / (ret * 2)
            if d - D >= 0: break
        return ret    
if True:
    f = BuildingBridge().howManyCards
    print(f(1, 1) == 4)
    print(f(9, 1) == 36865412)
    print(f(4, 6) == 2)

#### banklottery: https://community.topcoder.com/stat?c=problem_statement&pm=10988

In [ ]:
class BankLottery(object):

    def expectedAmount(self, b, p, w):
        return w * p * b[0] / sum(b) + b[0]
if True:
    f = BankLottery().expectedAmount
    print(math.isclose(f([100, 100], 100, 2), 200.0))
    print(math.isclose(f([2, 2, 2], 1, 2), 2.6666666666666665))
    print(math.isclose(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 100, 20), 37.36363636363636))
    print(math.isclose(f([0, 200, 200, 0, 300, 300, 600], 3, 776), 0.0))
    print(math.isclose(f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 1, 1000), 63.5))
    print(math.isclose(f([1, 100], 100, 1000), 991.09900990099))
    print(math.isclose(f([1, 0], 3, 789), 2368.0))
    print(math.isclose(f([91609, 99034, 57486, 66963, 20912, 22937, 70238, 77187], 283, 813), 133233.64918853162))
    print(math.isclose(f([86073, 769], 574, 1), 86641.91713686926))
    print(math.isclose(f([72744, 20973, 10106, 22274, 79560, 92181], 399, 812), 151874.88011603622))
    print(math.isclose(f([40832, 73970, 55327, 42687, 3328, 87546, 72037, 15960, 6732, 9680, 95387, 61281, 59718, 56342, 58328, 94553, 86113, 91198, 80611, 95908, 80529, 81116, 64574, 37448, 78082, 89648, 26608, 58791, 26686, 46577, 88187, 36873, 89793, 28930], 998, 767), 56294.4601569225))
    print(math.isclose(f([52003, 97545, 73128, 64331, 21393, 72606, 50113, 94943, 38974, 50632, 25344, 25015, 17425, 32203, 75740, 88258, 76730, 40345, 35172, 22229, 63998, 50240, 97267, 71923, 29574, 7261, 71838], 918, 149), 56921.3562407086))
    print(math.isclose(f([38723, 87300, 32586], 98, 779), 57361.23153793291))
    print(math.isclose(f([31077, 912, 5813, 48038, 17249, 6404, 69510, 87374, 72576], 35, 457), 32543.506020008674))
    print(math.isclose(f([73072, 12100, 79712, 80138, 89491, 87638, 52059, 54127, 79247, 70755, 56373, 49755, 1985, 77329, 40203, 7194, 14116, 8863, 36911, 64287, 35166], 495, 246), 81383.81960933041))
    print(math.isclose(f([22646, 48493, 20099, 10904, 8370, 94540, 51677, 46577, 21712, 66136, 25736, 25336, 1626, 21758, 83031, 5827, 84794, 32464, 82517, 56977, 12743, 15328, 37319, 8012, 8440, 80690, 27843, 74326, 16821, 1191, 49346, 29021, 61953, 39414, 80009, 29810, 78533, 62828, 69732, 77445, 99518, 74934, 16983, 47279, 40409, 81797, 17705], 270, 425), 23913.222474445894))
    print(math.isclose(f([4791, 6075], 928, 180), 78441.71231363887))
    print(math.isclose(f([92491, 52918, 35418, 11114, 72933, 77677, 88336, 87811, 32872, 91832, 28849, 44429, 64746], 33, 351), 93861.98490835984))
    print(math.isclose(f([91702, 18158, 23665, 34677, 10804, 69996, 32487, 11702, 34358, 12463, 5890, 6881, 2432, 31533, 94283, 36567, 81863, 77475, 67990, 95628, 20022, 3581, 23964, 73496, 55489, 79181, 50335, 63969, 57378, 9266, 69665, 20622, 42729, 44419, 53908, 43114, 6747, 12904, 2956, 24983, 54588, 56468, 65607], 698, 833), 121724.69815112518))
    print(math.isclose(f([24607, 12611, 97953, 28424, 61372, 85382, 62236, 55855, 48642, 20223, 28836, 40037, 59734, 87041, 9478, 7482, 31725, 87806], 244, 749), 29901.141687974723))
    print(math.isclose(f([7040, 22912, 14988, 61123, 31791, 84134, 95831, 45126, 64687, 81169, 39352, 45983, 81827, 69665, 67481, 84564, 86170, 62310, 53710, 23914, 13123, 72659, 64803, 38511, 22244, 83745, 72218, 34586, 4734, 81975, 45493, 61878, 8209, 4532, 56958], 849, 786), 9665.33386608697))
    print(math.isclose(f([28551, 30838, 52972, 73156, 22323, 2734, 4952, 14929, 32918, 55695, 28936, 76977, 7958, 88027, 65289, 91899, 59301, 97118, 32291, 30705, 56618, 33288, 66910, 93132, 4904, 7079, 82278, 28463, 82087, 91952, 37226, 59584, 43653, 40219, 13888, 78603, 71382, 28058, 64189], 300, 729), 31870.421322409125))
    print(math.isclose(f([93914, 96921, 62998, 75625, 3042, 86251, 21817], 796, 944), 254091.84663434475))
    print(math.isclose(f([80434, 67623, 36027, 47068, 5610, 57280, 17484, 49648, 7523, 55507], 857, 429), 150145.27429727206))
    print(math.isclose(f([53025, 91484, 93306, 54497, 4533, 32972], 948, 841), 181202.552097072))
    print(math.isclose(f([51301, 22272, 33511, 75581, 9901, 57339, 17617, 89750, 56114, 6335, 16188, 84365, 25627, 72784, 28026, 61579, 30900, 34874, 88663, 61351, 70008, 85579, 18168, 79019, 14595, 99171, 37274, 92708, 70910, 95782, 88153, 22944, 2603], 321, 761), 58668.38302178964))
    print(math.isclose(f([91300, 29536, 22129, 95818, 54688, 52269, 48316, 15024, 43011], 495, 402), 131486.1284564391))
    print(math.isclose(f([48596, 292], 769, 906), 741148.6416298478))
    print(math.isclose(f([3488, 275], 253, 866), 206574.32048897157))
    print(math.isclose(f([19702, 189], 8, 556), 24107.73606153537))
    print(math.isclose(f([5215, 436], 934, 895), 776649.2505751194))
    print(math.isclose(f([15961, 486], 471, 326), 164969.7983218824))
    print(math.isclose(f([36036, 309], 263, 220), 93404.08254230293))
    print(math.isclose(f([45937, 68], 566, 281), 204747.9140745571))
    print(math.isclose(f([51051, 44], 983, 677), 715968.9184068892))
    print(math.isclose(f([99637, 90], 646, 470), 402982.9939635204))
    print(math.isclose(f([98755, 390], 280, 318), 187444.74935700238))
    print(math.isclose(f([42461, 486], 699, 58), 82544.21563787924))
    print(math.isclose(f([75416, 165], 437, 760), 406810.9526997526))
    print(math.isclose(f([84981, 455], 464, 176), 166210.08825319537))
    print(math.isclose(f([17369, 401], 375, 451), 182677.5045019696))
    print(math.isclose(f([66992, 376], 290, 116), 100444.24557653486))
    print(math.isclose(f([97255, 212], 438, 845), 466559.97553017945))
    print(math.isclose(f([91255, 217], 483, 834), 493121.38107836276))
    print(math.isclose(f([20870, 360], 282, 586), 183319.7993405558))
    print(math.isclose(f([40696, 242], 131, 577), 115836.17665738435))
    print(math.isclose(f([13647, 138], 382, 911), 358165.1932535364))
    print(math.isclose(f([53568, 0, 0, 0, 0], 559, 882), 546606.0))
    print(math.isclose(f([28716, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 919, 594), 574602.0))
    print(math.isclose(f([37183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 296, 293), 123911.0))
    print(math.isclose(f([54287, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 469, 297), 193580.0))
    print(math.isclose(f([24769, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 671, 475), 343494.0))
    print(math.isclose(f([53004, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 555, 763), 476469.0))
    print(math.isclose(f([81060, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 555, 593), 410175.0))
    print(math.isclose(f([39045, 0, 0, 0, 0, 0, 0, 0, 0], 895, 956), 894665.0))
    print(math.isclose(f([10675, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 214, 685), 157265.0))
    print(math.isclose(f([57676, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 614, 463), 341958.0))
    print(math.isclose(f([0, 280, 693, 807, 779, 736, 829, 797, 583, 848, 303, 948, 385, 998, 686, 200, 103, 601, 390, 526, 118, 409, 629, 888, 750, 857, 528, 920, 654, 916, 253, 880, 436, 432, 650, 588, 439, 293, 296, 955, 354, 91, 900, 412], 326, 97), 0.0))
    print(math.isclose(f([0, 935, 746, 125, 793, 235, 581, 559, 121, 606, 931, 695, 362, 938, 829, 310, 777, 121, 712, 474, 719, 619, 542, 754, 55, 921, 893, 74, 983, 169, 478, 526, 427, 844, 470, 711, 215, 281, 46, 363, 99, 571, 256, 201], 542, 955), 0.0))
    print(math.isclose(f([0, 433, 474, 872, 532, 808, 833, 379, 191, 358, 173, 381, 13, 228, 141], 442, 180), 0.0))
    print(math.isclose(f([0, 31, 881], 511, 239), 0.0))
    print(math.isclose(f([0, 1000, 348, 750, 217, 969, 126, 366, 479, 543, 337, 738, 728, 204, 840, 209, 366, 454, 997, 369, 609, 542, 482, 552, 908, 272, 521, 75, 81, 601, 744, 552, 338, 204, 927, 840, 125, 248, 258, 491, 98, 893, 540, 36, 77, 840, 307, 408, 241, 205], 549, 488), 0.0))
    print(math.isclose(f([0, 994, 391, 198, 89, 323, 670, 471, 151, 248, 908, 429, 798, 59, 613, 375, 107, 197, 588, 124, 601, 555, 876, 810, 520, 901, 469, 393, 690, 164, 234, 589, 307, 542, 685, 836, 465, 260, 431, 929, 603, 624, 985, 622, 723, 655, 34, 554, 174, 317], 425, 95), 0.0))
    print(math.isclose(f([0, 129, 586, 453, 456, 295, 919, 399, 13], 599, 419), 0.0))
    print(math.isclose(f([0, 156, 603, 833, 434, 722, 406, 843, 680, 941, 738, 958, 147, 688, 203, 521, 392, 295, 552, 798, 663, 377, 497, 322, 697, 962, 110, 54, 710, 940, 368, 542, 140, 821, 901, 57, 762, 327, 842, 587, 539, 447, 998, 376, 47, 392, 265], 972, 996), 0.0))
    print(math.isclose(f([0, 587, 416, 571, 42, 294, 581, 391, 31, 85, 76, 142, 68, 497, 822, 715, 266, 8, 173, 114, 367, 712, 31, 393, 844, 777, 114, 928, 49, 305, 459, 231, 67, 354, 233, 6, 400], 306, 356), 0.0))
    print(math.isclose(f([0, 790, 809, 328, 166, 661, 900, 478, 521, 615, 122, 199, 121, 78, 628, 421, 557, 121, 161, 31, 626, 86, 584, 638, 458, 247, 524, 18, 682, 513, 255, 850, 166, 599, 834, 747, 337], 400, 572), 0.0))
    print(math.isclose(f([98272, 98572, 99578, 99528, 98224, 99355, 99076, 97884, 98157, 99336, 99365, 98561, 98848, 99302, 98197, 98577, 99452, 98016, 99352, 99219, 98683, 99609, 98593, 99747, 98136, 99799, 99199, 99365, 98770, 98029, 99322, 99199, 98463, 98060, 99929, 99076, 99017, 99314, 98449, 99057, 97796, 97857, 98229, 98511, 99334, 99960, 98041, 98098, 98369, 99218], 243, 265), 99552.47296493393))
    print(math.isclose(f([99691, 98990, 98013, 98879, 98815, 98995, 99778, 98064, 98435, 99867, 98883, 98432, 99785, 98946, 98276, 99103, 99096, 98566, 98391, 99260, 99489, 99742, 98080, 98699, 98090, 99517, 99002, 98346, 98402, 97841, 98924, 99901, 99655, 99450, 99954, 99333, 97847, 98063, 99484, 98805, 99342, 98240, 98074, 98042, 98093, 99457, 99269, 98895, 98083, 98145], 797, 24), 100076.81249558677))
    print(math.isclose(f([99786, 99789, 98174, 99109, 98807, 99470, 99423, 98080, 97970, 99713, 99763, 98063, 97941, 98687, 98340, 99163, 99259, 98038, 98966, 97833, 98946, 99180, 99754, 99705, 99756, 99372, 97853, 98987, 99603, 99810, 99191, 99872, 98508, 99603, 98170, 99390, 98124, 98576, 97786, 99496, 99784, 99314, 97926, 97974, 98854, 98217, 99299, 99634, 98759, 99541], 983, 422), 108152.8550438436))
    print(math.isclose(f([97972, 97909, 99191, 99304, 98751, 98078, 99257, 98925, 98331, 98514, 99846, 99281, 98896, 98993, 98965, 97949, 98126, 98762, 99571, 98205, 98361, 99019, 97826, 98626, 99524, 98951, 99524, 99528, 99851, 99003, 98446, 98923, 98845, 99582, 99616, 98271, 99628, 98357, 98988, 98070, 98073, 99462, 99364, 97850, 99931, 99175, 98531, 99572, 99731, 98997], 204, 585), 100336.66666976779))
    print(math.isclose(f([98755, 99017, 97896, 98136, 98117, 99445, 99269, 99424, 99776, 98239, 99403, 98661, 98813, 99457, 98285, 98088, 98227, 99777, 99589, 98875, 98826, 98895, 98774, 99213, 98536, 98600, 99810, 98376, 98622, 98888, 98029, 99992, 99263, 99016, 97827, 99689, 99464, 97866, 99167, 98290, 98423, 99539, 98844, 99167, 99607, 99553, 99241, 99268, 98253], 233, 803), 102567.60163605663))
    print(math.isclose(f([99911, 98118, 99134, 98686, 98058, 97911, 99802, 99155, 97928, 98564, 99237, 98139, 99122, 98309, 99112, 98602, 98374, 97856, 99611, 98509, 98045, 97851, 99079, 97887, 98459, 98608, 99028, 99675, 98405, 99762, 97875, 97827, 98471, 99717, 98604, 98826, 98337, 99062, 99009, 98864, 99079, 98685, 99891, 97930, 98113, 98195, 98571, 99347, 99464, 98439], 895, 965), 117395.57632683942))
    print(math.isclose(f([98200, 98534, 99535, 98010, 99329, 99109, 97872, 98884, 98769, 98016, 99184, 98994, 98329, 99004, 99396, 98663, 98222, 99863, 99033, 98512, 99013, 98794, 98314, 99742, 99023, 98789, 97935, 97834, 99103, 99765, 99253, 97787, 99551, 98133, 98326, 99442, 98182, 98246, 99072, 98039, 97809, 99466, 98749, 98009, 98461, 99140, 97937, 99996, 98205, 99316], 651, 689), 107121.9202214188))
    print(math.isclose(f([99763, 99928, 98184, 99652, 97871, 99347, 98425, 99533, 98838, 99806, 99638, 98382, 97931, 99293, 98133, 98210, 97997, 99409, 98942, 98906, 98393, 98589, 97811, 99782, 99712, 98550, 98512, 99864, 99096, 98415, 97954, 98952, 99545, 98599, 98546, 98094, 97883, 98391, 98924, 99968, 98222, 98767, 99891, 99570, 99806, 99779, 97814, 98516, 99744], 814, 98), 101405.28147681008))
    print(math.isclose(f([99207, 98121, 98972, 99150, 99764, 98133, 99279, 98089, 97885, 98278, 98565, 97887, 99099, 98625, 99808, 98809, 99232, 99784, 99300, 99623, 98920, 98908, 99326, 99569, 99506, 99368, 99957, 99699, 99072, 99819, 99455, 98711, 99265, 97991, 98713, 98398, 98481, 98988, 99810, 99134, 99275, 99211, 99030, 98270, 98153, 98816, 99485, 97882, 99014], 110, 192), 99639.02529735026))
    print(math.isclose(f([99239, 98361, 97880, 99931, 98402, 98813, 99532, 99714, 99141, 98027, 98578, 98109, 98432, 99677, 98398, 98195, 98931, 99585, 98845, 98265, 98265, 98721, 99471, 99827, 98283, 98274, 99788, 98602, 99768, 99195, 99102, 99581, 98294, 98660, 97853, 99986, 98550, 99592, 98526, 97852, 98300, 98821, 98964, 98289, 98959, 98148, 98211, 98453, 98627, 98167], 418, 688), 105017.19100806935))
    print(math.isclose(f([99827, 98396, 98222, 98695, 99461, 98072, 99991, 99629, 99693, 98447, 98974, 98751, 98259, 99781, 99953, 99212, 99064, 99779, 99523, 98988, 98536, 97863, 98437, 99158, 99333, 98152, 98732, 98354, 99491, 98600, 99076, 98710, 99311, 98469, 99311, 99818, 99627, 99186, 98795, 99273, 100000, 99189, 99301, 98307, 98091, 98774, 99552, 98212, 98393, 98234], 213, 969), 103990.26227368669))
    print(math.isclose(f([99675, 99332, 98869, 98353, 98131, 99128, 98493, 98990, 99774, 98676, 98021, 99015, 98613, 98476, 98923, 99228, 99747, 98232, 99670, 99688, 98625, 98500, 99442, 98989, 99718, 99902, 99316, 97991, 98188, 98731, 98251, 98276, 98340, 99463, 98513, 98624, 98562, 98917, 98526, 98613, 98149, 99290, 99646, 98627, 99355, 99955, 98165, 97928, 99305, 99894], 553, 135), 101179.85043990346))
    print(math.isclose(f([99718, 99506, 98361, 98096, 97945, 97795, 98326, 99059, 98859, 98302, 99051, 98783, 99327, 98129, 99276, 98431, 99944, 98512, 98401, 98293, 98364, 98246, 98893, 98283, 99006, 98983, 99201, 98091, 97900, 98303, 97793, 99317, 98089, 99923, 98520, 97818, 99357, 98236, 98014, 98506, 98324, 99425, 99082, 98559, 99248, 97996, 99615, 99705, 98848, 98886], 7, 433), 99779.2496457192))
    print(math.isclose(f([98245, 99931, 99656, 98154, 98672, 98450, 98291, 99875, 99524, 98421, 99179, 99090, 98984, 97795, 99350, 98056, 98623, 97947, 98953, 98937, 98052, 98486, 98913, 98698, 98779, 98780, 97807, 97928, 99614, 98297, 99619, 99172, 99924, 98344, 99169, 98272, 99569, 98602, 98757, 99431, 98352, 98838, 99178, 98420, 99813, 98871, 99432, 99164, 98244, 99260], 147, 273), 99042.80160152394))
    print(math.isclose(f([98102, 98533, 97891, 98347, 98210, 98609, 99611, 99495, 98202, 98285, 98480, 98517, 99734, 97797, 98527, 98826, 98852, 98373, 99979, 98202, 99025, 98758, 98884, 99500, 99143, 99963, 98018, 99463, 99049, 99996, 99150, 99457, 98728, 99934, 99407, 98380, 98270, 98157, 99164, 98824, 99297, 98196, 98819, 99376, 99766, 99562, 98174, 98117, 99426], 206, 306), 99378.47062373892))
    print(math.isclose(f([98357, 99920, 99928, 99481, 99097, 99190, 99704, 98098, 98651, 98572, 99950, 99281, 98401, 98232, 98741, 99268, 98513, 99902, 98055, 99575, 98152, 99564, 99280, 99768, 99245, 98777, 97806, 98509, 99108, 99284, 99552, 98723, 99483, 99572, 99888, 98210, 98834, 98327, 98790, 99985, 99077, 99952, 98642, 98240, 98967, 98413, 99605, 98746, 99018], 306, 606), 102115.71828668217))
    print(math.isclose(f([99539, 97941, 97803, 99226, 99285, 99887, 98941, 99778, 99392, 98311, 99377, 99090, 99813, 99861, 99816, 98614, 99850, 97986, 98716, 97997, 98872, 99944, 98407, 99128, 98614, 99549, 98251, 99534, 97952, 98038, 99210, 99257, 99586, 97896, 98720, 98903, 97825, 97980, 98933, 98234, 99593, 98077, 98182, 98342, 98480, 98106, 98626, 98980, 99204, 98612], 888, 253), 104063.82041528386))
    print(math.isclose(f([98902, 99881, 99539, 99750, 99566, 99117, 99244, 98445, 97839, 99149, 98046, 98856, 98445, 99017, 98818, 98820, 98591, 98737, 98168, 98527, 98737, 99978, 98382, 98541, 99120, 98092, 98629, 99608, 97911, 97972, 99124, 98097, 98222, 99260, 99381, 99401, 98748, 99755, 99029, 99442, 98589, 98771, 98412, 98262, 98515, 99667, 98713, 98744, 97862, 98470], 826, 862), 113154.37084242498))
    print(math.isclose(f([98368, 99872, 98641, 98811, 98487, 99777, 99889, 97972, 99459, 97880, 99375, 99349, 99681, 99992, 98753, 99637, 99505, 98193, 98334, 99660, 99136, 98301, 99370, 98798, 98661, 99513, 99658, 98772, 99034, 99405, 97977, 99623, 98344, 99345, 98031, 98928, 98417, 98601, 98712, 97941, 98273, 98211, 98420, 97854, 99704, 99787, 99701, 98657, 98406], 986, 88), 100128.84506752847))
    print(math.isclose(f([98871, 98585, 97901, 98856, 98701, 98970, 98735, 98307, 99061, 99807, 99612, 98923, 98169, 97847, 99654, 99204, 99536, 99929, 97809, 99496, 98018, 98766, 98274, 99190, 99192, 98619, 98601, 98222, 98173, 98506, 99084, 99648, 97953, 99701, 99648, 98386, 98903, 99366, 98220, 98273, 99175, 98761, 97851, 98763, 98015, 99843, 98985, 98800, 99626, 97912], 867, 86), 100363.17661924113))
    print(math.isclose(f([3, 4, 1, 4, 3], 350, 457), 31993.0))
    print(math.isclose(f([3, 1], 5, 809), 3036.75))
    print(math.isclose(f([3, 4, 3, 2, 4], 866, 360), 58458.0))
    print(math.isclose(f([1, 4, 4, 3, 1], 292, 722), 16218.230769230771))
    print(math.isclose(f([2, 5], 596, 692), 117839.71428571428))
    print(math.isclose(f([1, 2, 2, 2, 4], 288, 859), 22491.18181818182))
    print(math.isclose(f([2, 2, 1], 429, 292), 50109.200000000004))
    print(math.isclose(f([2, 1, 3], 500, 396), 66002.0))
    print(math.isclose(f([5, 5, 5, 2, 4, 5], 484, 601), 55944.23076923077))
    print(math.isclose(f([5, 4], 719, 666), 266035.0))
    print(math.isclose(f([4, 3], 668, 923), 352326.2857142857))
    print(math.isclose(f([1, 1, 2], 424, 777), 82363.0))
    print(math.isclose(f([5, 5, 3, 4, 2, 4], 999, 47), 10212.173913043478))
    print(math.isclose(f([1, 2, 1, 1, 5], 888, 669), 59408.200000000004))
    print(math.isclose(f([1, 4, 1, 1], 824, 896), 105473.0))
    print(math.isclose(f([5, 2, 4, 3], 523, 272), 50810.71428571428))
    print(math.isclose(f([3, 5, 3], 690, 671), 126272.99999999999))
    print(math.isclose(f([5, 2, 2], 730, 266), 107882.7777777778))
    print(math.isclose(f([4, 1, 2, 1], 562, 880), 247284.0))
    print(math.isclose(f([4, 1, 1, 2, 5], 163, 17), 856.6153846153846))
    print(math.isclose(f([4, 2, 2], 1, 4), 6.0))
    print(math.isclose(f([1, 5, 4, 3, 3], 3, 1), 1.1875))
    print(math.isclose(f([1, 4, 1], 2, 5), 2.6666666666666665))
    print(math.isclose(f([4, 2, 5], 5, 1), 5.818181818181818))
    print(math.isclose(f([2, 3, 1, 2, 4], 4, 4), 4.666666666666666))
    print(math.isclose(f([2, 1, 3, 1, 1, 4], 1, 1), 2.1666666666666665))
    print(math.isclose(f([1, 3, 4, 1, 1], 5, 5), 3.5))
    print(math.isclose(f([1, 5, 2, 2, 4, 2], 1, 2), 1.125))
    print(math.isclose(f([5, 5, 4, 3, 2, 2], 4, 1), 5.9523809523809526))
    print(math.isclose(f([2, 4, 2, 3, 2], 1, 4), 2.6153846153846154))
    print(math.isclose(f([4, 3], 5, 2), 9.714285714285714))
    print(math.isclose(f([2, 4, 3, 5], 4, 2), 3.142857142857143))
    print(math.isclose(f([2, 2, 5, 2], 1, 4), 2.7272727272727275))
    print(math.isclose(f([5, 2, 5], 3, 4), 10.0))
    print(math.isclose(f([5, 5, 3, 3, 1], 2, 3), 6.764705882352941))
    print(math.isclose(f([2, 1, 3], 4, 5), 8.666666666666666))
    print(math.isclose(f([2, 4], 4, 4), 7.333333333333333))
    print(math.isclose(f([5, 5, 1, 3, 2, 3], 2, 1), 5.526315789473684))
    print(math.isclose(f([5, 3, 3, 1], 3, 4), 10.0))
    print(math.isclose(f([2, 4, 3, 1, 4], 5, 4), 4.857142857142857))

#### bigburger: https://community.topcoder.com/stat?c=problem_solution&cr=277659&rd=4550&pm=1648

In [60]:
class BigBurger(object):

    def maxWait(self, arrival, service):
        """Return the maximum waiting time at BigBurger."""
        n = len(arrival)
        if n == 1: return 0
        wait = [0]
        done = [arrival[0] + service[0]]

        for i in range(1, n):
            a = arrival[i]
            w = done[i - 1] - a if a < done[i - 1] else 0
            d =  arrival[i] + w + service[i]
            wait.append(w)
            done.append(d)

        return max(wait)
if True:
    f = BigBurger().maxWait
    print(f([3, 3, 9], [2, 15, 14]) == 11)
    print(f([182], [11]) == 0)
    print(f([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]) == 735)
    print(f([2, 10, 11], [3, 4, 3]) == 3)
    print(f([2, 10, 12], [15, 1, 15]) == 7)
    print(f([720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]) == 735)
    print(f([5, 5, 100, 100, 200, 200, 300, 300, 400, 400], [3, 12, 9, 1, 4, 5, 1, 9, 8, 15]) == 9)
    print(f([3, 3, 3, 700, 700, 700], [12, 3, 9, 13, 1, 15]) == 15)
    print(f([3, 7, 15, 15, 20, 29, 40, 41, 42, 43, 100, 103, 103, 109, 344, 703, 707, 708], [3, 9, 6, 3, 6, 8, 2, 12, 5, 3, 6, 8, 8, 3, 9, 11, 1, 1]) == 16)
    print(f([9], [3]) == 0)
    print(f([100, 200, 300, 400, 500, 600, 700], [1, 2, 3, 4, 5, 6, 7]) == 0)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) == 14)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [15, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]) == 15)
    print(f([1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15], [15, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1]) == 15)
    print(f([1, 2, 3, 4, 5, 6, 7, 7, 7, 10, 11, 12, 13, 14, 15], [15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) == 16)
    print(f([1, 2, 3, 4, 7, 7, 7, 7, 10, 11, 12, 13, 14, 15], [15, 1, 1, 1, 3, 3, 2, 3, 1, 1, 1, 1, 1, 1]) == 20)
    print(f([2, 10, 12], [15, 1, 5]) == 7)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 1, 1, 1, 1, 10, 3, 6, 9]) == 16)
    print(f([7, 7, 7, 7], [10, 15, 8, 10]) == 33)
    print(f([1, 1, 1], [3, 2, 2]) == 5)
    print(f([3, 3, 5, 6, 6], [3, 3, 3, 10, 10]) == 16)
    print(f([720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 720], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]) == 735)
    print(f([3, 5, 5], [1, 3, 1]) == 3)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


# DP

In [186]:
class RandomPanckakeStack(object):

    def expectedDeliciousness(self, d):
        n = len(d)
        ix = [i for i in range(n)]
        ex = [0 for i in range(n)]
        ex[0] = (1 / n) * d[0]
        for i in ix[1:]:
            exg = d[i] * ((n - i + 1) / (n - 1))
            exs = 0
            for j in range(1, i):
                print(i - j, - 1 - j)
                exs += sum(d[i - j:i + 1]) / (n - 1 - j)
            ex[i] = (exg + exs) / n
        return sum(ex)

if True:
    f = RandomPanckakeStack().expectedDeliciousness
    print(f([1, 1, 1]))
    print(f([3, 6, 10, 9, 2]))

1 -2
1.8333333333333333
1 -2
2 -2
1 -3
3 -2
2 -3
1 -4
18.716666666666665


#### incrementanddoubling: https://community.topcoder.com/stat?c=problem_statement&pm=12790&rd=15708&rm=&cr=22922995

In [67]:
class IncrementAndDoubling:
    def getMin(self, array):
        a = array
        n = len(a)
        res = 0
        while any(a):
            if all(e % 2 == 0 for e in a):
                a = [e / 2 for e in a]
                res += 1
            for i in range(n):
                if a[i] % 2 == 1:
                    a[i] -= 1
                    res += 1
        return res
if True:
    f = IncrementAndDoubling().getMin
    print(f([2, 1]) == 3)
    print(f([16, 16, 16]) == 7)
    print(f([100]) == 9)
    print(f([0, 0, 1, 0, 1]) == 2)
    print(f([123, 234, 345, 456, 567, 789]) == 40)
    print(f([7, 5, 8, 1, 8, 6, 6, 5, 3, 5, 5, 2, 8, 9, 9, 4, 6, 9, 4, 4, 1, 9, 9, 2, 8, 4, 7, 4, 8, 8, 6, 3, 9, 4, 3, 4, 5, 1, 9, 8, 3, 8, 3, 7, 9, 3, 8, 4, 4, 7]) == 84)
    print(f([2]) == 2)
    print(f([4]) == 3)
    print(f([5]) == 4)
    print(f([12]) == 5)
    print(f([22]) == 7)
    print(f([56]) == 8)
    print(f([91]) == 11)
    print(f([139]) == 11)
    print(f([484]) == 13)
    print(f([725]) == 15)
    print(f([210, 236, 258]) == 19)
    print(f([642, 424, 49, 124]) == 24)
    print(f([314, 667, 264, 929, 353]) == 31)
    print(f([764, 204, 555, 239, 148, 706]) == 39)
    print(f([503, 159, 241, 843, 77, 156, 329]) == 46)
    print(f([14, 6, 9]) == 10)
    print(f([26, 31, 25, 11]) == 18)
    print(f([42, 22, 35, 52, 23]) == 21)
    print(f([3, 37, 54, 98, 64, 120]) == 23)
    print(f([233, 218, 36, 40, 133, 81, 116]) == 31)
    print(f([2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2]) == 51)
    print(f([2, 3, 4, 3, 3, 3, 4, 2, 2, 4, 2, 2, 4, 4, 2, 2, 4, 3, 2, 2, 2, 4, 3, 3, 2, 2, 4, 2, 2, 2, 3, 2, 4, 3, 4, 2, 3, 2, 3, 4, 3, 4, 2, 3, 4, 4, 2, 4, 4, 4]) == 65)
    print(f([5, 5, 8, 7, 5, 5, 4, 6, 8, 4, 5, 4, 6, 6, 4, 6, 5, 7, 8, 5, 8, 4, 4, 6, 5, 5, 5, 5, 7, 6, 5, 5, 7, 7, 6, 4, 4, 4, 7, 6, 8, 4, 6, 4, 4, 7, 8, 4, 8, 8]) == 89)
    print(f([9, 12, 11, 14, 14, 11, 16, 11, 13, 14, 10, 9, 11, 11, 10, 12, 12, 15, 10, 11, 10, 9, 11, 9, 16, 16, 12, 11, 14, 16, 13, 15, 13, 11, 9, 12, 13, 8, 8, 15, 8, 13, 12, 15, 10, 12, 15, 8, 11, 11]) == 126)
    print(f([29, 19, 27, 26, 22, 31, 31, 18, 27, 26, 29, 29, 25, 23, 19, 28, 29, 18, 26, 29, 16, 21, 19, 24, 17, 21, 27, 23, 24, 21, 18, 23, 18, 32, 18, 17, 16, 28, 30, 24, 29, 17, 17, 21, 21, 29, 18, 23, 24, 19]) == 154)
    print(f([36, 63, 34, 40, 43, 41, 47, 33, 38, 48, 56, 57, 51, 62, 34, 38, 62, 49, 53, 52, 37, 60, 32, 50, 34, 37, 36, 61, 42, 59, 58, 64, 35, 45, 52, 49, 56, 41, 63, 50, 61, 64, 61, 32, 55, 32, 63, 33, 47, 48]) == 170)
    print(f([110, 85, 91, 101, 118, 124, 73, 114, 121, 120, 119, 104, 80, 106, 82, 114, 80, 119, 91, 126, 89, 78, 120, 114, 78, 82, 119, 80, 92, 120, 81, 68, 70, 79, 65, 126, 115, 83, 125, 93, 125, 101, 87, 124, 106, 119, 98, 70, 106, 65]) == 213)
    print(f([146, 255, 202, 165, 202, 206, 186, 184, 185, 137, 187, 255, 234, 159, 226, 130, 186, 201, 254, 160, 203, 151, 130, 178, 224, 158, 154, 139, 136, 180, 253, 250, 157, 224, 199, 235, 140, 135, 222, 201, 194, 212, 235, 172, 244, 194, 151, 242, 205, 176]) == 231)
    print(f([303, 394, 382, 298, 407, 337, 384, 290, 317, 308, 442, 436, 487, 401, 480, 277, 381, 259, 380, 439, 323, 475, 353, 433, 258, 464, 421, 289, 330, 282, 292, 475, 396, 504, 347, 359, 283, 312, 449, 352, 383, 491, 415, 395, 464, 471, 446, 321, 278, 358]) == 254)
    print(f([750, 735, 689, 977, 924, 711, 837, 955, 652, 935, 651, 888, 555, 608, 948, 719, 734, 591, 540, 764, 581, 522, 544, 552, 677, 868, 949, 933, 645, 656, 727, 935, 653, 570, 808, 713, 869, 807, 714, 878, 541, 874, 994, 655, 961, 924, 585, 734, 827, 818]) == 283)
    print(f([685, 655, 114, 407, 212, 7, 161, 524, 200, 298, 142, 591, 407, 551, 260, 312, 894, 758, 936, 705, 822, 660, 652, 624, 69, 122, 932, 970, 911, 51, 598, 239, 827, 124, 974, 505, 926, 500, 969, 530, 413, 981, 509, 92, 555, 161, 278, 259, 886, 381]) == 264)
    print(f([0, 755, 905, 371, 867, 594, 581, 166, 449, 588, 702, 251, 960, 669, 97, 348, 298, 681, 763, 654, 7, 986, 845, 997, 829, 257, 102, 861, 255, 217, 45, 717, 372, 889, 375, 945, 760, 225, 553, 229, 268, 8, 141, 211, 234, 748, 992, 480, 271, 401]) == 259)
    print(f([351, 126, 469, 754, 634, 934, 35, 188, 153, 528, 474, 575, 125, 397, 710, 597, 624, 235, 485, 639, 691, 6, 545, 918, 902, 55, 56, 104, 384, 741, 707, 326, 669, 757, 316, 628, 763, 997, 491, 950, 348, 898, 359, 18, 364, 460, 743, 587, 341, 265]) == 267)
    print(f([849, 174, 389, 560, 640, 838, 837, 708, 968, 483, 539, 730, 722, 315, 733, 791, 130, 671, 803, 446, 79, 830, 834, 334, 467, 852, 159, 767, 534, 61, 310, 25, 638, 652, 666, 494, 148, 909, 923, 654, 233, 406, 422, 96, 344, 735, 627, 280, 194, 764]) == 265)
    print(f([862, 933, 642, 85, 742, 504, 954, 702, 368, 717, 319, 559, 497, 960, 413, 141, 81, 914, 422, 894, 541, 166, 177, 725, 271, 821, 553, 271, 714, 113, 845, 879, 154, 786, 866, 661, 290, 290, 536, 965, 691, 471, 882, 956, 379, 81, 555, 617, 728, 611]) == 270)
    print(f([8, 2, 12, 1, 16, 11, 10, 5, 3, 4, 0, 2, 0, 9, 13, 15, 7, 11, 0, 7, 0, 9, 11, 1, 8, 1, 6, 11, 15, 11, 8, 1, 13, 14, 12, 16, 1, 16, 13, 3, 11, 14, 15, 12, 12, 13, 8, 0, 6, 14]) == 100)
    print(f([30, 24, 30, 29, 27, 31, 17, 10, 0, 26, 3, 27, 20, 24, 9, 12, 23, 27, 32, 30, 16, 30, 27, 1, 15, 21, 7, 13, 29, 22, 29, 10, 30, 5, 5, 9, 22, 15, 16, 18, 25, 3, 16, 17, 22, 3, 18, 29, 13, 1]) == 141)
    print(f([54, 52, 2, 50, 51, 3, 46, 29, 35, 31, 63, 49, 16, 1, 23, 23, 28, 59, 21, 6, 5, 27, 22, 47, 15, 19, 1, 18, 58, 51, 54, 26, 20, 44, 33, 18, 50, 9, 35, 52, 24, 26, 1, 41, 5, 19, 39, 58, 15, 24]) == 157)
    print(f([97, 86, 6, 105, 36, 86, 84, 71, 88, 126, 10, 79, 111, 58, 98, 120, 17, 8, 16, 41, 31, 48, 45, 99, 75, 93, 40, 61, 119, 45, 108, 1, 68, 34, 91, 118, 100, 121, 102, 79, 45, 51, 1, 121, 64, 2, 77, 82, 77, 96]) == 178)
    print(f([90, 28, 94, 111, 12, 37, 161, 122, 207, 42, 230, 108, 253, 127, 156, 128, 123, 86, 189, 236, 191, 180, 75, 95, 184, 116, 136, 238, 170, 18, 142, 203, 117, 149, 88, 29, 120, 200, 78, 125, 63, 76, 101, 0, 171, 35, 124, 26, 89, 227]) == 219)
    print(f([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255]) == 407)
    print(f([511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511, 511]) == 458)
    print(f([767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767, 767]) == 459)
    print(f([895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895, 895]) == 459)
    print(f([959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959, 959]) == 459)
    print(f([0]) == 0)
    print(f([1]) == 1)
    print(f([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 0)
    print(f([0]) == 0)
    print(f([1, 1, 1, 1, 1, 1]) == 6)
    print(f([0, 0, 0]) == 0)
    print(f([0, 0]) == 0)
    print(f([0, 0, 0, 0]) == 0)
    print(f([512]) == 10)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


#### chessmetric: https://community.topcoder.com/stat?c=problem_statement&pm=1592&rd=4482&rm=&cr=269554

In [ ]:
class ChessMetric(object):

    def explore(self, path):
        """Recursively explore all possible positions from start."""
        moves = len(path) - 1
        start = path[-1]
            
        if moves == self.maxmoves:
            if start == self.end: self.count += 1
            return

        # explore onemove possibilities
        for i in range(-1, 2):
            for j in range(-1, 2):
                x, y = start[0] + i, start[1] + j
                if j == 0 and i == 0: continue
                if x < 0 or x >= self.size: continue 
                if y < 0 or y >= self.size: continue
                self.explore(path + [[x, y]])

        # explore L shaped possibilites
        for i in [-2, -1, 1, 2]:
            for j in [-2, -1, 1, 2]:
                x, y = start[0] + i, start[1] + j
                if abs(j) == abs(i): continue
                if x < 0 or x >= self.size: continue 
                if y < 0 or y >= self.size: continue
                self.explore(path + [[x, y]])

    def howMany(self, size, start, end, maxmoves):
        """Return possible ways to get from start to end in ``moves``."""
        self.mintoend = {}
        self.maxmoves = maxmoves
        self.size = size
        self.count = 0
        self.end = end
        self.explore([start])
        return self.count

    def howManyFast(self, size, start, end, maxmoves):
        """Dynamic programming implementation."""
        r = range
        b = [[[0 for m in r(maxmoves + 1)] for y in r(size)] for x in r(size)]
        sx, sy, ex, ey = start[0], start[1], end[0], end[1]
        b[sx][sy][0] = 1

        for m in r(1, maxmoves + 1): 
            for x in r(size):
                for y in r(size):       

                    # explore onemove possibilities
                    for i in range(-1, 2):
                        for j in range(-1, 2):
                            nx, ny = x + i, y + j
                            if j == 0 and i == 0: continue
                            if nx < 0 or nx >= size: continue 
                            if ny < 0 or ny >= size: continue
                            b[nx][ny][m] += b[x][y][m - 1]

                    # explore L shaped possibilites
                    for i in [-2, -1, 1, 2]:
                        for j in [-2, -1, 1, 2]:
                            nx, ny = x + i, y + j
                            if abs(j) == abs(i): continue
                            if nx < 0 or nx >= size: continue 
                            if ny < 0 or ny >= size: continue
                            b[nx][ny][m] += b[x][y][m - 1]

        return b[ex][ey][maxmoves]
if True:
    f = ChessMetric().howManyFast
    print(f(3, [0,0], [1,0], 1))
    print(f(3, [0,0],[0,0], 2))
    print(f(3, [0,0], [2,2], 1))
    print(f(100, [0,0], [0,99], 50))

#### coin example

In [ ]:
class HolyCoins(object):
    
    def get_coins(self, coins, total):
        """Get the minumum coins required to sum total."""
        # Set mincoins[i] equal to Infinity for all of i.
        mincoins = [math.inf for i in range(total + 1)]
        mincoins[0] = 0

        for i in range(1, total + 1):
            for c in coins:
                if c <= i:
                    if mincoins[i - c] + 1 < mincoins[i]:
                        mincoins[i] = mincoins[i - c] + 1

        return mincoins[total]
                    
if True:
    f = HolyCoins().get_coins
    print(f([1, 3, 5], 11))

# Recursion

#### inversehaar1d: https://community.topcoder.com/stat?c=problem_statement&pm=5896&rd=8072&rm=&cr=15179224

In [ ]:
class InverseHaar1D(object):

    def transform(self, seq, L):
        """"Return Haar original data."""
        if L == 0: return seq
        n = len(seq)
        div = n // (2 ** (L - 1))
        toinv = seq[:div]
        sums = toinv[:len(toinv) // 2]
        diffs = toinv[len(toinv) // 2:]
        invseq = []
        for i in range(len(sums)):
            a = (sums[i] + diffs[i]) // 2
            b = a - diffs[i]
            invseq += [a, b]
        invseq += seq[div:]
        return self.transform(invseq, L - 1)
        
if True:
    f = InverseHaar1D().transform
    print(f([101, -53], 1) == [24, 77])
    print(f([3, 8, -1, -2], 1) == [1, 2, 3, 5])
    print(f([11, -5, -1, -2], 2) == [1, 2, 3, 5])
    print(f([369, 477, 451, 262, 47, 135, -125, -2, 18, -23, 30, 101, -5, -18, 54, -20, 11, 45, -5, 70, -24, 2, -50, 15, 55, -62, -23, -17, 44, -8, -44, -52], 3) == [62, 51, 70, 25, 32, 37, 81, 11, 72, 96, 70, 68, 43, 93, 25, 10, 67, 12, 11, 73, 56, 79, 68, 85, 68, 24, 15, 23, 6, 50, 12, 64])
    print(f([20, 0, -4, 4, -1, -1, 1, 1], 3) == [1, 2, 3, 4, 4, 3, 2, 1])
    print(f([3, -1], 1) == [1, 2])
    print(f([58, -4], 1) == [27, 31])
    print(f([115, 160, 25, -16], 1) == [70, 45, 72, 88])
    print(f([190, -104, -1, -35], 2) == [21, 22, 56, 91])
    print(f([154, 83, 145, 98, -26, -79, 47, 84], 1) == [64, 90, 2, 81, 96, 49, 91, 7])
    print(f([125, 208, -39, -110, 17, 70, 17, 33], 2) == [30, 13, 76, 6, 33, 16, 96, 63])
    print(f([412, 34, 9, 95, 64, -55, -8, 17], 3) == [90, 26, 26, 81, 67, 75, 32, 15])
    print(f([77, 173, 37, 27, 22, 97, 127, 68, 47, -25, -29, 5, -20, 67, 53, 18], 1) == [62, 15, 74, 99, 4, 33, 16, 11, 1, 21, 82, 15, 90, 37, 43, 25])
    print(f([215, 207, 248, 194, 67, 49, -68, -16, 47, 18, -50, 71, -34, -20, -21, 57], 2) == [94, 47, 46, 28, 39, 89, 75, 4, 28, 62, 69, 89, 34, 55, 81, 24])
    print(f([429, 449, -145, 89, -56, 63, -45, -114, -21, 75, -23, -6, 14, -39, -15, 5], 3) == [11, 32, 87, 12, 76, 99, 53, 59, 63, 49, 59, 98, 9, 24, 76, 71])
    print(f([930, -44, 67, -61, 47, -130, 41, 44, -19, -64, -29, -39, 25, -80, 5, -25], 4) == [66, 85, 20, 84, 0, 29, 60, 99, 76, 51, 3, 83, 82, 77, 45, 70])
    print(f([171, 69, 129, 69, 99, 150, 129, 27, 117, 163, 160, 75, 88, 133, 51, 181, 25, -51, -9, 31, -51, -44, 37, -19, -43, -29, -12, -17, 46, -1, -3, -13], 1) == [98, 73, 9, 60, 60, 69, 50, 19, 24, 75, 53, 97, 83, 46, 4, 23, 37, 80, 67, 96, 74, 86, 29, 46, 67, 21, 66, 67, 24, 27, 84, 97])
    print(f([219, 221, 207, 171, 208, 215, 165, 262, -41, 65, 49, -73, -108, -19, 39, 14, -45, 22, -45, 74, 22, -31, -35, -50, -24, 30, 74, -21, -6, 17, -60, -24], 2) == [22, 67, 76, 54, 49, 94, 76, 2, 75, 53, 24, 55, 7, 42, 36, 86, 13, 37, 94, 64, 86, 12, 48, 69, 48, 54, 40, 23, 39, 99, 50, 74])
    print(f([619, 542, 332, 465, 75, 20, -38, -17, 41, -16, -53, -65, -87, -9, 110, 67, 2, -15, 70, 46, -72, -17, -14, -29, 0, -17, -40, 49, -1, 37, -16, 85], 3) == [98, 96, 69, 84, 99, 29, 95, 49, 21, 93, 75, 92, 42, 56, 67, 96, 15, 15, 50, 67, 24, 64, 73, 24, 83, 84, 47, 10, 69, 85, 86, 1])
    print(f([659, 785, -19, -15, 28, 47, 9, 132, 128, -20, 83, -52, 21, -100, 34, 82, -39, -13, 1, 31, 30, 37, -19, 71, 57, -36, 44, -40, -48, -22, -60, 22], 4) == [56, 95, 5, 18, 32, 31, 57, 26, 84, 54, 46, 9, 14, 33, 85, 14, 83, 26, 26, 62, 44, 0, 52, 92, 51, 99, 47, 69, 24, 84, 24, 2])
    print(f([1504, -368, -18, 50, 5, 17, -19, -151, 48, 61, -53, -62, -29, 42, 26, 53, -12, 44, -24, -23, -51, -62, -8, 36, 34, -33, -17, -59, 8, 30, -17, 54], 5) == [41, 53, 45, 1, 37, 61, 7, 30, 0, 51, 21, 83, 15, 23, 68, 32, 69, 35, 50, 83, 66, 83, 24, 83, 47, 39, 45, 15, 79, 96, 88, 34])
    print(f([25, 3], 1) == [14, 11])
    print(f([146, 150, 32, -14], 1) == [89, 57, 68, 82])
    print(f([296, -4, 32, -14], 2) == [89, 57, 68, 82])
    print(f([163, 73, 91, 78, -9, -29, -47, -74], 1) == [77, 86, 22, 51, 22, 69, 2, 76])
    print(f([236, 169, 90, 13, -9, -29, -47, -74], 2) == [77, 86, 22, 51, 22, 69, 2, 76])
    print(f([405, 67, 90, 13, -9, -29, -47, -74], 3) == [77, 86, 22, 51, 22, 69, 2, 76])
    print(f([152, 102, 135, 158, 53, 180, 128, 98, 18, -18, 51, 0, -17, -18, 40, 50], 1) == [85, 67, 42, 60, 93, 42, 79, 79, 18, 35, 81, 99, 84, 44, 74, 24])
    print(f([254, 293, 233, 226, 50, -23, -127, 30, 18, -18, 51, 0, -17, -18, 40, 50], 2) == [85, 67, 42, 60, 93, 42, 79, 79, 18, 35, 81, 99, 84, 44, 74, 24])
    print(f([547, 459, -39, 7, 50, -23, -127, 30, 18, -18, 51, 0, -17, -18, 40, 50], 3) == [85, 67, 42, 60, 93, 42, 79, 79, 18, 35, 81, 99, 84, 44, 74, 24])
    print(f([1006, 88, -39, 7, 50, -23, -127, 30, 18, -18, 51, 0, -17, -18, 40, 50], 4) == [85, 67, 42, 60, 93, 42, 79, 79, 18, 35, 81, 99, 84, 44, 74, 24])
    print(f([171, 116, 166, 106, 137, 94, 15, 49, 162, 159, 100, 87, 67, 104, 76, 90, 13, -12, -8, -18, 47, -58, 9, -49, 28, 15, 36, -21, 37, 54, 18, -60], 1) == [92, 79, 52, 64, 79, 87, 44, 62, 92, 45, 18, 76, 12, 3, 0, 49, 95, 67, 87, 72, 68, 32, 33, 54, 52, 15, 79, 25, 47, 29, 15, 75])
    print(f([287, 272, 231, 64, 321, 187, 171, 166, 55, 60, 43, -34, 3, 13, -37, -14, 13, -12, -8, -18, 47, -58, 9, -49, 28, 15, 36, -21, 37, 54, 18, -60], 2) == [92, 79, 52, 64, 79, 87, 44, 62, 92, 45, 18, 76, 12, 3, 0, 49, 95, 67, 87, 72, 68, 32, 33, 54, 52, 15, 79, 25, 47, 29, 15, 75])
    print(f([559, 295, 508, 337, 15, 167, 134, 5, 55, 60, 43, -34, 3, 13, -37, -14, 13, -12, -8, -18, 47, -58, 9, -49, 28, 15, 36, -21, 37, 54, 18, -60], 3) == [92, 79, 52, 64, 79, 87, 44, 62, 92, 45, 18, 76, 12, 3, 0, 49, 95, 67, 87, 72, 68, 32, 33, 54, 52, 15, 79, 25, 47, 29, 15, 75])
    print(f([854, 845, 264, 171, 15, 167, 134, 5, 55, 60, 43, -34, 3, 13, -37, -14, 13, -12, -8, -18, 47, -58, 9, -49, 28, 15, 36, -21, 37, 54, 18, -60], 4) == [92, 79, 52, 64, 79, 87, 44, 62, 92, 45, 18, 76, 12, 3, 0, 49, 95, 67, 87, 72, 68, 32, 33, 54, 52, 15, 79, 25, 47, 29, 15, 75])
    print(f([1699, 9, 264, 171, 15, 167, 134, 5, 55, 60, 43, -34, 3, 13, -37, -14, 13, -12, -8, -18, 47, -58, 9, -49, 28, 15, 36, -21, 37, 54, 18, -60], 5) == [92, 79, 52, 64, 79, 87, 44, 62, 92, 45, 18, 76, 12, 3, 0, 49, 95, 67, 87, 72, 68, 32, 33, 54, 52, 15, 79, 25, 47, 29, 15, 75])
    print(f([80, 0], 1) == [40, 40])
    print(f([94, 94, 0, 0], 1) == [47, 47, 47, 47])
    print(f([192, 0, 0, 0], 2) == [48, 48, 48, 48])
    print(f([120, 120, 120, 120, 0, 0, 0, 0], 1) == [60, 60, 60, 60, 60, 60, 60, 60])
    print(f([384, 384, 0, 0, 0, 0, 0, 0], 2) == [96, 96, 96, 96, 96, 96, 96, 96])
    print(f([576, 0, 0, 0, 0, 0, 0, 0], 3) == [72, 72, 72, 72, 72, 72, 72, 72])
    print(f([100, 100, 100, 100, 100, 100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0], 1) == [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50])
    print(f([284, 284, 284, 284, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2) == [71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71])
    print(f([544, 544, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3) == [68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68])
    print(f([1408, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 4) == [88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88, 88])
    print(f([144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 144, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1) == [72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72])
    print(f([388, 388, 388, 388, 388, 388, 388, 388, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2) == [97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97])
    print(f([776, 776, 776, 776, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3) == [97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97])
    print(f([1520, 1520, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 4) == [95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95])
    print(f([3200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 5) == [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])
    print(f([-171, -69, -129, -69, -99, -150, -129, -27, -117, -163, -160, -75, -88, -133, -51, -181, -25, 51, 9, -31, 51, 44, -37, 19, 43, 29, 12, 17, -46, 1, 3, 13], 1) == [-98, -73, -9, -60, -60, -69, -50, -19, -24, -75, -53, -97, -83, -46, -4, -23, -37, -80, -67, -96, -74, -86, -29, -46, -67, -21, -66, -67, -24, -27, -84, -97])
    print(f([-1408, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 4) == [-88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88, -88])
    print(f([-1504, 368, 18, -50, -5, -17, 19, 151, -48, -61, 53, 62, 29, -42, -26, -53, 12, -44, 24, 23, 51, 62, 8, -36, -34, 33, 17, 59, -8, -30, 17, -54], 5) == [-41, -53, -45, -1, -37, -61, -7, -30, 0, -51, -21, -83, -15, -23, -68, -32, -69, -35, -50, -83, -66, -83, -24, -83, -47, -39, -45, -15, -79, -96, -88, -34])
    print(f([-284, -284, -284, -284, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 3) == [-71, -71, -71, -71, 0, 0, 0, 0, -71, -71, -71, -71, 0, 0, 0, 0])

#### lotteryticket: https://community.topcoder.com/stat?c=problem_statement&pm=10860




 Problem Statement for LotteryTicket


Problem Statement
    	Nick likes to play the lottery. The cost of a single lottery ticket is price. Nick has exactly four banknotes with values b1, b2, b3 and b4 (some of the values may be equal). He wants to know if it's possible to buy a single lottery ticket without getting any change back. In other words, he wants to pay the exact price of a ticket using any subset of his banknotes. Return "POSSIBLE" if it is possible or "IMPOSSIBLE" if it is not (all quotes for clarity).
 
Definition
    	
Class:	LotteryTicket
Method:	buy
Parameters:	int, int, int, int, int
Returns:	String
Method signature:	String buy(int price, int b1, int b2, int b3, int b4)
(be sure your method is public)
    
 
Constraints
-	price will be between 1 and 4000, inclusive.
-	b1, b2, b3 and b4 will each be between 1 and 1000, inclusive.
 
Examples
0)	
    	
10
1
5
10
50
Returns: "POSSIBLE"
Nick can use the banknote with value b3.
1)	
    	
15
1
5
10
50
Returns: "POSSIBLE"
Here he can use the banknotes with values b2 and b3.
2)	
    	
65
1
5
10
50
Returns: "POSSIBLE"
b2 + b3 + b4 is 65.
3)	
    	
66
1
5
10
50
Returns: "POSSIBLE"
All four banknotes must be used.
4)	
    	
1000
999
998
997
996
Returns: "IMPOSSIBLE"
5)	
    	
20
5
5
5
5
Returns: "POSSIBLE"
Some of the banknote values may be equal.
6)	
    	
2
1
5
10
50
Returns: "IMPOSSIBLE"
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.

In [ ]:
import itertools
class LoteryTicket(object):
    
    def buy(self, p, b1, b2, b3, b4):
        """Return if any combination of bills sum p."""
        notes = [b1, b2, b3, b4]
        notes = [n for n in notes if n <= p]
        if sum(notes) == p: return "POSSIBLE"
        if sum(notes) < p: return "IMPOSSIBLE"
        
        for i in range(1, 4):
            combinations = itertools.combinations(notes, i)
            for c in combinations:
                if sum(c) == p: return "POSSIBLE"
                
        return "IMPOSSIBLE"
    
if True:
    f = LoteryTicket().buy
    print(f(10, 1, 5, 10, 50) == "POSSIBLE")
    print(f(15, 1, 5, 10, 50) == "POSSIBLE")
    print(f(65, 1, 5, 10, 50) == "POSSIBLE")
    print(f(66, 1, 5, 10, 50) == "POSSIBLE")
    print(f(1000, 999, 998, 997, 996) == "IMPOSSIBLE")
    print(f(20, 5, 5, 5, 5) == "POSSIBLE")
    print(f(20, 5, 5, 5, 6) == "IMPOSSIBLE")
    print(f(400, 1000, 1000, 1000, 1000) == "IMPOSSIBLE")
    print(f(10, 2, 8, 5, 1) == "POSSIBLE")
    print(f(65, 25, 79, 59, 63) == "IMPOSSIBLE")
    print(f(962, 706, 146, 282, 828) == "IMPOSSIBLE")
    print(f(2, 2, 6, 3, 8) == "POSSIBLE")
    print(f(37, 37, 92, 5, 3) == "POSSIBLE")
    print(f(154, 154, 293, 383, 422) == "POSSIBLE")
    print(f(9, 7, 9, 6, 8) == "POSSIBLE")
    print(f(72, 27, 72, 39, 70) == "POSSIBLE")
    print(f(668, 913, 668, 300, 36) == "POSSIBLE")
    print(f(9, 5, 4, 2, 3) == "POSSIBLE")
    print(f(108, 34, 74, 65, 42) == "POSSIBLE")
    print(f(966, 712, 254, 869, 548) == "POSSIBLE")
    print(f(8, 5, 3, 8, 8) == "POSSIBLE")
    print(f(42, 60, 24, 42, 30) == "POSSIBLE")
    print(f(36, 779, 317, 36, 191) == "POSSIBLE")
    print(f(10, 3, 9, 7, 1) == "POSSIBLE")
    print(f(92, 43, 65, 49, 47) == "POSSIBLE")
    print(f(1536, 806, 891, 730, 371) == "POSSIBLE")
    print(f(9, 1, 7, 2, 4) == "POSSIBLE")
    print(f(54, 49, 30, 24, 85) == "POSSIBLE")
    print(f(1598, 955, 757, 841, 967) == "POSSIBLE")
    print(f(18, 7, 2, 9, 5) == "POSSIBLE")
    print(f(91, 40, 27, 24, 38) == "POSSIBLE")
    print(f(741, 539, 119, 83, 930) == "POSSIBLE")
    print(f(10, 2, 4, 6, 10) == "POSSIBLE")
    print(f(78, 59, 5, 31, 78) == "POSSIBLE")
    print(f(22, 307, 674, 387, 22) == "POSSIBLE")
    print(f(7, 6, 5, 3, 1) == "POSSIBLE")
    print(f(128, 30, 78, 74, 98) == "POSSIBLE")
    print(f(675, 513, 987, 291, 162) == "POSSIBLE")
    print(f(12, 7, 6, 8, 6) == "POSSIBLE")
    print(f(83, 75, 32, 53, 51) == "POSSIBLE")
    print(f(1909, 151, 942, 725, 967) == "POSSIBLE")
    print(f(17, 1, 8, 2, 8) == "POSSIBLE")
    print(f(150, 38, 58, 88, 54) == "POSSIBLE")
    print(f(1540, 384, 946, 910, 210) == "POSSIBLE")
    print(f(12, 9, 2, 9, 3) == "POSSIBLE")
    print(f(45, 47, 7, 31, 14) == "POSSIBLE")
    print(f(1355, 169, 901, 592, 763) == "POSSIBLE")
    print(f(21, 6, 1, 10, 5) == "POSSIBLE")
    print(f(218, 38, 49, 84, 96) == "POSSIBLE")
    print(f(685, 42, 603, 351, 292) == "POSSIBLE")
    print(f(13, 7, 5, 1, 7) == "POSSIBLE")
    print(f(218, 22, 49, 100, 69) == "POSSIBLE")
    print(f(1071, 485, 282, 735, 54) == "POSSIBLE")
    print(f(29, 10, 9, 9, 1) == "POSSIBLE")
    print(f(214, 89, 28, 68, 29) == "POSSIBLE")
    print(f(2835, 894, 649, 484, 808) == "POSSIBLE")
    print(f(4000, 1000, 1000, 1000, 1000) == "POSSIBLE")
    print(f(4000, 1000, 999, 1000, 1000) == "IMPOSSIBLE")
    print(f(2, 1, 5, 10, 50) == "IMPOSSIBLE")
    print(f(20, 42, 18, 35, 1) == "IMPOSSIBLE")
    print(f(65, 25, 79, 59, 63) == "IMPOSSIBLE")
    print(f(62, 6, 96, 32, 28) == "IMPOSSIBLE")
    print(f(37, 92, 196, 143, 28) == "IMPOSSIBLE")
    print(f(43, 142, 105, 153, 154) == "IMPOSSIBLE")
    print(f(96, 83, 22, 117, 219) == "IMPOSSIBLE")
    print(f(120, 198, 27, 72, 339) == "IMPOSSIBLE")
    print(f(295, 313, 68, 300, 236) == "IMPOSSIBLE")
    print(f(124, 354, 412, 273, 184) == "IMPOSSIBLE")
    print(f(369, 165, 142, 212, 254) == "IMPOSSIBLE")
    print(f(238, 248, 145, 213, 308) == "IMPOSSIBLE")
    print(f(179, 260, 324, 142, 530) == "IMPOSSIBLE")
    print(f(289, 617, 436, 91, 543) == "IMPOSSIBLE")
    print(f(249, 7, 641, 543, 365) == "IMPOSSIBLE")
    print(f(371, 447, 556, 141, 730) == "IMPOSSIBLE")
    print(f(349, 151, 607, 702, 394) == "IMPOSSIBLE")
    print(f(57, 80, 724, 285, 405) == "IMPOSSIBLE")
    print(f(209, 141, 467, 177, 432) == "IMPOSSIBLE")
    print(f(788, 795, 140, 877, 874) == "IMPOSSIBLE")
    print(f(542, 539, 119, 83, 930) == "IMPOSSIBLE")
    print(f(8, 1, 1, 1, 1) == "IMPOSSIBLE")
    print(f(7, 1, 1, 3, 4) == "POSSIBLE")
    print(f(30, 10, 5, 10, 10) == "POSSIBLE")
    print(f(200, 1, 2, 199, 872) == "POSSIBLE")
    print(f(10, 3, 5, 5, 6) == "POSSIBLE")
    print(f(1000, 250, 1, 500, 250) == "POSSIBLE")
    print(f(8, 2, 6, 7, 10) == "POSSIBLE")
    print(f(1, 1, 1, 1, 1) == "POSSIBLE")
    print(f(18, 1, 1, 9, 9) == "POSSIBLE")
    print(f(5, 10, 10, 2, 3) == "POSSIBLE")
    print(f(8, 100, 100, 100, 2) == "IMPOSSIBLE")
    print(f(3, 1, 1, 1, 7) == "POSSIBLE")
    print(f(11, 2, 5, 10, 1) == "POSSIBLE")
    print(f(6, 1, 3, 1, 3) == "POSSIBLE")
    print(f(10, 1, 2, 3, 4) == "POSSIBLE")
    print(f(20, 1, 1, 1, 2) == "IMPOSSIBLE")
    print(f(15, 1, 2, 3, 15) == "POSSIBLE")
    print(f(9, 3, 4, 5, 6) == "POSSIBLE")
    print(f(10, 2, 100, 12, 8) == "POSSIBLE")
    print(f(6, 2, 2, 2, 5) == "POSSIBLE")
    print(f(100, 25, 25, 101, 50) == "POSSIBLE")
    print(f(100, 10, 20, 50, 40) == "POSSIBLE")
    print(f(1004, 1, 2, 3, 1000) == "POSSIBLE")
    print(f(10, 1, 2, 30, 7) == "POSSIBLE")
    print(f(10, 5, 5, 2, 3) == "POSSIBLE")
    print(f(4, 1, 2, 5, 3) == "POSSIBLE")
    print(f(14, 1, 3, 5, 10) == "POSSIBLE")
    print(f(18, 5, 6, 1, 7) == "POSSIBLE")
    print(f(60, 30, 1000, 30, 1000) == "POSSIBLE")
    print(f(2, 1, 10, 1, 10) == "POSSIBLE")
    print(f(5, 2, 9, 9, 3) == "POSSIBLE")
    print(f(6, 2, 2, 3, 4) == "POSSIBLE")
    print(f(18, 18, 1, 1, 1) == "POSSIBLE")
    print(f(10, 5, 1, 1, 5) == "POSSIBLE")
    print(f(16, 1, 5, 12, 15) == "POSSIBLE")
    print(f(100, 30, 1, 20, 50) == "POSSIBLE")
    print(f(20, 10, 1, 1, 10) == "POSSIBLE")
    print(f(500, 200, 100, 300, 1) == "POSSIBLE")
    print(f(2, 1, 2, 3, 4) == "POSSIBLE")
    print(f(3, 1, 2, 5, 6) == "POSSIBLE")
    print(f(67, 30, 80, 37, 80) == "POSSIBLE")
    print(f(3, 1, 55, 1, 1) == "POSSIBLE")
    print(f(50, 1, 5, 10, 50) == "POSSIBLE")
    print(f(17, 2, 3, 5, 7) == "POSSIBLE")
    print(f(10, 5, 4, 3, 2) == "POSSIBLE")
    print(f(1000, 900, 5, 100, 800) == "POSSIBLE")
    print(f(100, 1, 2, 3, 100) == "POSSIBLE")
    print(f(5, 1, 3, 122, 1) == "POSSIBLE")
    print(f(3, 1, 100, 1, 1) == "POSSIBLE")
    print(f(10, 1, 4, 9, 100) == "POSSIBLE")
    print(f(4, 1, 2, 100, 200) == "IMPOSSIBLE")
    print(f(100, 1, 50, 1, 50) == "POSSIBLE")
    print(f(10, 2, 3, 8, 11) == "POSSIBLE")
    print(f(8, 1, 2, 3, 4) == "POSSIBLE")
    print(f(30, 1, 9, 11, 20) == "POSSIBLE")
    print(f(20, 1, 8, 2, 11) == "POSSIBLE")
    print(f(3, 1, 4, 1, 1) == "POSSIBLE")
    print(f(100, 1, 2, 1, 1) == "IMPOSSIBLE")
    print(f(50, 5, 10, 20, 25) == "POSSIBLE")
    print(f(100, 90, 1, 1, 10) == "POSSIBLE")
    print(f(6, 2, 1, 2, 2) == "POSSIBLE")
    print(f(5, 1, 2, 4, 100) == "POSSIBLE")
    print(f(11, 2, 5, 4, 10) == "POSSIBLE")
    print(f(17, 10, 3, 4, 5) == "POSSIBLE")
    print(f(400, 100, 100, 100, 400) == "POSSIBLE")
    print(f(5, 4, 3, 2, 2) == "POSSIBLE")
    print(f(11, 5, 6, 10, 50) == "POSSIBLE")
    print(f(300, 100, 1, 100, 100) == "POSSIBLE")
    print(f(3, 1, 1, 7, 1) == "POSSIBLE")
    print(f(100, 50, 1, 50, 1) == "POSSIBLE")
    print(f(20, 1, 10, 1, 1) == "IMPOSSIBLE")
    print(f(50, 20, 20, 30, 40) == "POSSIBLE")
    print(f(50, 45, 1, 25, 25) == "POSSIBLE")
    print(f(5, 1, 2, 4, 9) == "POSSIBLE")
    print(f(67, 30, 80, 27, 10) == "POSSIBLE")
    print(f(6, 2, 4, 100, 100) == "POSSIBLE")
    print(f(11, 1, 5, 10, 2) == "POSSIBLE")
    print(f(100, 25, 101, 25, 50) == "POSSIBLE")
    print(f(10, 1, 2, 20, 7) == "POSSIBLE")
    print(f(30, 10, 1, 10, 10) == "POSSIBLE")
    print(f(10, 10, 1, 2, 3) == "POSSIBLE")
    print(f(20, 10, 7, 6, 4) == "POSSIBLE")
    print(f(1011, 1, 10, 100, 1000) == "POSSIBLE")
    print(f(11, 1, 5, 10, 50) == "POSSIBLE")
    print(f(15, 5, 5, 5, 9) == "POSSIBLE")
    print(f(17, 10, 5, 4, 3) == "POSSIBLE")
    print(f(20, 2, 3, 1, 15) == "POSSIBLE")
    print(f(7, 1, 4, 6, 9) == "POSSIBLE")
    print(f(30, 10, 1, 2, 20) == "POSSIBLE")
    print(f(4, 1, 2, 3, 5) == "POSSIBLE")
    print(f(300, 100, 100, 1, 100) == "POSSIBLE")
    print(f(6, 1, 100, 2, 3) == "POSSIBLE")
    print(f(5, 2, 2, 3, 4) == "POSSIBLE")
    print(f(11, 1, 2, 4, 8) == "POSSIBLE")
    print(f(50, 15, 15, 60, 20) == "POSSIBLE")
    print(f(22, 5, 6, 10, 11) == "POSSIBLE")
    print(f(11, 5, 10, 2, 1) == "POSSIBLE")
    print(f(1000, 50, 300, 100, 700) == "POSSIBLE")
    print(f(7, 1, 3, 77, 3) == "POSSIBLE")
    print(f(1001, 1, 500, 400, 1000) == "POSSIBLE")
    print(f(13, 3, 7, 10, 15) == "POSSIBLE")
    print(f(20, 16, 12, 11, 8) == "POSSIBLE")
    print(f(3, 1, 1, 4, 1) == "POSSIBLE")
    print(f(1101, 1, 10, 100, 1000) == "POSSIBLE")
    print(f(10, 3, 4, 6, 9) == "POSSIBLE")
    print(f(3, 1, 10, 1, 1) == "POSSIBLE")
    print(f(100, 50, 25, 1, 25) == "POSSIBLE")
    print(f(31, 1, 10, 13, 20) == "POSSIBLE")
    print(f(20, 30, 10, 5, 5) == "POSSIBLE")
    print(f(10, 6, 50, 2, 2) == "POSSIBLE")
    print(f(25, 1, 29, 9, 15) == "POSSIBLE")
    print(f(10, 3, 1, 3, 4) == "POSSIBLE")
    print(f(2, 1, 100, 100, 100) == "IMPOSSIBLE")

#### not solved matchcounts: https://community.topcoder.com/stat?c=problem_statement&pm=4798




 Problem Statement for MatchCounts


Problem Statement
    	The digit d will appear in element i of ability if person i can perform task d. You are required to assign each of the given people to exactly one of the 6 tasks (numbered '0' through '5'). Two people cannot be assigned to the same task. Return the number of assignments possible. Two assignments differ if there is a person who is assigned a different task in each assignment.
 
Definition
    	
Class:	MatchCounts
Method:	howMany
Parameters:	String[]
Returns:	int
Method signature:	int howMany(String[] ability)
(be sure your method is public)
    
 
Constraints
-	ability will contain between 1 and 6 elements, inclusive.
-	Each element of ability will contain between 1 and 6 characters, inclusive.
-	Each character in ability will be a digit between '0' and '5', inclusive.
-	No character will appear more than once in a single element of ability.
 
Examples
0)	
    	
{"045"}
Returns: 3
Person 0 can perform 3 different tasks.
1)	
    	
{"01","23"}
Returns: 4
Two choices for person 0 and two choices for person 1.
2)	
    	
{"01","01"}
Returns: 2
Both people have two choices, but not all 2*2 possibilities are feasible.
3)	
    	
{"0","0"}
Returns: 0
4)	
    	
{"012345","012345","012345","012345","012345","012345"}
Returns: 720
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       TCCC06 Online Round 2A - Division I, Level One

In [ ]:
from itertools import combinations

class MatchCounts(object):
    
    def howMany(self, ability):
        
        results = {}
        n = len(ability)
        m = len(ability[0])
        ret = 0
        for i in range(n):
            for d in range(m):
                current = ability[i][d]
                for k in range(n):
                    if k == i: continue
                    for j in range(m):
                        if j == d: continue
                        if ability[k][j] not in current:
                            current += ability[k][j]
                            break
                print(current)
                if current not in results and len(current) == n:
                    results[current] = ""
                    ret += 1


        return ret
    
if True:
    f = MatchCounts().howMany
    print(f(["045"]))
    print(f(["01","23"]))
    print(f(["0","0"]))
    print(f(["012345","012345","012345","012345","012345","012345"]))
    

In [ ]:
"j" in "juan"

#### very elegant: bearfills: https://community.topcoder.com/stat?c=problem_statement&pm=14011




 Problem Statement for BearFills


Problem Statement
    	Bear Limak has a rectangular grid that consist of H times W unit square cells. Limak also has N square stamps. The lengths of the stamps' sides are 2^0, 2^1, ..., 2^(N-1). 



A set of stamps is called good if it can be used to cover the entire grid. Each stamp must be placed along the grid lines. I.e., for each stamp each cell is either completely covered by the stamp or not covered at all. The stamps may overlap arbitrarily. The stamps may also cover areas outside of the grid. For example, you may use a 4x4 stamp just to cover a single cell in the corner of the grid. 



You are given the int N and the longs H and W. Limak wants to choose a subset of his stamps that will be good. Return the number of ways in which he can do so.
 
Definition
    	
Class:	BearFills
Method:	countSets
Parameters:	int, long, long
Returns:	long
Method signature:	long countSets(int N, long W, long H)
(be sure your method is public)
    
 
Notes
-	N will be between 1 and 60, inclusive.
-	H and W will be between 1 and 10^18, inclusive.
 
Examples
0)	
    	
3
1
3
Returns: 5
He has a 1x3 rectangle and 3 square stamps. The stamps' sides are 1, 2, and 4. The good sets of stamps are the following ones: (1,2), (1,2,4), (1,4), (2,4), and (4).
1)	
    	
3
3
5
Returns: 1
He has a 3x5 rectangle and 3 square stamps. The stamps' sides are 1, 2, and 4. He needs all of them.
2)	
    	
60
3
2
Returns: 1152921504606846972
He has a 3x2 rectangle and 60 square stamps. Only four sets are not good: (), (1), (2), and (1,2). The answer is 2^60-4.
3)	
    	
6
5
4
Returns: 56
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       2015 TopCoder Open Algorithm Round 3B - Division I, Level One 
       2015 TopCoder Open Algorithm Parallel Round 3B - Division I, Level One

In [ ]:
class BearFills(object):
    
    def count_sets(self, N, W, H):
        """Return max numbers of sets that will cover the grid."""
        if N == 0: 
            return 0 
        if H > W: 
            H, W = W, H 
        L = 2 ** (N - 1) 
        if L < H and L < W: 
            return 0 
        if L >= H and L >= W: 
            return 2 ** (N - 1) + self.count_sets(N - 1, H, W) 
        if H <= L: 
            return self.count_sets(N - 1, H, W - L) 
        return 0

if True:
    f = BearFills().count_sets
    print(f(60, 3, 2))

In [ ]:
2 % 3

#### truckloads: http://www.topcoder.com/tc?module=ProblemDetail&rd=8081&pm=6011

We have a pile of crates at our warehouse that we want to load onto trucks. Our plan is to divide the pile in half forming two smaller piles, then continuing dividing each of the small piles in half until we get piles that will fit on a truck. (Of course, when we divide an odd number of crates in "half", one of the resulting piles will have one more crate than the other.) Our problem is to determine how many trucks we will need to ship the crates.
Create a class Truckloads that contains a method numTrucks that is given numCrates (the number of crates at the warehouse) and loadSize (the maximum number of crates that will fit in a truck) and that returns the number of trucks required.

 
Definition
        
Class:  Truckloads
Method: numTrucks
Parameters: int, int
Returns:    int
Method signature:   int numTrucks(int numCrates, int loadSize)
(be sure your method is public)
    
 
Constraints
-   numCrates will be between 2 and 10,000, inclusive.
-   loadSize loadSize will be be between 1 and (numCrates - 1), inclusive.
 
Examples

0)  
        
14
3
Returns: 6
After the first division we have two piles each with 7 crates. Each of these piles must be divided giving us 2 piles of 3 and 2 piles of 4. The piles with 4 crates must be further divided giving us 2 piles of 3 and 4 piles of 2. Each of these piles fits into a truck, so we need 6 trucks.

1)  
        
15
1
Returns: 15
We will eventually end up with 15 piles, each with just 1 crate.

2)  
        
1024
5
Returns: 256
1024 divides in half very nicely. We eventually end up with 256 piles, each containing 4 crates.


In [ ]:
class TruckLoads(object):
    
    """Class track number of trucks."""

    def __init__(self):
        """Initialize the count of trucks."""
        self.trucks = 0
        self.initial = 0
    
    def numtrucks(self, crates, load):
        """Compute trucks needed to load crates."""
        if self.initial == 0:
            crates_error = "Crates must be between 2 and 10,000, inclusive."
            load_error = "Load must be between 1 and (crates - 1)."
            if crates < 2 or crates >= 10000: raise ValueError(crates_error)
            if load < 1 or load >= (crates - 1): raise ValueError(load_error)
            self.initial = crates
        if load >= crates:
            self.trucks += 1
        else:
            self.numtrucks(int(crates / 2) , load)
            self.numtrucks(int(crates / 2) + crates % 2, load)
        if crates == self.initial:
            trucks = self.trucks
            self.__init__()
            return trucks     
if True:
    loader = TruckLoads()
    print(loader.numtrucks(14, 3))
    print(loader.numtrucks(15, 1))
    print(loader.numtrucks(1024, 5))

#### odddivisors: https://community.topcoder.com/stat?c=problem_statement&pm=10547

Problem Statement
    	Let f(x) be the greatest odd divisor of x, where x is a positive integer. You are given a positive integer N. Return f(1)+f(2)+?+f(N).
 
Definition
    	
Class:	OddDivisors
Method:	findSum
Parameters:	int
Returns:	long
Method signature:	long findSum(int N)
(be sure your method is public)
    
 
Constraints
-	N will be between 1 and 1000000000, inclusive.
 
Examples
0)	
    	
7
Returns: 21
f(1)+f(2)+f(3)+f(4)+f(5)+f(6)+f(7)=1+1+3+1+5+3+7=21
1)	
    	
1
Returns: 1
2)	
    	
777
Returns: 201537


In [ ]:
class OddDivisors(object):
    
    """Sum of odd divisors."""
    
    def find_sum(self, n):
        if n == 0: return 0
        return int((n+1)/2)**2 + self.find_sum(int(n/2))
if True:
    divisor = OddDivisors()
    print(divisor.find_sum(7) == 21)
    print(divisor.find_sum(1) == 1)
    print(divisor.find_sum(777) == 201537)
    print(divisor.find_sum(1000000000) == 333333333334181226)
    print(divisor.find_sum(999999999) == 333333333332228101)
    print(divisor.find_sum(328) == 35878)
    print(divisor.find_sum(682) == 155194)
    print(divisor.find_sum(790) == 208250)
    print(divisor.find_sum(675) == 152223)
    print(divisor.find_sum(394) == 51830)
    print(divisor.find_sum(12571) == 52683771)
    print(divisor.find_sum(95176) == 3019495236)
    print(divisor.find_sum(32088) == 343215484)
    print(divisor.find_sum(11859) == 46884881)
    print(divisor.find_sum(77610) == 2007787872)
    print(divisor.find_sum(42066) == 589857576)
    print(divisor.find_sum(85768) == 2452053728)
    print(divisor.find_sum(14176) == 66986582)
    print(divisor.find_sum(45224) == 681739206)
    print(divisor.find_sum(50542) == 851513538)
    print(divisor.find_sum(939977424) == 294519185904320510)
    print(divisor.find_sum(704387306) == 165387159111074046)
    print(divisor.find_sum(914360241) == 278684883778351235)
    print(divisor.find_sum(907119090) == 274288348002106490)
    print(divisor.find_sum(368375588) == 45233524646880100)
    print(divisor.find_sum(458647306) == 70119117196979106)
    print(divisor.find_sum(266122812) == 23607117064139852)
    print(divisor.find_sum(259408181) == 22430868239780363)
    print(divisor.find_sum(227989863) == 17326459347169793)
    print(divisor.find_sum(666671192) == 148150159459544002)
    print(divisor.find_sum(535722582) == 95666228436645486)
    print(divisor.find_sum(739788206) == 182428863472190212)
    print(divisor.find_sum(206200429) == 14172872404581673)
    print(divisor.find_sum(889914681) == 263982713516094237)
    print(divisor.find_sum(567173363) == 107228541538022371)
    print(divisor.find_sum(2) == 2)
    print(divisor.find_sum(3) == 5)
    print(divisor.find_sum(4) == 6)
    print(divisor.find_sum(5) == 11)
    print(divisor.find_sum(6) == 14)
    print(divisor.find_sum(8) == 22)
    print(divisor.find_sum(9) == 31)
    print(divisor.find_sum(536870912) == 96076792050570582)
    print(divisor.find_sum(536870911) == 96076792050570581)
    print(divisor.find_sum(536870913) == 96076792587441495)
    print(divisor.find_sum(999898469) == 333265649870347453)
    print(divisor.find_sum(11000000) == 40333333422394)
    print(divisor.find_sum(999586148) == 333057489191684940)
    print(divisor.find_sum(999999989) == 333333326457228137)
    print(divisor.find_sum(110000000) == 4033333333886606)
    print(divisor.find_sum(99999999) == 3333333333080737)
    print(divisor.find_sum(100000000) == 3333333333471362)
    print(divisor.find_sum(999999997) == 333333331832228103)
    print(divisor.find_sum(900000000) == 270000000001383796)
    print(divisor.find_sum(987654321) == 325153686292870545)
    print(divisor.find_sum(999688885) == 333125956048712835)
    print(divisor.find_sum(999999998) == 333333332332228102)
    print(divisor.find_sum(999999996) == 333333330832228106)
    print(divisor.find_sum(2356725) == 1851385320119)

#### fibonacchi: https://community.topcoder.com/stat?c=problem_statement&pm=7262

solution: https://community.topcoder.com/tc?module=HSProblemSolution&cr=22654692&rd=10074&pm=7262

Problem Statement
    	A fibonacci sequence is defined recursively as follows: 

F1 = F2 = 1;
Fn = Fn-1 + Fn-2;
Given two integers a and b, return the number of fibonacci numbers between a and b, inclusive.

 
Definition
    	
Class:	FibonacciSequence
Method:	numFibs
Parameters:	int, int
Returns:	int
Method signature:	int numFibs(int a, int b)
(be sure your method is public)
    
 
Constraints
-	a will be between 2 and 1000, inclusive.
-	b will be between a and 1000, inclusive.
 
Examples
0)	
    	
2
5
Returns: 3
The fibonacci sequence is {1,1,2,3,5,...}. There are 3 fibonacci numbers between 2 and 5, inclusive: {2,3,5}.
1)	
    	
12
13
Returns: 1
2)	
    	
13
13
Returns: 1


In [ ]:
class FibonacciSequence(object):
    
    def num_fibs(self, a, b):
        """Total fibonachis between a and b, inclusive."""
        result = 0
        f0 = f1 = 1
        if a <= 1: result = 2 
        while True:
            current = f1 + f0
            f0, f1 = f1, current
            if current >= a and current <= b:
                result += 1
            if current > b:
                break
        
        return result
 
if True:
    f = FibonacciSequence().num_fibs
    print(f(2, 5) == 3)
    print(f(12, 13) == 1)
    print(f(13, 13) == 1)
    print(f(2, 144) == 10)
    print(f(2, 3) == 2)
    print(f(13, 233) == 7)
    print(f(2, 21) == 6)
    print(f(2, 5) == 3)
    print(f(2, 13) == 5)
    print(f(2, 2) == 1)
    print(f(144, 233) == 2)
    print(f(8, 8) == 1)
    print(f(2, 89) == 9)
    print(f(2, 8) == 4)
    print(f(13, 34) == 3)
    print(f(5, 13) == 3)
    print(f(89, 144) == 2)
    print(f(233, 233) == 1)
    print(f(15, 89) == 4)
    print(f(2, 89) == 9)
    print(f(2, 89) == 9)
    print(f(8, 89) == 6)
    print(f(377, 1000) == 3)
    print(f(2, 2) == 1)
    print(f(15, 89) == 4)
    print(f(5, 34) == 5)
    print(f(2, 144) == 10)
    print(f(3, 13) == 4)
    print(f(2, 55) == 8)
    print(f(2, 55) == 8)
    print(f(5, 377) == 10)
    print(f(377, 987) == 3)
    print(f(3, 377) == 11)
    print(f(3, 34) == 6)
    print(f(2, 3) == 2)
    print(f(34, 377) == 6)
    print(f(233, 233) == 1)
    print(f(3, 34) == 6)
    print(f(2, 13) == 5)
    print(f(21, 34) == 2)
    print(f(13, 21) == 2)
    print(f(21, 610) == 8)
    print(f(13, 21) == 2)
    print(f(2, 89) == 9)
    print(f(144, 233) == 2)
    print(f(89, 610) == 5)
    print(f(3, 55) == 7)
    print(f(21, 987) == 9)
    print(f(8, 377) == 9)
    print(f(34, 55) == 2)
    print(f(2, 1000) == 14)
    print(f(3, 55) == 7)
    print(f(13, 21) == 2)
    print(f(4, 4) == 0)
    print(f(7, 7) == 0)
    print(f(2, 3) == 2)
    print(f(3, 4) == 1)
    print(f(14, 14) == 0)
    print(f(986, 988) == 1)
    print(f(3, 147) == 9)
    print(f(2, 1000) == 14)
    print(f(37, 400) == 5)
    print(f(2, 14) == 5)

#### volumediscount:  https://community.topcoder.com/stat?c=problem_statement&pm=5945

Problem Statement
    	
When a customer buys large quantities of a product, frequently the seller will offer a volume discount. For instance, one unit might cost 10 dollars, but might be offered in packages of 5 for 45 dollars. In such a case, it always makes sense buy the bulk lots to save money. In some other cases, however, it might not always make sense. Suppose a single unit were on sale for 8 dollars. In such a case, purchasing single units would be less expensive than purchasing a 5-pack.

You are given a String[] priceList describing the number of units available in each bundle, and the cost of the bundle. Each element is of the form "units cost" (quotes added for clarity). You are also given an int quantity, the number of units you wish to purchase.

Return an int indicating the best possible cost to purchase at least the desired quantity of units.

 
Definition
    	
Class:	VolumeDiscount
Method:	bestDeal
Parameters:	String[], int
Returns:	int
Method signature:	int bestDeal(String[] priceList, int quantity)
(be sure your method is public)
    
 
Constraints
-	priceList will contain between 1 and 5 elements, inclusive.
-	Each element of priceList will be formatted as described in the problem statement.
-	units will be an integer between 1 and 99, inclusive, with no leading zeroes
-	cost will be an integer between 1 and 999, inclusive, with no leading zeroes.
-	No two values of units will be the same.
-	quantity will be between 1 and 99, inclusive.
 
Examples
0)	
    	
{"1 10", "5 45"}
10
Returns: 90
The first example suggested in the problem statement.
1)	
    	
{"1 8", "5 45"}
10
Returns: 80
The second example suggested in the problem statement.
2)	
    	
{"99 913", "97 173", "50 464", "80 565"} 	
18
Returns: 173
Here, every package has more units than we need, so we pick the cheapest one.
3)	
    	
{"2 272","1 166","10 993"}
81
Returns: 8110

In [ ]:
import math
class VolumeDiscount(object):
    
    def best_deal(self, prices, n):
        """Return best cost to purchase at least n units."""
        
        ps = [int(p.split(" ")[1]) for p in prices]
        qs = [int(p.split(" ")[0]) for p in prices]
        costs = []
        for i in range(len(ps)):
            times = 1
            if n <= qs[i]:
                costs.append(ps[i])
                continue
            while True:
                if n <= qs[i] * times: break
                times += 1
            costs.append(times * ps[i])
        return min(costs)
if True:
    print(f(["1 10", "5 45"], 10) == 90)
    print(f(["1 8", "5 45"], 10) == 80)
    print(f(["99 913", "97 173", "50 464", "80 565"], 18) == 173)
    print(f(["99 577", "97 796", "50 449"], 50) == 449)
    print(f(["99 577", "97 796", "50 449"], 21) == 449)
    print(f(["64 302", "3 363"], 74) == 604)
    print(f(["33 895", "13 838", "94 565", "83 761"], 21) == 565)
    print(f(["56 470", "71 775", "17 372", "5 895", "67 527"], 84) == 899)
    print(f(["32 235", "2 798", "3 855", "30 760", "94 892"], 99) == 940)
    print(f(["46 317", "99 906", "26 151"], 7) == 151)
    print(f(["72 290", "31 805"], 59) == 290)
    print(f(["2 272", "1 166", "10 993"], 81) == 8110)
    print(f(["78 34"], 66) == 34)
    print(f(["49 650", "90 90", "12 999", "39 279", "86 93"], 40) == 90)
    print(f(["88 994", "5 880", "84 281", "92 305"], 67) == 281)
    print(f(["97 114", "57 316", "67 995", "29 336"], 36) == 114)
    print(f(["63 937", "92 374", "23 787", "69 379", "55 29"], 54) == 29)
    print(f(["20 483", "32 898"], 34) == 966)
    print(f(["45 692"], 59) == 1384)
    print(f(["88 30", "94 841", "17 293", "5 111"], 88) == 30)
    print(f(["74 531", "29 903", "85 276", "15 274", "12 620"], 96) == 550)
    print(f(["53 147", "30 2", "84 194"], 46) == 4)
    print(f(["30 565", "33 568", "69 477", "58 77", "59 765"], 3) == 77)
    print(f(["88 335"], 26) == 335)
    print(f(["52 869", "53 620", "57 775", "94 232", "42 392"], 79) == 232)
    print(f(["2 272", "1 166", "10 993"], 81) == 8110)
    print(f(["2 272", "1 166", "10 993"], 85) == 8654)
    print(f(["10 1", "3 2", "1 100"], 12) == 2)
    print(f(["3 15", "7 34"], 11) == 60)
    print(f(["2 2", "3 3"], 5) == 5)
    print(f(["16 511", "3 783", "9 127", "21 759", "18 590"], 3) == 127)
    print(f(["5 100", "3 63", "6 126"], 9) == 189)
    print(f(["1 5", "98 1"], 99) == 2)
    print(f(["10 5", "6 4", "2 4"], 12) == 8)
    print(f(["3 200", "1 100"], 10) == 700)
    print(f(["2 5", "3 6", "4 7", "5 9"], 7) == 13)
    print(f(["1 3", "10 9", "4 4"], 12) == 12)
    print(f(["4 30", "20 100", "2 20"], 89) == 480)
    print(f(["5 5", "8 9"], 6) == 9)
    print(f(["15 37", "10 28"], 17) == 56)
    print(f(["99 913", "97 173", "50 464", "80 565"], 18) == 173)
    print(f(["10 500", "3 150"], 13) == 650)
    print(f(["99 198", "60 60"], 80) == 120)
    print(f(["3 10", "5 12"], 6) == 20)
    print(f(["3 6", "4 9"], 4) == 9)
    print(f(["3 100", "2 49"], 3) == 98)
    print(f(["3 3", "4 4"], 4) == 4)
    print(f(["2 272", "1 166", "10 993", "9 900"], 81) == 8100)
    print(f(["10 200", "20 100"], 10) == 100)
    print(f(["32 2", "97 7"], 97) == 7)
    print(f(["1 5", "5 14", "3 9"], 7) == 23)
    print(f(["50 500", "9 99"], 63) == 693)
    print(f(["5 10", "4 8", "1 500"], 97) == 194)
    print(f(["1 10", "5 45"], 6) == 55)
    print(f(["5 10", "6 11"], 14) == 30)
    print(f(["10 11", "9 7"], 19) == 18)
    print(f(["1 8", "5 20"], 8) == 40)
    print(f(["3 99", "2 80"], 7) == 259)
    print(f(["2 272", "7 166", "5 993"], 81) == 1992)
    print(f(["10 40", "50 100"], 20) == 80)
    print(f(["12 300", "8 196", "5 123"], 41) == 1007)
    print(f(["1 1", "5 1"], 4) == 1)
    print(f(["6 51", "5 50"], 10) == 100)
    print(f(["3 5"], 8) == 15)
    print(f(["20 1", "1 100"], 81) == 5)
    print(f(["3 3", "4 4"], 10) == 10)
    print(f(["1 8", "90 1"], 98) == 2)
    print(f(["10 86", "30 357", "89 890", "87 837", "1 11"], 89) == 774)
    print(f(["98 100", "97 100"], 99) == 200)
    print(f(["98 999"], 99) == 1998)
    print(f(["49 490", "1 100", "2 21"], 50) == 511)
    print(f(["1 999", "99 99"], 2) == 99)
    print(f(["8 6", "9 10"], 9) == 10)
    print(f(["5 4", "1 1"], 6) == 5)
    print(f(["1 3"], 99) == 297)
    print(f(["3 9", "4 8", "5 9"], 7) == 16)
    print(f(["98 1"], 99) == 2)
    print(f(["2 12", "3 20"], 5) == 32)
    print(f(["13 130", "23 198"], 62) == 588)
    print(f(["10 1", "2 1"], 11) == 2)
    print(f(["50 100", "26 60"], 52) == 120)
    print(f(["98 10", "99 15"], 99) == 15)
    print(f(["4 4", "13 14"], 13) == 14)
    print(f(["9 9", "6 6", "4 4"], 10) == 10)
    print(f(["1 90", "10 800", "11 999"], 11) == 890)
    print(f(["1 1", "2 2", "3 3", "4 4", "5 5"], 99) == 99)
    print(f(["3 99", "5 165", "1 50", "2 80"], 7) == 245)
    print(f(["20 30", "30 100", "21 800"], 99) == 150)
    print(f(["2 272", "1 166", "10 993"], 84) == 8488)
    print(f(["9 90", "8 88", "2 80", "1 79"], 10) == 168)
    print(f(["10 10", "11 12"], 11) == 12)
    print(f(["4 30", "20 100", "2 30"], 89) == 490)
    print(f(["1 7", "3 16", "4 20"], 14) == 72)
    print(f(["29 30", "30 30"], 60) == 60)
    print(f(["3 10", "5 14"], 9) == 28)
    print(f(["98 1", "97 1"], 99) == 2)
    print(f(["2 10", "4 15"], 3) == 15)
    print(f(["32 32", "33 34"], 97) == 98)
    print(f(["10 10", "20 200"], 13) == 20)
    print(f(["2 272", "1 166", "10 993", "99 999", "77 234"], 99) == 468)
    print(f(["2 5"], 1) == 5)
    print(f(["3 3"], 5) == 6)
    print(f(["5 10", "6 12"], 20) == 40)
    print(f(["5 45", "2 20"], 8) == 80)
    print(f(["5 45", "4 45"], 7) == 90)
    print(f(["98 99"], 99) == 198)
    print(f(["1 4", "5 15", "2 7", "10 20"], 99) == 200)
    print(f(["2 70", "1 100", "10 200"], 81) == 1670)
    print(f(["3 3", "4 4"], 8) == 8)
    print(f(["1 20", "2 1", "3 2"], 99) == 50)
    print(f(["2 30", "3 20"], 1) == 20)
    print(f(["1 4", "5 15", "2 7", "10 20"], 8) == 20)
    print(f(["5 80", "4 81"], 8) == 160)
    print(f(["1 99", "9 1", "5 8"], 10) == 2)
    print(f(["99 3", "2 2"], 1) == 2)
    print(f(["20 4", "60 3"], 80) == 6)
    print(f(["5 8", "3 5"], 19) == 31)
    print(f(["1 10", "2 9", "3 8", "4 7", "5 6"], 99) == 120)
    print(f(["1 1", "2 1", "3 1", "4 1", "5 1"], 99) == 20)
    print(f(["12 300", "8 100", "5 123"], 41) == 600)
    print(f(["2 272", "1 166", "10 993"], 87) == 8926)
    print(f(["5 5", "2 3"], 7) == 8)

# Greedy

#### equalizerstrings: https://community.topcoder.com/stat?c=problem_statement&pm=10933&rd=14279&rm=&cr=7452866

In [ ]:
class EqualizerStrings(object):

    def getEq(self, a, b):
        """
        Make a == b in the less possible number of moves.

        Arguments:
            a (str): a word.
            b (str): another word.
        """
        if a == b: return a
        abc = {}
        n = len(a)
        ret = ""

        # for each letter in alphabet find distance to other letter
        for i, l1 in enumerate(string.ascii_lowercase):
            abc[l1] = {}
            for j, l2 in enumerate(string.ascii_lowercase):
                direct = (j - i) % 26
                indirect = (i - j) % 26
                distance = min(direct, indirect)
                abc[l1][l2] = distance

        # find which character needs more moves to become the other
        for i in range(n):
            if abc[a[i]][b[i]] < abc[b[i]][a[i]]:
                ret += b[i]
            elif abc[a[i]][b[i]] > abc[b[i]][a[i]]:
                ret += a[i]
                
            # if equal, find the min character that moving equals the sum
            elif abc[a[i]][b[i]] == abc[b[i]][a[i]]:
                for c in string.ascii_lowercase:
                    if abc[a[i]][c] + abc[b[i]][c] == abc[a[i]][b[i]]:
                        ret += c
                        break
        return ret

if True:
    f = EqualizerStrings().getEq
    print(f("cat", "dog") == "caa")
    print(f("abcdefghijklmnopqrstuvwxyz", "bcdefghijklmnopqrstuvwxyza") == "abcdefghijklmnopqrstuvwxya")
    print(f("programmingcompetitionsrule", "programmingcompetitionsrule") == "programmingcompetitionsrule")
    print(f("topcoderopen", "onlinerounds") == "onlcndaoondn")
    print(f("aazanamaobnbmbobpxxxyyyzzz", "azanamaobnbmbobpbzyxzyxzyx") == "aaaaaaaaabbbbaaaaxxxyyxzyx")
    print(f("oxj", "imo") == "imj")
    print(f("tdnzvghl", "nvhhjmzs") == "nahajgal")
    print(f("gskstxncxwploxjebv", "yzjaiksblsjeeomhpx") == "asjaianblsjeeojeav")
    print(f("zzsjzcrhulejffoajqlxyfwqevdqkhudcizcmxxfshomk", "ksmjzoujccnbcazakzlhsmtfboteseqkninwqeviegumc") == "asmjzcrhacebcaoajqlasftfboaekeqdcinamavfagomc")
    print(f("ylygvkabtioalcsdwuotigpugjnmbkcmvbxoipwhavuvnbpke", "tfwpzlgxdqkwgpfjdyxqbhfbicwlbrzsqzxmfrxqjsnhbkmtl") == "tfwgvkaaaikagaadauoqbgfagcnlbkamqaxmfpwhasnabbmke")
    print(f("ocfmfrgrphpuqpvubnugufprdrwqssolndeveuhjtinztffgrc", "pxonbiolxlzjppbonivgllbherjsvbczglatewduemrmtxdftk") == "oafmbiglphpjppaobiuglfahdraqsacagdateudjainatadfrc")
    print(f("mltmv", "rukyd") == "mlkma")
    print(f("norkqfb", "pdpqvsi") == "ndpkqab")
    print(f("rdlaokqchxbocbmzhjtxp", "oqwvqvogifvnerkaxxojs") == "oalaokochaancakaaaoap")
    print(f("qqyuhokxrunltnvnekbbrjtzxmuyxqwwgimkoqkcxzz", "ngzpwjwajatycdmosbfqgyxlqprxidkitsaarcmrbfv") == "ngypajkajanaadmnabbagataqmrxaakaaiaaoakaaav")
    print(f("eolnscassggpwauarbydbfynejltrnzbgxglqdoaordsytfb", "xlaxnunkssqaabzrmdvzcninvpjmclcmnrdhhybchdfxrjra") == "alannaaksggaaauambvabfanajjmalabgrdhhaaahadsrjfa")
    print(f("eltemmzycsgscdiqcvsyumnqktzadmvjhaznpcwffilzyakyim", "wrydxbjbmtedhakjsfwafevlpvyunwklaxikdyuhvvvrdedqpi") == "altdmbaacseacaijaasaaenlktyadmkjaaakdaufaalraadqii")
    print(f("rzbvbk", "qqwutu") == "qqauak")
    print(f("qsfhrjtbi", "biyfknllt") == "aiafkjlbi")
    print(f("uhkwwiggjrbstk", "tygrowkcrygrfo") == "tagroagcjrbrak")
    print(f("ivwoltddkdypqeyezibbjdpwfyddkacreuiphnkvnwelqy", "onygtbgpxwkokycjexhsqslvwjhbyzasgihcoqfpcqrmme") == "inwgladdaaaokaaeaabajalvaadbaaareihahnfpcqalma")
    print(f("ftdijyhitvwpscpmarwsgtuexrxtmvlbzadqexjmagwmxful", "pwdcdonjulftqjitkancxasosxcgjgkegorwcwzhpvmogqmn") == "ftdcdohitlapqcimaanaaasesraajakbaaaqcwahaammafml")
    print(f("qgqfcgwspdojlsieixbyimduphsovejaiwkzsngkhmvanuyrvy", "xwqkizppkmaixgguyzylzcuduvosrztfrbipjjazsdiyzkpfwm") == "qaqfcappkdailggaaxaaacaapaoorajaiaipjjaahdaankpfvm")
    print(f("locgy", "oolbn") == "locbn")
    print(f("qwppatznd", "aghymptmc") == "aahpaptmc")
    print(f("tilgbbxsfrmorazhalkoquf", "hubjoupujbftzibklqnwugz") == "hibgaapsfaforaahalkoqaa")
    print(f("edjocwejlnhifmcvchvpknwptogtfjnc", "kxfffetsdaunugqcttavkucwzvlrfgec") == "eaffcaajdaaiagaaahapknaptogrfgec")
    print(f("cflzgaifhmyrzxqowhwvpevqerqaweesrhaxyrxpycxiujhn", "vgpwrtaebzgnxdtylmweeiyrmvyqzfqyehhuhwnqfdydhvpy") == "aflwgaaebaanxaqolhwaeevqerqaweesahauarnpacxdajhn")
    print(f("vmetkijzbisfkhpsyapaljmzdjgjdhwuyunxquz", "vvpseeaujlhthrnlysgdihudiwueochxueftiuo") == "vmeseeaubihahhnlyagaihmadaaedcauuaftiuo")
    print(f("a", "a") == "a")
    print(f("a", "z") == "a")
    print(f("b", "n") == "b")
    print(f("o", "b") == "a")
    print(f("b", "n") == "b")
    print(f("jinmys", "ysosru") == "ainmrs")
    print(f("bpanyo", "zonman") == "aoaman")
    print(f("wzxjrusb", "ihacfusz") == "aaacfusa")
    print(f("zyyoyann", "zzpnzzzm") == "zypnyanm")
    print(f("uqbpjfzshdzbxyoj", "azznxcwyzdhiaosw") == "aqanacwsadabaooa")
    print(f("zomybaaozmznmbmb", "amanpznoannaoynm") == "amanaaaoamnamamb")
    print(f("krzyzhqiqgpqgzohvkkqhoolaohgnj", "hxaixcqpkiwaaezplwbxninbvpsdvf") == "hraaxcqikgpaaaohlkbqhinbaohdnf")
    print(f("ynyybpazmzmbpamopooobpbyzopbmo", "ybzoaabazoznponaazopopnpmbbanp") == "ybyoaaaaaoabpamaaoooapbpaaaamo")
    print(f("wosqiomtnygzcvcynjbixipeimyqvjbrzviijwdwwjyygaxwj", "yejgtpqinhdwkwwiggrcjqzuozdowzzwfkdzatjncomnkivtq") == "wejgiominadwcvaaggacaipaiaaovaarakdaatdnajmngavtj")
    print(f("omznoponanomobzonyoppynmamzaaynonbzyymazypnnybyaz", "mpobzzbnzbnpmzzoyynbybzmobbpabpmzboybpbaympbnppam") == "mmobopanabnmmazonynapanmabaaaanmnboyamaaymnbnapaa")
    print(f("apvphrdiqnudqbtebqzeonovwcudzpczfzqmqtwhismavfexvd", "dlsthqydtxtywyxmmqikzsdltyrxxoddaxwjowmjtfhrfhetxo") == "alsphqadqntaqatebqaeondltaraxocaaxqjotmhiahaafetvd")
    print(f("nnapbbmobmpynppnopbybnonyppobnamoaaynmnmyzbaoymaoo", "zpymzoapabzaoobonmaoyyozzapnbbyayomybmmbpnnbymyayy") == "nnamaaaoabpanoannmaoanonyapnbbaaoaaybmmbpnbaommaoo")
    print(f("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa", "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa") == "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
    print(f("nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn", "nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn") == "nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn")
    print(f("b", "m") == "b")
    print(f("b", "o") == "a")
    print(f("abcdefghijklmnopqrstuvwxyznopqrstuvwxyzabcdefghijk", "nopqrstuvwxyzabcdefghijklmabcdefghijklmnopqrstuvwx") == "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
    print(f("abcdefghijklmnopqrstuvwxyzcdefghijklmnopqrstuvwxyz", "zyxwwwwwwwwbbbbbcccccccccczyxwwwwwwwwbbbbbcccccccc") == "aaaaaaaaaakbbbaaaaaaaaaaaaaaaaaaaaklmbaaaaaaaaaaaa")
    print(f("bcde", "zxvd") == "aaad")
    print(f("b", "y") == "a")
    print(f("b", "z") == "a")
    print(f("jaksdfhsdf", "jhgfdljksd") == "jagadfhkad")
    print(f("z", "c") == "a")
    print(f("bbbbb", "nopzy") == "baaaa")
    print(f("c", "o") == "c")
    print(f("z", "b") == "a")
    print(f("ab", "ac") == "ab")
    print(f("b", "p") == "a")
    print(f("bboo", "otch") == "aach")
    print(f("j", "z") == "a")
    print(f("x", "b") == "a")
    print(f("zzz", "bbb") == "aaa")
    print(f("c", "n") == "c")
    print(f("m", "z") == "a")
    print(f("j", "u") == "j")
    print(f("z", "z") == "z")

#### largestsubsequence: https://community.topcoder.com/stat?c=problem_statement&pm=11471

In [ ]:
class LargestSubsequence(object):

    def getLargest(self, s):
        if len(s) == 1: return s
        ret = ""
        maxc = -1
        maxi = 0
        while True:
            maxc = max(s)
            for i in range(len(s)):
                if s[i] == maxc:
                    maxi = i
                    ret += maxc
                    break
            if i == len(s) - 1: break
            s = s[maxi + 1:]
            if len(s) == 0: break
        return ret
if True:
    f = LargestSubsequence().getLargest
    print(f("test") == "tt")
    print(f("a") == "a")
    print(f("example") == "xple")
    print(f("aquickbrownfoxjumpsoverthelazydog") == "zyog")
    print(f("zzyyxxwwvvuuttssrrqqppoonnmmllkkjjiihhggffeeddccaa") == "zzyyxxwwvvuuttssrrqqppoonnmmllkkjjiihhggffeeddccaa")
    print(f("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa") == "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
    print(f("dtbsthudhqvqxxmqxusna") == "xxxusna")
    print(f("ddifmyqoojkeyiee") == "yyiee")
    print(f("mhdsrshovdancelcihqcrsfdfvsrgstsxvzh") == "zh")
    print(f("vslheaxwhc") == "xwhc")
    print(f("lumguibrvvnvwnwhzail") == "zl")
    print(f("hctiodxvzmllfgri") == "zri")
    print(f("zjxhngujgnhlqpkfwuseqzcrxzjsltdekwoiklfgkywdpj") == "zzzywpj")
    print(f("sw") == "w")
    print(f("xpmjapuoeqouquzjnuyxzajdrn") == "zzrn")
    print(f("rkkpfgidl") == "rpl")
    print(f("phqhlsucxtebkyemlykfmzlznmopuhgvdambgpiglo") == "zzvpo")
    print(f("bzslcgclmgzulhxdrhkogsbuofrjrapcdutisprkrvosug") == "zzxvug")
    print(f("puluqyrejeyurtkeaufs") == "yyuus")
    print(f("znm") == "znm")
    print(f("nxaiqgj") == "xqj")
    print(f("rsng") == "sng")
    print(f("sxcpfwwyjsjfkrxpycnvcwvcnghwphpxrlfmcccvyppfr") == "yyyr")
    print(f("ubvbsmbdayynthasljdikieqdjbnmr") == "yytsr")
    print(f("dvypuukxnjiqwavhfsypfghhqhumbfznmjn") == "znn")
    print(f("t") == "t")
    print(f("lohyungouxtdkoqncecbecamiglwxamgkyjoisgymbeynmpftc") == "yyyytc")
    print(f("mvcjkcbtzdyet") == "zyt")
    print(f("zjejurusuqdatseyswqghs") == "zyws")
    print(f("rkjqiepcwkgoobtawithjybepylvvwm") == "yywm")
    print(f("sggqrxlcadkjjxcomhqgqpzkyetpz") == "zz")
    print(f("oizojobmoeyncmosopractypj") == "zyypj")
    print(f("qxydejwry") == "yy")
    print(f("rnlrwcsgegebmwweybg") == "yg")
    print(f("xbulkgnapqrb") == "xurb")
    print(f("fqgq") == "qq")
    print(f("euthsnsugfxtmfkwtlldwenybjfafvecasosvnxbs") == "yxs")
    print(f("oxedpvevehtmfsletplcpnndvlxduxxwieibnthkhfnkpvhj") == "xxxxwvj")
    print(f("akvccoziowkmrzpdwsvybiszfccgowzlosn") == "zzzzsn")
    print(f("ixnaxxjgbmisqsjxjxgc") == "xxxxxgc")
    print(f("i") == "i")
    print(f("djouowzmqzafubqqenotqmwzfgcmtpveanvmb") == "zzzvvmb")
    print(f("qparcaihdnyk") == "yk")
    print(f("frraeazgnzdmhtbtaymysglanvdqlbmqicqw") == "zzyyw")
    print(f("jeottiolzlxcy") == "zy")
    print(f("gubnawecsnsekp") == "wssp")
    print(f("uiktpzkdxynlifewoqmmbilpddlpecxndkebucbghbjqan") == "zyxuqn")
    print(f("cjpxagblsqilexcqzjngqhnc") == "zqnc")
    print(f("jvgloimmvsdhfgdafmzhxbzwfhkntrjpaocnidhfzeooqp") == "zzzqp")
    print(f("mdvmgsxmxnvxgjtsheoxvmgoryhnkswbznzvfrdrqjm") == "zzvrrqm")
    print(f("aanhrzdsbbhdoezohvuhzoeaekxpwhnj") == "zzzxwnj")
    print(f("hhf") == "hhf")
    print(f("onltutgmnnpxiwue") == "xwue")
    print(f("ygng") == "yng")
    print(f("sgrmksiyzesherusczthyotrbtzvegbzkulsprvikhohlyg") == "zzzzyg")
    print(f("rgzoaitknjjioypjiaevejqldjgtw") == "zyw")
    print(f("iynwupnztulzbyk") == "zzyk")
    print(f("mrtnnhfvaqtctfuipndaye") == "ye")
    print(f("zegbcvvxxfqxo") == "zxxxo")
    print(f("kwpxplhnattmiuhdq") == "xuq")
    print(f("zkhpjjfztbnxs") == "zzxs")
    print(f("wdxxiqzyhhxsp") == "zyxsp")
    print(f("egdnuvamuhkucdfvssxavdjuymvzgfymr") == "zyr")
    print(f("kwjuffpavnbkypmtfuxvabwj") == "yxwj")
    print(f("jbxoquxgeettykaciacheofaqceidc") == "yqidc")
    print(f("pnmqqmkogrhghpqwcpzdspbtvlvpotgdikgm") == "zvvtm")
    print(f("vwqmytncvpxazwrjwroacwono") == "zwwwoo")
    print(f("oehulzriqbdtawyytjskhlemqmdshspq") == "zyytsssq")
    print(f("mlwlasmlxqriipmqglsvswhnxyirhaofgyhkiwpo") == "yywpo")
    print(f("eszpgnfapcsbftxagmmdcqi") == "zxqi")
    print(f("rbjopthedsdewdudlupyicpmdtpmqrf") == "ytrf")
    print(f("renwjvqjfdzywfxpajvemksbdb") == "zyxvsdb")
    print(f("hnrthrdkyalotlymzcumobmvzvstciqmnbxdlgftjgpfbm") == "zzxtpm")
    print(f("uzkbkzmdozxqmvvmbbioyljyemntpldykxwjgnllqrdugrahe") == "zzzyyyxwurhe")
    print(f("tfhsnjkvbndnnqjkwfd") == "wfd")
    print(f("iy") == "y")
    print(f("zyeqfuqghbnhhrboymupudbomtlql") == "zyyuutql")
    print(f("yougpkmhhdmkorkkincpcnhxunainwqhbmmxkhwfarlahdmoak") == "yxxwrok")
    print(f("ljmlijlnlfaaynvqyhtxck") == "yyxk")
    print(f("ezhidvylyuyqluoxozyv") == "zzyv")
    print(f("kjqhcdshyfeqiwrencjowhqksyqeyeutemfgmyxs") == "yyyyxs")
    print(f("ydikqgggygevjdeyxpnzvmdvfkxralhnuvywensvwe") == "zywwe")
    print(f("rdpwejzcdxxcimwm") == "zxxwm")
    print(f("jtlgzwjtyaewnvgezxcyntlj") == "zzytlj")
    print(f("hjypqtcafblpnlxeaavswvaijkflyrekr") == "yyrr")
    print(f("drmtfhglfcmsobeqibtjcskngttvmzzpgkvxwctv") == "zzxwv")
    print(f("wvttsspponkjjihfebaa") == "wvttsspponkjjihfebaa")
    print(f("xwssrplkihgfc") == "xwssrplkihgfc")
    print(f("zzzyyyxwwvtronnlllljiiigeddccbbaa") == "zzzyyyxwwvtronnlllljiiigeddccbbaa")
    print(f("l") == "l")
    print(f("zxrrmfda") == "zxrrmfda")
    print(f("xwwvvutttssqqpppoonnnmllkkkkjjjihhffccccbbbba") == "xwwvvutttssqqpppoonnnmllkkkkjjjihhffccccbbbba")
    print(f("zyxuuuttttssqpoommmlkkjihgeeddccaaa") == "zyxuuuttttssqpoommmlkkjihgeeddccaaa")
    print(f("yxwsssrllkc") == "yxwsssrllkc")
    print(f("yvqqoollkca") == "yvqqoollkca")
    print(f("zyutttsqqppponnnnnmlllkkkjihhhggeeeddccbbb") == "zyutttsqqppponnnnnmlllkkkjihhhggeeeddccbbb")
    print(f("zxxvsrponmmljiiihgba") == "zxxvsrponmmljiiihgba")
    print(f("xwvssqoljjjiiigfccbaa") == "xwvssqoljjjiiigfccbaa")
    print(f("zzywwutsssrrronmllljjiihhgggggfeeea") == "zzywwutsssrrronmllljjiihhgggggfeeea")
    print(f("wvsqonkifecb") == "wvsqonkifecb")
    print(f("zzyyyxxwvvutssrrrqqppponmllkjjiihgffdcbbbaa") == "zzyyyxxwvvutssrrrqqppponmllkjjiihgffdcbbbaa")
    print(f("zzywuuusssrrrqponmlkkkiigfedccb") == "zzywuuusssrrrqponmlkkkiigfedccb")
    print(f("yyyyyyxxwwwvutttssrqqppppoonnmkjihgggddddaa") == "yyyyyyxxwwwvutttssrqqppppoonnmkjihgggddddaa")
    print(f("zxooc") == "zxooc")
    print(f("utqp") == "utqp")
    print(f("yxuttssrrnmjida") == "yxuttssrrnmjida")
    print(f("ffffeeeeeddddccccccccccbbbbbbbbaaa") == "ffffeeeeeddddccccccccccbbbbbbbbaaa")
    print(f("ffeeedcccca") == "ffeeedcccca")
    print(f("fdda") == "fdda")
    print(f("ffffeeeeeeddddcccccccbbbbbbbaaaaaaaaa") == "ffffeeeeeeddddcccccccbbbbbbbaaaaaaaaa")
    print(f("fffeeeeeddddddddcccccccbbbbbbbbbaaaa") == "fffeeeeeddddddddcccccccbbbbbbbbbaaaa")
    print(f("ffffeeeeeedddccccccbbbaaaa") == "ffffeeeeeedddccccccbbbaaaa")
    print(f("ffffffeeeeedddddddcccccccbbbbbbbbaaaaaaaa") == "ffffffeeeeedddddddcccccccbbbbbbbbaaaaaaaa")
    print(f("ddba") == "ddba")
    print(f("fffbbaaa") == "fffbbaaa")
    print(f("fedddccccbbbaa") == "fedddccccbbbaa")
    print(f("ffffeeeeeddbaaa") == "ffffeeeeeddbaaa")
    print(f("fffffeeeeeeeeeeedddddcccccccccccbbbbbbbbbbbaaaaa") == "fffffeeeeeeeeeeedddddcccccccccccbbbbbbbbbbbaaaaa")
    print(f("ffebbb") == "ffebbb")
    print(f("ffeedddcbbbbaa") == "ffeedddcbbbbaa")
    print(f("fffeeeeddccccbbbbbbbbaaaaaa") == "fffeeeeddccccbbbbbbbbaaaaaa")
    print(f("bbbb") == "bbbb")
    print(f("fffedaaaa") == "fffedaaaa")
    print(f("bbaaaabaabbaabbbaaabbaaabaaabababaaababaaaabbbbba") == "bbbbbbbbbbbbbbbbbbbbba")
    print(f("bbabbabbabaababaababaaaaaabaabab") == "bbbbbbbbbbbbbb")
    print(f("abbbbabbbbbbabbbbabbababbabbabbba") == "bbbbbbbbbbbbbbbbbbbbbbbba")
    print(f("bbbbbaabbaaabaabbababaabbababaabba") == "bbbbbbbbbbbbbbbbbba")
    print(f("bbbabababbaa") == "bbbbbbbaa")
    print(f("abbbbabbabbbabaaaaabaabaabaaababbbbabbb") == "bbbbbbbbbbbbbbbbbbbbb")
    print(f("babbaaaa") == "bbbaaaa")
    print(f("abbaabbaaaaabbbbbabbaaaaa") == "bbbbbbbbbbbaaaaa")
    print(f("baaababbaaabbbabbaaaaababaaabaaaaa") == "bbbbbbbbbbbbaaaaa")
    print(f("baabbaba") == "bbbba")
    print(f("aedcbacdeacdedbebbc") == "eeeec")
    print(f("caefdadbeedbebdcbcdca") == "feeeddca")
    print(f("fbbabcccbbbbfcabf") == "fff")
    print(f("ceaeedacaffddcfcdaebacdeeeb") == "fffeeeeb")
    print(f("dcbecaded") == "eed")
    print(f("ccddabecdfcbdefeedbbdaacda") == "ffeeddda")
    print(f("dcffbbbbcfefedbbbfcdafddebfcdfdeeccfe") == "fffffffffe")
    print(f("ffebcdbfccedbfaeaedaaaeefebdcdfa") == "ffffffa")
    print(f("addcfafeaeefcbefcbbcedfbcfcdffcaaeeafdadcfca") == "ffffffffffca")
    print(f("fcb") == "fcb")
    print(f("fgfdsdgaasfdghfgffggffgfgf") == "sshgggggf")
    print(f("asxcsa") == "xsa")
    print(f("fhjtgui") == "ui")
    print(f("naa") == "naa")
    print(f("ttt") == "ttt")
    print(f("maxbm") == "xm")
    print(f("baaaf") == "f")
    print(f("zba") == "zba")
    print(f("zzzzzzzzzzddddddddddccccccccccbbbbbbbbbbaaaaaaaaaa") == "zzzzzzzzzzddddddddddccccccccccbbbbbbbbbbaaaaaaaaaa")
    print(f("hhhhhhh") == "hhhhhhh")
    print(f("tatttatttt") == "tttttttt")
    print(f("aaaaaaaaa") == "aaaaaaaaa")
    print(f("abcdefghgiabcdefghgiabcdefghgiabcdefghgiabcdefghgi") == "iiiii")
    print(f("zzzzz") == "zzzzz")
    print(f("aa") == "aa")
    print(f("bza") == "za")

#### unblur: http://topcoder.bgcoder.com/print.php?id=646

 Problem Statement for Unblur


Problem Statement
    	
A simple way to blur an image is to replace each pixel with the average of it and its neighboring pixels. The value of each pixel in the blurred image is computed by adding the values of the 3x3 region centered at the corresponding pixel of the original image, and dividing by 9.

When computing the value of pixels on the border, the 3x3 region will fall partially outside of the original image. Assume that pixels outside of the original image are black.

For example, given the following original image:


the algorithm described above results in the following blurred image:


Write a method that will, given a blurred image, return the original image. The original image will contain only black and white pixels. All pixels on the top and bottom rows and left and right columns of the original image will be black. All values of the blurred image will therefore be: 0 (black), 1/9, 2/9, 3/9, 4/9, 5/9, 6/9, 7/9, 8/9, or 9/9 (white).

The blurred image will be given as a String[]. Each character in the blurred image will be a character between '0' and '9', inclusive, giving the value of that pixel multiplied by nine. For example, the blurred image above would be given as:


    { "1233321000000000123332100000000000000000000",
      "1244422233222332334343323322232112332223321",
      "1255523344343443545343434434343233432334432",
      "0033303455465775633011445546454355753457753",
      "0033303333364543533011433336333364521346542",
      "0033303455464532445343545546454355753446542",
      "0022202344342200234343434434343233432323221",
      "0011101233221100123332223322232112332211111" }
  
Return the original image as a String[]. For each pixel in the original image, return a '.' if it is black and '#' if it is white. For example, the original image for the example above would be returned as:

    { "...........................................",
      ".#####...........#####.....................",
      "...#...####.####.#...#.####.###..####.####.",
      "...#...#..#.#..#.#.....#..#.#..#.#....#..#.",
      "...#...#..#.####.#.....#..#.#..#.###..####.",
      "...#...#..#.#....#...#.#..#.#..#.#....#.#..",
      "...#...####.#....#####.####.###..####.#..#.",
      "..........................................." } 
 
Definition
    	
Class:	Unblur
Method:	original
Parameters:	String[]
Returns:	String[]
Method signature:	String[] original(String[] blurred)
(be sure your method is public)
    
 
Constraints
-	blurred will contain between 1 and 50 elements, inclusive.
-	Each element of blurred will have a length between 1 and 50, inclusive, and contain only characters between '0' and '9', inclusive.
-	Each element of blurred will have the same length.
-	blurred will be the result of blurring a black and white image.
 
Examples
0)	
    	
    { "1221",
      "1221",
      "1221" }
  
  
Returns:

    { "....",
      ".##.",
      "...." }
    
All pixels in the center two columns are adjacent to two white pixels in the source image. The remaining pixels are adjacent to only one.

1)	
    	
        { "00000",
          "00000",
          "00000",
          "00000" }
  
Returns: 
    
     { ".....",  ".....",  ".....",  "....." }

A solid black image blurs to all zeros.
2)	
    	
    { "0011212121100",
      "0123333333210",
      "0123333333210",
      "1233333333321",
      "1233333333321",
      "1233333333321",
      "0112121212110" } 
  
  
Returns: 

    { ".............",
     "...#.#.#.#...",
     "..#.#.#.#.#..",
     ".............",
     ".#.#.#.#.#.#.",
     "..#.#.#.#.#..",
     "............." }
 
 
original:


blurred:


3)	
    	
        { "1233321000000000123332100000000000000000000",
          "1244422233222332334343323322232112332223321",
          "1255523344343443545343434434343233432334432",
          "0033303455465775633011445546454355753457753",
          "0033303333364543533011433336333364521346542",
          "0033303455464532445343545546454355753446542",
          "0022202344342200234343434434343233432323221",
          "0011101233221100123332223322232112332211111" } 
  
Returns: 

        { "...........................................",
         ".#####...........#####.....................",
         "...#...####.####.#...#.####.###..####.####.",
         "...#...#..#.#..#.#.....#..#.#..#.#....#..#.",
         "...#...#..#.####.#.....#..#.#..#.###..####.",
         "...#...#..#.#....#...#.#..#.#..#.#....#.#..",
         "...#...####.#....#####.####.###..####.#..#.",
         "..........................................." }
         
This is the example from the problem statement.
4)	
    	
        { "0000123210000",
          "0012456542100",
          "0135789875310",
          "0258988898520",
          "1479865689741",
          "2589742479852",
          "2589742479852",
          "1479865689741",
          "0258988898520",
          "0135789875310",
          "0012456542100",
          "0000123210000" }
Returns: 

        { ".............",
         ".....###.....",
         "...#######...",
         "..#########..",
         "..####.####..",
         ".####...####.",
         ".####...####.",
         "..####.####..",
         "..#########..",
         "...#######...",
         ".....###.....",
         "............." }

original:


blurred:


This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       2004 TCO Semifinal Room 3 - Division I, Level One


In [ ]:
class UnBlur(object):

    def original(self, image):
        
        B = [list(map(int, list(r))) for r in image]
        S = [["." for i in range(len(r))] for r in B]

        # skipping top and bottom rows
        for y in range(1, len(B) - 1):
            
            # skipping left and right columns
            for x in range(1, len(B[0]) - 1):    

                # only possible values are 0 and 1
                if B[y - 1][x - 1] == 1:
                    S[y][x] = "#"
                
                    # subtract 1 from 3x3 region of B centered at (x,y)
                    for i in range(y-1, y+2):
                        for j in range(x-1, x+2):
                            if i >= 0 and i < len(B) and j >= 0 and j < len(B[0]):
                                B[i][j] -= 1

        S = ["".join(map(str, r)) for r in S]
        S = "\n".join(S)
        return S
if True:
    f = UnBlur().original
    print(f(["1233321000000000123332100000000000000000000",
      "1244422233222332334343323322232112332223321",
      "1255523344343443545343434434343233432334432",
      "0033303455465775633011445546454355753457753",
      "0033303333364543533011433336333364521346542",
      "0033303455464532445343545546454355753446542",
      "0022202344342200234343434434343233432323221",
      "0011101233221100123332223322232112332211111"] ))

#### WTF apothecary: https://community.topcoder.com/stat?c=problem_statement&pm=2312&rd=5850




 Problem Statement for Apothecary


Problem Statement
    	
An accurate scale is one of the most important tools of the apothecary (an old-time pharmacist). To measure the weight of an object, the apothecary places the object on one pan of the scale, along with some weights of known size, and adds more weights of known size to the other pan until the scales balance. For example, if an object weighs 17 grains, the apothecary could balance the scales by placing a 1-grain weight and a 9-grain weight in the pan with the object, and a 27-grain weight in the other pan.

The apothecary owns weights in a range of sizes starting at 1 grain. In particular, he owns one weight for each power of 3: 1 grain, 3 grains, 9 grains, 27 grains, etc. Determine, for an object weighing W grains, how to distribute the weights among the pans to balance the object. This distribution will be unique. Return a int[] of the weights used. The sign of each weight should be negative if the weight goes in the same pan as the object, and positive if it goes in the other pan. The int[] should be arranged in increasing order.

 
Definition
    	
Class:	Apothecary
Method:	balance
Parameters:	int
Returns:	int[]
Method signature:	int[] balance(int W)
(be sure your method is public)
    
 
Constraints
-	W is between 1 and 1000000, inclusive.
 
Examples
0)	
    	
17
Returns: { -9,  -1,  27 }
The example above.
1)	
    	
1
Returns: { 1 }
A 1-grain weight is placed in the pan opposite the object being measured.
2)	
    	
2016
Returns: { -243,  -9,  81,  2187 }
A 9-grain weight and a 243-grain weight are placed in the pan with the object, and an 81-grain weight and a 2187-grain weight are placed in the opposite pan.
3)	
    	
1000000
Returns: { -531441,  -59049,  -6561,  -243,  -27,  1,  81,  729,  2187,  1594323 }
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 204 Round 1 - Division I, Level One 
       Single Round Match 204 Round 1 - Division II, Level Three


In [ ]:
class Apothecary(object):
    
    def balance(self, W):
        """Return weights needed to balance the scale."""
        current_size = 1
        answer = []
        while W > 0:
            if W % 3 == 1:
                W -= 1
                answer.append(current_size)
            if W % 3 == 2:
                W += 1;
                answer.append(-current_size)
            W /= 3
            current_size *= 3
        
        answer.sort()
        return answer
if True:
    f = Apothecary().balance
    print(f(17))
    print(f(2016))

#### WTF rockstar: https://community.topcoder.com/stat?c=problem_statement&pm=2984&rd=5862

Problem Statement
    	Inspired by the Dire Straits song, "Money for Nothing", you have decided to become a rock star. After a lengthy recording session, you have acquired:
a total of ff songs that start fast and end fast,
a total of fs songs that start fast and end slow,
a total of sf songs that start slow and end fast, and
a total of ss songs that start slow and end slow.
It remains only to determine which of these songs should go on your first album, and in what order they should appear. Of course, no song can appear on the album more than once. 



Unfortunately, your record company has placed several restrictions on your album:
1. A song that ends fast may only be immediately followed by a song that starts fast.
2. A song that ends slow may only be immediately followed by a song that starts slow.
3. If you have at least one song that starts fast, then the first song on the album must start fast. Otherwise, this restriction can be ignored.
At this stage in your artistic career, you must do what your record company has ordered, but you do want to place as many songs as possible on the album. 



Given ints ff, fs, sf, and ss, representing the quantities described above, return the maximum number of songs that can be placed on a single album without violating the record company's restrictions.
 
Definition
    	
Class:	RockStar
Method:	getNumSongs
Parameters:	int, int, int, int
Returns:	int
Method signature:	int getNumSongs(int ff, int fs, int sf, int ss)
(be sure your method is public)
    
 
Constraints
-	ff, fs, sf, and ss will each be between 0 and 1000 inclusive.
-	At least one of ff, fs, sf or ss will be greater than 0.
 
Examples
0)	
    	
100
0
0
200
Returns: 100
You must begin the album with one of your fast songs by the 3rd restriction. By the 1st restriction, each subsequent song must also now start fast.
1)	
    	
0
0
20
200
Returns: 201
Since you do not have any songs that start fast, you may begin the album with a song that starts slow. You can use 201 songs by first using the 200 songs that start slow and end slow, then finishing the album with one song that starts slow and ends fast.
2)	
    	
1
2
1
1
Returns: 5
3)	
    	
192
279
971
249
Returns: 999

In [ ]:
class RockStar(object):
    
    def get_songs(self, ff, fs, sf, ss):
        if fs or ff:
            if fs == 0:
                return ff
            else: 
                if sf >= fs:
                    return ff + ss + 2 * fs
                else:
                    return ff + ss + 2 * sf + 1
        else:
            return ss + min(sf, 1)
if True:
    f = RockStar().get_songs
    print(f(100, 0, 0, 200))
    print(f(0, 0, 20, 200))
    print(f(192, 279, 971, 249))

#### WTF schoolassembly: https://community.topcoder.com/stat?c=problem_statement&pm=1119&rd=5879




 Problem Statement for SchoolAssembly


Problem Statement
    	You are a director of a school, having an assembly for your kids. The assembly is on the topic of consistency, and as a demonstration, you want to hand out a number of jelly beans to each kid, and have all the jelly beans that an individual kid receives be the same color (one kid's color may be different than another kid's color). At the store, each bag of jelly beans contains exactly 20 beans, and each bean can be one of 5 colors. The bags are opaque, so you cannot determine ahead of time exactly how many of each color bean are in each bag (a bag could have any combination of the 5 colors, including 20 of one color). Given an int kids (the number of children) and an int quantity (the number of beans you want to give each kid), return how many bags of jelly beans you must buy to ensure that you have enough jelly beans to give each individual kid quantity same-colored beans.
 
Definition
    	
Class:	SchoolAssembly
Method:	getBeans
Parameters:	int, int
Returns:	int
Method signature:	int getBeans(int kids, int quantity)
(be sure your method is public)
    
 
Constraints
-	kids will be between 1 and 1000, inclusive.
-	quantity will be between 2 and 25, inclusive.
 
Examples
0)	
    	
1
5
Returns: 2
Only one kid to buy for. Since the first bag you buy could contain 4 of each color, you must buy a second bag to ensure the child has 5 of the same color.
1)	
    	
1
2
Returns: 1
2)	
    	
5
5
Returns: 3
If we buy 2 bags, we will be able to give 4 of the children 5 beans. However, we could be left with 4 beans of each color, requiring the third bag.
3)	
    	
223
15
Returns: 171

In [ ]:
#### with every bag I can give 4 beans to 5 kids

class SchoolAssembly(object):
    
    def get_beans(self, kids, q):
        """return required bags."""
        t = (q - 1) * 4
        c = kids * q
        y = t + c 
        if y % 20 == 0: return y / 20
        else: return (y // 20) + 1 
if True:
    f = SchoolAssembly().get_beans
    print(f(1, 5))
    print(f(5, 5))
    print(f(223, 15))
            

#### playgame: https://community.topcoder.com/stat?c=problem_statement&pm=3002&rd=5863

Problem Statement for PlayGame


Problem Statement
    	You are playing a computer game and a big fight is planned between two armies. You and your computer opponent will line up your respective units in two rows, with each of your units facing exactly one of your opponent's units and vice versa. Then, each pair of units, who face each other will fight and the stronger one will be victorious, while the weaker one will be captured. If two opposing units are equally strong, your unit will lose and be captured. You know how the computer will arrange its units, and must decide how to line up yours. You want to maximize the sum of the strengths of your units that are not captured during the battle. 

You will be given a int[] you and a int[] computer that specify the strengths of the units that you and the computer have, respectively. The return value should be an int, the maximum total strength of your units that are not captured.
 
Definition
    	
Class:	PlayGame
Method:	saveCreatures
Parameters:	int[], int[]
Returns:	int
Method signature:	int saveCreatures(int[] you, int[] computer)
(be sure your method is public)
    
 
Constraints
-	you and computer will have the same number of elements.
-	you and computer will contain between 1 and 50 elements, inclusive.
-	Each element of you and computer will be between 1 and 1000, inclusive.
 
Examples
0)	
    	
{5, 15, 100, 1, 5}
{5, 15, 100, 1, 5}
Returns: 120
Your units with strengths of 100 and 15, along with one of your strength 5 units, can avoid capture.
1)	
    	
{1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000}
{1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000}
Returns: 0
All units are equally strong, so all your units are captured.
2)	
    	
{1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
{2, 4, 6, 8, 10, 12, 14, 16, 18, 20}
Returns: 99
You assign your weakest unit to the computer's strongest unit. Then you assign all your other units in such a way that each of your units has a strength one higher than the opposing unit. So all your units except the weakest one avoid capture.
3)	
    	
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
{10, 9, 8, 7, 6, 5, 4, 3, 2, 1}
Returns: 65
All your units can win.
4)	
    	
{651, 321, 106, 503, 227, 290, 915, 549, 660, 115,
 491, 378, 495, 789, 507, 381, 685, 530, 603, 394,
 7, 704, 101, 620, 859, 490, 744, 495, 379, 781,
 550, 356, 950, 628, 177, 373, 132, 740, 946, 609,
 29, 329, 57, 636, 132, 843, 860, 594, 718, 849}
{16, 127, 704, 614, 218, 67, 169, 621, 340, 319,
 366, 658, 798, 803, 524, 608, 794, 896, 145, 627,
 401, 253, 137, 851, 67, 426, 571, 302, 546, 225,
 311, 111, 804, 135, 284, 784, 890, 786, 740, 612,
 360, 852, 228, 859, 229, 249, 540, 979, 55, 82}
Returns: 25084


In [ ]:
class PlayGame(object):
    
    def save_creatures(self, you, cpu):
        """Return my final strength."""
        you.sort()
        cpu.sort()
        n = pos = len(you)
        ret = 0
        for i in range(n - 1, -1, -1):
            for j in range(pos - 1, -1, -1):
                if you[j] > cpu[i]:
                    ret += you[j]
                    pos = j
                    break
                    
        return ret
if True:
    f = PlayGame().save_creatures
    print(f([5, 15, 100, 1, 5], [5, 15, 100, 1, 5]) == 120) 
    print(f([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]) == 0)   
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]) == 44)    
    print(f([1, 3, 5, 7, 9, 11, 13, 15, 17, 19], [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]) == 99)   
    print(f([2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]) == 65)   
    print(f([561, 193, 805], [582, 478, 349]) == 1366)  
    print(f([819, 743, 174, 854, 707, 511, 303, 16, 92, 363, 148, 166, 983, 444, 120, 6, 10, 377, 529, 569, 599, 604, 167, 660, 449, 351, 58, 605, 779, 798, 518, 301, 871, 723, 951, 921, 537, 143, 460, 235, 858, 210, 776, 839, 991, 994], [609, 391, 266, 297, 836, 25, 375, 93, 674, 57, 10, 914, 275, 272, 585, 688, 833, 723, 483, 205, 740, 467, 456, 944, 741, 109, 596, 384, 731, 606, 570, 360, 152, 225, 424, 799, 515, 984, 748, 345, 169, 654, 490, 65, 696, 503]) == 23300)  
    print(f([944, 142, 900, 689, 302, 425, 71, 961, 680, 154], [873, 817, 579, 191, 178, 813, 473, 156, 502, 728]) == 4901) 
    print(f([279, 566, 679, 752, 718, 473, 124, 367, 830, 36, 515, 660, 425, 105, 944, 916, 547, 345, 470, 374, 842, 316], [454, 271, 978, 297, 735, 565, 196, 757, 835, 396, 499, 885, 29, 989, 570, 52, 529, 194, 839, 624, 654, 198]) == 11142)  
    print(f([124, 111, 739, 313, 936, 286, 335, 141, 729, 830, 704, 597, 743, 252, 145, 3, 62, 802, 848, 211, 116, 551, 16, 114, 453, 748, 683, 541, 75, 435, 202, 693, 290, 435, 232, 575, 530, 626, 161, 502, 958, 692, 920, 190], [335, 179, 990, 456, 992, 98, 622, 95, 436, 926, 50, 890, 289, 227, 765, 409, 202, 625, 601, 450, 465, 595, 632, 850, 824, 622, 717, 563, 374, 184, 734, 553, 900, 243, 189, 602, 695, 582, 350, 492, 81, 737, 609, 617]) == 18887)    
    print(f([800, 150, 573, 863, 907, 612, 724, 44, 665, 971, 314, 567, 305, 174, 109, 864, 847, 741, 155, 326, 80, 78, 638, 816, 543, 447, 408, 298, 464, 499, 153, 322, 734, 313, 822, 954], [869, 721, 299, 939, 128, 67, 781, 522, 607, 951, 73, 871, 651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378, 495, 789, 507, 381, 685, 530, 603, 394, 7, 704, 101, 620]) == 18078)  
    print(f([490, 744, 495, 379, 781, 550, 356, 950, 628, 177, 373, 132, 740, 946, 609, 29, 329, 57, 636, 132, 843, 860, 594, 718, 849, 16, 127, 704, 614, 218, 67, 169, 621, 340, 319, 366, 658, 798, 803, 524, 608, 794, 896, 145, 627], [401, 253, 137, 851, 67, 426, 571, 302, 546, 225, 311, 111, 804, 135, 284, 784, 890, 786, 740, 612, 360, 852, 228, 859, 229, 249, 540, 979, 55, 82, 522, 425, 95, 258, 887, 233, 147, 126, 927, 81, 48, 60, 336, 910, 397]) == 22642)    
    print(f([941, 833, 532, 838, 690, 396, 259, 6, 523, 950, 397, 241, 583, 255, 681, 940, 434, 885, 9, 936, 599, 782, 574, 143], [222, 382, 6, 416, 83, 657, 851, 66, 807, 659, 688, 799, 528, 682, 143, 686, 724, 774, 32, 864, 641, 703, 86, 550]) == 13269) 
    print(f([60, 275, 146, 976, 617, 292, 917, 366, 691, 219, 156, 240, 519, 897, 107, 898, 440, 81, 778, 172, 969, 772, 866, 211, 455, 5, 747, 115, 403, 310, 987, 40, 252, 189, 247, 154, 617, 881, 934, 196, 776, 642, 654, 904, 454, 916, 661, 151, 633], [567, 421, 938, 538, 576, 534, 255, 396, 349, 38, 791, 197, 71, 388, 588, 72, 198, 156, 641, 453, 602, 694, 440, 681, 395, 831, 590, 200, 944, 871, 390, 982, 185, 891, 572, 440, 624, 705, 51, 285, 260, 406, 890, 707, 725, 891, 392, 396, 899]) == 23129)  
    print(f([387, 568, 353, 730, 742, 733, 736, 140, 200, 272, 677, 906, 367, 515, 110, 903, 203, 516], [652, 437, 684, 91, 80, 77, 29, 354, 307, 694, 143, 394, 69, 672, 721, 199, 691, 613]) == 9058) 
    print(f([542, 99, 543, 51, 969, 462, 965, 724, 529, 677, 689, 371, 387, 51, 806, 170, 62, 310, 780, 212, 321, 483, 11, 438, 506, 603, 755, 400, 349, 838, 937, 620, 992, 55, 560], [859, 386, 306, 284, 28, 610, 390, 20, 32, 454, 853, 932, 401, 724, 609, 804, 871, 580, 592, 80, 789, 460, 588, 587, 567, 812, 669, 343, 12, 267, 341, 232, 767, 371, 865]) == 17037)    
    print(f([722, 626, 889, 183, 533, 897, 776, 960], [197, 897, 793, 948, 844, 886, 626, 186]) == 4870)    
    print(f([265, 250, 729, 784, 171, 710, 479, 82, 802, 851, 125, 307, 571, 471, 758, 192, 724, 989, 17, 645, 688, 174, 73, 275, 673, 834, 554, 370, 775, 232, 635, 232, 205], [966, 281, 24, 765, 152, 48, 11, 348, 641, 342, 413, 415, 92, 543, 696, 337, 654, 872, 533, 567, 55, 464, 260, 425, 859, 485, 88, 563, 706, 774, 860, 846, 292]) == 15174)  
    print(f([642, 743, 824, 762, 867, 903, 908, 520, 439, 108, 353, 819, 426, 170, 2, 654, 770, 65, 91, 384, 681, 803, 154, 143, 50, 345, 20, 973, 645, 55, 909, 634, 430, 22, 242, 391, 651, 778, 24, 822, 139, 631, 548, 978, 462], [215, 300, 778, 342, 479, 605, 103, 979, 752, 151, 455, 708, 993, 772, 639, 618, 941, 685, 289, 565, 290, 546, 709, 246, 544, 491, 947, 667, 761, 620, 444, 795, 742, 983, 205, 627, 905, 589, 913, 503, 487, 448, 885, 935, 183]) == 21261)  
    print(f([203, 727, 936, 622, 43, 638, 36, 554, 542, 445, 62, 990, 128, 598, 51], [173, 657, 332, 315, 650, 409, 426, 222, 675, 167, 496, 506, 799, 433, 6]) == 6383)    
    print(f([375, 129, 823, 730, 886, 640, 647, 512, 730, 733, 952, 464, 362, 113, 900, 817, 453, 109, 865], [757, 40, 415, 931, 69, 165, 211, 118, 588, 119, 615, 577, 687, 728, 911, 666, 298, 805, 377]) == 11131)   
    print(f([520, 727, 994, 964, 863, 836, 147, 373, 284, 171, 968, 738, 713, 32, 642, 808, 732, 814, 125, 715, 374, 647, 910, 885, 33, 770, 447, 9, 228, 417, 157, 914, 500, 243, 751, 902, 213, 33, 767, 228, 554, 802, 847, 331, 537, 986, 865], [644, 228, 809, 370, 366, 36, 559, 397, 120, 324, 894, 202, 935, 126, 845, 889, 444, 222, 632, 558, 621, 422, 18, 860, 140, 267, 291, 678, 559, 954, 604, 784, 421, 814, 736, 136, 514, 21, 8, 745, 333, 897, 460, 764, 52, 628, 25]) == 26516) 
    print(f([964, 498, 703, 266, 818, 644, 594, 263, 840, 458, 744, 351, 16, 475, 624, 403, 382, 29, 288, 594, 879, 431, 926, 382, 531, 11, 969, 861, 436, 696, 332, 600, 497, 557, 40, 377], [233, 932, 855, 959, 606, 581, 77, 406, 305, 497, 81, 870, 304, 777, 928, 586, 216, 279, 558, 870, 250, 301, 723, 862, 231, 191, 445, 886, 572, 30, 990, 60, 616, 257, 170, 117]) == 18383)   
    print(f([493, 955, 50, 81, 337, 279, 513, 489, 411, 920, 598, 292, 474, 400], [946, 546, 831, 790, 797, 590, 865, 317, 934, 268, 938, 715, 128, 771]) == 3968)  
    print(f([176, 28, 43, 453, 858, 137, 16, 213, 814, 793, 776, 655, 298, 777, 916, 838, 242, 497, 24, 77, 32, 355, 712, 408, 181, 297], [182, 741, 135, 59, 523, 358, 803, 159, 895, 132, 31, 182, 814, 249, 568, 346, 538, 662, 866, 927, 940, 106, 154, 556, 796, 890]) == 10473)   
    print(f([868, 818, 387, 546, 669, 449], [281, 944, 977, 724, 592, 272]) == 2901)    
    print(f([198, 598, 485, 354, 230, 477, 688, 943, 668], [977, 237, 511, 236, 893, 482, 326, 535, 568]) == 4213)  
    print(f([347, 170, 290, 219, 918, 736, 803, 647, 683, 270, 763, 65, 718, 854, 341, 46, 482, 22, 834], [616, 538, 601, 566, 773, 637, 693, 756, 863, 158, 422, 782, 23, 437, 43, 504, 269, 69, 245]) == 9075)    
    print(f([858, 759, 49, 481, 57, 956, 86, 563, 957, 571, 874, 407], [402, 412, 655, 819, 940, 283, 4, 613, 51, 653, 420, 949]) == 6569)  
    print(f([201, 413, 340, 620, 804, 6, 106, 393, 276, 316, 820, 849, 311, 198, 562, 467, 990, 432, 367, 417, 384, 578, 873, 337, 392, 946, 507, 417, 605, 773, 85, 152], [208, 88, 317, 470, 626, 332, 80, 612, 143, 280, 821, 437, 989, 221, 108, 517, 264, 9, 574, 767, 108, 964, 368, 899, 520, 223, 302, 477, 310, 413, 761, 837]) == 14846)  
    print(f([426, 721, 731, 92, 83, 749, 842, 635, 359, 473, 95, 414, 189, 342, 908, 229, 310, 155, 341, 182, 272, 206], [173, 334, 66, 500, 958, 805, 445, 908, 179, 480, 470, 696, 248, 932, 277, 299, 818, 815, 627, 791, 9, 261]) == 7958)  
    print(f([948, 855, 515, 704, 256, 916, 489, 94, 822, 78, 282, 383, 640, 513, 430, 141, 970], [76, 7, 910, 410, 751, 188, 102, 422, 675, 525, 820, 833, 271, 497, 866, 149, 965]) == 8958)   
    print(f([221, 68, 205, 198, 193, 200, 952, 591, 735, 294, 677, 782, 916, 923, 840, 259, 106, 63, 899, 149, 888, 358, 951, 380, 331, 681, 883, 379, 432], [671, 267, 481, 526, 798, 790, 743, 485, 823, 557, 278, 346, 55, 906, 759, 367, 282, 211, 324, 51, 877, 249, 402, 145, 871, 605, 780, 197, 450]) == 14168) 
    print(f([432, 633, 800, 175, 322, 958, 706, 198, 858, 48, 739, 37, 131, 83, 550, 566, 531, 219, 78, 304, 65, 410, 134, 251, 816, 507, 268, 982, 912, 122, 284, 703, 469, 294, 344, 523, 807, 515, 68, 825], [552, 104, 507, 941, 719, 472, 450, 914, 546, 364, 880, 657, 123, 505, 113, 279, 544, 787, 368, 854, 741, 462, 359, 440, 583, 378, 629, 555, 481, 832, 186, 11, 110, 42, 222, 256, 409, 704, 441, 134]) == 17035)   
    print(f([852, 802, 218, 919, 313, 144, 604, 12, 950, 535, 386, 14, 237, 881, 590, 153, 18, 533, 912, 650, 224, 404, 960, 98, 398, 504, 334, 328, 622], [250, 289, 531, 894, 476, 327, 671, 459, 580, 411, 917, 122, 662, 149, 247, 112, 103, 686, 822, 489, 452, 650, 417, 637, 100, 826, 368, 873, 518]) == 13309) 
    print(f([393, 722, 628, 373, 809, 848, 65, 740, 361, 235, 815, 780, 273, 561, 175, 125, 753, 412, 279, 171, 542, 241, 318, 225, 655, 695, 667, 640, 57, 329, 947, 320, 307, 163, 367, 276, 956, 236, 260, 622, 752, 580, 271, 709, 337, 711, 23, 697, 628, 127], [716, 380, 522, 860, 589, 493, 32, 669, 551, 349, 570, 760, 24, 2, 545, 744, 282, 790, 831, 720, 296, 536, 404, 103, 388, 98, 665, 162, 688, 668, 735, 883, 753, 567, 265, 112, 267, 730, 972, 486, 215, 797, 610, 348, 481, 993, 373, 943, 221, 627]) == 21810)   
    print(f([146, 925, 456, 639, 50, 653, 739, 101, 648, 200, 234, 728, 989, 750, 888, 545, 887, 864, 613, 716, 727, 152, 752, 153, 403, 612, 713, 826, 881, 65, 393], [601, 486, 450, 758, 576, 335, 580, 359, 746, 82, 96, 992, 46, 395, 109, 414, 432, 269, 451, 225, 261, 639, 888, 874, 448, 410, 232, 959, 736, 341, 719]) == 17232)  
    print(f([619, 889, 679, 235, 110, 529, 712, 565, 16, 509, 457, 965, 885, 699, 246, 104, 266, 464, 604, 50, 385, 117, 993, 736, 42, 6, 677, 951, 703, 747, 142, 675], [610, 660, 971, 72, 902, 857, 626, 884, 573, 153, 894, 571, 263, 176, 622, 830, 667, 854, 447, 978, 657, 705, 59, 812, 743, 969, 831, 313, 118, 206, 291, 943]) == 15332)  
    print(f([249, 216, 862, 429, 206, 924, 749, 389, 374, 556, 628, 936, 733, 732, 387, 548, 975, 625, 327, 428, 624, 692, 488, 665, 268, 105, 66, 140, 452, 180, 643, 852, 266, 813, 622, 912, 103, 254, 495, 698, 701, 110, 371, 480], [956, 100, 453, 94, 519, 325, 786, 820, 734, 561, 53, 188, 985, 426, 640, 718, 896, 838, 910, 481, 478, 724, 754, 383, 741, 274, 362, 32, 818, 754, 433, 224, 83, 265, 623, 763, 325, 254, 483, 748, 559, 347, 693, 962]) == 21363)    
    print(f([272, 884, 590, 604, 346, 797, 825, 811, 422, 82, 629, 977, 685, 497, 184, 565, 238, 857, 288, 984, 816, 401], [321, 397, 654, 926, 899, 277, 760, 523, 947, 912, 639, 49, 263, 288, 788, 258, 286, 351, 909, 814, 243, 321]) == 12250) 
    print(f([956, 245, 928, 814, 104, 575, 227, 375, 192, 783, 892, 309, 55, 748], [716, 556, 546, 367, 110, 70, 769, 29, 319, 308, 145, 524, 326, 737]) == 7148)   
    print(f([577, 921, 657, 706, 579, 73, 294, 16, 768, 992, 577, 305, 473, 135, 513, 187, 547, 376, 509, 674, 720, 704, 613, 124, 952, 727, 57, 228, 497, 134, 969, 455], [781, 609, 771, 610, 835, 288, 992, 853, 337, 158, 108, 958, 973, 582, 115, 69, 330, 397, 429, 731, 800, 594, 873, 561, 902, 206, 229, 179, 438, 320, 217, 830]) == 15333)   
    print(f([852, 397, 442, 944, 49, 784, 300, 517, 221, 665, 289, 173, 868, 470, 838, 907, 583, 160, 444, 679, 814, 977, 960, 512, 89, 52, 574, 858, 590, 601, 578, 282, 785, 386, 877, 120, 963, 59, 417, 343, 358], [3, 316, 212, 778, 489, 62, 884, 396, 566, 444, 703, 566, 582, 603, 196, 374, 772, 901, 562, 808, 795, 943, 766, 197, 627, 512, 839, 821, 726, 687, 616, 395, 874, 977, 531, 253, 597, 677, 638, 945, 560]) == 20854)    
    print(f([508, 203, 871, 170, 149, 718, 688, 554, 619, 991, 994, 595, 512, 523, 47, 828], [888, 549, 257, 7, 928, 813, 101, 131, 850, 380, 758, 66, 326, 18, 76, 369]) == 8923)  
    print(f([230, 572, 743], [354, 438, 115]) == 1545)  
    print(f([393, 319, 929, 594, 562, 868, 582, 236, 321, 234, 624, 652, 655, 757, 498, 509, 176, 418, 469, 979, 882, 23, 74, 93, 388, 316, 927, 828, 571, 492, 393, 915, 520, 113, 856, 416, 749, 990, 229, 31, 464, 119, 354], [339, 939, 22, 937, 777, 773, 772, 747, 611, 45, 54, 552, 694, 136, 286, 728, 329, 347, 508, 907, 939, 920, 857, 568, 935, 688, 306, 985, 424, 935, 602, 489, 166, 721, 50, 323, 501, 220, 17, 158, 534, 300, 844]) == 20889)  
    print(f([348, 3, 117, 285, 546, 2, 762, 77, 913, 802, 209, 382, 446, 771, 869, 910, 536, 641, 250, 610, 640, 710, 948, 568, 146, 346, 854, 774, 737], [937, 802, 110, 119, 27, 937, 439, 203, 634, 198, 331, 757, 646, 949, 154, 631, 178, 421, 78, 393, 752, 763, 827, 689, 406, 496, 638, 84, 284]) == 15197) 
    print(f([231, 618, 34, 974, 333, 976, 525, 57, 569, 413, 144], [397, 153, 302, 219, 54, 265, 594, 851, 333, 580, 918]) == 4783) 
    print(f([27, 785, 633, 82, 104, 523, 597, 434, 22, 664, 973, 419, 77, 342, 817, 109, 899, 583, 707, 187, 612, 44, 277, 559, 613, 630, 834, 545, 642, 569, 954, 325, 52, 655, 205, 789, 841, 243, 876], [486, 561, 762, 22, 729, 460, 290, 426, 646, 91, 918, 26, 215, 748, 537, 325, 5, 906, 700, 271, 767, 51, 211, 246, 545, 27, 144, 535, 381, 455, 843, 23, 409, 156, 736, 360, 379, 160, 359]) == 19227)   
    print(f([314, 373, 551, 695, 457, 370, 209, 990, 156, 362, 452, 211, 822, 892, 699, 432, 702, 287, 479, 176, 468, 325, 419, 23, 450, 299, 332], [20, 550, 842, 765, 624, 547, 671, 972, 475, 923, 686, 649, 736, 901, 550, 393, 380, 791, 194, 277, 75, 383, 329, 435, 71, 456, 497]) == 10270) 
    print(f([784, 142, 652, 814, 419, 221], [852, 760, 330, 79, 62, 227]) == 2890)  
    print(f([415, 577, 496, 914, 424, 215, 562, 906, 538, 600, 199, 628, 450, 801, 309, 572, 525, 673, 960, 133, 292, 37, 5, 397, 141, 181, 620, 21], [903, 759, 862, 364, 494, 569, 407, 793, 168, 244, 534, 333, 702, 441, 758, 469, 56, 517, 923, 452, 640, 216, 258, 771, 992, 412, 225, 191]) == 12254)    
    print(f([464, 654, 18, 328, 167, 986, 993, 520, 847, 76, 546, 862, 685, 411, 592, 711, 683, 290, 863, 619, 434, 650, 868, 548, 134, 167, 340, 627, 909, 350, 442, 645, 16, 207, 544, 903, 809, 228, 781, 628, 260, 222, 249, 533], [750, 822, 40, 771, 962, 142, 3, 860, 670, 376, 407, 773, 97, 925, 360, 843, 817, 778, 206, 646, 783, 641, 738, 496, 397, 729, 501, 735, 296, 603, 880, 470, 797, 939, 419, 869, 569, 700, 952, 769, 505, 520, 42, 626]) == 21325)   
    print(f([561, 193, 805, 582, 478, 349, 891, 819, 743, 174, 854, 707, 511, 303, 16, 92, 363, 148, 166, 983, 444, 120, 6, 10, 377, 529, 569, 599, 604, 167, 660, 449, 351, 58, 605, 779, 798, 518, 301, 871, 723, 951, 921, 537, 143, 460, 235, 858, 210, 776], [839, 991, 994, 609, 391, 266, 297, 836, 25, 375, 93, 674, 57, 10, 914, 275, 272, 585, 688, 833, 723, 483, 205, 740, 467, 456, 944, 741, 109, 596, 384, 731, 606, 570, 360, 152, 225, 424, 799, 515, 984, 748, 345, 169, 654, 490, 65, 696, 503, 148]) == 24277)  
    print(f([142, 900, 689, 302, 425, 71, 961, 680, 154, 873, 817, 579, 191, 178, 813, 473, 156, 502, 728, 404, 279, 566, 679, 752, 718, 473, 124, 367, 830, 36, 515, 660, 425, 105, 944, 916, 547, 345, 470, 374, 842, 316, 454, 271, 978, 297, 735, 565, 196, 757], [835, 396, 499, 885, 29, 989, 570, 52, 529, 194, 839, 624, 654, 198, 838, 124, 111, 739, 313, 936, 286, 335, 141, 729, 830, 704, 597, 743, 252, 145, 3, 62, 802, 848, 211, 116, 551, 16, 114, 453, 748, 683, 541, 75, 435, 202, 693, 290, 435, 232]) == 25538)    
    print(f([530, 626, 161, 502, 958, 692, 920, 190, 335, 179, 990, 456, 992, 98, 622, 95, 436, 926, 50, 890, 289, 227, 765, 409, 202, 625, 601, 450, 465, 595, 632, 850, 824, 622, 717, 563, 374, 184, 734, 553, 900, 243, 189, 602, 695, 582, 350, 492, 81, 737], [609, 617, 688, 800, 150, 573, 863, 907, 612, 724, 44, 665, 971, 314, 567, 305, 174, 109, 864, 847, 741, 155, 326, 80, 78, 638, 816, 543, 447, 408, 298, 464, 499, 153, 322, 734, 313, 822, 954, 869, 721, 299, 939, 128, 67, 781, 522, 607, 951, 73]) == 25715)    
    print(f([651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378, 495, 789, 507, 381, 685, 530, 603, 394, 7, 704, 101, 620, 859, 490, 744, 495, 379, 781, 550, 356, 950, 628, 177, 373, 132, 740, 946, 609, 29, 329, 57, 636, 132, 843, 860, 594, 718, 849], [16, 127, 704, 614, 218, 67, 169, 621, 340, 319, 366, 658, 798, 803, 524, 608, 794, 896, 145, 627, 401, 253, 137, 851, 67, 426, 571, 302, 546, 225, 311, 111, 804, 135, 284, 784, 890, 786, 740, 612, 360, 852, 228, 859, 229, 249, 540, 979, 55, 82]) == 25084)    
    print(f([425, 95, 258, 887, 233, 147, 126, 927, 81, 48, 60, 336, 910, 397, 431, 941, 833, 532, 838, 690, 396, 259, 6, 523, 950, 397, 241, 583, 255, 681, 940, 434, 885, 9, 936, 599, 782, 574, 143, 222, 382, 6, 416, 83, 657, 851, 66, 807, 659, 688], [799, 528, 682, 143, 686, 724, 774, 32, 864, 641, 703, 86, 550, 943, 60, 275, 146, 976, 617, 292, 917, 366, 691, 219, 156, 240, 519, 897, 107, 898, 440, 81, 778, 172, 969, 772, 866, 211, 455, 5, 747, 115, 403, 310, 987, 40, 252, 189, 247, 154]) == 23556)  
    print(f([578, 198], [829, 600]) == 0)   
    print(f([359, 918, 843, 765, 179, 880, 728, 526, 312, 16, 94, 374, 151, 170, 457, 122, 5, 10, 387, 545, 585, 617, 622, 171, 679], [462, 361, 59, 623, 803, 822, 533, 310, 897, 745, 979, 948, 553, 146, 474, 241, 883, 215, 799, 864, 627, 402, 273, 305, 861]) == 9947)    
    print(f([385, 95, 694], [58, 10, 941]) == 1079) 
    print(f([280, 603, 708, 858, 744, 497, 211, 762, 480, 469, 972, 763, 111, 614, 395], [753, 624, 587, 371, 156, 231, 436, 823, 530, 770, 354, 174, 674, 504, 66]) == 8467)   
    print(f([517, 152, 973, 145, 927, 710, 311, 437, 73, 990, 700, 157, 899, 842, 597, 196, 183, 837, 487, 160, 517, 750, 416, 287, 583, 699, 774, 740, 487, 126, 377, 855, 36, 530, 679, 437], [108, 973, 944, 563, 355, 484, 384, 868, 325, 468, 279, 305, 757, 581, 201, 780, 860, 408, 513, 912, 29, 587, 52, 545, 199, 864, 642, 674, 203, 863, 127, 113, 761, 322, 964, 293]) == 18206)   
    print(f([651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378, 495, 789, 507, 381, 685, 530, 603, 394, 7, 704, 101, 620, 859, 490, 744, 495, 379, 781, 550, 356, 950, 628, 177, 373, 132, 740, 946, 609, 29, 329, 57, 636, 132, 843, 860, 594, 718, 849], [16, 127, 704, 614, 218, 67, 169, 621, 340, 319, 366, 658, 798, 803, 524, 608, 794, 896, 145, 627, 401, 253, 137, 851, 67, 426, 571, 302, 546, 225, 311, 111, 804, 135, 284, 784, 890, 786, 740, 612, 360, 852, 228, 859, 229, 249, 540, 979, 55, 82]) == 25084)    
    print(f([5, 15, 100, 1, 5], [5, 15, 100, 1, 5]) == 120) 
    print(f([651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378], [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]) == 4468)    
    print(f([11, 12], [10, 20]) == 12)  
    print(f([1, 4, 5, 15, 100], [1, 5, 5, 15, 100]) == 120) 
    print(f([651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378, 495, 789, 507, 381, 685, 530, 603, 394, 7, 704, 101, 620, 859, 490, 744, 495, 379, 781, 550, 356, 950, 628, 177, 373, 132, 740, 946, 609, 29, 329, 57, 636, 132, 843, 860, 594, 718, 849], [16, 127, 704, 614, 218, 67, 169, 621, 340, 319, 366, 658, 798, 803, 524, 608, 794, 896, 145, 627, 401, 253, 137, 851, 67, 426, 571, 302, 546, 225, 311, 111, 804, 135, 284, 784, 890, 786, 740, 612, 360, 852, 228, 859, 229, 249, 540, 979, 55, 82]) == 25084)    
    print(f([5, 15, 100, 1, 5], [5, 15, 100, 1, 5]) == 120) 
    print(f([651, 321, 106, 503, 227, 290, 915, 549, 660, 115, 491, 378], [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]) == 4468)    
    print(f([11, 12], [10, 20]) == 12)  
    print(f([1, 4, 5, 15, 100], [1, 5, 5, 15, 100]) == 120)

#### review: fanfailure: https://community.topcoder.com/stat?c=problem_statement&pm=2235&rd=5070




 Problem Statement for FanFailure


Problem Statement
    	
In a robust computer system, one of the most important pieces is the cooling. Without proper cooling, processors can heat up to over 400 degrees C. The reliability of a system can be measured by determining how many fans can fail without risking the system processor. Each fan can be assigned a value indicating how much capacity it has to cool the system, and we can define a minimum cooling capacity, which the sum of the fan capacities must exceed to properly cool the system. We define a Failure Set as a set of fans which are not necessary to cool the system. In other words, if the fans in a Failure Set break, the system can still be properly cooled by the remaining fans. The count of a Failure Set is the number of fans in the set.

To measure the reliability, we will define two values, the Maximum Failure Set (MFS) and the Maximum Failure Count (MFC). A MFS is a Failure Set of fans with the largest count possible. A set of fans may have more than one MFS (see below). A Failure Set is an MFS if and only if there are no Failure Sets with a higher count. The MFC is the largest value such that all fan sets with count <= MFC are Failure Sets. In other words, any set of fans of size MFC or less can fail, and the system will still be properly cooled by the remaining fans.

Consider the fan set with capacities 1, 2, 3, and a cooling requirement of 2. Two MFSs with a count of 2 exist: fans 1 and 3, or fans 1 and 2. However, the MFC is not 2 because fans 2 and 3 is not a Failure set (fan 1 could not cool the system properly by itself). Thus, the MFC is 1, because if any single fan fails, the system can still be cooled.

You will be given a int[] capacities, which designates how many units of cooling each fan provides, and an int minCooling, which designates the minimum units of cooling required to cool the system. Your method should return a int[], where the first value should be the number of fans in the Maximum Failure Set (MFS), and the second value should be the Maximum Failure Count (MFC).

 
Definition
    	
Class:	FanFailure
Method:	getRange
Parameters:	int[], int
Returns:	int[]
Method signature:	int[] getRange(int[] capacities, int minCooling)
(be sure your method is public)
    
 
Constraints
-	capacities has between 1 and 50 elements, inclusive.
-	each element of capacities is between 1 and 1000, inclusive.
-	minCooling is between 1 and 50000, inclusive.
-	The sum of all elements in capacities will be greater than or equal to minCooling.
 
Examples
0)	
    	
{1,2,3}
2
Returns: { 2,  1 }
Example from the problem statement.
1)	
    	
{8,5,6,7}
22
Returns: { 0,  0 }
No fans can fail in this system.
2)	
    	
{676, 11, 223, 413, 823, 122, 547, 187, 28}
1000
Returns: { 7,  2 }
If you eliminate fans with 676, 11, 413, 122, 547, 187, and 28, you are left with 223 + 823 = 1046 units of cooling, which is sufficient, yielding an MFS size of 7. If you eliminate 676, 823, and 547, you are left with only 984 units of cooling. All combinations of 2 or less fans breaking leaves sufficient cooling, so the MFC is 2.
3)	
    	
{955, 96, 161, 259, 642, 242, 772, 369, 311, 785,
 92, 991, 620, 394, 128, 774, 973, 94, 681, 771,
 916, 373, 523, 100, 220, 993, 472, 798, 132, 361,
 33, 362, 573, 624, 722, 520, 451, 231, 37, 921,
 408, 170, 303, 559, 866, 412, 339, 757, 822, 192}
3619
Returns: { 46,  30 }

In [ ]:
class FailureCount(object):
    
    def get_range(self, capacities, minc):
        """Return (len(MFS), MFC)."""
        capacities = list(capacities)
        tcap = sum(capacities)
        n = len(capacities)
        capacities.sort()
        mfs = mfc = 0

        currentcap = 0
        for i in range(n):
            currentcap += capacities[i]
            if tcap - currentcap >= minc:
                mfs += 1
            else:
                break

        currentcap = 0
        for i in range(n - 1, -1, -1):
            currentcap += capacities[i];
            if tcap - currentcap >= minc:
                mfc += 1
            else:
                break

        return [mfs, mfc]
            
if True:
    f = FailureCount().get_range
    print(f([1, 2, 3], 2) == [2, 1])
    print(f([8, 5, 6, 7], 22) == [0, 0])
    print(f([676, 11, 223, 413, 823, 122, 547, 187, 28], 1000) == [7, 2])
    print(f([1000, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 50) == [49, 0])
    print(f([460, 191, 332, 693, 617, 837, 468, 983, 909, 214, 112, 586], 2278) == [9, 5])
    print(f([521, 16, 655, 906, 653, 957, 625, 842, 792, 15, 739], 6313) == [2, 0])
    print(f([516], 241) == [0, 0])
    print(f([89, 886, 545, 819, 567, 586, 340, 824, 148, 384, 845, 523, 259, 727, 520, 980, 426, 494, 533, 736], 7586) == [9, 4])
    print(f([229, 102, 817, 103, 486, 65, 543, 360, 289, 397, 995, 896, 347, 201, 733, 90, 966, 39, 843, 49, 851, 668, 904, 741, 500, 287, 463, 45, 639, 753], 8219) == [20, 6])
    print(f([120, 33, 225, 827, 144, 542, 592, 152, 998, 428, 235, 281, 192, 822, 733, 559, 705, 709, 78, 213, 614, 660, 504, 524, 880, 286, 304, 467, 546, 803, 939, 339, 369], 5123) == [27, 14])
    print(f([736, 678, 982, 498, 809, 32, 89, 324, 642, 354, 531, 624, 859], 5733) == [5, 1])
    print(f([177, 749, 10, 86, 993, 461, 952, 518, 823, 608, 160, 255, 914, 798, 33, 736, 264, 481, 446, 842, 194, 365], 1410) == [20, 13])
    print(f([192], 45) == [0, 0])
    print(f([328, 973, 135, 66, 802], 719) == [4, 1])
    print(f([450, 53, 454, 971, 270, 835, 320, 436, 319, 739, 19, 809, 625, 345, 107, 960, 647, 922, 136, 655, 651, 275, 80, 994, 353, 675, 637, 843, 206, 294, 729, 486, 452, 86, 586, 165, 121, 997, 734, 260, 553, 183, 492, 947, 204, 334, 611, 755, 962], 12193) == [35, 14])
    print(f([460, 425, 524, 776, 94, 392, 15, 513, 40, 757, 996, 662, 984, 192, 250, 140, 380, 753, 159, 916, 644, 706], 3630) == [18, 8])
    print(f([452, 367, 916, 53, 433, 219, 29, 730], 1937) == [5, 1])
    print(f([947, 965, 650, 863, 156, 507, 740, 895, 823, 55, 550, 986, 505, 326, 135, 779, 238, 46, 375, 264, 190, 857, 611, 17, 474, 929, 725, 679, 355, 52, 487, 676, 121, 342, 570, 62, 894, 921, 648, 532, 484], 20675) == [8, 0])
    print(f([959, 460, 721, 446, 808, 705, 464, 258, 343, 534, 982, 588, 695, 762, 559, 967, 256, 802, 270, 780, 608, 930, 82, 207, 678, 554, 810, 746, 340, 968, 423, 97, 870, 158, 566], 2677) == [32, 24])
    print(f([217, 518, 774, 447, 926, 352, 927, 173], 378) == [7, 6])
    print(f([325, 886, 304, 268, 370, 361, 948, 255, 239, 486, 215, 547, 967, 222, 742, 692, 79, 748, 680, 359, 718, 309, 378, 288, 971, 525, 63, 634, 258, 784, 236, 895, 305, 754, 376], 12252) == [18, 5])
    print(f([468, 768, 877, 394, 883, 16, 743, 380, 774, 880, 608, 704, 745, 677, 988, 442, 413, 555, 372, 427, 423, 299, 284, 347, 868, 492, 965, 171, 696, 496, 367, 144, 390, 524, 906, 865, 534, 497, 813, 409], 3000) == [36, 29])
    print(f([253, 959, 619, 780, 232, 494, 395, 102, 742, 565, 423, 75, 643, 914, 234, 32, 803, 790, 499, 319, 216, 936, 263, 530, 836, 853, 295, 162, 999, 730, 886, 811, 224, 590], 15953) == [11, 2])
    print(f([549, 440, 161, 964, 135, 92, 322, 859, 13, 576, 82, 636, 780, 871, 408, 685, 980], 4775) == [11, 4])
    print(f([670, 624, 589, 897, 343, 763, 730, 43, 104, 801, 226, 975, 393, 260, 315, 406, 24, 216, 957, 149, 736, 331, 672, 852, 599, 497, 221, 129, 881, 800, 788, 953, 286, 31, 733, 991, 106, 748, 282], 7926) == [30, 14])
    print(f([736, 746, 606, 778, 175, 124, 54, 343, 212, 304, 151, 508, 844, 394, 940, 863, 847, 980, 3, 744, 942, 361, 493, 374, 384, 397, 975, 738, 28, 534, 311, 934, 748, 628, 611, 49, 367, 155, 150, 137, 590, 555, 729, 505, 380], 4071) == [40, 25])
    print(f([122, 160, 238, 220, 856, 425, 442, 143, 579, 115, 469, 566, 484, 133, 165, 13, 503, 380, 984, 342, 411, 936, 387, 597, 423, 650, 312, 752, 871, 111, 179, 487, 71, 976, 456, 813, 51, 235, 540, 145, 177, 86, 660, 618], 8275) == [33, 13])
    print(f([710, 802, 948, 866, 943, 129, 369, 759, 168, 728, 930, 583, 740, 944, 278, 766, 339, 915, 393, 152, 807, 80, 922, 306], 5995) == [17, 9])
    print(f([603, 874, 198, 502, 142, 650, 65, 180, 518, 476, 973, 529, 15, 779, 102, 562, 387, 643, 842, 323, 257, 805, 470, 309, 713, 138, 204, 296, 770, 259, 334, 797, 43, 719, 405, 341], 6804) == [27, 12])
    print(f([674, 415, 799, 675, 61, 881, 259, 561, 543, 84, 745, 163, 560, 861, 646, 238, 463, 330], 8362) == [4, 0])
    print(f([580, 635, 543, 167, 994, 386, 608, 298, 984, 276, 659, 485, 297, 946, 154, 426, 420, 906, 707, 786, 238, 959, 288, 807, 651, 943, 112, 827, 794], 625) == [28, 25])
    print(f([11, 106, 522, 987, 535, 452, 90, 944, 878, 305, 247, 826, 394, 929, 619, 111, 472, 961, 322], 313) == [18, 15])
    print(f([304, 706, 398, 697, 73, 679, 826, 117, 232, 40, 222, 745, 681, 94, 895, 747, 720, 516, 917, 116, 77, 609, 30, 283, 150, 600, 854, 26, 684, 627], 1793) == [28, 16])
    print(f([221, 386, 146, 708, 540, 86, 505, 653, 726, 603, 740, 737, 240], 3925) == [7, 3])
    print(f([855, 156, 869, 623, 538, 913, 386, 795, 392, 62, 346, 185, 541, 231, 373, 751, 271, 949, 885, 507, 792, 138, 493, 789, 821, 34, 948, 370, 61, 914, 329, 38, 858, 532, 814], 2994) == [31, 20])
    print(f([542, 273, 6, 20, 71, 198, 417], 635) == [5, 1])
    print(f([194, 549, 740, 519, 874, 582], 2381) == [2, 1])
    print(f([135, 860, 780, 741, 151, 778, 482, 650, 626, 69, 484, 897, 544, 821, 389, 83, 356, 116, 979, 813, 987, 718, 292, 365, 448, 214, 402, 393, 139, 344, 153, 655, 818, 668, 82, 851, 776, 735, 455, 125, 437, 578, 377, 348, 555, 439, 939, 321], 22891) == [10, 1])
    print(f([847, 877, 269], 1020) == [1, 1])
    print(f([349, 735, 687, 160, 84, 478, 405, 396, 841, 479, 301, 13, 152, 246, 949, 849, 657, 318, 264, 552, 491, 730, 259, 844, 972, 427, 340, 736, 655, 944, 935, 700, 932, 748, 430, 767, 808, 753, 752, 30, 504, 876, 447, 793, 417], 10722) == [32, 17])
    print(f([881, 581, 768, 151, 706, 164, 261, 312, 789, 410, 576, 712, 590, 59, 823, 867, 51, 246, 393, 661], 4233) == [14, 7])
    print(f([219, 585, 301, 793, 864, 687, 259, 436], 1458) == [6, 3])
    print(f([549, 35, 233, 569, 558, 885, 994, 991, 516, 696, 22, 692, 749, 659, 307, 63, 122, 435, 910], 1792) == [17, 10])
    print(f([736, 480, 235, 134, 713, 421, 286, 780, 429, 425, 837, 76, 349, 775, 339, 766, 830, 707, 40, 490, 25, 919, 825, 358, 483, 782, 280, 301, 907, 972, 450, 13, 327, 302, 47, 500, 187, 298, 178, 401, 866, 179, 192, 874, 22, 42], 11710) == [31, 10])
    print(f([269], 88) == [0, 0])
    print(f([546, 790, 728, 560, 415, 847, 393, 500, 752, 289, 974, 114, 355, 352, 602, 729, 127, 834, 536, 54, 523, 502, 521, 439, 11, 963, 600, 208, 87, 708, 389, 737, 9], 11549) == [16, 5])
    print(f([311, 50, 706, 527, 636, 366, 80, 485, 238, 302, 99, 861, 312, 806, 812, 848, 488, 295, 610, 817, 765, 694, 854, 35, 753, 304, 152, 16], 11675) == [9, 1])
    print(f([39, 338, 527, 254, 869, 457, 548, 856, 998, 37, 910, 242, 526, 573, 42, 57, 673, 960, 748, 771], 10091) == [4, 0])
    print(f([273, 405, 691, 916, 917, 813, 285, 665, 616, 827, 118, 714, 942, 93, 444, 780, 762], 4373) == [12, 6])
    print(f([527, 311, 180, 573, 922, 513, 724, 282, 886, 415, 151, 692], 601) == [11, 9])
    print(f([649, 908, 976, 281, 487, 269, 428, 984, 402, 663, 716, 183, 528, 979, 703, 231, 778], 6066) == [9, 4])
    print(f([637, 161, 135, 520, 349, 702, 880, 429, 717, 607, 480, 479, 275, 993, 501, 515, 116, 291, 199, 810, 555, 260, 489, 206, 634, 955, 322, 770, 529, 268, 518, 224, 692, 129, 39, 188, 517, 265, 44, 254, 152], 17415) == [4, 0])
    print(f([48, 306, 331, 100, 316, 142, 805, 899], 2287) == [4, 0])
    print(f([955, 96, 161, 259, 642, 242, 772, 369, 311, 785, 92, 991, 620, 394, 128, 774, 973, 94, 681, 771, 916, 373, 523, 100, 220, 993, 472, 798, 132, 361, 33, 362, 573, 624, 722, 520, 451, 231, 37, 921, 408, 170, 303, 559, 866, 412, 339, 757, 822, 192], 3619) == [46, 30])
    print(f([1000], 1) == [0, 0])
    print(f([10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 45) == [15, 7])
    print(f([676, 11, 223, 413, 823, 122, 547, 187, 28], 1000) == [7, 2])
    print(f([2, 4, 6, 8], 1) == [3, 3])

#### grocerybagger: https://community.topcoder.com/stat?c=problem_statement&pm=3450&rd=5868




 Problem Statement for GroceryBagger


Problem Statement
    	
You have just begun working as a grocery bagger at the local TopGrocer food store. Your job is to place all of a customer's items into bags, so they can be carried from the store. Your manager has instructed you to use as few bags as possible, to minimize the store's overall cost. However, for the customer's convenience, you are instructed that only items of the same type can be placed in the same bag. For instance, a produce item can be bagged with any other produce items, but not with dairy items.

You are given a String[] itemType indicating the type of each item that needs to be bagged. You are also given an int strength indicating the maximum number of items that can be placed in each bag. You are to return an int indicating the minimum number of bags required to package the customer's items.

 
Definition
    	
Class:	GroceryBagger
Method:	minimumBags
Parameters:	int, String[]
Returns:	int
Method signature:	int minimumBags(int strength, String[] itemType)
(be sure your method is public)
    
 
Constraints
-	strength will be between 1 and 50, inclusive.
-	itemType will contain between 0 and 50 elements, inclusive.
-	Each element of itemType will contain between 1 and 50 characters, inclusive.
-	Each element of itemType will contain only the characters 'A'-'Z'.
 
Examples
0)	
    	
2
{"DAIRY",
 "DAIRY",
 "PRODUCE",
 "PRODUCE",
 "PRODUCE",
 "MEAT"}
Returns: 4
Here, you have six items. You could put two items in each bag, but you would have to mix item types. The single meat item must get its own bag. The two dairy items fit in one bag. The three produce items will require two bags.
1)	
    	
3
{"DAIRY",
 "DAIRY",
 "PRODUCE",
 "PRODUCE",
 "PRODUCE",
 "MEAT"}
Returns: 3
Similar to above, but now we have stronger bags. Note again, though, that if we were allowed to mix item types, we could get away with only 2 bags. But since item types cannot be mixed, we need 3 bags.
2)	
    	
10
{}
Returns: 0
The bags are really strong, but we didn't buy anything.
3)	
    	
5
{"CANNED",   "CANNED",  "PRODUCE",
 "DAIRY",    "MEAT",    "BREAD",
 "HOUSEHOLD","PRODUCE", "FROZEN",
 "PRODUCE", "DAIRY"}
Returns: 7
Notice that a customer doesn't necessarily pay for the items in any particular order, but the bagger still has to be responsible for sorting them out. In this case, though, one bag for each item type suffices.
4)	
    	
2
{"CANNED",   "CANNED",  "PRODUCE",
 "DAIRY",    "MEAT",    "BREAD",
 "HOUSEHOLD","PRODUCE", "FROZEN",
 "PRODUCE", "DAIRY"}
Returns: 8
As above, but our produce requires two bags now.


In [ ]:
class GroceryBagger(object):
    
    def minimum_bags(self, s, items):
        """Return the minimum number of bags required to package the items."""
        d = {}
        for i in items:
            if i not in d:
                d[i] = 1
            else:
                d[i] += 1
            
        res = 0
        for v in d.values():
            res += v // s + (v % s != 0)

        return res
        
if True:
    f = GroceryBagger().minimum_bags
    print(f(2, ["DAIRY", "DAIRY", "PRODUCE", "PRODUCE", "PRODUCE", "MEAT"]) == 4)
    print(f(3, ["DAIRY", "DAIRY", "PRODUCE", "PRODUCE", "PRODUCE", "MEAT"]) == 3)
    print(f(10, []) == 0)
    print(f(5, ["CANNED", "CANNED", "PRODUCE", "DAIRY", "MEAT", "BREAD", "HOUSEHOLD", "PRODUCE", "FROZEN", "PRODUCE", "DAIRY"]) == 7)
    print(f(2, ["CANNED", "CANNED", "PRODUCE", "DAIRY", "MEAT", "BREAD", "HOUSEHOLD", "PRODUCE", "FROZEN", "PRODUCE", "DAIRY"]) == 8)
    print(f(11, ["GTIIHVQ", "GTIIHVQ", "PFPDCBOEJ", "PFPDCBOEJ", "JFRIOPWZXTAWOBPSMLNFMBQATNVWC", "GTIIHVQ", "JFRIOPWZXTAWOBPSMLNFMBQATNVWC", "GTIIHVQ", "PFPDCBOEJ", "GTIIHVQ", "GTIIHVQ", "PFPDCBOEJ", "FXOKNOCHJPFKXOXZNBTQKFBSNBHAOGYQIGFDHNL", "GTIIHVQ", "GTIIHVQ", "PFPDCBOEJ", "GTIIHVQ", "PFPDCBOEJ", "JFRIOPWZXTAWOBPSMLNFMBQATNVWC", "GTIIHVQ", "GTIIHVQ", "GTIIHVQ", "PFPDCBOEJ", "GTIIHVQ", "GTIIHVQ", "GTIIHVQ", "GTIIHVQ", "GTIIHVQ", "PFPDCBOEJ", "GTIIHVQ", "PFPDCBOEJ", "PFPDCBOEJ"]) == 5)
    print(f(37, ["VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT", "VSUGSVKPCPQWBXJPBKT"]) == 1)
    print(f(27, ["WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "TNQAGBBDWOBVXIC", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "UVPXOPN", "TNQAGBBDWOBVXIC", "ECNAUROLXFIJZWJSTJ", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "ECNAUROLXFIJZWJSTJ", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "OJBQHUQYRVXDFDNROLCDBIBIKWBSGUIAJNGFBZPMQYKWJRTOKM", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "TNQAGBBDWOBVXIC", "ECNAUROLXFIJZWJSTJ", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "TNQAGBBDWOBVXIC", "TNQAGBBDWOBVXIC", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "PFDCDKEEMCFXWHUAIVXUBRGK", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "TNQAGBBDWOBVXIC", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "AOLGNYQPVAPNYRZJNPLORJWQRXWCBKKHAHXM", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "WBYKLAISBNYQHGZLULPEURFPXRSYUMXI", "TNQAGBBDWOBVXIC", "OJBQHUQYRVXDFDNROLCDBIBIKWBSGUIAJNGFBZPMQYKWJRTOKM"]) == 7)
    print(f(19, ["CTBNGESALQIGSYEPBYFQTRWJWQHNGEXHMJZC", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "NBOGFFSENJKVOKCRQSPAELQOXTSPF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "NBOGFFSENJKVOKCRQSPAELQOXTSPF", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "NBOGFFSENJKVOKCRQSPAELQOXTSPF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "CTBNGESALQIGSYEPBYFQTRWJWQHNGEXHMJZC", "CTBNGESALQIGSYEPBYFQTRWJWQHNGEXHMJZC", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "SNRVIRFILQWIASMWGQFOAYIDSBWNZF", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX", "FGXAFVMMGPXEMEMZSQXPPGLAMJEWGMPIRTMAUKYKMYSCQBUX"]) == 4)
    print(f(14, ["GIY", "GIY", "GDKEUGBSDLVIQBYFIHYTPBVNHJXOTMJYIQOOBYGZSNCWXWSK", "C", "GIY", "GIY", "C", "C", "GIY", "C", "C", "C", "GIY", "GIY", "GIY"]) == 3)
    print(f(45, ["XEUXDJXUYMWCXQUKTYMMCFAGDLPAQXUEFFWAWAKTSLHXCET", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "RUKJASCCIGF", "HVIIZKRLPFTYBCO", "RUKJASCCIGF", "XEUXDJXUYMWCXQUKTYMMCFAGDLPAQXUEFFWAWAKTSLHXCET", "RUKJASCCIGF", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "UEWDPQBHJ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "RUKJASCCIGF", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ", "XQERREFZWJKEAZPRITSQCYVLNKPZOOUVDVBQRJFESWFZ"]) == 5)
    print(f(6, ["FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "BVAEUFCHUVYCKYVAKZV", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "YEPDMDYH", "YEPDMDYH", "WUOWLXYQNTKQOOKNQQMSBQVTBMCOUDVWTULGBOFCLXYXD", "BVAEUFCHUVYCKYVAKZV", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "YEPDMDYH", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "YEPDMDYH", "BVAEUFCHUVYCKYVAKZV", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "BVAEUFCHUVYCKYVAKZV", "FLGULCPGRLQHJHSLCVEFQVJBAJA", "YEPDMDYH", "YEPDMDYH", "YEPDMDYH", "BVAEUFCHUVYCKYVAKZV", "BVAEUFCHUVYCKYVAKZV"]) == 7)
    print(f(6, ["WNOZQPDWMCAHZJDMGMIMNAXKCIZOSSNZRJGGX", "WNOZQPDWMCAHZJDMGMIMNAXKCIZOSSNZRJGGX", "WNOZQPDWMCAHZJDMGMIMNAXKCIZOSSNZRJGGX", "WNOZQPDWMCAHZJDMGMIMNAXKCIZOSSNZRJGGX", "WNOZQPDWMCAHZJDMGMIMNAXKCIZOSSNZRJGGX"]) == 1)
    print(f(24, ["TOJRFIDS", "DXJHGHYMLDUDWRWFGZRMNHPNDKXCGQJJRYBTLDHVDOYOFUYRT", "TOJRFIDS", "TOJRFIDS", "DXJHGHYMLDUDWRWFGZRMNHPNDKXCGQJJRYBTLDHVDOYOFUYRT", "DRPQYQWVSQHITYHZGK", "PGLJNCANRZUIFTJZGTFFOINMLXKPGCDQSHZVUFBDPAJOTIEF", "TOJRFIDS", "DVTBJRYSGUMPABCWJCEFUTPCQ", "DFQ", "TOJRFIDS", "DFQ", "DFQ", "DRPQYQWVSQHITYHZGK", "TOJRFIDS", "DFQ", "GBVRPLKY", "TOJRFIDS", "TOJRFIDS", "PGLJNCANRZUIFTJZGTFFOINMLXKPGCDQSHZVUFBDPAJOTIEF", "DXJHGHYMLDUDWRWFGZRMNHPNDKXCGQJJRYBTLDHVDOYOFUYRT", "DFQ", "DXJHGHYMLDUDWRWFGZRMNHPNDKXCGQJJRYBTLDHVDOYOFUYRT", "TOJRFIDS", "TOJRFIDS", "DVTBJRYSGUMPABCWJCEFUTPCQ", "TOJRFIDS", "DFQ", "DFQ", "DFQ", "DRPQYQWVSQHITYHZGK", "PGLJNCANRZUIFTJZGTFFOINMLXKPGCDQSHZVUFBDPAJOTIEF", "DFQ", "DFQ", "TOJRFIDS", "DXJHGHYMLDUDWRWFGZRMNHPNDKXCGQJJRYBTLDHVDOYOFUYRT", "TOJRFIDS", "TOJRFIDS", "PGLJNCANRZUIFTJZGTFFOINMLXKPGCDQSHZVUFBDPAJOTIEF", "TOJRFIDS", "TOJRFIDS"]) == 7)
    print(f(7, ["KHZF", "KHZF", "KHZF", "KHZF", "KHZF", "KHZF", "KHZF", "GQYQEMSYWWL", "KHZF", "GQYQEMSYWWL", "GQYQEMSYWWL", "GQYQEMSYWWL", "KHZF", "KHZF", "KHZF", "KHZF", "KHZF", "KHZF", "GQYQEMSYWWL"]) == 3)
    print(f(3, ["TGQYBCTPETXB", "TGQYBCTPETXB", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "TGQYBCTPETXB", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "TGQYBCTPETXB", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "TGQYBCTPETXB", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "ZIUIRQZRDEUHLDQRQMTZZBTG", "YDEUSXZLTAFNVICQINK", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD", "AISRSLIGANJQJFIWSLCLMTODPEFRVEEZKAJRTCALRXFEOYBBD"]) == 9)
    print(f(7, ["OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ", "OHYQDNLTEJFPFVSBLQTPDUZ"]) == 2)
    print(f(15, ["KVREIURMMGTKRDYNQNFPPBRLXZP", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ", "TUULOLANMRPGATGVAEAMMOUSFJNEEO", "LXVMSITFOKFZXIRHDNUZ", "DCNPRDMK", "TUULOLANMRPGATGVAEAMMOUSFJNEEO", "LXVMSITFOKFZXIRHDNUZ", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ", "LXVMSITFOKFZXIRHDNUZ", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ", "LXVMSITFOKFZXIRHDNUZ", "KVREIURMMGTKRDYNQNFPPBRLXZP", "KVREIURMMGTKRDYNQNFPPBRLXZP", "TUULOLANMRPGATGVAEAMMOUSFJNEEO", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ", "KVREIURMMGTKRDYNQNFPPBRLXZP", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ", "LXVMSITFOKFZXIRHDNUZ", "LXVMSITFOKFZXIRHDNUZ", "KVREIURMMGTKRDYNQNFPPBRLXZP", "LXVMSITFOKFZXIRHDNUZ"]) == 4)
    print(f(41, ["EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "GTGLSOJQRFDQDTAMHQTSTAJQXXWUOZ", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "BKTMDDEUYXJVCUUQOUYAEDRS", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "BKTMDDEUYXJVCUUQOUYAEDRS", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO", "EBFCQZUOWSLHYWYFDNQQSSWUXGSUOVXSHBBDSDCOFPYLLIO"]) == 3)
    print(f(12, ["WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "NTFMGVFYQONSIQKDGPLJPQNZPKWGBILINDSZKGKDSCPKNDIW", "ZLSYRWDEXVNEMJITBESKSKMXERER", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW", "WKPDXGOCQRWUCTJIQVAQABR", "GRYIPVSKPRIQAJQTSFJXJBOHNBIYZSRCGEW"]) == 6)
    print(f(39, ["ARYQRTRPIMVSUYGFFEHFYTMGM", "FJ", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "ARYQRTRPIMVSUYGFFEHFYTMGM", "GLABAMYFUOYWJPMMY", "GLABAMYFUOYWJPMMY", "ARYQRTRPIMVSUYGFFEHFYTMGM", "ARYQRTRPIMVSUYGFFEHFYTMGM", "FJ", "ARYQRTRPIMVSUYGFFEHFYTMGM", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "FDBTFNCMZVRRTJNVQBYLKKAZULQG", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "UBCHVHKLIEW", "QLXCBHDBKRWNKCTNLSTKRXYVRHTO", "ARYQRTRPIMVSUYGFFEHFYTMGM", "LQWQZABGNJIQEMFQNMBRSGERIXUDAUXWWI", "FJ", "TJJWIWIWTMDSYXEZAHEXBGPUWQGONVJPVCHS", "UBCHVHKLIEW", "TJJWIWIWTMDSYXEZAHEXBGPUWQGONVJPVCHS", "AYQVHTAMSGILWFJRBAC", "ARYQRTRPIMVSUYGFFEHFYTMGM", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "ARYQRTRPIMVSUYGFFEHFYTMGM", "TJJWIWIWTMDSYXEZAHEXBGPUWQGONVJPVCHS", "UBCHVHKLIEW", "UBCHVHKLIEW", "ARYQRTRPIMVSUYGFFEHFYTMGM", "ARYQRTRPIMVSUYGFFEHFYTMGM", "ARYQRTRPIMVSUYGFFEHFYTMGM", "ARYQRTRPIMVSUYGFFEHFYTMGM", "GLABAMYFUOYWJPMMY", "ARYQRTRPIMVSUYGFFEHFYTMGM", "GLABAMYFUOYWJPMMY", "UBCHVHKLIEW", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "GLABAMYFUOYWJPMMY", "UBCHVHKLIEW", "TJJWIWIWTMDSYXEZAHEXBGPUWQGONVJPVCHS", "UBCHVHKLIEW", "GLABAMYFUOYWJPMMY", "GLABAMYFUOYWJPMMY"]) == 9)
    print(f(27, ["MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX", "MGCMX"]) == 1)
    print(f(10, ["WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "ITC", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP", "WOVDHMTQLGADAGZXESBNWHQBRTTKXWJSRSSKPZXEOUXGMP"]) == 3)
    print(f(16, ["CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "WEYCYJGNSEYNWSOGJNQUOTRORACEQB", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS", "CPILIIDEYNUUQTCBKCKKWLONEAOS"]) == 2)
    print(f(4, ["TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "CGNEPQCMWHPPUYUZHQHAOZYAYZ", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "FPPZBRYSVOSBZUE", "CGNEPQCMWHPPUYUZHQHAOZYAYZ", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "EGACKTEUCTBVHRMLUBTUYKMNDWECSPRTKXBFNANMVAASKYPMY", "EGACKTEUCTBVHRMLUBTUYKMNDWECSPRTKXBFNANMVAASKYPMY", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "CGNEPQCMWHPPUYUZHQHAOZYAYZ", "FMMJLRNPDVNLFVDBHLJOPHOUMC", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "CGNEPQCMWHPPUYUZHQHAOZYAYZ", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH", "TEYWTWCXHKJTCBEDVQNBDOLSKJKMH"]) == 12)
    print(f(2, ["CANNED", "CANNED", "PRODUCE", "DAIRY", "MEAT", "BREAD", "HOUSEHOLD", "PRODUCE", "FROZEN", "PRODUCE", "DAIRY"]) == 8)
    print(f(5, ["A", "A", "A", "A", "A", "A", "A", "A"]) == 2)
    print(f(3, ["A", "A", "A", "A", "A"]) == 2)
    print(f(3, ["DAIRY", "PRODUCE", "WHEAT", "DAIRY", "PRODUCE", "WHEAT", "DAIRY", "PRODUCE", "WHEAT"]) == 3)
    print(f(3, ["A", "A", "A", "A", "B", "B", "B", "B", "C", "C", "C", "C"]) == 6)
    print(f(10, []) == 0)
    print(f(50, ["DAIRY", "DAIRY", "DAIRY"]) == 1)
    print(f(1, ["A", "A", "A", "A", "B", "B", "B"]) == 7)
    print(f(2, ["A", "A", "A", "A", "A", "B"]) == 4)
    print(f(1, ["A"]) == 1)

#### boxing: https://community.topcoder.com/stat?c=problem_statement&pm=2977&rd=5880

Problem Statement
    	
In Olympic boxing, there are five judges who press a button when they think that a particular boxer has just landed a punch. The times of the button presses are recorded, and the boxer receives credit for a punch if at least three of the judges press their buttons within 1 second of each other.
This "algorithm" needs a lot of refinement. Here is the version that you should implement. Find the maximum number of disjoint time intervals that can be chosen such that each interval is of duration 1 second or less and contains button presses from at least 3 different judges. Two intervals are disjoint if every time within one interval is strictly less than every time in the other. We give the boxer credit for one punch during each interval.

The duration of an interval is the amount of time between the instant when it starts and when it ends, and a punch may be included in an interval if its recorded time is at the start, end, or in between. So, for example, the interval from 1 to 4 has duration 3, and a punch recorded at time 1, 2, 3, or 4 is in that interval. The intervals 1 to 4 and 5 to 22 are disjoint, but the intervals 1 to 4 and 4 to 22 are not disjoint.

Create a class Boxing that contains a method maxCredit that is given five int[]s, a, b, c, d, and e, representing the times of the button presses of the five judges in milliseconds. The method returns the maximum credits that can be given to the boxer.

 
Definition
    	
Class:	Boxing
Method:	maxCredit
Parameters:	int[], int[], int[], int[], int[]
Returns:	int
Method signature:	int maxCredit(int[] a, int[] b, int[] c, int[] d, int[] e)
(be sure your method is public)
    
 
Constraints
-	Each of the five arguments will contain between 0 and 50 elements inclusive.
-	Each element of each of the arguments will be between 0 and 180,000 inclusive.
-	The elements within each of the arguments will be in strictly increasing order.
 
Examples
0)	
    	
{1,2,3,4,5,6}
{1,2,3,4,5,6,7}
{1,2,3,4,5,6}
{0,1,2}
{1,2,3,4,5,6,7,8}
Returns: 6
This match had a fast start, with 6 punches credited in the first 6 milliseconds of the match! One way to choose 6 disjoint intervals is to choose [1,1], [2,2], [3,3], [4,4], [5,5], [6,6] each of which contains button presses from judges a, b, and c. There are three button presses in the time interval from 7 through 8, but only from two different judges so no additional credit can be given.
1)	
    	
{100,200,300,1200,6000}
{}
{900,902,1200,4000,5000,6001}
{0,2000,6002}
{1,2,3,4,5,6,7,8}
Returns: 3
One way to form three intervals is [0,1000], [1001,2000], [6000,6002]
2)	
    	
{5000,6500}
{6000}
{6500}
{6000}
{0,5800,6000}
Returns: 1
Any interval that doesn't include 6000 will not have enough button presses, so we can form only one disjoint interval with credit for a punch.


In [ ]:
class Boxing(object):
    
    def max_credit(self, *presses):
        """Return the maximum credits that can be given to the boxer."""
        if all([len(p) == 0 for p in presses]): return 0
        end = max([i[-1] for i in presses if len(i) > 0])
        current = min([i[0] for i in presses if len(i) > 0])
        judge = [False for i in range(5)]
        pos = [0 for i in range(5)]
        credits = 0
        while current <= end:

            for i in range(5):
                if pos[i] >= len(presses[i]): continue
                if presses[i][pos[i]] <= current:
                    judge[i] = presses[i][pos[i]]
                    pos[i] += 1

            hits = 0
            for i in range(5):
                if judge[i] is False: continue
                if current - judge[i] <= 1000: hits += 1
                        
            if hits >= 3:
                credits += 1
                judge = [False for i in range(5)]

            current += 1

        return credits
if True:
    f = Boxing().max_credit
    print(f([1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6], [0, 1, 2], [1, 2, 3, 4, 5, 6, 7, 8]) , 6)
    print(f([100, 200, 300, 1200, 6000], [], [900, 902, 1200, 4000, 5000, 6001], [0, 2000, 6002], [1, 2, 3, 4, 5, 6, 7, 8]) , 3)
    print(f([5000, 6500], [6000], [6500], [6000], [0, 5800, 6000]) , 1)
    print(f([], [], [], [], []) , 0)
    print(f([180000], [1, 2, 3, 4, 5, 6, 179000, 179500, 179800], [5, 10, 1000, 10000], [5, 10, 1000, 10000], [6, 9000]) , 3)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]) , 50)
    print(f([29000, 29005], [30000, 35000], [29815, 180000], [29999, 31000], [3000, 30500]) , 2)
    print(f([2699, 10692, 13011, 15965, 17741, 19186, 23392, 32495, 32890, 33294, 38153, 41040, 48392, 54352, 55114, 55658, 55960, 56201, 64961, 65481, 65515, 75634, 78801, 79643, 83789, 89641, 96181, 96216, 104873, 106920, 117908, 118745, 120013, 122753, 124907, 125091, 126876, 129105, 133341, 135052, 136473, 137495, 139277, 142103, 143967, 144712, 150713, 153393, 158784, 161182], [1595, 8642, 8954, 11045, 11249, 20771, 27970, 31274, 36528, 36892, 41520, 44978, 54877, 57368, 65224, 69869, 73501, 75231, 76272, 76423, 78302, 83535, 87222, 88010, 88361, 88945, 92000, 93747, 97352, 98511, 98960, 102550, 102990, 106636, 115539, 120722, 122265, 122430, 126235, 134111, 143167, 149823, 151733, 153758, 155472, 156083, 158202, 166624, 172914, 175590], [3780, 9366, 13367, 23559, 24191, 24831, 25786, 26391, 26794, 26852, 31798, 37596, 38421, 48740, 49811, 56623, 62882, 77821, 82774, 85865, 87313, 91804, 92493, 92818, 95129, 95923, 104007, 110764, 113758, 114294, 115821, 122485, 126932, 133926, 136982, 138418, 138999, 145410, 149647, 150652, 151485, 153397, 156791, 156801, 159556, 160288, 161454, 161636, 166235, 178516], [5054, 10277, 11234, 13012, 17143, 20271, 22171, 23615, 28055, 35313, 48476, 53922, 59118, 61729, 64751, 65548, 68138, 74352, 85808, 87073, 90894, 90962, 95082, 101435, 102635, 103405, 104385, 104656, 104760, 110629, 130051, 134159, 135014, 136227, 140978, 141735, 143163, 149823, 149917, 154906, 158519, 160314, 162052, 168874, 170552, 172221, 175517, 176894, 177644, 177679], [2388, 3005, 5419, 7320, 8811, 8870, 10630, 21793, 22133, 23869, 35905, 37104, 37474, 37490, 37978, 39589, 43939, 44835, 49094, 50658, 57209, 59529, 70009, 74916, 81029, 87448, 89111, 89635, 91942, 97418, 100352, 104035, 107523, 113736, 114910, 123639, 124246, 126071, 135449, 136225, 141088, 142377, 155102, 156125, 163533, 163921, 164128, 164941, 167598, 168811]) , 32)
    print(f([2699, 10692, 13011, 15965, 17741, 19186, 23392, 32495, 32890, 33294, 38153, 41040, 48392, 54352, 55114, 55658, 55960, 56201, 64961, 65481, 65515, 75634, 78801, 79643, 83789, 89641, 96181, 96216, 104873, 106920, 117908, 118745, 120013, 122753, 124907, 125091, 126876, 129105, 133341, 135052, 136473, 137495, 139277, 142103, 143967, 144712, 150713, 153393, 158784, 161182], [1595, 8642, 8954, 11045, 11249, 20771, 27970, 31274, 36528, 36892, 41520, 44978, 54877, 57368, 65224, 69869, 73501, 75231, 76272, 76423, 78302, 83535, 87222, 88010, 88361, 88945, 92000, 93747, 97352, 98511, 98960, 102550, 102990, 106636, 115539, 120722, 122265, 122430, 126235, 134111, 143167, 149823, 151733, 153758, 155472, 156083, 158202, 166624, 172914, 175590], [3780, 9366, 13367, 23559, 24191, 24831, 25786, 26391, 26794, 26852, 31798, 37596, 38421, 48740, 49811, 56623, 62882, 77821, 82774, 85865, 87313, 91804, 92493, 92818, 95129, 95923, 104007, 110764, 113758, 114294, 115821, 122485, 126932, 133926, 136982, 138418, 138999, 145410, 149647, 150652, 151485, 153397, 156791, 156801, 159556, 160288, 161454, 161636, 166235, 178516], [5054, 10277, 11234, 13012, 17143, 20271, 22171, 23615, 28055, 35313, 48476, 53922, 59118, 61729, 64751, 65548, 68138, 74352, 85808, 87073, 90894, 90962, 95082, 101435, 102635, 103405, 104385, 104656, 104760, 110629, 130051, 134159, 135014, 136227, 140978, 141735, 143163, 149823, 149917, 154906, 158519, 160314, 162052, 168874, 170552, 172221, 175517, 176894, 177644, 177679], [2388, 3005, 5419, 7320, 8811, 8870, 10630, 21793, 22133, 23869, 35905, 37104, 37474, 37490, 37978, 39589, 43939, 44835, 49094, 50658, 57209, 59529, 70009, 74916, 81029, 87448, 89111, 89635, 91942, 97418, 100352, 104035, 107523, 113736, 114910, 123639, 124246, 126071, 135449, 136225, 141088, 142377, 155102, 156125, 163533, 163921, 164128, 164941, 167598, 168811]) , 32)
    print(f([1520, 1846, 7333, 8112, 20247, 27916, 39498, 41761, 46386, 47793, 56674, 57549, 61237, 61738, 61807, 65566, 69075, 71904, 74585, 80694, 84588, 86505, 88568, 89512, 91125, 99605, 104067, 105132, 109113, 111132, 112180, 112192, 124246, 125499, 132535, 136731, 137175, 143796, 144887, 145500, 146398, 150481, 154549, 154625, 159865, 163845, 164236, 175503, 175900, 176126], [2573, 2723, 5513, 5935, 8194, 12985, 23445, 29571, 31423, 31487, 40780, 51677, 56402, 59271, 60194, 60478, 66049, 70507, 73577, 74547, 75331, 77356, 80258, 81166, 83826, 87223, 88214, 89516, 97263, 102691, 111577, 117020, 118327, 119370, 119660, 125980, 130251, 139707, 142443, 148094, 157423, 166045, 167681, 167759, 167818, 168658, 169568, 170767, 175423, 175720], [959, 4395, 7172, 8529, 9331, 14385, 23960, 25489, 31966, 35544, 36902, 43935, 45973, 50360, 51612, 60847, 62728, 64504, 69043, 69729, 70709, 70853, 73450, 73670, 78050, 78098, 84248, 85529, 94807, 95321, 96076, 99179, 100936, 104392, 106920, 107507, 110199, 113120, 129867, 131702, 140539, 142123, 142408, 145029, 146598, 158440, 162276, 163556, 165301, 176549], [6720, 7433, 7788, 11690, 17426, 20105, 23451, 26032, 36674, 38158, 39144, 41710, 42534, 44682, 48068, 52603, 58238, 59746, 60568, 68524, 80900, 83011, 87298, 88995, 91297, 94327, 95120, 96803, 106949, 108191, 108590, 116531, 120351, 122883, 124316, 129214, 129599, 135157, 140186, 142868, 143710, 143892, 146237, 150549, 151505, 153595, 154854, 155600, 156528, 170935], [2234, 15006, 18258, 19351, 20627, 25780, 27578, 28999, 29014, 30624, 35196, 35837, 36317, 41041, 51208, 58629, 62003, 62009, 65021, 68516, 68598, 83627, 92390, 97218, 103272, 110723, 119850, 120544, 122313, 124948, 125626, 127582, 128559, 132029, 132666, 136231, 137211, 140195, 143206, 143535, 146053, 148029, 160479, 161179, 163014, 165505, 169279, 171856, 176947, 179620]) , 31)
    print(f([0, 4244, 8627, 9903, 31479, 40959, 44573, 45119, 45998, 46078, 47150, 47833, 49210, 50070, 56274, 56982, 58247, 59485, 77334, 81949, 91133, 98080, 102696, 105045, 105079, 111421, 113613, 118602, 124653, 134545, 136564, 137813, 138060, 141547, 151108, 151348, 156210, 166236, 174986, 177342, 177710], [0, 2430, 7853, 13088, 21832, 24275, 30409, 36233, 49393, 51785, 54662, 55725, 56628, 60462, 63882, 66856, 67212, 74681, 74957, 87425, 89567, 97859, 97958, 105136, 106213, 107656, 108554, 111029, 113202, 119091, 119684, 124691, 126694, 132821, 144116, 146492, 147893, 151953, 155306, 164568, 164748], [0, 6842, 13439, 13936, 16259, 24061, 24247, 24852, 35161, 40211, 41246, 43182, 46628, 46750, 56598, 60779, 62036, 62447, 65024, 70049, 74411, 78506, 79298, 80462, 80797, 83909, 89970, 99032, 103158, 116140, 118861, 121203, 121884, 133335, 145983, 150611, 154531, 160319, 166847, 176077, 176980], [0, 4278, 4521, 14703, 15142, 15530, 19172, 19265, 21880, 25585, 26615, 28314, 31481, 31977, 33593, 37475, 39154, 49434, 60148, 63581, 68149, 68737, 73341, 73367, 84830, 84848, 89726, 91857, 94920, 109318, 110430, 112129, 115111, 119117, 122739, 126402, 147254, 149053, 151834, 168627, 174675], [0, 2148, 4367, 10640, 18035, 18487, 21629, 26109, 31283, 40380, 40730, 52166, 59667, 60313, 64014, 66510, 68348, 68609, 71314, 75624, 88972, 96476, 98137, 98788, 106141, 109176, 114246, 117649, 126005, 127893, 128583, 131962, 132113, 134437, 140498, 144765, 150567, 151156, 151584, 161092, 179359]) , 18)
    print(f([58247, 59485, 77334, 81949, 91133, 98080, 102696, 105045, 105079, 111421, 113613, 118602, 124653, 134545, 136564, 137813, 138060, 141547, 151108, 151348, 156210, 166236, 174986, 177342, 177710], [0, 4278, 4521, 14703, 15142, 15530, 19172, 19265, 21880, 25585, 26615, 28314, 31481, 31977, 33593, 37475, 39154, 49434, 60148, 63581, 68149, 68737, 73341, 73367, 84830, 84848, 89726, 91857, 94920, 109318, 110430, 112129, 115111, 119117, 122739, 126402, 147254, 149053, 151834, 168627, 174675], [0, 6842, 13439, 13936, 16259, 24061, 24247, 24852, 35161, 40211, 41246, 43182, 46628, 46750, 56598, 60779, 62036, 62447, 65024, 70049, 74411, 78506, 79298, 80462, 80797, 83909, 89970, 99032, 103158, 116140, 118861, 121203, 121884, 133335, 145983, 150611, 154531, 160319, 166847, 176077, 176980], [0, 2430, 7853, 13088, 21832, 24275, 30409, 36233, 49393, 51785, 54662, 55725, 56628, 60462, 63882, 66856, 67212, 74681, 74957, 87425, 89567, 97859, 97958, 105136, 106213, 107656, 108554, 111029, 113202, 119091, 119684, 124691, 126694, 132821, 144116, 146492, 147893, 151953, 155306, 164568, 164748], [68348, 68609, 71314, 75624, 88972, 96476, 98137, 98788, 106141, 109176, 114246, 117649, 126005, 127893, 128583, 131962, 132113, 134437, 140498, 144765, 150567, 151156, 151584, 161092, 179359]) , 10)
    print(f([0, 981, 3613, 26000, 26176, 37010, 38435, 39040, 40685, 42425, 46970, 59261, 67421, 67920, 82003, 85701, 87573, 91240, 93491, 107579, 112368, 113824, 116747, 121687, 125755, 127324, 132032, 140824, 143746, 144757, 146730, 155636, 156018, 157475, 162649, 163964, 165479, 170919, 177957, 179764, 179965], [0, 1217, 4153, 5425, 17053, 19129, 43622, 46089, 48103, 50301, 53492, 56102, 68174, 84843, 85637, 87136, 87185, 88702, 95409, 95797, 101068, 104223, 106991, 108238, 108955, 117189, 119639, 121061, 124599, 125297, 126114, 134470, 138386, 145528, 147670, 152412, 162297, 164351, 165708, 168293, 172580], [0, 11051, 13830, 19854, 30421, 36371, 36535, 36897, 37306, 38723, 38925, 41350, 46265, 48062, 58292, 64193, 68744, 70567, 72253, 76984, 82827, 95247, 101210, 101222, 102896, 108648, 109826, 110615, 119751, 127846, 131990, 138495, 141718, 145700, 146602, 146685, 147124, 150007, 172043, 177263, 179134], [0, 1348, 7290, 9987, 12606, 13130, 15817, 22158, 30099, 30741, 37397, 38311, 59532, 72507, 82249, 84112, 84427, 85130, 92552, 95084, 95409, 97456, 101816, 104606, 106746, 111065, 112492, 113485, 116757, 117245, 118106, 118496, 124949, 135786, 136212, 151069, 157902, 161266, 167618, 175037, 179129], [0, 6109, 7876, 8417, 9118, 15800, 15935, 21154, 26751, 30028, 34785, 35328, 38739, 39685, 42901, 49852, 56923, 63140, 67867, 83832, 92868, 93990, 95129, 99411, 100246, 106167, 106358, 109043, 109910, 118188, 118844, 123839, 128368, 137816, 154556, 154889, 158067, 168585, 173068, 175219, 175501]) , 24)
    print(f([5990, 6985, 17723, 18784, 20958, 26433, 27925, 29068, 34574, 39404, 39961, 52385, 61619, 67571, 69079, 71683, 76016, 77748, 79929, 79942, 83772, 86008, 90871, 92029, 92895, 95040, 99406, 106093, 109832, 111714, 114326, 115461, 118375, 123873, 124070, 127204, 130167, 134872, 135251, 139682, 141782, 142837, 151104, 158085, 160856, 163874, 170734, 173038, 173124, 179615], [12605, 13835, 16403, 18397, 18568, 19172, 19645, 23229, 25607, 31723, 32089, 32436, 33012, 42898, 44695, 55663, 60634, 64704, 65858, 66595, 67057, 67500, 69060, 69671, 70288, 75940, 79076, 79608, 94714, 114741, 118019, 123599, 130401, 131280, 136012, 138743, 141284, 142172, 148411, 149675, 150070, 152978, 153206, 157338, 157356, 159875, 162145, 163073, 173452, 174169], [6314, 11673, 18740, 32338, 36256, 38728, 39063, 42030, 45585, 46830, 52297, 61046, 61085, 66019, 72246, 73199, 73617, 85000, 91849, 97412, 97723, 100716, 100927, 102447, 103616, 116332, 120475, 123782, 139454, 143663, 144075, 150883, 151120, 152183, 153475, 153661, 154452, 157076, 159166, 161365, 165992, 171242, 171868, 172309, 172784, 175837, 175902, 178248, 178859], [7540, 13808, 17898, 19074, 19394, 27970, 40304, 48853, 49506, 50631, 52266, 52473, 54344, 55260, 59663, 60235, 62136, 63861, 67360, 67520, 67660, 72177, 74169, 77411, 83827, 88394, 89716, 93588, 98557, 104525, 106318, 110903, 116276, 118109, 119318, 119392, 122442, 122623, 125319, 129273, 137152, 137260, 141872, 147669, 148620, 151715, 154407, 163392, 163960, 168826], [1307, 3302, 3646, 6741, 10217, 24069, 25045, 25617, 36166, 38588, 40843, 49777, 51433, 51524, 63389, 65744, 69457, 70002, 71456, 73600, 74296, 74620, 75943, 78866, 82864, 83867, 85497, 85517, 90879, 94100, 95914, 105759, 108018, 110401, 112565, 114407, 115743, 116012, 119734, 126263, 135184, 137207, 144413, 145286, 146748, 147420, 150406, 151795, 154445, 170406]) , 32)
    print(f([0, 33, 61, 64, 74, 78, 83, 87, 98, 111, 115, 122, 126, 133, 135, 141, 155, 163, 170], [20, 39, 46, 53, 58, 66, 74, 80, 95, 110, 123, 124, 125, 127, 133, 134, 138, 144, 145, 168], [3, 4, 9, 21, 29, 54, 76, 84, 86, 88, 94, 102, 112, 115, 127, 134, 140, 152, 169, 175], [8, 9, 20, 21, 22, 23, 34, 45, 57, 59, 87, 90, 97, 117, 120, 139, 149, 152, 153, 171], [12, 15, 20, 28, 34, 43, 45, 52, 64, 71, 73, 76, 88, 91, 127, 138, 167]) , 26)
    print(f([1, 1008], [1, 3], [1, 3], [], []) , 1)
    print(f([0], [0], [1000], [], []) , 1)
    print(f([5], [5555], [9999, 10000], [], []) , 0)
    print(f([0], [2000], [4000], [6000], [8000]) , 0)
    print(f([1], [1], [6000], [], []) , 0)
    print(f([10000], [20000], [30000], [40000], [50000]) , 0)
    print(f([0, 10000, 20000, 30000, 40000], [0, 10000, 20000, 30000, 40000], [1000, 11001, 21000, 31001, 41000], [], []) , 3)
    print(f([0], [0], [], [20000], [30000]) , 0)
    print(f([1000], [3000], [5000], [7000], [9000]) , 0)

#### listeningin: https://community.topcoder.com/stat?c=problem_statement&pm=4670

solution: http://www.topcoder.com/tc?module=Static&d1=match_editorials&d2=srm254

Problem Statement
    	You are creating an online multiplayer cooperative game. Players on a team may chat with each other during the game, and you intend to take advantage of this when building the AI to handle opponents. Part of the AI includes determining whether a given phrase is part of a player's chat. Of course, many variations of a given phrase are possible, and you want to detect as many as you can. Shorthand is the most common example: instead of typing 'capture', a player might type 'cptr', or 'port to me' instead of 'teleport to me'. You will be provided with a String typed typed by a player and a phrase that you wish to check against. Return the characters removed from phrase to obtain typed in the order they appear in phrase or "UNMATCHED" if there is no way to obtain typed from phrase by simply removing characters. The constraints ensure that the return is unique (there is only one option for which String is returned).
 
Definition
    	
Class:	ListeningIn
Method:	probableMatch
Parameters:	String, String
Returns:	String
Method signature:	String probableMatch(String typed, String phrase)
(be sure your method is public)
    
 
Constraints
-	typed and phrase will contain only lowercase letters ('a'-'z') and spaces
-	typed and phrase will be between 1 and 50 characters long, inclusive.
-	All valid groups of characters that could be removed to turn phrase into typed will give the same output.
 
Examples

0)	
    	
"cptr"
"capture"
Returns: "aue"
The example given in the problem statement.

1)	

"port to me"
"teleport to me"
Returns: "tele"
The other example from the statement.

2)	
    	
"back  to base"
"back to base"
Returns: "UNMATCHED"
An extra space has been added; we do not account for additions, only deletions.


In [ ]:
class ListeningIn(object):
    
    def probable_match(self, typed, phrase):
        """
        Return the characters removed from phrase to obtain typed.
        
        Characters are returned in the order they appear in phrase.
        "UNMATCHED" is returned if there is no way to obtain typed from phrase
        by simply removing characters.
        """
        p = 0
        ret = ""
        for i in range(len(phrase)):
            if p < len(typed) and phrase[i] == typed[p]:
                p+=1
            else:
                ret += phrase[i]

        if p == len(typed):
            return ret
        return "UNMATCHED"
if True:
    f = ListeningIn().probable_match
    print(f("cptr", "capture"))

# Sorting and Search

#### buildteam: https://community.topcoder.com/stat?c=problem_solution&cr=15179224&rd=10731&pm=7627

In [5]:
class BuildTeam(object):

    def maximalStrength(self, skills, teams):
        """Return skill of the strongest team."""
        skills.sort()
        player = len(skills) - 1
        tskills = [0 for i in range(teams)]
        even = list(range(teams - 1, -1, -1))
        odd = list(range(teams))
        order = 1
        while player >= 0:
            if order % 2 == 1:
                order = 2
                for i in odd:
                    tskills[i] += skills[player]
                    player -= 1
            else:
                order = 1
                for i in even:
                    tskills[i] += skills[player]
                    player -= 1

        return max(tskills)
if True:
    f = BuildTeam().maximalStrength
    print(f([10, 9, 7, 3, 3, 2], 3) == 12)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 4) == 21)
    print(f([10, 10, 10, 9, 8, 8, 3, 1], 2) == 31)
    print(f([10, 8, 10, 1, 10, 9, 3, 8], 2) == 31)
    print(f([98, 19, 11, 11, 11, 11, 23, 55, 1, 4, 3, 4, 6, 8], 7) == 99)
    print(f([2, 3, 4, 6, 9, 12, 14, 15, 15, 15, 17, 17, 19, 22, 26, 27, 32, 35, 36, 36, 38, 41, 45, 45, 50, 51, 52, 52, 60, 64, 69, 70, 70, 72, 80, 81, 81, 82, 83, 84, 85, 88, 91, 97], 44) == 97)
    print(f([1, 7, 11, 15, 18, 20, 33, 34, 34, 36, 39, 41, 45, 46, 54, 55, 64, 70, 78, 80, 81, 88, 96, 97, 98, 99], 26) == 99)
    print(f([5, 10, 12, 13, 17, 19, 23, 25, 26, 28, 31, 31, 34, 39, 42, 45, 46, 49, 52, 52, 55, 58, 62, 65, 69, 69, 75, 81, 85, 87, 92, 98], 32) == 98)
    print(f([2, 3, 3, 6, 7, 8, 9, 11, 19, 20, 20, 25, 25, 25, 26, 26, 32, 40, 41, 41, 42, 43, 45, 45, 45, 46, 46, 57, 59, 62, 63, 64, 65, 68, 69, 70, 75, 76, 78, 79, 81, 83, 88, 90, 91, 91, 91, 97, 98], 49) == 98)
    print(f([1, 3, 4, 5, 5, 5, 6, 7, 9, 13, 15, 21, 22, 24, 26, 26, 27, 28, 28, 30, 44, 53, 59, 63, 65, 70, 70, 70, 70, 73, 74, 75, 77, 82, 88, 91, 96, 99], 38) == 99)
    print(f([3, 4, 7, 15, 16, 16, 19, 20, 20, 30, 31, 33, 38, 40, 40, 41, 45, 47, 47, 48, 48, 49, 52, 52, 52, 53, 55, 68, 72, 74, 75, 75, 77, 79, 81, 83, 84, 87, 91, 91, 91, 98], 42) == 98)
    print(f([6, 7, 7, 9, 11, 12, 19, 20, 25, 41, 61, 67, 68, 69, 70, 73, 74, 74, 77, 81, 84, 85, 89, 90, 90, 93, 96, 96, 97], 29) == 97)
    print(f([5, 7, 7, 11, 15, 16, 18, 22, 24, 25, 40, 52, 55, 56, 57, 65, 65, 67, 92, 97, 99], 21) == 99)
    print(f([1, 6, 8, 9, 10, 10, 13, 17, 25, 28, 29, 30, 31, 41, 42, 45, 45, 46, 47, 62, 64, 64, 65, 68, 70, 70, 71, 74, 75, 75, 79, 80, 86, 87, 91, 91, 95], 37) == 95)
    print(f([3, 5, 5, 14, 16, 18, 20, 20, 23, 32, 33, 34, 40, 45, 48, 49, 51, 54, 58, 61, 64, 65, 65, 67, 68, 68, 76, 79, 79, 82, 83, 84, 85, 99], 34) == 99)
    print(f([2, 4, 6, 10, 10, 11, 22, 23, 27, 28, 31, 33, 45, 49, 51, 52, 53, 54, 57, 71, 72, 73, 80, 80, 94, 96], 26) == 96)
    print(f([3, 6, 6, 14, 15, 16, 24, 44, 45, 48, 56, 57, 59, 77, 78, 83, 89, 91, 100, 100], 20) == 100)
    print(f([2, 4, 4, 10, 16, 16, 21, 22, 23, 25, 27, 29, 29, 30, 31, 36, 36, 41, 42, 43, 46, 47, 49, 52, 55, 55, 55, 57, 59, 62, 71, 73, 78, 79, 79, 80, 80, 84, 85, 87, 89, 92, 95, 95], 22) == 107)
    print(f([1, 2, 2, 6, 9, 14, 42, 48, 49, 54, 55, 70, 72, 79, 87], 15) == 87)
    print(f([2, 8, 9, 10, 11, 15, 19, 19, 23, 24, 29, 31, 34, 36, 36, 41, 44, 48, 52, 57, 58, 59, 60, 61, 66, 70, 76, 78, 79, 80, 80, 81, 83, 86, 86, 87, 88, 93, 94, 95], 40) == 95)
    print(f([4, 6, 7, 10, 10, 12, 17, 18, 22, 23, 25, 25, 26, 33, 35, 39, 40, 44, 46, 48, 50, 55, 55, 56, 59, 59, 60, 61, 64, 78, 80, 82, 84, 92, 99], 35) == 99)
    print(f([19, 20, 22, 23, 25, 25, 29, 38, 43, 48, 59, 63, 66, 71, 73, 73, 75, 75, 76, 82, 87, 94, 94, 99], 24) == 99)
    print(f([2, 4, 4, 4, 5, 9, 12, 25, 26, 29, 32, 35, 35, 37, 40, 43, 44, 48, 53, 60, 60, 61, 62, 64, 68, 73, 74, 79, 79, 84, 84, 86, 89, 90, 94, 95, 96, 97], 38) == 97)
    print(f([1, 8, 9, 10, 15, 17, 17, 19, 21, 21, 23, 26, 27, 29, 30, 32, 35, 43, 44, 44, 49, 55, 58, 59, 63, 63, 63, 64, 65, 66, 73, 80, 83, 93, 95, 95, 97, 97, 100], 39) == 100)
    print(f([1, 3, 5, 25, 33, 42, 47, 48, 55, 62, 81, 82, 84, 91, 92, 93, 95], 17) == 95)
    print(f([1, 2, 4, 11, 16, 17, 18, 19, 23, 28, 32, 33, 35, 36, 38, 38, 39, 40, 41, 44, 44, 45, 46, 50, 52, 52, 53, 54, 57, 60, 63, 69, 69, 70, 72, 72, 74, 83, 86, 86, 86, 88, 88, 90, 95], 45) == 95)
    print(f([4, 4, 5, 9, 9, 12, 17, 17, 18, 22, 30, 32, 37, 38, 40, 46, 50, 55, 55, 56, 57, 69, 76, 82, 82, 90, 93, 96, 96, 99, 99, 99], 16) == 107)
    print(f([1, 12, 19, 25, 27, 30, 31, 42, 43, 48, 49, 64, 71, 99], 14) == 99)
    print(f([5, 10, 12, 12, 14, 16, 16, 16, 19, 19, 27, 35, 35, 41, 43, 46, 48, 50, 53, 54, 55, 61, 67, 70, 71, 73, 74, 74, 74, 80, 83, 87, 88, 91, 92, 93, 94, 97, 99, 100], 40) == 100)
    print(f([1, 2, 3, 3, 6, 7, 11, 12, 14, 15, 15, 22, 25, 27, 31, 31, 33, 36, 36, 36, 38, 40, 44, 46, 47, 48, 49, 52, 56, 57, 62, 62, 63, 63, 64, 66, 67, 67, 71, 72, 76, 79, 88, 94, 95, 97, 98, 100], 16) == 165)
    print(f([4, 11, 19, 38, 41, 42, 46, 47, 52, 54, 54, 58, 65, 69, 70, 70, 73, 74, 74, 77, 83, 87, 90, 94, 97, 98], 2) == 799)
    print(f([1, 4, 4, 7, 7, 8, 11, 13, 17, 19, 22, 22, 23, 25, 26, 28, 29, 33, 35, 35, 38, 40, 44, 47, 51, 53, 53, 61, 62, 65, 66, 72, 73, 75, 81, 81, 82, 85, 86, 95, 96, 99], 3) == 628)
    print(f([2, 5, 7, 14, 16, 16, 16, 25, 28, 33, 39, 41, 45, 46, 63, 64, 66, 67, 69, 77, 81, 82, 82, 85, 87, 88, 91, 93, 93, 94], 3) == 556)
    print(f([15, 21, 22, 24, 37, 43, 44, 51, 63, 70, 77, 77, 87, 90, 91, 93, 95, 98, 98, 100], 4) == 330)
    print(f([6, 6, 7, 7, 9, 11, 14, 17, 20, 34, 34, 37, 40, 42, 43, 47, 48, 49, 56, 56, 63, 65, 75, 77, 78, 79, 81, 84, 85, 88, 92, 93, 93, 93, 94, 95, 98, 98, 98, 100], 4) == 583)
    print(f([1, 4, 9, 21, 36, 45, 45, 68, 70, 87, 87, 91, 92, 95], 2) == 379)
    print(f([5, 11, 12, 17, 18, 22, 22, 24, 25, 27, 27, 28, 28, 29, 33, 38, 39, 40, 42, 45, 47, 48, 49, 65, 66, 67, 68, 71, 73, 73, 78, 78, 80, 82, 83, 86, 87, 88, 90, 91, 92, 93, 97, 97, 99], 5) == 512)
    print(f([6, 9, 10, 37, 40, 42, 43, 49, 66, 74, 77, 90, 90, 90, 97], 5) == 179)
    print(f([3, 34, 44, 53, 75, 80, 81, 88, 90], 3) == 197)
    print(f([4, 6, 9, 12, 19, 29, 30, 30, 30, 32, 32, 35, 42, 42, 48, 50, 51, 53, 55, 66, 68, 74, 76, 77, 77, 78, 79, 79, 86, 87, 94, 98], 4) == 417)
    print(f([10, 1, 3, 10], 2) == 13)
    print(f([4, 1, 4, 6, 4, 5], 3) == 9)
    print(f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 50) == 1)
    print(f([98, 19, 11, 11, 11, 11, 23, 55, 1, 4, 3, 4, 6, 8], 7) == 99)
    print(f([3, 1, 1, 3], 2) == 4)
    print(f([1, 2, 3, 4, 5, 6, 7, 8], 2) == 18)
    print(f([1, 10, 30, 20], 2) == 31)
    print(f([10, 8, 10, 1, 10, 9, 3, 8], 2) == 31)
    print(f([32, 1, 2, 4, 8, 16], 2) == 38)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 3) == 41)
    print(f([10, 10, 10, 9, 8, 8, 3, 1], 2) == 31)
    print(f([98, 19, 11, 11, 11, 11, 23, 55, 1, 4, 3, 4, 6, 8, 54, 23, 43, 90, 19, 43, 34, 13, 34, 15, 41, 22, 34, 11, 80, 80, 70, 67, 34, 37, 56], 7) == 188)
    print(f([1, 2, 2, 1], 2) == 3)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 3) == 26)
    print(f([1, 2, 3, 4, 5, 6, 7, 7, 65, 5, 4, 23, 2, 3, 2, 3, 2, 1, 2, 3, 2, 55], 11) == 66)
    print(f([1, 100, 100, 1, 1, 100], 2) == 201)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


#### wordcompositiongame: https://community.topcoder.com/stat?c=problem_statement&pm=4483&rd=7228&rm=&cr=260828

In [ ]:
class WordCompositionGame(object):

    def score(self, a, b, c):
        players = [a, b, c]
        scores = [len(a) * 3, len(b) * 3, len(c) * 3]
        keys = [{i: i for i in a}, {i: i for i in b}, {i: i for i in c}]
        for i in range(3):
            for j in range(3):
                if i == j: continue
                for key in keys[j]:
                    try:
                        keys[i][key]
                        scores[i] -= 1
                    except KeyError:
                        pass

        return "%s/%s/%s" % tuple(scores)
if True:
    f = WordCompositionGame().score
    print(f(["cat", "dog", "pig", "mouse"], ["cat", "pig"], ["dog", "cat"]) == "8/3/3")
    print(f(["mouse"], ["cat", "pig"], ["dog", "cat"]) == "3/5/5")
    print(f(["dog", "mouse"], ["dog", "pig"], ["dog", "cat"]) == "4/4/4")
    print(f(["bcdbb", "aaccd", "dacbc", "bcbda", "cdedc", "bbaaa", "aecae"], ["bcdbb", "ddacb", "aaccd", "adcab", "edbee", "aecae", "bcbda"], ["dcaab", "aadbe", "bbaaa", "ebeec", "eaecb", "bcbba", "aecae", "adcab", "bcbda"]) == "14/14/21")
    print(f(["baacd", "aaebe", "dbacc", "aaeca", "aacdd", "eacdb", "eddce", "acabd"], ["aaebe", "eaeac", "adbda", "bcedc", "eddce", "abcac", "aaabc", "deaab", "acabc", "dccce"], ["dbdad", "deaab", "aacdd", "aacce", "acabc", "eacdb", "bedba", "aaebe", "abcac", "ccbec"]) == "19/24/23")
    print(f(["ccaea", "ecece", "badea", "ddbae", "bccdd", "eedae", "cccee"], ["aeaac", "ecdeb", "eedae", "ddbae", "adadb", "eceda", "cbaba", "badea", "bbcde"], ["eacbe", "adadb", "aebea", "ecdeb", "eedae", "eceda", "ccaea", "decea", "badea", "bbcab"]) == "15/19/22")
    print(f(["beccd", "decae", "eedcc", "ccebe", "bcdcd", "ebeeb", "deaeb", "ddbdd"], ["eceea", "cccdc", "ccebe", "ecaec", "ddbdd", "eaacc", "ecddb", "accdc"], ["dccba", "beabe", "ecaec", "badbe", "daeaa", "eaacc", "adebc", "cccdc", "dddce", "badbd"]) == "22/19/27")
    print(f(["eaaec", "dcabb", "eaecd", "aedee", "adbbd", "adbde", "acded", "bbbcb", "acbde"], ["acded", "bbaaa", "caaab", "ddbca", "ddbea", "bcdda", "adbde", "dadbc", "eecaa", "caeee"], ["ddbcd", "adbbd", "bbbcb", "acded", "eadda", "ddbca", "abbde", "caeee", "baeab", "eaaec"]) == "21/25/23")
    print(f(["ebedb", "eeadc", "eadee", "beaac", "abdcd", "dbcda", "cadea"], ["edebc", "beaac", "aaecd", "cadea", "eeadc", "edddd", "abcab", "daced", "cbabb"], ["eeadc", "edebc", "beaac", "ecebe", "dcead", "abbdc", "aaecd", "cadea", "aebcc", "caeab"]) == "15/19/22")
    print(f(["aaceb", "aecad", "beebe", "cebaa", "ededd", "decea", "babbc"], ["babbc", "aecad", "aaeba", "ededd", "abcea", "ddcdb", "bdadc", "cebaa"], ["deaae", "aaebe", "abcea", "dcebb", "adaab", "bbbbd", "bbded", "ddcdb", "aadae"]) == "17/18/25")
    print(f(["eadac", "bddee", "aeabb", "dedcc", "dbabe", "aeccc", "dcaae", "ebaad", "ccada"], ["cbacb", "dcaae", "ebbac", "aacdd", "bddee", "ccedd", "dceed", "eadeb", "aeabb"], ["eeaee", "eecbd", "dedcc", "cabcb", "dcaae", "aacdd", "aeabb", "eadcc", "dbdaa"]) == "21/21/21")
    print(f(["cadbb", "bdaad", "ebadb", "dcaae", "ebecc", "aadbc", "cbead", "bbdbc"], ["dcebc", "bdedb", "bdeed", "ebccb", "ebecc", "bdbbb", "beecb", "dcaae", "bbcca"], ["ebadb", "bbdbc", "aadbc", "dcaec", "bdedb", "eceeb", "bdeed", "ecdbd"]) == "19/23/19")
    print(f(["bdaed", "ecaae", "ccdeb", "aecac", "edccd", "ababc", "dcdbe"], ["aaadd", "dbcac", "cbcad", "ecdbc", "cbaed", "dbaec", "ecaae", "eeead", "dcdbe"], ["edcea", "dacaa", "bddbe", "cdcbc", "dbcac", "deced", "abbae", "edecc"]) == "19/24/23")
    print(f(["iebccgbhfd", "kkecekehbj", "iahkbjjfkh", "gcjhbagjib"], ["dbkdfedicd", "kkecekehbj", "bbhbbgdgfi", "abikddbdfg", "jfiakchhac", "iebccgbhfd"], ["kkecekehbj", "jfiakchhac", "iahkbjjfkh", "bbhbbgdgfi", "iebccgbhfd", "abikddbdfg"]) == "7/11/10")
    print(f(["dhbjhedhkf", "bkbjjfdcci", "hkbjciighf", "eaegeghjhe", "ibkcffafij", "dcjkfbejcg", "hafhjagiie", "agejckjefd"], ["hkbjciighf", "bkbjjfdcci", "dhbjhedhkf", "kbebghfhbg", "agejckjefd", "eaegeghjhe", "dcjkfbejcg"], ["bkbjjfdcci", "ibkcffafij", "hafhjagiie", "eceicafhbd", "dcjkfbejcg", "eaegeghjhe", "kbebghfhbg", "agejckjefd"]) == "12/10/13")
    print(f(["bcghfbfbek", "cbikebigdj", "ibbecgibgk", "iigegjeiff", "ddgaegakdh", "dfbfdkchha", "daifeegbii"], ["ejdkfbdakh", "iigegjeiff", "hihceedcfg", "daifeegbii", "cbikebigdj", "ddgaegakdh", "ibbecgibgk", "eahkjjjefd", "dfbfdkchha"], ["bcghfbfbek", "ejdkfbdakh", "daifeegbii", "dfbfdkchha", "cbikebigdj", "ddgaegakdh", "eahkjjjefd"]) == "10/15/10")
    print(f(["abibikjabk", "fiaicekhdg", "aegcjfhiah", "ikgahddcdh", "fchjaigaki", "gebgfecfib"], ["abibikjabk", "ggcbbkcghb", "ikgahddcdh", "dciiijjbbf", "fchjaigaki", "bcikkbiead", "fiaicekhdg"], ["keffkgbaab", "ggcbbkcghb", "bcikkbiead", "abibikjabk", "fiaicekhdg", "fchjaigaki"]) == "11/12/10")
    print(f(["gidbccfifa", "djfdjfikce", "eagidabcbj", "kjciikbejb", "bcibeckgjh"], ["dghggchkfa", "gidbccfifa", "hcbcaifkcd", "kjciikbejb", "eagidabcbj"], ["hcbcaifkcd", "bcibeckgjh", "kjciikbejb", "ggcakedbhf", "dghggchkfa", "iegebdchhj"]) == "10/9/13")
    print(f(["kcifefaeeg", "ahagffhafb", "ifkbfgckgi", "faeackabad", "fhbhfjidak", "keiaikbgef", "abiffekcfd"], ["ahagffhafb", "keiaikbgef", "abgbackihc", "acjfdcebfe", "abiffekcfd", "faeackabad", "fhbhfjidak", "dehkjbefkb"], ["kcifefaeeg", "fhbhfjidak", "abgbackihc", "acjfdcebfe", "ifkbfgckgi"]) == "13/16/9")
    print(f(["adahicgdcj", "iiadkjihkj", "idfgkkaabd", "ekhiidfgbh", "agakiacbki", "hjehhddagf", "afhbckkegc", "bjaicedijb"], ["idfgkkaabd", "iiadkjihkj", "hjehhddagf", "fcfijjbejc", "hjjieddeki", "ekhiidfgbh"], ["idfgkkaabd", "adahicgdcj", "hjehhddagf", "agakiacbki", "ekhiidfgbh", "iiadkjihkj"]) == "14/10/8")
    print(f(["ifiefjhebd", "begdhjajce", "dgdakdhcea", "gdgeidhecf", "bdbjbcjgab", "eajjkbjaij"], ["gdgeidhecf", "bdbjbcjgab", "hfjckcijjf", "ifiefjhebd", "dgdakdhcea", "begdhjajce", "bccaebcfdk", "adfdjkjdig"], ["adfdjkjdig", "eajjkbjaij", "bccaebcfdk", "ifiefjhebd", "dgdakdhcea", "hfjckcijjf", "begdhjajce"]) == "9/13/11")
    print(f(["cbcfccfgjd", "idgggghchc", "hihgbjkhff", "ccggegdgbc", "hcjfgddccj", "ifibejbbag", "bajkbfkkcf", "cgjjdgjbeg"], ["addcacebhk", "idgggghchc", "ifibejbbag", "bajkbfkkcf", "dfhifffkij", "ccggegdgbc", "hihgbjkhff"], ["ifibejbbag", "addcacebhk", "bajkbfkkcf", "dfhifffkij", "hcjfgddccj"]) == "16/12/8")
    print(f(["dbejjidkgk", "icdikieigk", "ijkidchhgi", "jfaaekjdec", "babkebdibf", "kdeffcgeec", "egbdhhiack"], ["icdikieigk", "kjhcjjjffg", "dbejjidkgk", "khjbhfebah", "babkebdibf", "ijkidchhgi"], ["egbdhhiack", "dbejjidkgk", "kdeffcgeec", "kjhcjjjffg", "khjbhfebah", "icdikieigk", "kfjghjbhih"]) == "13/10/13")
    print(f(["jwctsngdtguigmmwsgeo", "gdxjcvzjrfbwtlgedmdy", "gtrpalrvgmzukdzuoeaj", "sjhucnkvajjsvhiezslh", "mionyycxthwujllzqlhj", "ctaycqanpztjeynperfj", "reycjmndcpdbgwwbntis", "fqrayqgyoamlqvjvwcsr", "bszbbfbywcovqqaqjwfj", "agriunkfdydzhqvtoxaj", "mavzburbmuwdkheooyae", "pfetoqtwqcuwnhbalsrx", "zkvqcmmqmcvaymkbexpt", "egypyvdqcnfbeestukca", "mppduwxodztjmjlgctbl", "mnrhdjkvvkmdmcsgrlwi", "ildhsznbhssobvtxaevf", "kdoyxaxhjsdwljgktgmn", "ldjcedyzzzbyohsyywmg", "lbnirbzsojrpjxprupic", "dqtnedsybrnyjdpviive", "rrblydzdwknqvgnnmofd", "aqhusaxbfaagsnrnyoqj", "bvhucavuncgijsvtvbhl", "xzzmerxynghwljpvmvci", "zhpkxzssfxrzuulhlfhc", "omrypmuicoruoscrmhiy", "broshsprqxcuvrjxrdkk", "vrxrywqzkipikktxxpwk", "swqhmghqqilbgbumecns", "enkrqgvehvaicmozkozv", "bnlakkwqtatdikcjzlqx", "sblcovbtarnnqjgqbzzb", "afhncgpiwxvenrjjukzt", "apkcxyihpcbluhcvcjht", "qdkbwfbedqjviqafurdl", "pxiymzefszpoketjhpci", "oszhjqeyifjqctwdiucp"], ["teqrruzrbtmjcopcpuqv", "zkvqcmmqmcvaymkbexpt", "rwanuiwpnyjxbawwdtsr", "edhteadtdxcmrbyxlzdp", "dsgjpgfopzagvdctvsfb", "bgdywnpyqqbgjzxyvhsm", "lqeoszwsoqdkcwskrkde", "tzarnxmsoadeimduwjlv", "lxfjzvzyhvyhjsnzjhft", "rfqthtsfkzddvkfqtmhp", "oogosehycnazftypexwq", "bnlakkwqtatdikcjzlqx", "apkcxyihpcbluhcvcjht", "lbnirbzsojrpjxprupic", "jsafstisvnrkvxbaurjs", "xlxxnsuoutjmjgyfcxxx", "fifzdgnrarjqepaxmjng", "aoadgrjwcttdnzqdnrzu", "agriunkfdydzhqvtoxaj", "jtcxioqordrlieriugde", "broshsprqxcuvrjxrdkk", "nzpcrduvdmtotzxcpktt", "ggclopyujkrmbtwxyhkb", "omrypmuicoruoscrmhiy", "bhwlpqbdegtzhaebgwhh", "qxeqjnakijcegftkztcq", "pxiymzefszpoketjhpci", "pfetoqtwqcuwnhbalsrx", "nuapowrvptolyoejipvf", "enkrqgvehvaicmozkozv", "xccyghfsqzavoowjxbpa", "zhpkxzssfxrzuulhlfhc", "lzmzoehpjhiwnfptazpp", "knlvmuwpdiyxfttmivpo", "tuihmqlcvqwbhjksaorv", "oxnipvxscszrheonkqze", "atynxzjueolgyaobklfz", "rxhgdjqhiesquyedzyoz"], ["agriunkfdydzhqvtoxaj", "rrblydzdwknqvgnnmofd", "pxiymzefszpoketjhpci", "reycjmndcpdbgwwbntis", "ylecznhnoxdscfzmyeff", "ctaycqanpztjeynperfj", "yowldhkaqautgztiktsa", "bnlakkwqtatdikcjzlqx", "gmfvardcfgwyxlcaoozz", "egypyvdqcnfbeestukca", "fwnshwkxmqucemxplosp", "zkvqcmmqmcvaymkbexpt", "akvtqnvpqdlfbrkaocef", "afhncgpiwxvenrjjukzt", "uznxsmmoiavtbmiodxoj", "sjhucnkvajjsvhiezslh", "bghiovcjhfsnsvefhdgv", "lqeoszwsoqdkcwskrkde", "fifzdgnrarjqepaxmjng", "thfxxbazbwtyjnchgsrk", "oogosehycnazftypexwq", "edhteadtdxcmrbyxlzdp", "fqrayqgyoamlqvjvwcsr", "pyxcnvvodqxitmxupafu", "oxnipvxscszrheonkqze", "qdkbwfbedqjviqafurdl", "apkcxyihpcbluhcvcjht", "xafhajeeqbhjhnvferoz", "udetdnbneaxzgkolvaqx", "knlvmuwpdiyxfttmivpo", "ycvcgbciykywlcvgetzq", "xzzmerxynghwljpvmvci", "pfetoqtwqcuwnhbalsrx", "nuapowrvptolyoejipvf", "dsgjpgfopzagvdctvsfb", "oszhjqeyifjqctwdiucp", "jsomqahnpxqenxrvbukz", "rynuqmifvhusgwadyggh", "kdoyxaxhjsdwljgktgmn", "rxhgdjqhiesquyedzyoz", "lzmzoehpjhiwnfptazpp"]) == "86/87/90")
    print(f(["zdvfnpvyntzceucphsgo", "tqziofrzhbubawwahizq", "mbgxpedapvlvrqkzqzkg", "dpkhezbbckdsjdqybrjl", "mxfwuanawegicfehrgmw", "qrmjrjpxdqukqnfwdhte", "oktmpapfivwvjxgqrhcy", "lnqmmzobdkulwoytxzye", "xswhfxceagvludgwudej", "sbziqvpomuttpkofszri", "czshbfycjfjphzjnpoqf", "agkhsshzzpkircpsucum", "sysfjqggrhrnylnlndai", "isttodlbplrxiblhiupl", "rkexbhtqxpafkhlgwaxw", "mlinopnyzexnleukasjf", "lpmjaxtwwrrvzlszeynb", "koklsfsyqpisbelkaior", "vuuasarndfiahlgjvxgs", "tebeesxvootqgtpulori", "liawozjfzncndzauxocv", "ovtedqjzoumidxmhndcz", "obgbozjmgzolnitkiymr", "rhayaxgphidjxfokyqiv", "ovmxamdonwyeykidmqaz", "cyvsxdpiarpzjoemhpch", "libujmarstbmbaoyundf", "wuzynaaalmejmbacimiz", "nusyregaivdnigjiobvh", "kidrtpblxoyrnnttrsrv", "hfvdjntjnzuuwrnderdd", "wjkwlxzbrwzbcfojpzaw", "mpqnpidxdtyyljuwgetd", "tgwnsppeppuvvvxkzidp", "axygfydmqkzilgaxlfla", "jxyrqzlqottsuylerrds", "jcubthrrvshljimcdphf", "fnlwbhlnzqocamnyizas", "ychvamgzejypwbqbswap", "ifneytrznkamorjbtcet", "netuvsdqekrbaavworns"], ["rtiqplurjmronyyoakba", "iiaoyyitgosoedoxanzj", "libujmarstbmbaoyundf", "hrkekrzolyfkfsrkzlhy", "bcnslqfbiuqiothmovyk", "hwaazfyozsdxvdnwcjgb", "rkexbhtqxpafkhlgwaxw", "gnmgymeeamqaosdfsmsy", "cjsbezzgmcxfozowttfq", "wjkwlxzbrwzbcfojpzaw", "lppxdsiugdmhtfxhscdx", "feuzifahwobvwefopvqx", "sryopehzdnenymlmslfk", "uqfmavqkemeuyaywuokd", "gdhnwlsvpbfsryffzxpz", "vuuasarndfiahlgjvxgs", "uhbxcplzamoiabsshiyg", "hodkafmwnlvfueuspxhx", "bvrqexrlkrpiccbquhfj", "nusyregaivdnigjiobvh", "axygfydmqkzilgaxlfla", "qqqgasbcgvdefrktsgia", "mocxxlwemeyteswqtxko", "hfvdjntjnzuuwrnderdd", "tebeesxvootqgtpulori", "rnfnwffyaudmwalkymoo", "pchttkhgrieenatrougz", "xcvqcdkuzuisgcufnylx", "zdvfnpvyntzceucphsgo", "jxyrqzlqottsuylerrds", "qrmjrjpxdqukqnfwdhte", "cypmxctywmjkmsyuikeu", "hhuvdzvehldpesaxahij", "yahsrxjvvvcsvrwkqxtb", "wuwtoddnvxajkkowznqs", "grvqstbiuchzzbcrpvzg", "koklsfsyqpisbelkaior", "mpqnpidxdtyyljuwgetd", "tldxfuugqcmckwwngejx", "ovqnirmimnrsrpvpvjed"], ["rhayaxgphidjxfokyqiv", "haodbwbcrudycxphlhhw", "hrqxwvvrdmscxrrpmidf", "eyrehtlxnlekmvafiqze", "isttodlbplrxiblhiupl", "obgbozjmgzolnitkiymr", "wclwsjvcohdkcfktanwu", "rnfnwffyaudmwalkymoo", "vakftrjbeqyotopeyxwv", "bcnslqfbiuqiothmovyk", "sxkzfoveqgjkvnhfrmzl", "hwaazfyozsdxvdnwcjgb", "xswhfxceagvludgwudej", "lnqmmzobdkulwoytxzye", "zdvfnpvyntzceucphsgo", "pchttkhgrieenatrougz", "uhbxcplzamoiabsshiyg", "tebeesxvootqgtpulori", "thiyrvrtbufwftjjmaas", "sryopehzdnenymlmslfk", "bxneeykaqdhexqjdnibx", "vuuasarndfiahlgjvxgs", "mjpsfxbzimohuaatmqwv", "wuwtoddnvxajkkowznqs", "oktmpapfivwvjxgqrhcy", "tqziofrzhbubawwahizq", "gfrmsvtwjrwpzosyacmy", "jpsynztosydozzpjkmwn", "yahsrxjvvvcsvrwkqxtb", "mtdxdlxytlehvalsuvkh", "wjkwlxzbrwzbcfojpzaw", "sqdadnjftfealnfxoutg", "sysfjqggrhrnylnlndai", "hhuvdzvehldpesaxahij", "mlinopnyzexnleukasjf", "etnedbncpaskqffcmvxf", "pdppybsyiluknndcxgda", "aoyvandgtqmbhpgfmtix", "lfnufjyznzgpwswjewvy", "cypmxctywmjkmsyuikeu", "koklsfsyqpisbelkaior", "netuvsdqekrbaavworns"]) == "95/92/96")
    print(f(["zzesnfpruklrzfdisuwt", "uxpsjfjerkmmfycciiln", "fhhcqczkeiwhstvxxayf", "ykpuoqagbprgvwttozgq", "yphhptpixrpfosltzslg", "myecngrpksktaqfaaglv", "yvfhryuerchqptttrvhp", "ivyydfdkndhkqtroxsao", "gccjfacpdfngspnyasaz", "onackuhnpjdkaipfcdqa", "bckfyiomlonfkxztokji", "korsbfpnajrksnppfpfk", "vccwekenbtkjcsbuqsid", "holihzsmlkadirmagdxk", "bzqnhjvxgcwclkjyswnt", "czisxahstzsdqnotrbud", "gzwvahbklboytspxcqud", "hiipsaghkaqmoqgnpbjy", "vkwegsbskuclojaieqrq", "aqgpyyhsqqpvlpiukqos", "seezyorpvihctlvssxal", "pealoiqlnvgehxyoiqvs", "qatiwqaeyhiahxesphuz", "kcxoyaffurcaomvwfkay", "bbqkuuifmnzsixmmodlf", "emhgxtistszdmlkqevsn", "johmjzvnvnbrzomzygse", "ozpeoijbvztuffgpnaji", "rnnyiekhkpckspwauwuv", "gqsyqqguivpmwgpzxtco", "pbkaopwmjlacdyelesdk", "cbteohrvdtmrfnxcnjja", "offkgnbulvwintuwaour", "edyadtvufffbkktmvagg", "mdyktxrtwtkhylsepqhk", "abxhuhfpchqghxtfdylq", "kefulxqgaqqnwcvzcqmj", "fxbrnftszwfipdeznebl", "xiwxwcwyaonxnyutekga", "zbcuivskrnygrlfdpfql", "nhxaylloprxdrrwwaktd", "ybgpdcblpanhkbrsqkne"], ["ztpinsnqnkhihovsydtj", "izycufwrehaatbbdbgmg", "myecngrpksktaqfaaglv", "nutdfcvoblektwcbtnih", "dzuiasaprgphffqvwhfn", "kcxoyaffurcaomvwfkay", "johmjzvnvnbrzomzygse", "cyiccanjcxdvsscokpdk", "qhjtiydgrqnzkcdlubzq", "zzesnfpruklrzfdisuwt", "uxpsjfjerkmmfycciiln", "pjjgzfxwwxhvyfmvuppv", "aoqcfeixmyttyesesebs", "smnjulhdrofmnhnimdlh", "rnnyiekhkpckspwauwuv", "emhgxtistszdmlkqevsn", "fxbrnftszwfipdeznebl", "vccwekenbtkjcsbuqsid", "holihzsmlkadirmagdxk", "ajaqiugarapsejjydfhf", "kyefmikgfkeaachkletn", "kyksnijluioldfejmton", "mqkkuagfuqmfsfgjolwu", "amoekcyirtxrpxdtxtpb", "gqyahkdyrhrapsnruqjh", "qydoilcnzzcpwjisdqfw", "klzeglmfucclbntincqd", "bckfyiomlonfkxztokji", "ptgavpidfrqcdlrxvisb", "abxhuhfpchqghxtfdylq", "hiipsaghkaqmoqgnpbjy", "nyrtixbbyawvxfmsnfjo", "ukzwlkocyibppgsledbi", "jnsiwvxyeqbcrmkxdecq", "cauwpydncwwygpmkpdcm", "qcfexsefffkuekhdycid", "jzbkuqrepjfpvdhuvcqj", "bzqnhjvxgcwclkjyswnt"], ["nzivwevqynwqlhtalovx", "kefulxqgaqqnwcvzcqmj", "kyefmikgfkeaachkletn", "sngcopndrhjjkpqdzsjt", "coeaoaholzvvubpbvven", "onackuhnpjdkaipfcdqa", "bkmzrtzjgzxzybwunmkf", "myecngrpksktaqfaaglv", "zaqldcppihgdoxxbklar", "smnjulhdrofmnhnimdlh", "fxbrnftszwfipdeznebl", "ajaqiugarapsejjydfhf", "hiipsaghkaqmoqgnpbjy", "xsjzvrwhsqvmbqyrrufk", "nuhcuycgsdxmtyrdmyda", "xrnpafcvbhegasdkwovt", "seezyorpvihctlvssxal", "pealoiqlnvgehxyoiqvs", "lqaoujbsgpwfewihupbv", "nhxaylloprxdrrwwaktd", "bzqnhjvxgcwclkjyswnt", "qfcdkddaacctqabaqlft", "abxhuhfpchqghxtfdylq", "offkgnbulvwintuwaour", "jnsiwvxyeqbcrmkxdecq", "ynpumbbzrrepsrbrblcn", "vkwegsbskuclojaieqrq", "gjpzvjdsfjqmxmjipafw", "mqkkuagfuqmfsfgjolwu", "qbzthcjtavxbisuwgnyi", "phpkotshtefsjahkefqv", "pbkaopwmjlacdyelesdk", "vccwekenbtkjcsbuqsid", "nacumxjgdoolarljqoxt", "ivyydfdkndhkqtroxsao", "amoekcyirtxrpxdtxtpb", "cyiccanjcxdvsscokpdk"]) == "97/87/83")
    print(f(["lquagbpymfolaynxwmcm", "cpwgncetdwuhdpzdlume", "sueipazzykbsrjkcayaf", "cgbasmqyprunvdbikjgk", "bcnairnksrrnhfsevnga", "nmjehdbpmonhtdfvwfea", "nxgidfwdhfxlfmpbyjew", "varihwnmldfeoaflxcrv", "mijwsxkenvccedztkkdv", "cnsammnmwaxklnsxyrxj", "xnzrufkgwtcaletxkbft", "tawbwsgizwkpwcdljbqy", "dgkhhpvbjkhmhgceunnm", "dvhwbbzcpwfjvvrxzgip", "lrnitbmihunwhkqhrmgi", "yebcibundqslycahfcjl", "yglwxcgosxrfnwmxgpds", "yfkddhfqjfxcafrmlwsf", "fglynxbgdvifkwkeplhq", "mthrljfkqsimgzanonzo", "tafjnjzfuwrojwtnoxbx", "svncjexyumucybecbgaq", "izblbxrjvoawjvdycgwu", "zseereknhgomrelnmibg", "voddloamfhktybstkobr", "zcghkhdxrkrwscxhjxmo", "xufesnbzduocesoqhzzx", "fwqxxeqoylpedvphmxdf", "zwoaakdvxnzwgqiuadlo", "mlxkerrshtmwsbowtysh", "zmpcgpokdtrivcqderca", "feiafowpmuycftysrmrd", "jtlponglpmbbcdovdzqw", "cwypxetmzqpwuuksjdox", "sihokwtnrbbzrduafxxl", "jgrcdohjuorcrjfdeadh", "melgahfvfctnabslojts", "nsmbzlcqpkazuimvirlz", "xeiyjyqclkvxugsgcyjt", "bkjvedeibariammaaofj", "oobnjafuapsoczbxozsj", "jqqibviglqrxybqpirpv"], ["xeiyjyqclkvxugsgcyjt", "vdsmratnjfwcmxoucysi", "lquagbpymfolaynxwmcm", "varihwnmldfeoaflxcrv", "vfjkkqwzrlgkkezsiyzx", "yfkddhfqjfxcafrmlwsf", "owlzqfqhimuyzkldsjcw", "sihokwtnrbbzrduafxxl", "jqqibviglqrxybqpirpv", "xslhehpubrzracfdvtym", "qbsrzklrxjxbudponkxx", "xugpjbyuqbfgbbvtmrvj", "xcoohubsbfylsruwmpqr", "cgbasmqyprunvdbikjgk", "eyuuiuhwjkhbyavxjyzn", "buhtmlckrxmgyqgvclud", "mtfcygdxoqhzoblvtzzg", "nsmbzlcqpkazuimvirlz", "lrnitbmihunwhkqhrmgi", "frljaidtdjyzvjxjkdsv", "fmvtrarhcspvuvjznuld", "rqpupmnhdzidmdokfdpc", "zgnozfrrotjusopwjmaf", "xzfosaeufpbzxelhnhyw", "gacqybdswhfnwliwdoqu", "mtwxfbguenwlxppsbvbt", "bfdgbzgweqvxtcngckza", "svncjexyumucybecbgaq", "izblbxrjvoawjvdycgwu", "fspdjwfmqvybuanwrteo", "xtnistokuvqdajpdlmnd", "dokrwlgruesmfuvkgzcg", "cewmevcwdlanbobtvemn", "kfoqlwuskufrvrbvzgik", "yglwxcgosxrfnwmxgpds", "uhfimihpqwuuhycnnmpf", "ufcrhurjzolpnrfftqpl", "wnqkwsnnzggdglhoozho", "cpwgncetdwuhdpzdlume", "nmjehdbpmonhtdfvwfea", "mijwsxkenvccedztkkdv", "sueipazzykbsrjkcayaf"], ["dugpihkmekkjuiuspcqn", "fjniuiqhbabdljqjtpxj", "varihwnmldfeoaflxcrv", "fspdjwfmqvybuanwrteo", "vnlfcjlohpneghpheuld", "szhhicduczchdtsncuxg", "dvhwbbzcpwfjvvrxzgip", "tmsjcwbhpjgsrtnocoqu", "fglynxbgdvifkwkeplhq", "dokrwlgruesmfuvkgzcg", "wtvgfdsbcmjeboshwlxr", "melgahfvfctnabslojts", "vdsmratnjfwcmxoucysi", "nsmbzlcqpkazuimvirlz", "nmjehdbpmonhtdfvwfea", "tlwimgnwjbjwhpktdqht", "oobnjafuapsoczbxozsj", "cewmevcwdlanbobtvemn", "bcwzwvidebhewynaxnrx", "jgrcdohjuorcrjfdeadh", "flsxteyrazaytzsvtcoz", "qhngxgxrmymggjcnlezk", "tawbwsgizwkpwcdljbqy", "yebcibundqslycahfcjl", "wtsqfpmhucivxbgjeqqn", "lpehliicvsxmtulhlflw", "xcoohubsbfylsruwmpqr", "jtlponglpmbbcdovdzqw", "frljaidtdjyzvjxjkdsv", "uhfimihpqwuuhycnnmpf", "ufcrhurjzolpnrfftqpl", "qijltleupwhzqcaotxqt", "svncjexyumucybecbgaq", "pftghxolcenesgfdogas", "jcotmogljmilckvjtdxv", "feiafowpmuycftysrmrd", "wbsurzrnkdjrybujvkvs", "mlxkerrshtmwsbowtysh", "mthrljfkqsimgzanonzo", "nxgidfwdhfxlfmpbyjew", "jbudjjyekjxfbidnbcmi"]) == "94/98/95")
    print(f(["xsigurhuromzadgobppd", "rbjewlapagaqrmlfnlvu", "fanaaxgocslczmcmpkpf", "moytqsmqeftihgaydvdy", "iamkeyqklhuafehbmrrr", "lrdocjdnilqqgmatttqs", "ztcwmexyntpzjnntkwpx", "ickhsnbdbeluwenrffdv", "agzdtwrpbfmmyjtxffpg", "gurilgufxpxdewiprtnr", "mdoitkqpdffxgtjjflxv", "avnxbuzumymdqgxdfejr", "pceoiedujhmpsstgwyzu", "ycsjqdzvbjnkifimsyww", "ohpgahkigkyuejwrjeow", "atvcvoknqcjpwihrxhyu", "uqppgcxenlrllslbtold", "xfggugcukzxejtrsnzfx", "lmvzfinxtbdpnfmhirda", "zwntarcdgdhibocfpztl", "rmrznklbxlmqymlqeojy", "pohcnmptbcbtkmledtzb", "bhqhykgohyvulzmmwqya", "ghqrlzxzdgythcyaqaul", "gkexcfibahzcntuhlvod", "kucnqluuedfithpzmrru", "wgeskstmndtyhzplazxm", "mxbtejxbbptyhgmpphwj", "tuyxqbjcfdvdjbdgopqr", "aqtkbvjudjtznlvkywad", "gsgywrleqdyjnmqfknud", "qtndseevvcgstrezmvcl", "rqrinymonechhxnuopuf", "ufpmbiojdvgxbmhvcsis", "vozrqasdcnxbpxwrdifd", "qpwepmememydxzxbohwe", "nlpsmbqwbbwsnewugtmo", "elsemtulvsuvptpyksci", "rmxplrnebaanmpjsvako", "kwtesopyrbndwlvttcrv", "pcgirhbswibibbsdhjom", "zqcphbammpgkoklpnxwg", "wtycowtpldcajeeclguc"], ["ickhsnbdbeluwenrffdv", "mzjladkxacvoenorrscn", "tiepwxtbcmzijgpotiov", "uqppgcxenlrllslbtold", "bravnauguzdkjtjabwmr", "ptnabmpxljxyhhbeagcj", "pcgirhbswibibbsdhjom", "nlpsmbqwbbwsnewugtmo", "hhtxvlbqiookknjwibgf", "gurilgufxpxdewiprtnr", "piknuxoyijlmniwnohhw", "euzxwjhwihvrarzfzzae", "zwntarcdgdhibocfpztl", "pqgzcexinanxmegxgprt", "vozrqasdcnxbpxwrdifd", "lrdocjdnilqqgmatttqs", "jbhqtlafnpcijskaeols", "lmvzfinxtbdpnfmhirda", "xqcifvdfokbuktznxsra", "zqcphbammpgkoklpnxwg", "qpqzcnymfzmqblscylms", "rmxplrnebaanmpjsvako", "ghqrlzxzdgythcyaqaul", "czbkusnspemxhbrphilt", "mzctteooifptpcswsesn", "aqtkbvjudjtznlvkywad", "hpwloayzerydhjusdcga", "vjzlnowacancrqfejoir", "atvcvoknqcjpwihrxhyu", "cpzzntugughjzrdsoszj", "wtycowtpldcajeeclguc", "wstqacyggkajhnxxkgvn", "mdoitkqpdffxgtjjflxv", "pivizbiteocjqgvcddjg", "pohcnmptbcbtkmledtzb", "esstpekivivnoqieqhlk", "rmrznklbxlmqymlqeojy", "bhqhykgohyvulzmmwqya", "kwtesopyrbndwlvttcrv", "chitvkfymdnkxpnkdigb"], ["kwtesopyrbndwlvttcrv", "fwqybalqvennpwkgqxme", "rbiljsqnwumcnmoghmxh", "ptnabmpxljxyhhbeagcj", "ltvpdpbdbbujpgzqwcnc", "fhzlhpuokwfngjmkwvga", "wgeskstmndtyhzplazxm", "metxjkykwykwmeuklovd", "piknuxoyijlmniwnohhw", "hhtxvlbqiookknjwibgf", "mdoitkqpdffxgtjjflxv", "geobolttatcyidvtthnk", "ickhsnbdbeluwenrffdv", "xfggugcukzxejtrsnzfx", "hpwloayzerydhjusdcga", "ponxbshpzmmifglmaoju", "agzdtwrpbfmmyjtxffpg", "vsgvqyujmiiwvboxknuo", "chitvkfymdnkxpnkdigb", "tuyxqbjcfdvdjbdgopqr", "bravnauguzdkjtjabwmr", "tfuavreunxphgdbgzges", "cdlrizfzyctvwwnknocb", "ownrjlsbuctmklnrqhrq", "aqtkbvjudjtznlvkywad", "hdzwjonalsbuoenebdyb", "wtycowtpldcajeeclguc", "aojfvmirqbybtkchakss", "ccastrutzsuylmanwfgo", "euzxwjhwihvrarzfzzae", "gsgywrleqdyjnmqfknud", "rhfdsstdivuqmujvubkx", "tiepwxtbcmzijgpotiov", "mxbtejxbbptyhgmpphwj", "mzjladkxacvoenorrscn", "qpqzcnymfzmqblscylms", "qtndseevvcgstrezmvcl", "gkexcfibahzcntuhlvod", "oqrchisuiemitocatvwb", "pohcnmptbcbtkmledtzb", "rmrznklbxlmqymlqeojy"]) == "94/83/91")
    print(f(["jgcchrgrbrjwsqcqxscc", "dyluxfnsczxcujcwzasf", "jjdpfcgphzqnjzrmuydj", "ngdcqvzynapwjokmnaon", "ktioqdbexyxqepjvisdc", "tykbeltkyhvhyfdtnmsu", "rrxyqacqjufqokmdtdoo", "dlpxiyiokkpemdjlevfv", "yavzuxcydpmmwhnrcfwa", "stqqevaqlxggsavqmemu", "olnzdqslornwesigngvz", "hxbxdwarzwoygtemzkxy", "dytfdcuzitjatdqjbvdv", "nvbnxxwqjrynbjajoimw", "khzwyatxtjrfqhxgefmk", "ronlwgurovujqtbtiwrx", "akufyymthiebybajarhm", "swyjqgrxrgurdzgcakdm", "ovztsxtmsuiowdeijyab", "zcmhtvxcarvgfmcjifuc", "rulqtzoqbvzepsozfmzf", "ecswugbexwgxgxzfcpqq", "mkzfuiscthmutegjwxcp", "cztthxnojudsqhrszgef", "pkunjdlcpsvzffuwtgyt", "efqlertkdxuesnagezdi", "mzpmpwvryqicxfzavbkd", "kzdkfchpaxusaeymzbfc", "xzlvhhdjitqycnlyyeij", "fxnmvdokygcuaeigrxfc", "mmthxahrszbbktdrkzaf", "uaoelbiftuwuukyvyciy", "tfwkduvaxtkpwwiyfmtf", "igopreouwcutnjpgpoma", "tagatuapvgcqyykxowgj", "ihezzdaakfvumqjcglnh"], ["dytfdcuzitjatdqjbvdv", "yluwefbhcxyymjxccjbv", "brpixghyahakmzapbzvg", "ovztsxtmsuiowdeijyab", "zcmhtvxcarvgfmcjifuc", "kpddxwzaxdgrudntyfmp", "jyvtirbxslyyoogidxmw", "tagatuapvgcqyykxowgj", "bopvepgewicyudiusndh", "jvieqspryzwtoglwjaum", "judjppzxpivbecsorvkx", "aolouodhswkqeectlcmq", "uzormvkectqrjmuxzadk", "euxpryhiertmyuqlfjpk", "ghbeeyxkpvtbtnzlvznn", "fzjnhylahgsdgwwglncy", "guazjcihsproljsiyczz", "nvbnxxwqjrynbjajoimw", "qmoqztkdyihrnfwzoucw", "wucrrfvmxpqzaypuswmk", "rwbsdyzgqkpunrrnozqq", "olnzdqslornwesigngvz", "phbecqpowonemuvblbna", "ruyekuwcrqxhhvrkqvmu", "hxbxdwarzwoygtemzkxy", "xjbbnulxgdhhaebasxkn", "xzlvhhdjitqycnlyyeij", "dyluxfnsczxcujcwzasf", "rhudvwyanrgygwdftzmj", "cztthxnojudsqhrszgef", "xnlistjngjkrbrmnfehi", "ywogatqvgcgzwhifhlyt", "ascopfdutqjsdrrgkzga", "akufyymthiebybajarhm", "nprhcebqcvvpkvvdlzre", "pmcvnjdozwsuzxmgwegy", "rezhybwotmnkwprakyoa", "ecswugbexwgxgxzfcpqq"], ["zfatpjaxrrysjrqxaeoy", "icclnwcmwjswtauaperh", "rmzjepvmifjsybptamxc", "qwohxqqgsetqrvlipxyi", "pmcvnjdozwsuzxmgwegy", "fzjnhylahgsdgwwglncy", "jvieqspryzwtoglwjaum", "mzpmpwvryqicxfzavbkd", "zcmhtvxcarvgfmcjifuc", "mmthxahrszbbktdrkzaf", "kphufpgcbwzoyorxcjqs", "ascopfdutqjsdrrgkzga", "ywogatqvgcgzwhifhlyt", "ruyekuwcrqxhhvrkqvmu", "yluwefbhcxyymjxccjbv", "jyvtirbxslyyoogidxmw", "jjdpfcgphzqnjzrmuydj", "pkunjdlcpsvzffuwtgyt", "zruipvcrkrnzkkprotdu", "rnalghsgbomygpkqkabb", "sppealqaizzfhdtiedgh", "olnzdqslornwesigngvz", "uaoelbiftuwuukyvyciy", "phbecqpowonemuvblbna", "ihezzdaakfvumqjcglnh", "hucdvlapyfadmenxzlpk", "efqlertkdxuesnagezdi", "jysknrvikbeeicnzturx", "fxnmvdokygcuaeigrxfc", "ovztsxtmsuiowdeijyab", "ulegknoirxzymuyrodnd", "tagatuapvgcqyykxowgj", "pknbjqyirxrqzumzqkqs", "khzwyatxtjrfqhxgefmk", "deeikfuavvpcqrmrupcj", "dyluxfnsczxcujcwzasf", "vdaglhkokemeofaarkit"]) == "82/88/83")
    print(f(["gagoraehqymjtpdihoow", "wfvnxxfgauuwpaafazju", "jrueotzumsdcszfqszgr", "amwjybgbzzbuhwgohkgy", "hkerfzfbcgboxwzkxqws", "tsjijsgsqmlrgxlisggt", "fwvcjfafvsbqghluecuu", "ccuqedorzckmrndxyyqj", "vqpnuinqupbqttavxfjw", "gvezostmzvxblsjyifdu", "hdvrcflnnvdhiacguymq", "fxahifwkgooqohyxibjp", "sunsmktlelsoykmopjzj", "qxtqcwmfbvrjfpszpkgp", "dksxqijcmfrnjtdqlozk", "jtpnqrtpiihqpmpawmoz", "cgpkfnhobbgrhjzjbfwk", "ooicyifoefsjdirnyobb", "oaazxouptufrefudtdfr", "nsebykxypzausiwgoikv", "hlqniubipejwjcacsvjg", "xzxkgyhckfocfiymoxms", "dbivamxzpasbkelzbohi", "qkyigirfdaqrvpbqfkqa", "tdemsjfosjnfwejwjgrk", "khxpnwumlhintwvmybkf", "glxyxvbgrammpjahcvnr", "enlweksfxldtxlcwhhiu", "kzcfvakwvsrqpuheqxjc", "bjmjtzjpdcayqqjozlcr", "cgzisclbqveuwfrdqyev", "udtdhushgitititflxnd", "awicbwamcfyyqvhryuwu", "swvgilaldnkbjgsiqdys", "pfvimoyrfdsurflywlhj", "ihwcmuimtuqvegvlwxax", "dtgjbpyovcyarcroqlnd", "ezubmhfkweuxosdmzrbs", "divoashjotlflvepckcy", "fjovuvoibrtwccpqhzlu", "wvcnxnboeyvchgjlpkre"], ["sdgpchugkyhikpkpdmaj", "lupahgbztnciiwyjaldj", "seonwzrotxgammfzprpu", "ujvniifwqpzapxnaccka", "pogtlldhdpskcmmycfoa", "rplknrxcmfyttpbmnxiv", "vozwyrbwjajzuljjxwbr", "tbxzwteegjkwehgwyved", "gagoraehqymjtpdihoow", "awsoqjccytnmlhizxuiz", "hlqniubipejwjcacsvjg", "ihwcmuimtuqvegvlwxax", "fxahifwkgooqohyxibjp", "xzxkgyhckfocfiymoxms", "qmqkjjmpzziuwbxcnfcp", "vqpnuinqupbqttavxfjw", "hdvrcflnnvdhiacguymq", "urwggsrieazbdcbzmwnb", "yyhqgmhtzcelftixfyie", "glxyxvbgrammpjahcvnr", "rjiuqmdfhopouweggroo", "mbzjtgtjwvrrpvvdndvd", "olxmtcauzebyezwyjgou", "enlweksfxldtxlcwhhiu", "tvzxgdvzcvcziojkkguh", "ooicyifoefsjdirnyobb", "uxkajgxtncrgqwtdtfkb", "rmlfuewnnmyjekyxffcg", "qkyigirfdaqrvpbqfkqa", "ccuqedorzckmrndxyyqj", "toruwpkecfybymokzimi", "zlhlimegujlrpsqddmgc", "vvafoefpcjyovftqybpq", "fbirdwhvhiuazmchbwwc", "mtzizaqxigcxkzvauqwf", "ayzwmgemmwnetjyfuvff", "udtdhushgitititflxnd", "ksshhinnhswvokmriizo", "tbtqydnzqtyrioopdheh"], ["tsjijsgsqmlrgxlisggt", "armrfohdfqljfcrutgur", "dksxqijcmfrnjtdqlozk", "hlqniubipejwjcacsvjg", "dbivamxzpasbkelzbohi", "vgiiihnwmvpmhgflbrsq", "mbzjtgtjwvrrpvvdndvd", "ayzwmgemmwnetjyfuvff", "fbirdwhvhiuazmchbwwc", "enlweksfxldtxlcwhhiu", "ktsszqrueafmemhowzfq", "oaazxouptufrefudtdfr", "pogtlldhdpskcmmycfoa", "ezubmhfkweuxosdmzrbs", "mtzizaqxigcxkzvauqwf", "gaxhgriibhclrkzntxda", "wfvnxxfgauuwpaafazju", "hkerfzfbcgboxwzkxqws", "sunsmktlelsoykmopjzj", "rjiuqmdfhopouweggroo", "nsebykxypzausiwgoikv", "hpecvapjqwtpbeiblqjm", "qyfugacxxvwdyrrzsdro", "bpmejwqdppxreuqugjal", "swvgilaldnkbjgsiqdys", "vzctgzkonckwelcczmay", "rplknrxcmfyttpbmnxiv", "ngyjgqdfyhapxvvywghr", "ertqsyjgksthhgdphspy", "jrueotzumsdcszfqszgr", "hkyyvxrvpajncpowucnc", "gvezostmzvxblsjyifdu", "qxtqcwmfbvrjfpszpkgp", "ucylayaiqwjfonjviqdz", "sugzvycsoskcvucpjpna", "glxyxvbgrammpjahcvnr", "rmlfuewnnmyjekyxffcg", "ksshhinnhswvokmriizo", "urwggsrieazbdcbzmwnb", "wvcnxnboeyvchgjlpkre", "qmqkjjmpzziuwbxcnfcp", "pvhrfqlwyekyvjysdiht", "ujvniifwqpzapxnaccka", "zlhlimegujlrpsqddmgc", "dtgjbpyovcyarcroqlnd", "amwjybgbzzbuhwgohkgy", "seonwzrotxgammfzprpu"]) == "91/87/105")
    print(f(["hjdwemgbdrpjjkbndzkt", "csowhgqanqvjzsfnxjtr", "siraylxhrjuvybalzuya", "tzbeetcwpuazrbcrplor", "xenyxkihkhmsneekarss", "udliqmucqciuzjqszuzn", "ndmudfclsuupagbdjrht", "iqwvjxdqwvundcrsqjli", "cvjzanxrvxujuuerhdro", "dexhpypqxltjrpbmsydu", "gyycpkdlgdsyxcmbjvwa", "zjxmfmzhkmvfakhvqxjy", "ithiapmltngoyjmtrtbi", "sdoxbpvsljwegwcpueju", "wfibsvkochvvuoclxamo", "piakewqhtlkdwvrimxvc", "uppxwxmparwaggixhqvr", "slczcrzkezyaiudhpbhp", "edkswlhqisgucfczwdmh", "edtquxqjqhotwbdkqjmu", "lnfyqeesnldhpyamwwrj", "vvvsxhydpgynbebvzhro", "diewulrfwhkhnnbdoadh", "obvbzpmtmyanamxhgzjh", "mkxmbjchrxvruzolbtrh", "ljggnffguqpylkufgarw", "pvlefvorzrxfzfswwmeh", "htseuktrzydolmkfyrmg", "nhjqzzbklicsveuetrkt", "npvgchmkeqtjsdslnmvb", "plbpupoxwhekquklhchn", "gmrqqhwxgcmzlaytjuai", "ujnqgblathhqieujgeig", "ardplgzdsttfiroqfxcn", "uuurpjabttsxrakzuqac", "garjfadzbrxbpajmqdkd", "irtgctkymidbxjcadkrf", "awpcwzkwrxtlczmmmhvz", "rwczcaxwdxopujetprlc", "dnbrfskwmocofdlgrknl", "wluqnukzsxvrmoemsdoq", "ppxmahosodbzyfefxeee"], ["rcfxrllxvmxspfnrwcqe", "ppxmahosodbzyfefxeee", "lbxdupryshejpjqkgfuq", "edtquxqjqhotwbdkqjmu", "mkxmbjchrxvruzolbtrh", "ardplgzdsttfiroqfxcn", "iqwvjxdqwvundcrsqjli", "tzbeetcwpuazrbcrplor", "usnqbsrcmtvhtejvhcxl", "wfibsvkochvvuoclxamo", "mtjbzhecxfnsvjusnbvt", "csowhgqanqvjzsfnxjtr", "edkswlhqisgucfczwdmh", "wwzntqudhdnqswkvpseu", "rwczcaxwdxopujetprlc", "npvgchmkeqtjsdslnmvb", "vvvsxhydpgynbebvzhro", "ndmudfclsuupagbdjrht", "qadwmgatzorxmpyffitk", "cvjzanxrvxujuuerhdro", "nhjqzzbklicsveuetrkt", "bxblwhhopouvkeosmhow", "awpcwzkwrxtlczmmmhvz", "zzhrlhcxymlcmnmsjdhq", "dexhpypqxltjrpbmsydu", "qsqpwmsxpaipsfiyqgwa", "suhaymeylnqsyzrrolsk", "cvxbxilebxleowopemjk", "vqprhfenulvugovmmctw", "xpqupvjcfzwhobmlupvw", "kwgjofibcdndsfjfquqr", "gwvlwiqquczsfnvwfusw", "yhzaokijezlftybtfxbg", "nlulvmynodsvhidkfwev", "qssgwobwknffcflkmfsy", "cwctulmypusvbkuhrdal", "guhdsedzlskewkqclcqf", "dnexpoudnyejnuphtrtf", "ithiapmltngoyjmtrtbi"], ["hweznfakmggmiqururle", "hfudzholymrghtggedrl", "uuurpjabttsxrakzuqac", "mtjbzhecxfnsvjusnbvt", "edkswlhqisgucfczwdmh", "zjxmfmzhkmvfakhvqxjy", "hnmlvqbxnqpcmhtemgzx", "uppxwxmparwaggixhqvr", "oqxrkgtgzejhwgkexlsu", "ljggnffguqpylkufgarw", "pnpclvxlvqkudqhisvli", "cwctulmypusvbkuhrdal", "nhjqzzbklicsveuetrkt", "cvxbxilebxleowopemjk", "ujnqgblathhqieujgeig", "ithiapmltngoyjmtrtbi", "htseuktrzydolmkfyrmg", "wwzntqudhdnqswkvpseu", "oibiqxtatcncegmxwuxm", "deptkdznhpjjzwhtjkfs", "hkmvjoomxdocjnaovvyx", "dkdaxosyridyvbgausil", "garjfadzbrxbpajmqdkd", "usnqbsrcmtvhtejvhcxl", "gmrqqhwxgcmzlaytjuai", "zzhrlhcxymlcmnmsjdhq", "lbxdupryshejpjqkgfuq", "puqdouvsfipvcrawpnak", "rkkjypzwxtxivoeyttce", "yhzaokijezlftybtfxbg", "rcfxrllxvmxspfnrwcqe", "nkdrvfseqkvkgilvfxya", "suhaymeylnqsyzrrolsk", "hjdwemgbdrpjjkbndzkt", "rnlahcawtdywdayvthkd", "ecmzqoeymcikhdzkbfqm", "xpqupvjcfzwhobmlupvw", "dnexpoudnyejnuphtrtf", "bsmxfhmjnopzsrjrpxgs", "gytqhbejcperqfrqvnru", "xxmoziddfqchoeygydga", "prteniedkfowhmnkrdzs"]) == "96/84/99")
    print(f(["dbgxeojobdhujiygbrzv", "newghmfdvstpsbbjbhnv", "voanerufkgimtksjufds", "eyztakutvwajfxsytwpw", "ilzjahqxaslnzysopulf", "mbivznzmaqfhczfuvqsj", "xilyhmqasalgaqdfbtza", "fzysobsjsdrphovgmify", "obdvdmnrhmwbuzzpvqhp", "ipsmknmlrirtugaevhpz", "dbuwksdoayixqaibrjks", "xiinbvbqifwexupimopi", "whefzarqmoxafykzqctc", "lvgllgokuampejamjywo", "uouglympqbvtujhuwscl", "gydelopbpscbzfkglmbb", "iwpkmpgudctfkuouoioi", "dsevuyastwumznhghbmw", "apsdwskabyiqddpjdkuo", "suaibhutoufvoxxktbtx", "qsjrqorurpoubvxqinry", "wiwfzelnbczaemvzongz", "ioopoirwbmyapumolokj", "oticgtebdwzmwcmhypjr", "dibpuxhfumhqsopqvgmp", "ayseyzdeidzaxjilevxr", "icjfhkborhevyknfwwxq", "ztevfiaoatfffhnfmgrt", "wmdplyywhycvmmrgomeb", "kxylvprycorobifxhrfm", "pzkayiwrqamdtyzryrbq", "lyiesxwksumgprszbxao", "mfomkwxgquabrwztezug", "bevvrwaxtvxaraqepfnm", "tpxdgxjvxllmgseqdcxr", "jydfbuafzlelgnskknht"], ["dcytzuewrewqjphoxdpk", "bufqevzzpbaetivkluhc", "dzngdjuylkcrpofrqlzz", "qxzlicvmwffllfugyyil", "colpzvbdqgdoftqqffdp", "dihefdywvqafkvcybmos", "okfembkabuijztfnpanq", "tyyouugnezheowbjhxew", "xbbmtgegjhartlpranoq", "ayseyzdeidzaxjilevxr", "dndxonnafdxcwgfcilnm", "ioopoirwbmyapumolokj", "mbifffhcmvyyhufwnbth", "cjjeqsljrxfcdjcxkmms", "nvcsjoqkcowsbajdcfqh", "mbivznzmaqfhczfuvqsj", "eslmvxigkjnmqtoestxd", "ubpaclywflujonjbnqiw", "xilyhmqasalgaqdfbtza", "txfutejfutloxgqbfqek", "twvudqonsesbcujyydov", "vnqnsavhaehpcgfyjggl", "iwpkmpgudctfkuouoioi", "cmvtghgcflmdiexrdqwx", "kzvynssbcykpuriqmmzq", "obdvdmnrhmwbuzzpvqhp", "pjnuroulvsbwnrqndvlg", "uhuqgsrtsrhzaeizvedz", "vybolnahkaqntgoiyjnm", "newghmfdvstpsbbjbhnv", "wmdplyywhycvmmrgomeb", "lyiesxwksumgprszbxao", "aphplvylxhcyvmxwnkic", "kcyalcqivjozkgabfpuk", "pzkayiwrqamdtyzryrbq", "zdadnphipzghrnzzjzpx", "oticgtebdwzmwcmhypjr", "mfomkwxgquabrwztezug", "rcdxloaymxtdmafprugh", "ejqqteieffmvgumcmstx", "dibpuxhfumhqsopqvgmp"], ["fqaooqgarbvokvhfpubo", "smorozngprvknzxyxgpa", "mjzvjxuvfjexypkiliey", "mbifffhcmvyyhufwnbth", "mbivznzmaqfhczfuvqsj", "ucjjmnuurvhgcbqurtjy", "suaibhutoufvoxxktbtx", "vnqnsavhaehpcgfyjggl", "puwvjjfxfseycubyajmf", "kzvynssbcykpuriqmmzq", "vgwjyeztidkxkupqrwfz", "qsjrqorurpoubvxqinry", "pzkayiwrqamdtyzryrbq", "ulwxuzlwyapstwvkdswf", "ckczevgizyfdjlnwrocx", "fzysobsjsdrphovgmify", "aphplvylxhcyvmxwnkic", "dzngdjuylkcrpofrqlzz", "vybolnahkaqntgoiyjnm", "okfembkabuijztfnpanq", "dtfvyrsfuwfikqupkadi", "qxzlicvmwffllfugyyil", "yoqikhdymsmgeydwqltb", "dsevuyastwumznhghbmw", "iabsdebgsmgylnohjscc", "ubpaclywflujonjbnqiw", "zdadnphipzghrnzzjzpx", "ejqqteieffmvgumcmstx", "pjnuroulvsbwnrqndvlg", "uiomleqwbbmkteqimiwh", "yxphkbdolpqqqkvtulbh", "lopdtpozgkmpksgjkgsd", "ncbojwzudxftmawgwbmj", "wmdplyywhycvmmrgomeb", "kcyalcqivjozkgabfpuk", "cksixwuwxbxqiltdwdle", "xggcuegslsynwtvwdpyw"]) == "88/94/88")
    print(f(["vxigkkvtnsaiwimwtwno", "baxtpzsfgtsspfmbkdzw", "eqijkclgasuxzdgoljpy", "kdushfbmfngxeardcnxw", "fdbfkkqrngessbufpcja", "bozygretcfjnvlkyibta", "svwrlivglsoknyoskhql", "ssensxladhknciesbccc", "akokypwqdpsjaknrszti", "wxcoewydvrthvjiqnrvz", "vgimramqvhwzlwameojm", "hzkwxirqlbkqoinelzub", "kkhgtixkresavylenllx", "tjvgkwmkyogcbhpogktk", "yfhoxehephkbwhwvijve", "etzsatwnjwvxamhewajx", "xfltwzfrkrrpyzaknrkq", "clmkqgakshqjettsslrc", "lqjyyuwpgubjgxdzgdvv", "szkdibozcghjldehqcas", "zsvrggshdhuitwxadqyd", "hfqiczwicvcpxqvosxoi", "endtugnzficydxaxstkt", "fqkuyzonzypjtzhulcaa", "rtxbxmaecmamthrevees", "ntqsfcxupnjvmoebdzxz", "oqjsunysyemnkyjwynbj", "qqhsnhacwwohpfdmsdxu", "jpodaixruovlifpufxyk", "lfzbcyywkcfobqkdwpzv", "wbzvcltmyibjqabsvjkw", "nztrshtoadfliybcmhfw", "kicillpyzwlsgbolrbyv", "lokjrfstsejcqvxivukn", "thebbgoqmhvnbyrxwslc", "ifbzptjzujaibfvkszks"], ["unaqhxivbvnggkvucbcv", "zsvrggshdhuitwxadqyd", "tjvgkwmkyogcbhpogktk", "gmvulvwlkixzcutsnrxv", "aceytmcljgqdevrnludh", "fdbfkkqrngessbufpcja", "heylaoczofzrjjbaxuli", "qgmibriibeehdbsjzgbu", "bozygretcfjnvlkyibta", "lbgviiijumiybtsznfxx", "lvvmqaixiqgvfzxjvpuc", "baxtpzsfgtsspfmbkdzw", "fdutihgbnskufkwrecuu", "szovlszpqckzjgpfitxq", "xewhtptbrjgzhledaroa", "kkhgtixkresavylenllx", "ntqsfcxupnjvmoebdzxz", "kdushfbmfngxeardcnxw", "ophctiecfxdezbarvguz", "ljvpprqrvemursptswqb", "vgimramqvhwzlwameojm", "rtxbxmaecmamthrevees", "knnqqppfxfkveejwojvw", "lnxkvfnzyghuhjlxscyv", "vzfryjxngmzbpafxbnig", "vfvhxduigvicnhzfzlvq", "typfjdtnvthryufxajtt", "miqjjygocznvmtoilcui", "lljuexqeuhqsncfbyens", "aicvbddcwvgqkzmwnyiz", "yfhoxehephkbwhwvijve", "vrrcivfeaqgcoletwqsa", "liunkzycliydurzxkntc", "cdcdaetyvwtukfmglpgu", "lokjrfstsejcqvxivukn", "ktoekqxulhyvrputnqrl", "tlmgrkzfxbnvzhukcjyr"], ["kofdircweyujjivhfvyx", "hfqiczwicvcpxqvosxoi", "bkjverjzgcdjbbpymdym", "iygbxanmhcqckjgblich", "siquepfwjxfqjaqvnyit", "ssensxladhknciesbccc", "rtvqbratgpxuposcicuc", "fjvrdowgbmxhwbxzapbj", "wbzvcltmyibjqabsvjkw", "tjvgkwmkyogcbhpogktk", "oqjsunysyemnkyjwynbj", "xewhtptbrjgzhledaroa", "qgmibriibeehdbsjzgbu", "cdcdaetyvwtukfmglpgu", "ifbzptjzujaibfvkszks", "defhybcreapvovwyhnzd", "aicvbddcwvgqkzmwnyiz", "fqkuyzonzypjtzhulcaa", "kdushfbmfngxeardcnxw", "cqrwqzfqtnmnijairzku", "cshduwhcfifczhpifoah", "rtxbxmaecmamthrevees", "liunkzycliydurzxkntc", "knnqqppfxfkveejwojvw", "fdutihgbnskufkwrecuu", "lnxkvfnzyghuhjlxscyv", "clmkqgakshqjettsslrc", "wxcoewydvrthvjiqnrvz", "uvdnvfavdecgcrmbljog", "rolzppvlgfenpesllrvf", "typfjdtnvthryufxajtt", "dypngkzfccspauobdagm", "sjjxkyiuvezgvxpavztc", "ntqsfcxupnjvmoebdzxz", "zsvrggshdhuitwxadqyd", "eszgpevlzchdjowuwiij", "umrpklaebzltwauohcnn"]) == "83/85/84")
    print(f(["s", "r", "a", "p", "i", "v", "c"], ["o", "q", "v", "a", "p", "f", "t", "c"], ["k", "c", "t", "r", "p", "e", "a", "q"]) == "13/15/15")
    print(f(["t", "l", "x", "p", "e", "g", "s", "r", "u"], ["s", "x", "t", "l", "e", "b", "q", "g"], ["c", "f", "q", "u", "l", "w", "t", "p", "e"]) == "16/14/18")
    print(f(["u", "g", "y", "f", "l", "v", "d", "h"], ["h", "l", "y", "g", "z", "q"], ["g", "q", "h", "v", "y", "u"]) == "15/10/9")
    print(f(["i", "b", "q", "v", "x", "g"], ["f", "d", "o", "v", "s", "i", "g", "q", "l"], ["a", "s", "d", "b", "q", "f", "v"]) == "11/18/13")
    print(f(["i", "w", "e", "u", "r", "q", "g", "j"], ["r", "q", "l", "t", "a", "e", "j"], ["w", "a", "e", "q", "z", "r", "t", "l"]) == "16/11/14")
    print(f(["a", "c", "f", "b"], ["b", "c", "e", "g", "f"], ["a", "b", "f", "c", "g"]) == "5/8/7")
    print(f(["b", "g", "h", "f", "d", "e"], ["h", "d", "b", "f", "a", "e", "c"], ["f", "h", "c", "g", "d", "b"]) == "8/11/8")
    print(f(["e", "h", "d", "b", "a", "c", "f", "g"], ["a", "c", "d", "h", "g", "e"], ["b", "e", "c", "d", "g"]) == "13/8/6")
    print(f(["d", "h", "c", "a", "e", "f"], ["g", "a", "c", "f"], ["a", "g", "d", "c", "f", "h"]) == "10/5/9")
    print(f(["h", "a", "f"], ["c", "f"], ["a"]) == "7/5/2")
    print(f(["jxiufpifqaluzjjoxmii", "botkegnhwkeppoyoimnn", "docsmqttvzujhsfxttez", "jcinklwvhsksgzyvezms", "ewhscmrtmvoupxnxirtm", "orqsnbrfdyfmdjkmuoqa", "rdxgecmjdvxogoosydak", "epxlnozrhxxjeeclitsi", "xuwsmpocgnmpifxofznr", "aadwvrmnftahiamzwkir", "giefeppsutusuikwfpxi", "zgpiucdrunjqukslqigj", "wgfvidprvixasvowmgjw", "jikawmtnzuwsqzufhdur", "eekyfnmzwddnmumcmfwo", "flyhhqfwgvhlpkhjkpcq", "avtacddcbufbqdlbcakd", "yipxabhorlamzxhtxmid", "ccoirhrlwlhjmlbplpal", "wkxpdrzycokixgywnakx", "mnbseiieexvnznsvlzjb", "wlodoiycaqvabnmdowcc", "qbeufppiwijqmgfpcpis", "oczujnsbzltybedubzve", "yylhookgxjeftylcekru", "eskldcgyggsogcmtdiap", "qypcqlufcpzpmzrplybi", "icxgjmbeasxlhuzuizzi", "neldpsxspniudbkqfbsp", "gystblovppwmnagydfyz", "adhktpkbaimtealvzuxv", "ajwtzcidaptoytwqdcvx", "dfqtfpeikgffiwglcgby", "dlblviildstysepurkqd", "wgsctvydjbvrcjzcjibv", "dsbdvwcoprgqzknbscjn", "qztwcipcswvdffichzuj", "iegmrrmxvuolflavhmja", "sxsxxgbsxaaexqxreyca", "opzfbjcbquhxbkgstfby", "lazzysfmscoqvxdcpkap", "akjggbdhexsaamnoouav", "jytponrpqfanenrivchy", "awrkkktmfuqydvhyhnxi", "tnwcrwzbtjhdzozceapa", "rsjukjnrrtotokhstuds", "exzpjgwkrqounqedilss", "zwjilpdsgzttpdambzoq", "xqnwwijjmcnwdllkdxhl", "tuzyhqzmpzflvtwpiayr"], ["sxsxxgbsxaaexqxreyca", "jikawmtnzuwsqzufhdur", "avtacddcbufbqdlbcakd", "akjggbdhexsaamnoouav", "jcinklwvhsksgzyvezms", "xqnwwijjmcnwdllkdxhl", "jxiufpifqaluzjjoxmii", "lazzysfmscoqvxdcpkap", "orqsnbrfdyfmdjkmuoqa", "neldpsxspniudbkqfbsp", "wkxpdrzycokixgywnakx", "epxlnozrhxxjeeclitsi", "qztwcipcswvdffichzuj", "icxgjmbeasxlhuzuizzi", "yipxabhorlamzxhtxmid", "ccoirhrlwlhjmlbplpal", "dfqtfpeikgffiwglcgby", "yylhookgxjeftylcekru", "rsjukjnrrtotokhstuds", "zgpiucdrunjqukslqigj", "dsbdvwcoprgqzknbscjn", "tnwcrwzbtjhdzozceapa", "docsmqttvzujhsfxttez", "jytponrpqfanenrivchy", "iegmrrmxvuolflavhmja", "dlblviildstysepurkqd", "awrkkktmfuqydvhyhnxi", "rdxgecmjdvxogoosydak", "exzpjgwkrqounqedilss", "qypcqlufcpzpmzrplybi", "eekyfnmzwddnmumcmfwo", "ajwtzcidaptoytwqdcvx", "giefeppsutusuikwfpxi", "xuwsmpocgnmpifxofznr", "botkegnhwkeppoyoimnn", "flyhhqfwgvhlpkhjkpcq", "aadwvrmnftahiamzwkir", "zwjilpdsgzttpdambzoq", "oczujnsbzltybedubzve", "wgsctvydjbvrcjzcjibv", "tuzyhqzmpzflvtwpiayr", "ewhscmrtmvoupxnxirtm", "opzfbjcbquhxbkgstfby", "wlodoiycaqvabnmdowcc", "eskldcgyggsogcmtdiap", "mnbseiieexvnznsvlzjb", "adhktpkbaimtealvzuxv", "qbeufppiwijqmgfpcpis", "wgfvidprvixasvowmgjw", "gystblovppwmnagydfyz"], ["wkxpdrzycokixgywnakx", "giefeppsutusuikwfpxi", "wgsctvydjbvrcjzcjibv", "tnwcrwzbtjhdzozceapa", "docsmqttvzujhsfxttez", "xqnwwijjmcnwdllkdxhl", "opzfbjcbquhxbkgstfby", "gystblovppwmnagydfyz", "yipxabhorlamzxhtxmid", "avtacddcbufbqdlbcakd", "jytponrpqfanenrivchy", "ewhscmrtmvoupxnxirtm", "jikawmtnzuwsqzufhdur", "wlodoiycaqvabnmdowcc", "aadwvrmnftahiamzwkir", "eekyfnmzwddnmumcmfwo", "jxiufpifqaluzjjoxmii", "adhktpkbaimtealvzuxv", "zgpiucdrunjqukslqigj", "rdxgecmjdvxogoosydak", "epxlnozrhxxjeeclitsi", "neldpsxspniudbkqfbsp", "sxsxxgbsxaaexqxreyca", "yylhookgxjeftylcekru", "qztwcipcswvdffichzuj", "awrkkktmfuqydvhyhnxi", "ajwtzcidaptoytwqdcvx", "jcinklwvhsksgzyvezms", "icxgjmbeasxlhuzuizzi", "mnbseiieexvnznsvlzjb", "dfqtfpeikgffiwglcgby", "eskldcgyggsogcmtdiap", "lazzysfmscoqvxdcpkap", "exzpjgwkrqounqedilss", "dlblviildstysepurkqd", "tuzyhqzmpzflvtwpiayr", "ccoirhrlwlhjmlbplpal", "qypcqlufcpzpmzrplybi", "wgfvidprvixasvowmgjw", "oczujnsbzltybedubzve", "zwjilpdsgzttpdambzoq", "qbeufppiwijqmgfpcpis", "akjggbdhexsaamnoouav", "rsjukjnrrtotokhstuds", "orqsnbrfdyfmdjkmuoqa", "iegmrrmxvuolflavhmja", "botkegnhwkeppoyoimnn", "xuwsmpocgnmpifxofznr", "dsbdvwcoprgqzknbscjn", "flyhhqfwgvhlpkhjkpcq"]) == "50/50/50")
    print(f(["xfkrtcvpbnfcntonrici", "qvommquotwvumdxgwoue", "qgkxmnycaaeqvbzwculz", "gubgukzmwcqclmbukivo", "hvweqtuxbcofssvfvnur", "fchqylglcpnlujrjfyxu", "rhbqwdtlpygelebnixhr", "ndmrucvglhgeflypcesg", "ocklxjfctzxoewmmusvj", "uqucymnqihfrgveopnlk", "vpnmirabkxnjhsbkgtji", "qekonxtgmrzztkzwipqh", "adtasomumypsimhabozd", "dqgjhrvxsqwlbycpgvab", "plrqhjfgmnhclspowpsb", "aqkirspcjlzncgtksqma", "hweahxviwczdgzsiqeor", "nbwmxedybhluquzxssyt", "zuoojrsdloisgeublgzc", "kelhizkdcgvpjfivsibm", "twnbquknyurebjsspwmz", "qirldvqkkvhdnrxjunmn", "vlfqkcknyxbmssbhvzlp", "dtbuoyudfynbtehvuxre", "mysnfjtrzxdhmgkvzjml", "ojknipbduftmdndlgogz", "oxyilltsoqlbangfyntk", "kfirkyeqedgrjlroshkc", "mgxskvhnrmwqshsmpzop", "zetbmegizmexadjmmktr", "drfcmayffbcwpouazcnj", "meygheyxmpxhhrucnwva", "hbdoditsxfqpminnnmhg", "qwqhauikjhozgyyqfnls", "jedwoonemxdpswyqvvuo", "epbxgdgzzjcofqopmbdr", "dgucdmohnxsuftmryafn", "ptvynceajxalyvrzligz", "tvnxuytxirjcggtegbsx", "sfufxepircuqimfjjwcb", "qqwlbimyfhzartquqqtr", "rbqxyaxvdnfegowjzesz", "nrrzyfzcwnjwrqiqihoq", "hduxexzbrtchpylnedhb", "wflrxvnbhyffbjtaafor", "gwxipchqypbtsxvojhlj", "fdhnvcgryeuhqzjwsjds", "nktkgbltleqqcmzyhdsp", "ynbimknabhiblakfkvnk", "pehjmifygssnllmnjkjo"], ["fzhkjoypysjojnbnefjn", "rakosenkgxljbktqomvb", "hofhnrlekgcyaojtuesi", "jxakjefzuoycddvngria", "qubovwwvzenmbdndhqiv", "konxlzbuwdlcsfdkgaru", "iawxbxtpxkrqsdgphjda", "vrwhqniuoraqsznlxnfk", "tsmfjwdyczwmtjejzzgi", "uiawvkkozkcotojrewdr", "gnyuhmlgdvymvetvquga", "dmzscbaevbvqkluyhikz", "nodsledmhhyewaladhfv", "nqxoifnxaufbbemxhpkr", "cbrvmrtjyexeapltbfcz", "emmkncsbzamedncbvsef", "zslhxhgpiqzksglkybwy", "kxpydsvvajbkzcmuzogv", "qbhxdyhycxuoaqgduzdb", "cokwflbbdfizqjdufmkq", "nblvqoatcjgvfudumure", "sgtafvdwzffjygvweecq", "frvahedrirvijbibuddv", "muyzxsznxkpbuniufnid", "nryllsyhuszkvspvoitl", "yyzfsxgtocgcrptsaucy", "bampcnilqadkduvizwgl", "akmeqyljhexxkycygcuz", "cuocerpgfvorvtifveod", "whfmjpitkkuyrepxzqvo", "wvootvkqrnjervxkgvtd", "couiufzhkpecveikfukw", "raktvmgmkkhyrnyrdoit", "veqifnicwrnbyibsxube", "gsqxbcijtlhrzyzolndd", "becacwcejkvlkuetfyld", "ftsafuzltfbogvyokehw", "phtupzcvxrjowgvjayqx", "srmvrxpcpxicbuocbmka", "xjyvifivohxzqncxegqe", "eunnkjayaiuuquvndgbk", "bvndugtizbdqvlncydah", "mfrvvpvkpozqfgiolwfv", "yomtwsnkhuczgtbzrrvl", "cicozocahnuwolaggcnx", "verpxppfhyvwvyunzqer", "lueopdrhaxmbwoqnasqv", "gxoqyyxijgnnmerodcsf", "bhzfjmdboafiblqobygv", "oouqmrnrlxrlyiqhatbp"], ["yurmlviyzxyehuulyazh", "kzpbuothwtiudcqunpqw", "cnvqbjjgzuwgqmebylun", "qmnblcdtfuhcgjetvorr", "fuchbroqzueqoujiuzaw", "kolqloagqyjfychzxqyu", "usnzlwcrddryjsvmltmu", "xahiemslbatqrizboddo", "wpjgkuonopumoowpnbpa", "qurlotbmuhykviiggfnw", "sdfsxsuqnkxafxtoqodk", "webnlqdmtrspfgejmepz", "hhrammoekaxmjdawmgrr", "sjmaeokuskcqxtgijyuq", "miwjbtjdanohykyfndwc", "ixuyieyzwksuehdsqtoy", "obimvacwlhcioesrlkth", "xlrfdnfnniuetlhvwmhn", "zfuwtzvikwdyspyhcemk", "rgqhbnafxnlioivxlehe", "eiaeoexkphzsndshcuxw", "vdndlexwnkwymncscatr", "fwhodcfbkcbirmqtyvzo", "kloufqtokttacwhwpieo", "dkkmlsfhghoplofijccd", "ddjmjwvimimrdztwdjnw", "xqgmbidnnhecyivxnrtp", "nembwhbfagidcaxswflw", "vlioacwyojdrggnlihbw", "qhbhbpmmrfgnugmhtjix", "lwhjoyyfpiwbpswrrzqd", "mqfxjvfzdascgfoelner", "swssnejwqtsjofhiztxc", "bzawhfcrrxgxlshoofic", "cucetmimphgnliwdyzeq", "jlvvqafysbczkvjgmwhq", "bpngucynxkfacogpnwur", "caekvcazqzeovrqkvmpt", "nwjslbargxurxedvcgbt", "whffxkavdzpztjlzunjt", "wyjhpcgdndpribzipyrd", "vnklnvkteooowbnjdrsa", "ivgjuhtutglviylbnzle", "jcxbiatjrvnkvyugueau", "vvairhyrmvkfeoazovrk", "ajyqqzefvsprbywkrnia", "xdqjkcovtanuubnwlrmp", "imueyfccwzghyoxpyqkh", "dihuodtwiwfsnpkknzuh", "klypkqyyasgawicttbru"]) == "150/150/150")
    print(f(["aaaaaaaaaaaaaaaaaaao", "aaaaaaaaaaaaaaaaaazn", "aaaaaaaaaaaaaaaaaanh", "aaaaaaaaaaaaaaaaaavw", "aaaaaaaaaaaaaaaaaarf", "aaaaaaaaaaaaaaaaaaam", "aaaaaaaaaaaaaaaaaatj", "aaaaaaaaaaaaaaaaaatf", "aaaaaaaaaaaaaaaaaass", "aaaaaaaaaaaaaaaaaabh", "aaaaaaaaaaaaaaaaaafb", "aaaaaaaaaaaaaaaaaahg", "aaaaaaaaaaaaaaaaaacz", "aaaaaaaaaaaaaaaaaadb", "aaaaaaaaaaaaaaaaaahd", "aaaaaaaaaaaaaaaaaaix", "aaaaaaaaaaaaaaaaaapf", "aaaaaaaaaaaaaaaaaadg", "aaaaaaaaaaaaaaaaaabb", "aaaaaaaaaaaaaaaaaabo", "aaaaaaaaaaaaaaaaaatm", "aaaaaaaaaaaaaaaaaalw", "aaaaaaaaaaaaaaaaaasc", "aaaaaaaaaaaaaaaaaafq", "aaaaaaaaaaaaaaaaaaim", "aaaaaaaaaaaaaaaaaays", "aaaaaaaaaaaaaaaaaaoy", "aaaaaaaaaaaaaaaaaazh", "aaaaaaaaaaaaaaaaaalx", "aaaaaaaaaaaaaaaaaanq", "aaaaaaaaaaaaaaaaaakv", "aaaaaaaaaaaaaaaaaagp", "aaaaaaaaaaaaaaaaaaep", "aaaaaaaaaaaaaaaaaajv", "aaaaaaaaaaaaaaaaaavf", "aaaaaaaaaaaaaaaaaayh", "aaaaaaaaaaaaaaaaaauy", "aaaaaaaaaaaaaaaaaaga", "aaaaaaaaaaaaaaaaaaqo", "aaaaaaaaaaaaaaaaaavz", "aaaaaaaaaaaaaaaaaatb", "aaaaaaaaaaaaaaaaaajl", "aaaaaaaaaaaaaaaaaast", "aaaaaaaaaaaaaaaaaadk", "aaaaaaaaaaaaaaaaaaod", "aaaaaaaaaaaaaaaaaagc", "aaaaaaaaaaaaaaaaaaea", "aaaaaaaaaaaaaaaaaawy", "aaaaaaaaaaaaaaaaaawc", "aaaaaaaaaaaaaaaaabdu"], ["aaaaaaaaaaaaaaaaaaaf", "aaaaaaaaaaaaaaaaaaut", "aaaaaaaaaaaaaaaaaaeh", "aaaaaaaaaaaaaaaaaasa", "aaaaaaaaaaaaaaaaaany", "aaaaaaaaaaaaaaaaaayl", "aaaaaaaaaaaaaaaaaaop", "aaaaaaaaaaaaaaaaaalq", "aaaaaaaaaaaaaaaaaacm", "aaaaaaaaaaaaaaaaaaue", "aaaaaaaaaaaaaaaaaank", "aaaaaaaaaaaaaaaaaafo", "aaaaaaaaaaaaaaaaaarg", "aaaaaaaaaaaaaaaaaalf", "aaaaaaaaaaaaaaaaaajt", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaajw", "aaaaaaaaaaaaaaaaaary", "aaaaaaaaaaaaaaaaaalc", "aaaaaaaaaaaaaaaaaaqv", "aaaaaaaaaaaaaaaaaafx", "aaaaaaaaaaaaaaaaaagy", "aaaaaaaaaaaaaaaaaanr", "aaaaaaaaaaaaaaaaaavl", "aaaaaaaaaaaaaaaaaaxa", "aaaaaaaaaaaaaaaaaayu", "aaaaaaaaaaaaaaaaaabz", "aaaaaaaaaaaaaaaaaavw", "aaaaaaaaaaaaaaaaaapo", "aaaaaaaaaaaaaaaaaalj", "aaaaaaaaaaaaaaaaaatl", "aaaaaaaaaaaaaaaaaaja", "aaaaaaaaaaaaaaaaaajl", "aaaaaaaaaaaaaaaaaavs", "aaaaaaaaaaaaaaaaaaqi", "aaaaaaaaaaaaaaaaaafw", "aaaaaaaaaaaaaaaaaare", "aaaaaaaaaaaaaaaaaauj", "aaaaaaaaaaaaaaaaaawa", "aaaaaaaaaaaaaaaaaaak", "aaaaaaaaaaaaaaaaaatz", "aaaaaaaaaaaaaaaaaala", "aaaaaaaaaaaaaaaaaajj", "aaaaaaaaaaaaaaaaaaaq", "aaaaaaaaaaaaaaaaaasd", "aaaaaaaaaaaaaaaaaaiz", "aaaaaaaaaaaaaaaaaavh", "aaaaaaaaaaaaaaaaabvh", "aaaaaaaaaaaaaaaaabvi", "aaaaaaaaaaaaaaaaacdu"], ["aaaaaaaaaaaaaaaaaaxi", "aaaaaaaaaaaaaaaaaaua", "aaaaaaaaaaaaaaaaaaix", "aaaaaaaaaaaaaaaaaabk", "aaaaaaaaaaaaaaaaaaxe", "aaaaaaaaaaaaaaaaaarm", "aaaaaaaaaaaaaaaaaacg", "aaaaaaaaaaaaaaaaaajf", "aaaaaaaaaaaaaaaaaaaq", "aaaaaaaaaaaaaaaaaayv", "aaaaaaaaaaaaaaaaaaui", "aaaaaaaaaaaaaaaaaakb", "aaaaaaaaaaaaaaaaaaau", "aaaaaaaaaaaaaaaaaaag", "aaaaaaaaaaaaaaaaaamo", "aaaaaaaaaaaaaaaaaari", "aaaaaaaaaaaaaaaaaami", "aaaaaaaaaaaaaaaaaasj", "aaaaaaaaaaaaaaaaaaec", "aaaaaaaaaaaaaaaaaaxf", "aaaaaaaaaaaaaaaaaaiz", "aaaaaaaaaaaaaaaaaayw", "aaaaaaaaaaaaaaaaaahk", "aaaaaaaaaaaaaaaaaati", "aaaaaaaaaaaaaaaaaahs", "aaaaaaaaaaaaaaaaaaqo", "aaaaaaaaaaaaaaaaaaqj", "aaaaaaaaaaaaaaaaaayp", "aaaaaaaaaaaaaaaaaabl", "aaaaaaaaaaaaaaaaaafq", "aaaaaaaaaaaaaaaaaakf", "aaaaaaaaaaaaaaaaaapg", "aaaaaaaaaaaaaaaaaabm", "aaaaaaaaaaaaaaaaaamx", "aaaaaaaaaaaaaaaaaaij", "aaaaaaaaaaaaaaaaaald", "aaaaaaaaaaaaaaaaaaed", "aaaaaaaaaaaaaaaaaaxy", "aaaaaaaaaaaaaaaaaapb", "aaaaaaaaaaaaaaaaaadm", "aaaaaaaaaaaaaaaaaauz", "aaaaaaaaaaaaaaaaaaqe", "aaaaaaaaaaaaaaaaaagu", "aaaaaaaaaaaaaaaaaavi", "aaaaaaaaaaaaaaaaaatr", "aaaaaaaaaaaaaaaaaany", "aaaaaaaaaaaaaaaaaaws", "aaaaaaaaaaaaaaaaaayr", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaaddu"]) == "145/145/144")
    print(f(["aaaaaaaaaaaaaaaaaagf", "aaaaaaaaaaaaaaaaaamt", "aaaaaaaaaaaaaaaaaaxe", "aaaaaaaaaaaaaaaaaahy", "aaaaaaaaaaaaaaaaaaqs", "aaaaaaaaaaaaaaaaaaih", "aaaaaaaaaaaaaaaaaabj", "aaaaaaaaaaaaaaaaaaze", "aaaaaaaaaaaaaaaaaalg", "aaaaaaaaaaaaaaaaaaah", "aaaaaaaaaaaaaaaaaarg", "aaaaaaaaaaaaaaaaaaqr", "aaaaaaaaaaaaaaaaaazu", "aaaaaaaaaaaaaaaaaawz", "aaaaaaaaaaaaaaaaaavb", "aaaaaaaaaaaaaaaaaave", "aaaaaaaaaaaaaaaaaahd", "aaaaaaaaaaaaaaaaaavm", "aaaaaaaaaaaaaaaaaaeq", "aaaaaaaaaaaaaaaaaads", "aaaaaaaaaaaaaaaaaack", "aaaaaaaaaaaaaaaaaaxg", "aaaaaaaaaaaaaaaaaakb", "aaaaaaaaaaaaaaaaaaqc", "aaaaaaaaaaaaaaaaaatk", "aaaaaaaaaaaaaaaaaaum", "aaaaaaaaaaaaaaaaaakg", "aaaaaaaaaaaaaaaaaajb", "aaaaaaaaaaaaaaaaaahl", "aaaaaaaaaaaaaaaaaams", "aaaaaaaaaaaaaaaaaazn", "aaaaaaaaaaaaaaaaaalk", "aaaaaaaaaaaaaaaaaalc", "aaaaaaaaaaaaaaaaaaub", "aaaaaaaaaaaaaaaaaakv", "aaaaaaaaaaaaaaaaaafm", "aaaaaaaaaaaaaaaaaamr", "aaaaaaaaaaaaaaaaaawb", "aaaaaaaaaaaaaaaaaawq", "aaaaaaaaaaaaaaaaaauk", "aaaaaaaaaaaaaaaaaalb", "aaaaaaaaaaaaaaaaaarj", "aaaaaaaaaaaaaaaaaahk", "aaaaaaaaaaaaaaaaaaaj", "aaaaaaaaaaaaaaaaaaui", "aaaaaaaaaaaaaaaaaaga", "aaaaaaaaaaaaaaaaaadt", "aaaaaaaaaaaaaaaaaazg", "aaaaaaaaaaaaaaaaaajf", "aaaaaaaaaaaaaaaaabdu"], ["aaaaaaaaaaaaaaaaaaul", "aaaaaaaaaaaaaaaaaawc", "aaaaaaaaaaaaaaaaaarx", "aaaaaaaaaaaaaaaaaazr", "aaaaaaaaaaaaaaaaaatl", "aaaaaaaaaaaaaaaaaafc", "aaaaaaaaaaaaaaaaaaig", "aaaaaaaaaaaaaaaaaabm", "aaaaaaaaaaaaaaaaaakd", "aaaaaaaaaaaaaaaaaaik", "aaaaaaaaaaaaaaaaaaid", "aaaaaaaaaaaaaaaaaawm", "aaaaaaaaaaaaaaaaaafw", "aaaaaaaaaaaaaaaaaafy", "aaaaaaaaaaaaaaaaaagc", "aaaaaaaaaaaaaaaaaapl", "aaaaaaaaaaaaaaaaaaff", "aaaaaaaaaaaaaaaaaajq", "aaaaaaaaaaaaaaaaaalv", "aaaaaaaaaaaaaaaaaagh", "aaaaaaaaaaaaaaaaaaja", "aaaaaaaaaaaaaaaaaacq", "aaaaaaaaaaaaaaaaaaiu", "aaaaaaaaaaaaaaaaaanv", "aaaaaaaaaaaaaaaaaaso", "aaaaaaaaaaaaaaaaaadp", "aaaaaaaaaaaaaaaaaaqu", "aaaaaaaaaaaaaaaaaahd", "aaaaaaaaaaaaaaaaaaug", "aaaaaaaaaaaaaaaaaaaj", "aaaaaaaaaaaaaaaaaake", "aaaaaaaaaaaaaaaaaacf", "aaaaaaaaaaaaaaaaaauc", "aaaaaaaaaaaaaaaaaawv", "aaaaaaaaaaaaaaaaaamb", "aaaaaaaaaaaaaaaaaauw", "aaaaaaaaaaaaaaaaaaif", "aaaaaaaaaaaaaaaaaald", "aaaaaaaaaaaaaaaaaabc", "aaaaaaaaaaaaaaaaaakb", "aaaaaaaaaaaaaaaaaaex", "aaaaaaaaaaaaaaaaaaev", "aaaaaaaaaaaaaaaaaaxk", "aaaaaaaaaaaaaaaaaara", "aaaaaaaaaaaaaaaaaalx", "aaaaaaaaaaaaaaaaaagz", "aaaaaaaaaaaaaaaaaaus", "aaaaaaaaaaaaaaaaaatr", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaacdu"], ["aaaaaaaaaaaaaaaaaasx", "aaaaaaaaaaaaaaaaaayb", "aaaaaaaaaaaaaaaaaabv", "aaaaaaaaaaaaaaaaaaxb", "aaaaaaaaaaaaaaaaaara", "aaaaaaaaaaaaaaaaaafm", "aaaaaaaaaaaaaaaaaand", "aaaaaaaaaaaaaaaaaaru", "aaaaaaaaaaaaaaaaaajd", "aaaaaaaaaaaaaaaaaafo", "aaaaaaaaaaaaaaaaaayr", "aaaaaaaaaaaaaaaaaaox", "aaaaaaaaaaaaaaaaaakj", "aaaaaaaaaaaaaaaaaakn", "aaaaaaaaaaaaaaaaaale", "aaaaaaaaaaaaaaaaaavt", "aaaaaaaaaaaaaaaaaaps", "aaaaaaaaaaaaaaaaaaow", "aaaaaaaaaaaaaaaaaahv", "aaaaaaaaaaaaaaaaaalz", "aaaaaaaaaaaaaaaaaabk", "aaaaaaaaaaaaaaaaaavd", "aaaaaaaaaaaaaaaaaaff", "aaaaaaaaaaaaaaaaaall", "aaaaaaaaaaaaaaaaaakx", "aaaaaaaaaaaaaaaaaatx", "aaaaaaaaaaaaaaaaaaqc", "aaaaaaaaaaaaaaaaaayq", "aaaaaaaaaaaaaaaaaaqm", "aaaaaaaaaaaaaaaaaazh", "aaaaaaaaaaaaaaaaaafy", "aaaaaaaaaaaaaaaaaawp", "aaaaaaaaaaaaaaaaaasz", "aaaaaaaaaaaaaaaaaayy", "aaaaaaaaaaaaaaaaaavz", "aaaaaaaaaaaaaaaaaazq", "aaaaaaaaaaaaaaaaaahg", "aaaaaaaaaaaaaaaaaaun", "aaaaaaaaaaaaaaaaaalx", "aaaaaaaaaaaaaaaaaapu", "aaaaaaaaaaaaaaaaaayl", "aaaaaaaaaaaaaaaaaaca", "aaaaaaaaaaaaaaaaaaxn", "aaaaaaaaaaaaaaaaaadm", "aaaaaaaaaaaaaaaaaazx", "aaaaaaaaaaaaaaaaaamj", "aaaaaaaaaaaaaaaaaatp", "aaaaaaaaaaaaaaaaaaoa", "aaaaaaaaaaaaaaaaaaob", "aaaaaaaaaaaaaaaaaddu"]) == "145/143/144")
    print(f(["aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaaes", "aaaaaaaaaaaaaaaaaaxn", "aaaaaaaaaaaaaaaaaaie", "aaaaaaaaaaaaaaaaaaeq", "aaaaaaaaaaaaaaaaaaph", "aaaaaaaaaaaaaaaaaafk", "aaaaaaaaaaaaaaaaaacs", "aaaaaaaaaaaaaaaaaaaw", "aaaaaaaaaaaaaaaaaaur", "aaaaaaaaaaaaaaaaaamw", "aaaaaaaaaaaaaaaaaazn", "aaaaaaaaaaaaaaaaaavy", "aaaaaaaaaaaaaaaaaatv", "aaaaaaaaaaaaaaaaaapv", "aaaaaaaaaaaaaaaaaapn", "aaaaaaaaaaaaaaaaaagc", "aaaaaaaaaaaaaaaaaazl", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaajs", "aaaaaaaaaaaaaaaaaakh", "aaaaaaaaaaaaaaaaaakk", "aaaaaaaaaaaaaaaaaadt", "aaaaaaaaaaaaaaaaaaqj", "aaaaaaaaaaaaaaaaaada", "aaaaaaaaaaaaaaaaaayj", "aaaaaaaaaaaaaaaaaawi", "aaaaaaaaaaaaaaaaaaop", "aaaaaaaaaaaaaaaaaapp", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaahi", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaaot", "aaaaaaaaaaaaaaaaaary", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaadc", "aaaaaaaaaaaaaaaaaaqo", "aaaaaaaaaaaaaaaaaaeg", "aaaaaaaaaaaaaaaaaadd", "aaaaaaaaaaaaaaaaaatc", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaazv", "aaaaaaaaaaaaaaaaaaam", "aaaaaaaaaaaaaaaaaazm", "aaaaaaaaaaaaaaaaaaof", "aaaaaaaaaaaaaaaaaapl", "aaaaaaaaaaaaaaaaaahm", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"], ["aaaaaaaaaaaaaaaaaagc", "aaaaaaaaaaaaaaaaaazl", "aaaaaaaaaaaaaaaaaaeg", "aaaaaaaaaaaaaaaaaatc", "aaaaaaaaaaaaaaaaaawi", "aaaaaaaaaaaaaaaaaadc", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaavy", "aaaaaaaaaaaaaaaaaaur", "aaaaaaaaaaaaaaaaaaof", "aaaaaaaaaaaaaaaaaapl", "aaaaaaaaaaaaaaaaaapp", "aaaaaaaaaaaaaaaaaacs", "aaaaaaaaaaaaaaaaaahi", "aaaaaaaaaaaaaaaaaaaw", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaadd", "aaaaaaaaaaaaaaaaaaph", "aaaaaaaaaaaaaaaaaakh", "aaaaaaaaaaaaaaaaaaam", "aaaaaaaaaaaaaaaaaaop", "aaaaaaaaaaaaaaaaaaot", "aaaaaaaaaaaaaaaaaazv", "aaaaaaaaaaaaaaaaaazn", "aaaaaaaaaaaaaaaaaaeq", "aaaaaaaaaaaaaaaaaajs", "aaaaaaaaaaaaaaaaaaxn", "aaaaaaaaaaaaaaaaaapv", "aaaaaaaaaaaaaaaaaapn", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaaqo", "aaaaaaaaaaaaaaaaaada", "aaaaaaaaaaaaaaaaaatv", "aaaaaaaaaaaaaaaaaaqj", "aaaaaaaaaaaaaaaaaaie", "aaaaaaaaaaaaaaaaaazm", "aaaaaaaaaaaaaaaaaafk", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaaes", "aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaamw", "aaaaaaaaaaaaaaaaaayj", "aaaaaaaaaaaaaaaaaakk", "aaaaaaaaaaaaaaaaaadt", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaahm", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaary", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"], ["aaaaaaaaaaaaaaaaaapl", "aaaaaaaaaaaaaaaaaada", "aaaaaaaaaaaaaaaaaaur", "aaaaaaaaaaaaaaaaaawi", "aaaaaaaaaaaaaaaaaadt", "aaaaaaaaaaaaaaaaaaie", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaapp", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaakk", "aaaaaaaaaaaaaaaaaamw", "aaaaaaaaaaaaaaaaaaqj", "aaaaaaaaaaaaaaaaaapv", "aaaaaaaaaaaaaaaaaary", "aaaaaaaaaaaaaaaaaaeq", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaazn", "aaaaaaaaaaaaaaaaaazm", "aaaaaaaaaaaaaaaaaaop", "aaaaaaaaaaaaaaaaaaof", "aaaaaaaaaaaaaaaaaahi", "aaaaaaaaaaaaaaaaaakh", "aaaaaaaaaaaaaaaaaahm", "aaaaaaaaaaaaaaaaaazl", "aaaaaaaaaaaaaaaaaatv", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaaqo", "aaaaaaaaaaaaaaaaaadc", "aaaaaaaaaaaaaaaaaafk", "aaaaaaaaaaaaaaaaaacs", "aaaaaaaaaaaaaaaaaaot", "aaaaaaaaaaaaaaaaaatc", "aaaaaaaaaaaaaaaaaaaw", "aaaaaaaaaaaaaaaaaazv", "aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaagc", "aaaaaaaaaaaaaaaaaadd", "aaaaaaaaaaaaaaaaaavy", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaayj", "aaaaaaaaaaaaaaaaaaph", "aaaaaaaaaaaaaaaaaajs", "aaaaaaaaaaaaaaaaaaeg", "aaaaaaaaaaaaaaaaaaxn", "aaaaaaaaaaaaaaaaaaam", "aaaaaaaaaaaaaaaaaaes", "aaaaaaaaaaaaaaaaaapn", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"]) == "50/50/50")
    print(f(["aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaaue", "aaaaaaaaaaaaaaaaaato", "aaaaaaaaaaaaaaaaaafq", "aaaaaaaaaaaaaaaaaake", "aaaaaaaaaaaaaaaaaaqh", "aaaaaaaaaaaaaaaaaayp", "aaaaaaaaaaaaaaaaaajd", "aaaaaaaaaaaaaaaaaazk", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaazs", "aaaaaaaaaaaaaaaaaann", "aaaaaaaaaaaaaaaaaazq", "aaaaaaaaaaaaaaaaaaus", "aaaaaaaaaaaaaaaaaazc", "aaaaaaaaaaaaaaaaaauv", "aaaaaaaaaaaaaaaaaagh", "aaaaaaaaaaaaaaaaaawy", "aaaaaaaaaaaaaaaaaasx", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaayv", "aaaaaaaaaaaaaaaaaait", "aaaaaaaaaaaaaaaaaacc", "aaaaaaaaaaaaaaaaaaxy", "aaaaaaaaaaaaaaaaaaaa", "aaaaaaaaaaaaaaaaaaom", "aaaaaaaaaaaaaaaaaabe", "aaaaaaaaaaaaaaaaaavj", "aaaaaaaaaaaaaaaaaaqr", "aaaaaaaaaaaaaaaaaara", "aaaaaaaaaaaaaaaaaajk", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaabz", "aaaaaaaaaaaaaaaaaayz", "aaaaaaaaaaaaaaaaaaex", "aaaaaaaaaaaaaaaaaanb", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaans", "aaaaaaaaaaaaaaaaaaud", "aaaaaaaaaaaaaaaaaasc", "aaaaaaaaaaaaaaaaaauy", "aaaaaaaaaaaaaaaaaagb", "aaaaaaaaaaaaaaaaaamh", "aaaaaaaaaaaaaaaaaajg", "aaaaaaaaaaaaaaaaaadv", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaamm", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"], ["aaaaaaaaaaaaaaaaaacc", "aaaaaaaaaaaaaaaaaayv", "aaaaaaaaaaaaaaaaaaex", "aaaaaaaaaaaaaaaaaaue", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaavj", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaayp", "aaaaaaaaaaaaaaaaaaom", "aaaaaaaaaaaaaaaaaabe", "aaaaaaaaaaaaaaaaaaaa", "aaaaaaaaaaaaaaaaaayz", "aaaaaaaaaaaaaaaaaaud", "aaaaaaaaaaaaaaaaaauy", "aaaaaaaaaaaaaaaaaann", "aaaaaaaaaaaaaaaaaajd", "aaaaaaaaaaaaaaaaaait", "aaaaaaaaaaaaaaaaaamm", "aaaaaaaaaaaaaaaaaasx", "aaaaaaaaaaaaaaaaaazq", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaauv", "aaaaaaaaaaaaaaaaaawy", "aaaaaaaaaaaaaaaaaazs", "aaaaaaaaaaaaaaaaaajg", "aaaaaaaaaaaaaaaaaagb", "aaaaaaaaaaaaaaaaaajk", "aaaaaaaaaaaaaaaaaato", "aaaaaaaaaaaaaaaaaaxy", "aaaaaaaaaaaaaaaaaazc", "aaaaaaaaaaaaaaaaaasc", "aaaaaaaaaaaaaaaaaake", "aaaaaaaaaaaaaaaaaaus", "aaaaaaaaaaaaaaaaaafq", "aaaaaaaaaaaaaaaaaaqh", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaabz", "aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaara", "aaaaaaaaaaaaaaaaaaqr", "aaaaaaaaaaaaaaaaaanb", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaagh", "aaaaaaaaaaaaaaaaaazk", "aaaaaaaaaaaaaaaaaamh", "aaaaaaaaaaaaaaaaaadv", "aaaaaaaaaaaaaaaaaans", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"], ["aaaaaaaaaaaaaaaaaacc", "aaaaaaaaaaaaaaaaaajk", "aaaaaaaaaaaaaaaaaanz", "aaaaaaaaaaaaaaaaaamh", "aaaaaaaaaaaaaaaaaato", "aaaaaaaaaaaaaaaaaabe", "aaaaaaaaaaaaaaaaaajd", "aaaaaaaaaaaaaaaaaajg", "aaaaaaaaaaaaaaaaaauy", "aaaaaaaaaaaaaaaaaazk", "aaaaaaaaaaaaaaaaaagb", "aaaaaaaaaaaaaaaaaagh", "aaaaaaaaaaaaaaaaaayp", "aaaaaaaaaaaaaaaaaauu", "aaaaaaaaaaaaaaaaaayz", "aaaaaaaaaaaaaaaaaafq", "aaaaaaaaaaaaaaaaaavj", "aaaaaaaaaaaaaaaaaaav", "aaaaaaaaaaaaaaaaaaus", "aaaaaaaaaaaaaaaaaait", "aaaaaaaaaaaaaaaaaawy", "aaaaaaaaaaaaaaaaaaip", "aaaaaaaaaaaaaaaaaahf", "aaaaaaaaaaaaaaaaaamm", "aaaaaaaaaaaaaaaaaavv", "aaaaaaaaaaaaaaaaaann", "aaaaaaaaaaaaaaaaaaud", "aaaaaaaaaaaaaaaaaans", "aaaaaaaaaaaaaaaaaaxy", "aaaaaaaaaaaaaaaaaake", "aaaaaaaaaaaaaaaaaadv", "aaaaaaaaaaaaaaaaaanb", "aaaaaaaaaaaaaaaaaasc", "aaaaaaaaaaaaaaaaaara", "aaaaaaaaaaaaaaaaaabz", "aaaaaaaaaaaaaaaaaauv", "aaaaaaaaaaaaaaaaaazc", "aaaaaaaaaaaaaaaaaaex", "aaaaaaaaaaaaaaaaaazs", "aaaaaaaaaaaaaaaaaaqh", "aaaaaaaaaaaaaaaaaaaa", "aaaaaaaaaaaaaaaaaazq", "aaaaaaaaaaaaaaaaaaom", "aaaaaaaaaaaaaaaaaaue", "aaaaaaaaaaaaaaaaaayv", "aaaaaaaaaaaaaaaaaaqr", "aaaaaaaaaaaaaaaaaafn", "aaaaaaaaaaaaaaaaaasx", "aaaaaaaaaaaaaaaaabyh", "aaaaaaaaaaaaaaaaardu"]) == "50/50/50")
    print(f(["aaaaaaaaaaaaaaaaaodb", "aaaaaaaaaaaaaaaaaykg", "aaaaaaaaaaaaaaaaabgq", "aaaaaaaaaaaaaaaaaani", "aaaaaaaaaaaaaaaaansm", "aaaaaaaaaaaaaaaaawwj", "aaaaaaaaaaaaaaaaaoir", "aaaaaaaaaaaaaaaaanbp", "aaaaaaaaaaaaaaaaapjj", "aaaaaaaaaaaaaaaaakfi", "aaaaaaaaaaaaaaaaakvu", "aaaaaaaaaaaaaaaaatlw", "aaaaaaaaaaaaaaaaaail", "aaaaaaaaaaaaaaaaafqv", "aaaaaaaaaaaaaaaaahuo", "aaaaaaaaaaaaaaaaadnh", "aaaaaaaaaaaaaaaaauhv", "aaaaaaaaaaaaaaaaallw", "aaaaaaaaaaaaaaaaagmi", "aaaaaaaaaaaaaaaaakoa", "aaaaaaaaaaaaaaaaacim", "aaaaaaaaaaaaaaaaasfw", "aaaaaaaaaaaaaaaaablv", "aaaaaaaaaaaaaaaaanix", "aaaaaaaaaaaaaaaaaldt", "aaaaaaaaaaaaaaaaaqsq", "aaaaaaaaaaaaaaaaajyr", "aaaaaaaaaaaaaaaaaoja", "aaaaaaaaaaaaaaaaasbs", "aaaaaaaaaaaaaaaaautc", "aaaaaaaaaaaaaaaaaolb", "aaaaaaaaaaaaaaaaaquu", "aaaaaaaaaaaaaaaaaniy", "aaaaaaaaaaaaaaaaaepz", "aaaaaaaaaaaaaaaaadpv", "aaaaaaaaaaaaaaaaasgw", "aaaaaaaaaaaaaaaaasht", "aaaaaaaaaaaaaaaaajln", "aaaaaaaaaaaaaaaaadwa", "aaaaaaaaaaaaaaaaagqu", "aaaaaaaaaaaaaaaaafeb", "aaaaaaaaaaaaaaaaacmb", "aaaaaaaaaaaaaaaaajxg", "aaaaaaaaaaaaaaaaaygf", "aaaaaaaaaaaaaaaaazat", "aaaaaaaaaaaaaaaaabfk", "aaaaaaaaaaaaaaaaakgq", "aaaaaaaaaaaaaaaaaxau", "aaaaaaaaaaaaaaaaardt", "aaaaaaaaaaaaaaaaardu"], ["aaaaaaaaaaaaaaaaagmi", "aaaaaaaaaaaaaaaaajyr", "aaaaaaaaaaaaaaaaasgw", "aaaaaaaaaaaaaaaaansm", "aaaaaaaaaaaaaaaaadpv", "aaaaaaaaaaaaaaaaabgq", "aaaaaaaaaaaaaaaaagqu", "aaaaaaaaaaaaaaaaatlw", "aaaaaaaaaaaaaaaaaldt", "aaaaaaaaaaaaaaaaaquu", "aaaaaaaaaaaaaaaaasbs", "aaaaaaaaaaaaaaaaardt", "aaaaaaaaaaaaaaaaaail", "aaaaaaaaaaaaaaaaanix", "aaaaaaaaaaaaaaaaajxg", "aaaaaaaaaaaaaaaaallw", "aaaaaaaaaaaaaaaaaani", "aaaaaaaaaaaaaaaaahuo", "aaaaaaaaaaaaaaaaanbp", "aaaaaaaaaaaaaaaaaykg", "aaaaaaaaaaaaaaaaautc", "aaaaaaaaaaaaaaaaakgq", "aaaaaaaaaaaaaaaaadnh", "aaaaaaaaaaaaaaaaaniy", "aaaaaaaaaaaaaaaaaodb", "aaaaaaaaaaaaaaaaaolb", "aaaaaaaaaaaaaaaaaygf", "aaaaaaaaaaaaaaaaauhv", "aaaaaaaaaaaaaaaaawwj", "aaaaaaaaaaaaaaaaafqv", "aaaaaaaaaaaaaaaaablv", "aaaaaaaaaaaaaaaaasfw", "aaaaaaaaaaaaaaaaaoja", "aaaaaaaaaaaaaaaaasht", "aaaaaaaaaaaaaaaaaoir", "aaaaaaaaaaaaaaaaaxau", "aaaaaaaaaaaaaaaaakfi", "aaaaaaaaaaaaaaaaafeb", "aaaaaaaaaaaaaaaaaqsq", "aaaaaaaaaaaaaaaaabfk", "aaaaaaaaaaaaaaaaazat", "aaaaaaaaaaaaaaaaakoa", "aaaaaaaaaaaaaaaaacmb", "aaaaaaaaaaaaaaaaapjj", "aaaaaaaaaaaaaaaaadwa", "aaaaaaaaaaaaaaaaakvu", "aaaaaaaaaaaaaaaaardu", "aaaaaaaaaaaaaaaaaepz", "aaaaaaaaaaaaaaaaacim", "aaaaaaaaaaaaaaaaajln"], ["aaaaaaaaaaaaaaaaaepz", "aaaaaaaaaaaaaaaaadnh", "aaaaaaaaaaaaaaaaasgw", "aaaaaaaaaaaaaaaaafeb", "aaaaaaaaaaaaaaaaaoir", "aaaaaaaaaaaaaaaaaoja", "aaaaaaaaaaaaaaaaahuo", "aaaaaaaaaaaaaaaaaygf", "aaaaaaaaaaaaaaaaasbs", "aaaaaaaaaaaaaaaaautc", "aaaaaaaaaaaaaaaaardt", "aaaaaaaaaaaaaaaaansm", "aaaaaaaaaaaaaaaaaail", "aaaaaaaaaaaaaaaaauhv", "aaaaaaaaaaaaaaaaagmi", "aaaaaaaaaaaaaaaaakgq", "aaaaaaaaaaaaaaaaaquu", "aaaaaaaaaaaaaaaaaolb", "aaaaaaaaaaaaaaaaafqv", "aaaaaaaaaaaaaaaaaani", "aaaaaaaaaaaaaaaaadwa", "aaaaaaaaaaaaaaaaacim", "aaaaaaaaaaaaaaaaacmb", "aaaaaaaaaaaaaaaaazat", "aaaaaaaaaaaaaaaaatlw", "aaaaaaaaaaaaaaaaawwj", "aaaaaaaaaaaaaaaaadpv", "aaaaaaaaaaaaaaaaajxg", "aaaaaaaaaaaaaaaaaldt", "aaaaaaaaaaaaaaaaakfi", "aaaaaaaaaaaaaaaaajyr", "aaaaaaaaaaaaaaaaajln", "aaaaaaaaaaaaaaaaakvu", "aaaaaaaaaaaaaaaaaqsq", "aaaaaaaaaaaaaaaaabgq", "aaaaaaaaaaaaaaaaallw", "aaaaaaaaaaaaaaaaanbp", "aaaaaaaaaaaaaaaaaniy", "aaaaaaaaaaaaaaaaanix", "aaaaaaaaaaaaaaaaasfw", "aaaaaaaaaaaaaaaaaxau", "aaaaaaaaaaaaaaaaaykg", "aaaaaaaaaaaaaaaaardu", "aaaaaaaaaaaaaaaaabfk", "aaaaaaaaaaaaaaaaasht", "aaaaaaaaaaaaaaaaagqu", "aaaaaaaaaaaaaaaaapjj", "aaaaaaaaaaaaaaaaaodb", "aaaaaaaaaaaaaaaaablv", "aaaaaaaaaaaaaaaaakoa"]) == "50/50/50")
    print(f(["aaaaaaaaaaaaaaaaavqq", "aaaaaaaaaaaaaaaaadzl", "aaaaaaaaaaaaaaaaacpk", "aaaaaaaaaaaaaaaaabgt", "aaaaaaaaaaaaaaaaamsn", "aaaaaaaaaaaaaaaaaoid", "aaaaaaaaaaaaaaaaabjs", "aaaaaaaaaaaaaaaaaonl", "aaaaaaaaaaaaaaaaaoon", "aaaaaaaaaaaaaaaaabvu", "aaaaaaaaaaaaaaaaayde", "aaaaaaaaaaaaaaaaabmv", "aaaaaaaaaaaaaaaaachi", "aaaaaaaaaaaaaaaaacwz", "aaaaaaaaaaaaaaaaadjl", "aaaaaaaaaaaaaaaaacfk", "aaaaaaaaaaaaaaaaavqf", "aaaaaaaaaaaaaaaaaozf", "aaaaaaaaaaaaaaaaaiiy", "aaaaaaaaaaaaaaaaaqhv", "aaaaaaaaaaaaaaaaakfj", "aaaaaaaaaaaaaaaaahzr", "aaaaaaaaaaaaaaaaafwv", "aaaaaaaaaaaaaaaaaqrq", "aaaaaaaaaaaaaaaaavfj", "aaaaaaaaaaaaaaaaapsr", "aaaaaaaaaaaaaaaaavmi", "aaaaaaaaaaaaaaaaahgo", "aaaaaaaaaaaaaaaaamcw", "aaaaaaaaaaaaaaaaapyy", "aaaaaaaaaaaaaaaaabcq", "aaaaaaaaaaaaaaaaatnj", "aaaaaaaaaaaaaaaaaklk", "aaaaaaaaaaaaaaaaaalk", "aaaaaaaaaaaaaaaaaosf", "aaaaaaaaaaaaaaaaatzb", "aaaaaaaaaaaaaaaaaktm", "aaaaaaaaaaaaaaaaacpx", "aaaaaaaaaaaaaaaaamma", "aaaaaaaaaaaaaaaaapjv", "aaaaaaaaaaaaaaaaawma", "aaaaaaaaaaaaaaaaawer", "aaaaaaaaaaaaaaaaaxbn", "aaaaaaaaaaaaaaaaawtc", "aaaaaaaaaaaaaaaaanet", "aaaaaaaaaaaaaaaaamqo", "aaaaaaaaaaaaaaaaaivv", "aaaaaaaaaaaaaaaaaido", "aaaaaaaaaaaaaaaaamva", "aaaaaaaaaaaaaaaaaqck"], ["aaaaaaaaaaaaaaaaapsr", "aaaaaaaaaaaaaaaaacpk", "aaaaaaaaaaaaaaaaaklk", "aaaaaaaaaaaaaaaaacfk", "aaaaaaaaaaaaaaaaacpx", "aaaaaaaaaaaaaaaaavmi", "aaaaaaaaaaaaaaaaaoid", "aaaaaaaaaaaaaaaaaiiy", "aaaaaaaaaaaaaaaaaktm", "aaaaaaaaaaaaaaaaadzl", "aaaaaaaaaaaaaaaaachi", "aaaaaaaaaaaaaaaaawer", "aaaaaaaaaaaaaaaaayde", "aaaaaaaaaaaaaaaaafwv", "aaaaaaaaaaaaaaaaabcq", "aaaaaaaaaaaaaaaaaivv", "aaaaaaaaaaaaaaaaahgo", "aaaaaaaaaaaaaaaaamqo", "aaaaaaaaaaaaaaaaakfj", "aaaaaaaaaaaaaaaaacwz", "aaaaaaaaaaaaaaaaabjs", "aaaaaaaaaaaaaaaaawtc", "aaaaaaaaaaaaaaaaavqq", "aaaaaaaaaaaaaaaaapyy", "aaaaaaaaaaaaaaaaaido", "aaaaaaaaaaaaaaaaanet", "aaaaaaaaaaaaaaaaapjv", "aaaaaaaaaaaaaaaaaqck", "aaaaaaaaaaaaaaaaavqf", "aaaaaaaaaaaaaaaaadjl", "aaaaaaaaaaaaaaaaaqhv", "aaaaaaaaaaaaaaaaamma", "aaaaaaaaaaaaaaaaatzb", "aaaaaaaaaaaaaaaaaqrq", "aaaaaaaaaaaaaaaaaonl", "aaaaaaaaaaaaaaaaaalk", "aaaaaaaaaaaaaaaaawma", "aaaaaaaaaaaaaaaaaozf", "aaaaaaaaaaaaaaaaamsn", "aaaaaaaaaaaaaaaaabvu", "aaaaaaaaaaaaaaaaamcw", "aaaaaaaaaaaaaaaaahzr", "aaaaaaaaaaaaaaaaaosf", "aaaaaaaaaaaaaaaaamva", "aaaaaaaaaaaaaaaaavfj", "aaaaaaaaaaaaaaaaaxbn", "aaaaaaaaaaaaaaaaaoon", "aaaaaaaaaaaaaaaaatnj", "aaaaaaaaaaaaaaaaabmv", "aaaaaaaaaaaaaaaaabgt"], ["aaaaaaaaaaaaaaaaakfj", "aaaaaaaaaaaaaaaaamva", "aaaaaaaaaaaaaaaaadzl", "aaaaaaaaaaaaaaaaawtc", "aaaaaaaaaaaaaaaaabjs", "aaaaaaaaaaaaaaaaacwz", "aaaaaaaaaaaaaaaaapjv", "aaaaaaaaaaaaaaaaapyy", "aaaaaaaaaaaaaaaaamcw", "aaaaaaaaaaaaaaaaatzb", "aaaaaaaaaaaaaaaaafwv", "aaaaaaaaaaaaaaaaamqo", "aaaaaaaaaaaaaaaaacpk", "aaaaaaaaaaaaaaaaamsn", "aaaaaaaaaaaaaaaaahzr", "aaaaaaaaaaaaaaaaavmi", "aaaaaaaaaaaaaaaaawma", "aaaaaaaaaaaaaaaaaozf", "aaaaaaaaaaaaaaaaaqrq", "aaaaaaaaaaaaaaaaabgt", "aaaaaaaaaaaaaaaaanet", "aaaaaaaaaaaaaaaaaktm", "aaaaaaaaaaaaaaaaaxbn", "aaaaaaaaaaaaaaaaabvu", "aaaaaaaaaaaaaaaaawer", "aaaaaaaaaaaaaaaaacfk", "aaaaaaaaaaaaaaaaaqck", "aaaaaaaaaaaaaaaaaosf", "aaaaaaaaaaaaaaaaaonl", "aaaaaaaaaaaaaaaaadjl", "aaaaaaaaaaaaaaaaaklk", "aaaaaaaaaaaaaaaaaqhv", "aaaaaaaaaaaaaaaaavfj", "aaaaaaaaaaaaaaaaachi", "aaaaaaaaaaaaaaaaatnj", "aaaaaaaaaaaaaaaaaoid", "aaaaaaaaaaaaaaaaavqq", "aaaaaaaaaaaaaaaaahgo", "aaaaaaaaaaaaaaaaaiiy", "aaaaaaaaaaaaaaaaaido", "aaaaaaaaaaaaaaaaaalk", "aaaaaaaaaaaaaaaaaivv", "aaaaaaaaaaaaaaaaapsr", "aaaaaaaaaaaaaaaaaoon", "aaaaaaaaaaaaaaaaayde", "aaaaaaaaaaaaaaaaacpx", "aaaaaaaaaaaaaaaaabmv", "aaaaaaaaaaaaaaaaavqf", "aaaaaaaaaaaaaaaaabcq", "aaaaaaaaaaaaaaaaamma"]) == "50/50/50")
    print(f(["bcdbb", "aaccd", "dacbc", "bcbda", "cdedc", "bbaaa", "aecae"], ["bcdbb", "ddacb", "aaccd", "adcab", "edbee", "aecae", "bcbda"], ["dcaab", "aadbe", "bbaaa", "ebeec", "eaecb", "bcbba", "aecae", "adcab", "bcbda"]) == "14/14/21")
    print(f(["sswezzfbzouxthsqiwbf", "nqhnrdcsxawcfk", "akifvg", "wmsayjpobdk", "gisfbwwafmfquwds", "nucckksdnwesvp", "hkjhshdcpacna", "exqkb", "jqexrtpmsdwgzgvncuk"], ["shvzfwmgrfhnyplfe", "hdkdulp", "azztedbwwygrj", "qdzjwfjwrvvggagny", "ysjtudbsysfbera", "qnyensjztjgoqz", "jqknyxpdpiymftcv", "nemeoefugelc", "hclyojwgugsaqe", "teqzfvlmk", "bwbovtoylcjm", "nqhnrdcsxawcfk", "jzlkquethyer", "qvkyna", "nucckksdnwesvp", "uek", "mdrn", "xqmtxp", "bcgpcegwqycakeauf", "efoldl", "hfgrxhkmslvuhcq"], ["dpijzgmlzj", "xqmtxp", "knmkapqhuwayrluykqy", "upflqzzwxantuj", "zuf", "ebuzxtlaua", "ptwsefmhjlnp", "jrjy", "dfgrpp", "uiblwgbesshpsltm", "nxkiq", "qdzjwfjwrvvggagny", "qvkyna"]) == "25/58/36")
    print(f(["cat", "dog", "pig", "mouse"], ["cat", "pig"], ["dog", "cat"]) == "8/3/3")
    print(f(["ca", "cab"], ["cat"], ["cat"]) == "6/2/2")
    print(f(["q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l", "z", "x", "c", "v", "b", "n", "qq", "ww", "ee", "rr", "tt", "yy", "uu", "ii", "oo", "pp", "aa", "ss", "dd", "ff", "gg", "hh", "jj", "kk", "ll", "zz", "xx", "cc", "vv", "bb", "nn"], ["asdf"], ["qwer"]) == "150/3/3")

#### huffmandecoding: https://community.topcoder.com/stat?c=problem_statement&pm=6477

In [ ]:
class HuffmanDecoing(object):

    def decode(self, archive, dictionary):
        """Decode archive using dictionary and HuffmanDecoding method."""
        alphabet = string.ascii_uppercase
        ret = ""
        ix = list(range(len(dictionary)))
        jx = 0
        while archive:
            for i in ix:
                code = dictionary[i]
                if archive.startswith(code):
                    ret += alphabet[i]
                    archive = archive[len(code):]

        return ret
if True:
    f = HuffmanDecoing().decode
    print(f("101101", ["00", "10", "01", "11"]) == "BDC")
    print(f("10111010", ["0", "111", "10"]) == "CBAC")
    print(f("0001001100100111001", ["1", "0"]) == "BBBABBAABBABBAAABBA")
    print(f("111011011000100110", ["010", "00", "0110", "0111", "11", "100", "101"]) == "EGGFAC")
    print(f("001101100101100110111101011001011001010", ["110", "011", "10", "0011", "00011", "111", "00010", "0010", "010", "0000"]) == "DBHABBACAIAIC")
    print(f("00000000000000000000000000000000000000000000000000", ["00000000000000000000000000000000000000000000000000", "11111111111111111111111111111111111111111111111111"]) == "A")
    print(f("00000000000000000000000000000000000000000000000000", ["0", "11111111111111111111111111111111111111111111111111"]) == "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    print(f("00000000000000000000000000000000000000000000000000", ["0"]) == "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    print(f("11111111111111111111111111111111111111111111111111", ["1"]) == "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    print(f("10101010101010101010101010101010101010101010101010", ["1", "0"]) == "ABABABABABABABABABABABABABABABABABABABABABABABABAB")
    print(f("0011101011111011001011001111001000111101011111111", ["010", "1110", "1101", "000", "0111", "1111", "0110", "001", "1100", "10"]) == "HCECJAIFHDFAFF")
    print(f("010011010010001010001001000100100100110110101000", ["1", "000", "010", "011", "001"]) == "CDCCECEEBAEEEADCAB")
    print(f("0110000011111100010110011110110011110111011101101", ["100", "111", "110", "01", "00", "101"]) == "DAEDBCEFABFABFCBDF")
    print(f("0100110000111000010000000100000101010101000001001", ["0001", "0101", "11", "011", "10", "0000", "0100", "001"]) == "GCFCEAFAFBBGAH")
    print(f("1110101100010011001100000111110011010110110001010", ["1101", "00", "011", "1100", "10", "111", "010"]) == "FGDGCBDBCFBACCBGE")
    print(f("100010011110000100111110010101000110100", ["101", "100", "11", "0"]) == "BDBCCDDDDBCCBADBDCDB")
    print(f("000110001110110100110001101001001110110100101110", ["0111", "01100", "10", "1110", "010", "110", "011010", "1111", "00", "011011"]) == "IBAGBGEAGED")
    print(f("101001110100000110001111111011111100110011001000", ["1010", "1110", "01", "000", "1000", "110", "1011", "1111", "1001", "001"]) == "ACFEJEHBHFCIID")
    print(f("1001101001110001000101101010001011000100111001000", ["1001", "11", "1000", "011", "10110", "00", "1010", "010", "10111"]) == "AGDCCEGFEFABFC")
    print(f("0110011101100111011001100110011001100110010011111", ["0111", "1", "00", "0110", "010"]) == "DADADDDDDDEABB")
    print(f("00001010100101001011", ["0", "1"]) == "AAAABABABAABABAABABB")
    print(f("0001001011001000000110011010100011010100100001001", ["110", "0000", "100", "0110", "0001", "101", "0111", "001", "010", "111"]) == "EHDIBDDFEFIIEH")
    print(f("1010110101110010111010110001111110001010110110", ["00", "111", "01", "110", "10"]) == "EEDEBAEBCCEABBACCCED")
    print(f("0011010011100000001000010010010111110010011011100", ["101", "0000", "1101", "100", "0001", "111", "01", "001", "1100"]) == "HAHIBGBDDAFDDCI")
    print(f("110110011101110011101100001011110110011110001010", ["110", "011", "00", "111", "01010", "01011", "0100", "100", "101"]) == "AABIABIHCIDBCDHE")
    print(f("110011100111011101110011100101100110011011111000", ["0", "11000", "1101", "111", "101", "100", "11001"]) == "GGCCGGAGFCDFA")
    print(f("111111101011010101011110101101001011", ["0", "11", "10"]) == "BBBCCBACCCBBACBACACB")
    print(f("000010001111000000110001000110101", ["1", "00", "01"]) == "BBABCAAABBBAABCBCACC")
    print(f("1110010110111101011000111101011110111101010111100", ["00", "010", "1100", "1110", "011", "1111", "10", "11011", "11010"]) == "DBHICEIFEIGFA")
    print(f("0011001100110100001100100110101000001000110010011", ["0101", "011", "0000", "0100", "0001", "1", "0010", "0011"]) == "HHHDHGBACDBGB")
    print(f("0011101111010010110011010100110000011010110101100", ["11010110101", "111", "110100", "110000100", "0000", "100", "1100011", "110011", "1101011011", "1100000", "1100010", "11010111", "11010100", "110101011", "001", "110101010", "11011", "110101100", "0001", "01", "101", "110010", "11010110100", "11000011", "110000101"]) == "OQCUFMJAF")
    print(f("0011101100010110111100100100101011011100101100110", ["01011", "001001", "0010100", "00110", "001000", "111", "001010100", "00101100", "00111", "01010", "0010111", "10", "00101101", "0110", "01001", "00101011", "000", "001010101", "110", "0111", "01000"]) == "INMFBPTHS")
    print(f("101101001010001000111011100101011100010111000010", ["101110011", "010", "11", "100110", "100001", "10110", "001", "011", "101110010", "1000000", "10111000", "000", "101111", "1000101", "10010", "100011", "100111", "1000100", "10111011", "1000001", "10111010", "1010"]) == "FORHIKKB")
    print(f("0101111101010111001010001000110100100001010101111", ["0011001", "0110", "11", "0010", "0011000", "01010", "01011001", "0111", "001101", "01000", "001110", "01011000", "100", "0101101", "0101111", "000", "101", "001111", "0101110", "01001"]) == "OQSFDIDPQO")
    print(f("110010100110010110010011110111000101010010101001", ["101000", "101001", "1100", "1000", "1111", "0011", "0001", "0000101", "1110", "010", "0010", "1101", "10010", "101010", "01100", "100111", "100110", "00000", "1011", "01101", "0000100", "000011", "0111", "101011"]) == "CBMCPSDNJB")
    print(f("1101000011100001100100011001111011101110111011011", ["001", "10101", "110101", "10100", "10110", "01", "1000", "000", "10111101", "1101000", "10011", "10111100", "110001", "1100101", "1111", "10010", "101110", "110000", "10111110", "1101001", "11011", "1100100", "1110", "110011", "10111111"]) == "JFRVFKUQWU")
    print(f("1111001000011100001011000101000010111111010011011", ["1000011", "1011", "111010", "100010", "1110110", "100000", "00", "10000101", "111100", "111111", "1010", "111110", "111101", "1110111", "111001", "111000", "110", "01", "1001", "10000100", "100011"]) == "IAHDHLSB")
    print(f("1010100111101111011000110110101101100011011111100", ["101101", "11101", "10001100", "10111", "1000110110", "10001110", "111101", "111100", "10011011", "10011010", "11111", "100011010", "101100", "1001111", "10001111", "1000110111", "10010", "1010", "10000", "1001110", "0", "110", "1001100", "100010", "11100"]) == "RNUGEAPY")
    print(f("111111111101011111111110001100110110110001000111", ["0111", "1011", "1110", "0100", "11010", "1001", "00", "111100", "1111010", "0101", "011010", "110111", "1100", "1000", "111110", "01100", "011011", "1010", "1111011", "111111", "110110"]) == "TITHPUMDA")
    print(f("0100010101110001010100000101010000111000101010101", ["0010101111", "00100", "00000", "0111", "0011", "0110", "0001", "00001110", "000010", "0010101110", "001011", "001010110", "000011111", "100", "0010100", "0000110", "010", "00101010", "11", "101", "000011110"]) == "QJRITHRT")
    print(f("10111101100110001100111011011100011100001101001", ["1101111", "011", "111010", "1011", "0011", "1101001", "1111", "000", "110001", "110010", "1101100", "1001", "11100", "1101000", "0010", "10001", "1010", "110011", "10000", "1101101", "010", "111011", "110000", "1101110", "110101"]) == "DKILTIWF")
    print(f("0000100000100100000100110000010010000101000110111", ["0001001000", "0001001001", "0000011", "0100", "0101", "00011", "0000010", "000000", "0011", "0001001100", "000100111", "0001001101", "1101", "1100", "000010", "0001000", "0001001011", "0001001010", "0000111", "0111", "111", "0000110", "10", "000101", "0010", "0110"]) == "OODJAEFT")
    print(f("001110000011000011100100111000001010111010001001", ["110", "11101", "11110", "00010", "00111000", "00100", "001111", "00011", "00111001", "001100", "0011101", "00001", "001010", "11111", "00000", "001101", "01", "10", "001011", "11100"]) == "EJIEMBDQ")
    print(f("0000001110011111110010000110010000101001110101101", ["000000110", "101", "100", "0010110", "001010", "0111", "000001", "0001", "0110", "00100", "0101", "0000000", "0100", "00110", "00000010", "000011", "0010111", "000000111", "000010", "00111", "11"]) == "RTUUJNMEFKB")
    print(f("1011010001111100101101110001011001011000110001100", ["1110", "0", "110100", "110101", "110010", "1111", "100000", "1000111", "110001", "1000010", "1000110", "100010", "10110", "110011", "11011", "1000011", "1001", "1010", "110000", "10111"]) == "MHEOLEIKB")
    print(f("1011100100011101101110100011111011101101100010101", ["1010001", "101110000", "101110001", "1011000", "000", "10101", "011", "110", "1011111", "1011001", "101001", "111", "1000", "1011101", "010", "1011011", "1011100101", "1011010", "1010000", "1001", "1011100100", "1011110", "101110011", "001"]) == "UGPALNDF")
    print(f("011100000110100000110101100101110000111101000111", ["0001001", "0001000", "1000010", "1010", "0000111", "001", "00001100", "1000011", "10011", "000110", "1011", "000010", "000101", "000111", "011", "10001", "010", "1101", "10010", "000000", "000001", "1100", "00001101", "100000", "111"]) == "OXRUDVKHRN")
    print(f("1001101100001010101000011111101110100000100001000", ["110", "1000111", "100000", "10111", "100111", "10000110", "101101", "1011001", "1000101", "1001101", "10010", "1111", "1110", "1000110", "1011000", "1000100", "0", "1010", "1000010", "10000111", "1001100"]) == "JSRTMMCSQQ")
    print(f("110001110011110100011111110110011000110011001100", ["1111011", "00011", "0010", "0100", "1111000", "0000", "011000", "001110", "0011111", "1111001", "0011110", "101", "110", "11100", "1111010", "01111", "0101", "01101", "011001", "01110", "111010", "11111", "00010", "100", "00110", "111011"]) == "MHPDPZYYSX")
    print(f("01001111010010010011001000001010001101001000010", ["01101", "01110", "01001110", "0100110", "00010", "01000", "0101", "0000", "001001", "111", "010011111", "1010", "100", "0100111101", "00101", "01100", "00011", "010010", "1011", "0011", "1101", "0100111100", "01111", "001000", "1100"]) == "NITXOQRE")
    print(f("11111111111111111111111111111111111111111111111111", ["0", "1"]) == "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB")
    print(f("0001001100100111001", ["1", "0"]) == "BBBABBAABBABBAAABBA")
    print(f("0000100111001001111001", ["1", "0"]) == "BBBBABBAAABBABBAAAABBA")
    print(f("001101100101100110111101011001011001010", ["110", "011", "10", "0011", "00011", "111", "00010", "0010", "010", "0000"]) == "DBHABBACAIAIC")
    print(f("000100110010011", ["1", "0"]) == "BBBABBAABBABBAA")

#### bombsweeper: https://community.topcoder.com/stat?c=problem_statement&pm=1778&rd=4585&rm=&cr=309786

Problem Statement for BombSweeper


Problem Statement
    	
The game of BombSweeper is a single-player game played on a rectangular grid. Each square in the grid is either a bomb (represented by 'B') or empty space (represented by '.'). The true identity of each square is hidden when the game begins. The object of the game is to correctly determine both the number of bombs on the board and their positions.

You are trying to write a program which predicts the percent likelihood that you will win a given game of BombSweeper. You've recently become so good at the game that your success or failure depends only on your first few moves. To start, you pick a random square on the gameboard and uncover it. If the square you uncover is a bomb, you lose the game. If the square is not a bomb, but one or more of its horizontal, vertical, and diagonal neighbors is, you are no better off than when you started and you must take another turn. If the square is not a bomb, and none of its (up to) eight neighbors are bombs either, then you win the game.

Given a String[] board, representing the game board, return a number between 0.0 and 100.0, inclusive, representing the percent likelihood of you winning the game.

 
Definition
    	
Class:	BombSweeper
Method:	winPercentage
Parameters:	String[]
Returns:	double
Method signature:	double winPercentage(String[] board)
(be sure your method is public)
    
 
Notes
-	In calculating the probability, let wins be the number of non-bomb squares which have no bomb neighbors, and let losses be the number of bombs on the board. The odds of winning are then (wins / (wins + losses)), which returns a number between 0.0 and 1.0, inclusive.
-	As long as you calculate wins and losses correctly, you need not worry about minor double imprecisions.
-	Your solution need only be accurate to 1e-9 (relative or absolute). Thus, if your result is within 1e-9 (relative or absolute) of the result shown, your result will be judged correct.
 
Constraints
-	board will contain between 1 and 50 elements, inclusive.
-	Each element of board will contain between 1 and 50 characters, inclusive.
-	Each element of board will contain the same number of characters.
-	Each character in board will be either '.' or 'B'.
 
Examples
0)	
    	
{".....",
 ".....",
 "..B..",
 ".....",
 "....."}
Returns: 94.11764705882354
If you uncover the bomb in the center of the gameboard, you lose. If you uncover one of the eight squares surrounding the bomb, you would need to take another turn. If you uncover one of the 16 squares on the perimeter of the gameboard, you would win, since none of these squares has a neighboring square which is a bomb. Since there are 16 ways to win and only 1 way to die, your odds of winning are 16/17, or about 94.1%.

1)	
    	
{"BBBBB",
 "B...B",
 "B...B",
 "B...B",
 "BBBBB"}
Returns: 5.882352941176471
As far as your odds of winning are concerned, this board is the opposite of the previous one. The only way to win is to uncover the square in the exact center of the gameboard, while any of the 16 bombs on the perimeter will cause you to lose. 1/17 is about 0.0588, or about 5.88%.

2)	
    	
{".........",
 ".B..B..B.",
 ".........",
 ".........",
 ".B..B..B.",
 ".........",
 ".........",
 ".B..B..B.",
 "........."}
Returns: 0.0
Every square on this board is either a bomb, or has a bomb as a neighbor. Therefore, there is no way to win.

3)	
    	
{".........................",
 ".........................",
 ".........................",
 "........................."}
Returns: 100.0
Now there are no bombs, so any square you uncover will cause you to win.

4)	
    	
{"......B.......B..B...........................B....",
 "..............B..................BB..B............",
 "B.B.B.............B.....B..............B..........",
 "...................B...B....................BB....",
 "...B.....B.........................B.......B.....B",
 "B.B.........B.....B.......B..B......B.B...B.BB....",
 "..B...................BB...............B..........",
 ".........B...B................B..B................",
 ".......BB.......B....B................B.....BBB...",
 ".......BB..........B..............B......BB.......",
 "...................BB.....................B.......",
 "...B.B.B......B..............B...B......B.........",
 "B................B................................",
 "....B..........B.....B..BB....B...............BB..",
 "..B....B.....B.............B.....B............B...",
 "...................B.B........B..B.........B.B....",
 ".....B.....B......................................",
 "...........BB......BB...B.B........B...B..........",
 ".....BBB..........................................",
 ".B...........B....B...BB......B......B...B.B......",
 "..................B........BB................B....",
 "...............................B..B....BBB.B....B.",
 "..........B.......................................",
 ".....B..........B....BB......B.B......B......B....",
 ".....B..................B........B................",
 "............B.....B..B....BB...B....BB........B...",
 "..B.................B.........B...................",
 ".BB..............B................................",
 "...B....B..................B.................B....",
 "......B...B......B......................B.B.......",
 "..............B..................B.......B........",
 ".....B........BB...B.....B........................",
 ".......B......B.B..B..........B...........B....B..",
 "B...B...........B...B...B......B.B...B..B......B..",
 "....B..B.....B.B.......BB..B............B.B....B..",
 "B.......B..........B.........B...B.BB......B......",
 "....B...............................B.............",
 ".....B.B..........................................",
 "..........B............B......B.B..B....B.........",
 "....B...B.......................B.................",
 "B.................B...........B..B....B...........",
 "...B.....B........................................",
 "...B..B......................................BBB..",
 ".B...B....................................B....B..",
 "...B...B..........B...B.B.........................",
 ".....B.............B...BB..........B..BBB.BB......",
 "....................B.....B.......................",
 "........B..BB..........B.B....B...........B......B",
 ".........B.....BB..B.............B....BB..........",
 "....B..B..............B...B..B..........B........."}
Returns: 77.67558528428094
This board has 267 bombs, 1304 empty spaces with bomb neighbors, and 929 empty spaces without bomb neighbors. (929 / (929 + 267)) is about 0.7768.

In [ ]:
class BombSweeper(object):

    def winPercentage(self, board):
        """Return the likelihood of winning the game."""
        l, w = 0, 0
        rows = len(board)
        cols = len(board[0])
        for r in range(rows):
            for c in range(cols):
                if board[r][c] == "B":
                    l += 1
                    continue
                sr = 0 if r - 1 < 0 else r - 1
                sc = 0 if c - 1 < 0 else c - 1
                values = "".join([e[sc: c + 2] for e in board[sr:r + 2]])
                if "B" not in values: w += 1

        return (w / (w + l)) * 100
if True:
    f = BombSweeper().winPercentage
    print(math.isclose(f([".....", ".....", "..B..", ".....", "....."]), 94.11764705882354))
    print(math.isclose(f(["BBBBB", "B...B", "B...B", "B...B", "BBBBB"]), 5.882352941176471))
    print(math.isclose(f([".........", ".B..B..B.", ".........", ".........", ".B..B..B.", ".........", ".........", ".B..B..B.", "........."]), 0.0))
    print(math.isclose(f([".........................", ".........................", ".........................", "........................."]), 100.0))
    print(math.isclose(f(["......B.......B..B...........................B....", "..............B..................BB..B............", "B.B.B.............B.....B..............B..........", "...................B...B....................BB....", "...B.....B.........................B.......B.....B", "B.B.........B.....B.......B..B......B.B...B.BB....", "..B...................BB...............B..........", ".........B...B................B..B................", ".......BB.......B....B................B.....BBB...", ".......BB..........B..............B......BB.......", "...................BB.....................B.......", "...B.B.B......B..............B...B......B.........", "B................B................................", "....B..........B.....B..BB....B...............BB..", "..B....B.....B.............B.....B............B...", "...................B.B........B..B.........B.B....", ".....B.....B......................................", "...........BB......BB...B.B........B...B..........", ".....BBB..........................................", ".B...........B....B...BB......B......B...B.B......", "..................B........BB................B....", "...............................B..B....BBB.B....B.", "..........B.......................................", ".....B..........B....BB......B.B......B......B....", ".....B..................B........B................", "............B.....B..B....BB...B....BB........B...", "..B.................B.........B...................", ".BB..............B................................", "...B....B..................B.................B....", "......B...B......B......................B.B.......", "..............B..................B.......B........", ".....B........BB...B.....B........................", ".......B......B.B..B..........B...........B....B..", "B...B...........B...B...B......B.B...B..B......B..", "....B..B.....B.B.......BB..B............B.B....B..", "B.......B..........B.........B...B.BB......B......", "....B...............................B.............", ".....B.B..........................................", "..........B............B......B.B..B....B.........", "....B...B.......................B.................", "B.................B...........B..B....B...........", "...B.....B........................................", "...B..B......................................BBB..", ".B...B....................................B....B..", "...B...B..........B...B.B.........................", ".....B.............B...BB..........B..BBB.BB......", "....................B.....B.......................", "........B..BB..........B.B....B...........B......B", ".........B.....BB..B.............B....BB..........", "....B..B..............B...B..B..........B........."]), 77.67558528428094))
    print(math.isclose(f(["B.................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", "..................................................", ".................................................."]), 99.9599519423308))
    print(math.isclose(f(["..BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "..BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB"]), 0.040048057669203045))
    print(math.isclose(f(["..B...B.BB.B..B...B.......B.........B.B..B...", ".......B...B..B...B..B.B..B.B..B.......B.....", "...B.B..B.B.B.B.....B.....B...B......BBB.....", ".B....B....B..........B.BB.....B.BB....B..B.B", "B...BB..BBBB....B....B..B.B....B.....BB......", "B.....BBB.B.B...BB.B.B....B..B.B.B....B.B.BB.", "B...B.BB......B....B........B.B.B.BB.B....B..", "...B......B...B...B.BB...B....B..B....BBB....", ".........BB.....BB..B.BB.B..BBBB...B.....B..B", "..BBBB......BBB...BB............B.BB...B.B...", "...B...BB..B...B.....B.B..B..B...B.......B...", "...B..B....B...B.....B..BB.B....B..B....B....", ".B.....B......B.B...B.......BBB.B.....B.BBB..", "........B....B.B....B.B.BB.......B....B...B..", ".B............B.BBBB.B..BB.BB....B.........B.", "...B......B....B.......BB.B.B.B..............", "B.B............BB......B.B..B.........BBB..B.", "..BB...B....B...B..B........B.....B.B..B.....", "..B...BB......B.B.B....B....B.BB...B.....B...", "...BB......B............B...BB..B.B.....B..B.", "...B.B....BBB.....B...BB...B....BB.B.......B.", "B.B.B...BB.B.B...B..BBB........B......B..B...", "B..BBBBB..B..B......B.......B...B............", "....B..B.....B.B..B..B..B..B.B.......B.B.....", "B.....B.B.....BB................BB....B..B.B.", "....BB.BBB.B.B.B..........B.B..B..B..B..B....", "..B.BB.......B.....B..B....B..B..BB.B...B..BB", "......B..BB..B...B........B.B..B..B...BBB...B", ".....B.....BBB......B...........B.B.BB......B", "....B.....B...B.B.B.B.B.....B...B.B.B...BB...", ".B.B....BBB....................B......B..B...", "..B..BB.BBBBB..B.BBBB..BB........BBB.BB......", "B...B..B....B.................B.B........B.B.", ".....B..........B....B......BBB.B....B..BB..B", "B.B.BB.B.B....BB............B.....BBB.B.....B", "B...B......B...B..B..B.B...B...B.....B.......", "......BB......B.B...B......BB......B.B..B.B.B", "BB..B........B.......BB.B................BBBB", "BB......B.B.....B.B..........BB......B..B....", "B...B..B..BB.BB.B......B.B.B.B...BB...B...B..", "B...BBB.BB.B......BB..B.BB.............B....B"]), 18.12191103789127))
    print(math.isclose(f(["BBBB.B.BB..B....BB.B.BB..B.B....B..B"]), 19.047619047619047))
    print(math.isclose(f([".B.BB.....B.B..B..B..B..B..B.BB.....B.B.B..B.BB.B"]), 24.0))
    print(math.isclose(f(["BBBBB.BBBB...BBBBB.B.BBBB.BB", "B..BBBB.BBB.BBB.B..B.B.BB.BB"]), 0.0))
    print(math.isclose(f(["..B..B..B..B...B.", "....BBB.........."]), 33.333333333333336))
    print(math.isclose(f(["B", "B", "B", "B", "B", ".", "B", "B", ".", ".", "B", ".", "B", "B", "B", "B", "B", "B", "B", "B", "B", "B", "B", "B", "B", "B", "."]), 0.0))
    print(math.isclose(f(["B", "B", "B", ".", ".", "B", ".", "B", ".", "B", ".", ".", ".", "B", "B", "B", ".", ".", ".", "B", ".", ".", ".", "B", ".", "B", ".", "B", ".", ".", ".", "B", ".", ".", ".", "B", "B", ".", "B", ".", ".", "."]), 29.166666666666668))
    print(math.isclose(f(["..", "B.", "BB", "..", "BB", "..", ".B", "B.", ".B", "BB", "..", ".B", ".B", ".B", ".B", "BB", "..", "B.", "BB", ".B", "BB", "BB", "..", ".B", "BB"]), 0.0))
    print(math.isclose(f(["BB", "BB", ".B", "B.", "BB", "..", "..", "BB", "B.", ".B", "BB", "BB", "B.", "B.", "..", "B.", "B.", ".B", "BB", "..", "..", "BB", ".B", ".B", "BB", "..", "BB", ".B", "B.", "B.", ".B", "BB", ".B", "BB", "..", "..", "B.", "BB", ".B", ".B", ".B", "B.", "BB", "BB"]), 0.0))
    print(math.isclose(f(["B.......B..B...B.B.....B.B.B...B.......BB.", "BB...........BBBB........B...BBBBB........", "..B.B..B..B.B.B...............B.BB.B.B.B.B", ".......B..B..B......B....B....B...B..B...B", "BBBBB..B..BB..B...B..B.B......B......BB.B.", "B......B.BB..B....B..B......B.........B..B", "......B...B........BB...B.BB.......BB..BBB", "..B..B..BB.BB..B.....B..............B..BB.", ".B...B..B...BB.....B.....BB.........B.B...", "...B......B..B.B......BBB...B.B.....B.B.B.", "...B..........BB..B.B..........B.....B.BB.", "BB.B..B..B..B...B...B....B...BBB.....BB..B", "....B.....B...BBB..B......BBBB.......B....", "....B..B.B......B..B..BBBB...B....BB....B.", ".BB..BBBB.B...B....B....B.B..B.B...BB..B..", ".B........B.B..B.....BBB.B.BB.BBB...B.....", "....BB.......BB.....B.........B....B.BB...", "....BB......B....BB.........B.B....B.B.B..", "BB.BBBBBBBB........B......B..B..BB........", "B......BBB.BBB......B..B......B...........", "...B......BB.B..B..BBBB..B..B....BB....B..", "B....BBB.....B...B........BBB..BB.B...BB..", "....B...B..B.B....B...BB.B..BB..BB...B...B", "..B....B..............B.............B.....", "BBBB...B...........B...BB..B..........B.B.", ".........B.B.B.B.B..........B...B....B.BB.", ".BB..B.....B......B....BB....B.BB...B.B..B", "B...B..B.BB..B...B.B....B..BB..BB.BBB.B.B.", ".....BBBB.......B.......B..B......BB......", "B........B...B.B..BB...B...B..B.B......B..", ".....BBB....B..B.B..B..B....BB.B..B...BB..", "...BBBBB...B.B..B...BB..B......BBBB..B.B..", "...B...B.....BBBB.......B.B...B...BBB.BBB.", "....BB..B.B.B.B..BB......BB..B..........B.", ".B..B......B...B.BB..BB..B.BB.........BBB.", "........B..B...B...BBB...B.......B..B..B..", "..BB.B......B...BB...B..BB......B.B..B.B.B"]), 17.36745886654479))
    print(math.isclose(f(["BBBBBBB..BBBBBBBBBBBBB..BBBBB.BB.B.BBBBBBBBBB", ".BBBB.B.B.BBBBBBBBBBBBB.BB.BBBBBBBBBBBBBBBB.B", "BBBBBBBB.BBBBB....B.BBBBBBBBBBBBBBBBBB.BBBB.B", "BBBBBB.BBB.BBBB.BBBB.BBB.BBB.BBBBBBBBBBBBBB..", "BBBBBBBBBBBBB.BBBBBBB.BBBBBBBBBBB.BBBBB.BBBBB", "BBBBB...BB.BBBB.BB.BBBBBBBB.BBBBBB.BBBBB..BBB", "BBBBBBBBBBBB.BBBBBBBBBBBBBBB...B.B...BBBBBB.B", ".B..B..BBBBBBBBB.BB.BBBBB.BBBB.BBBB..BBBBBBBB", ".B.BBBB..B.BBBBBBBBB.BB.BB.B.B.BBB.BBBB.BBB.B", ".BB.B..BBBBBBBBBBBBBB.BBBBBBBBBBBB.BBB.B.BBBB", "BBBBBBB..BBB..BBBBBBB.BB.BBBB.BBB.BBBBBBB.B..", "BBBBBBBBBBBBBBBBB.BBBB.B.BB.BB..BBB.BB.BBB...", "..BBBBBBBBBB.BBB.BBBBBB.BBB.BBBB.B.BBBBBBBBBB", ".BBBBBBBBB.B..BBBB.BB.B.BBBB..BB.BBB..BBBBBBB", ".BBB..BBBB.BBBBBBBBB.BBBBBBBBBBBBBBBBB.BB.BBB", "BB.B.BBBBBB.B..BBB.BB.BB.BB.BBBBBB.B..BBBBBBB", "BBBB.BBBBBB.BBBB.BBBBBBBBB..BBBB.B.B..B.BBBBB", "BBB.BBB.BB.B...BBBBBBBBBB.BBBBBBB.BB.BB.BBBBB", "BBBBBBBBBBBBBBB.BBBBBBBBBB.BBBBBBBBBBBBBBBBB.", ".B.BBBB.BBBB.BBBB.B.B.BBBBBB.BBBBBBBBBBBB.B.B", "B.BBBBBBBBBB.BBBBBBB.BBBBBBBBBB.BBBBBBB.BB..B", "BBBBBBBBB.BBBB.B..BBBB.BBBBBBBBBBBBBBB..BB...", "BBBBBBBBBB.BBBBBBBBB.B.BBBBB.BBBBBB.B.BBBBBBB", "BBBBBBB.BBBBB.B.BBBBBBB.BBBB.BBBBBBBBB.B.BB.B", "BBB..BBBBBBBBBB.BBB.B.B.BBBB.B.BBBB.BBB.BBBBB", ".B.BBB.BBBBBBB..B.BBBB.BBBBBBBBBBBBBBBB.BBBB.", "..BBBB.BBBBBBBB.B.BBBB.B...BB.BB.BBB.B.BBBBBB", "BBB.B.BB..BBBB.BBBBBBBBBBB..BB.B.BBBBBBBBBBBB", "BB..BBB.BBBBBBBBBBBBBBBB.BBBB.BBBBBBBB.BBB.BB", "BBBBBBBBB.BBBBBBBBBBB..BBBBBBBBB..BBBBBB..BB.", "B.BBBB.BB.BBB.BBBB.BBBBB.B.BBBB.B.BBBBBBBBBB.", "B.BBBB...B.B.BB...BBBBB.B.BBBBB.BBBBBBBBB..BB", "BBBBBBBBBBBB.BB.B.BB.BBBBBBBBBB.B..BBBBBB.BBB", "BBBBBBBBBB..BBBB..B.BBBBBBBBBBBB.BBBBBBBBB.BB", "B...B.BBBBB.BBBBBBBBBBBB.BB.BBB.BBBBBBBBBBBBB", "BBBBBB.BB.BBBBBBBBBBBBBBBBBBBBBBBBBBB.BB..BBB", "B.BBBBBBBBBB..BBBBBBBBBB.BBBB.BB...BB...BB.BB", "BBBBBBBBBBBBBBBBBB.B..BBBBBBBBBBBBBBBBBBBBB.B", "BBBB.BBBB..BB.BB.BB.BBBB.BB.B.BBB.B.BB.BBBB.B", ".BB.BBBB.BBBBB.B.BBBB.BBB.BBBBBB.BB.BB.BBB.BB", ".BBB.BBB..BBB.B.BBBBBB.BBB.B.B.BB.BBBBBBBB.BB", "BB.B..BBBBBBBBBB..BBB..BBBBB.B.BBB.BB.BB.BB.B"]), 0.0))
    print(math.isclose(f([".B..........B.B....BB..B.BBB...B.BB...BB...B", "...B...B.BB....BB..B.B.B.B...B..B.....BBB..B", ".BB.BBB.BB...BB.B.BBB.B.B......B.B..........", "B..B.B.B.BB...B.BB.....B.B.B................", "B..BB..BB.......B.B......B.....B...........B", "B...B..B.B.B.B.B.B..B.BB.B...B.BB....B..BB.B", ".B.....B.BB...BB..B..B.B.BBB......B......B.B", ".BBB.B.B..BB...B.B..BB.BB.............BBBB.B", "...B........B.B.......B.......B....BB....B.B", ".B..BBBB...B..BB.....B..B..BB.B.....BB...B..", "B.B..B.B.B..B..B....BB.B..............B.....", ".B..BB..B..B.BB..B.......B.......B..BBBB...B", ".B..B......B.B...BBBB....B.B...BBBBB.......B", "B...BBB.....BBBBBB...B...B.....B.B..........", ".B.........B...B.B.BB.B.....BBBB.B..B....B..", ".B.BB.B.B..B....B.BBBB.B.BB..B.B......B...BB", "BB.....BB.....B..B....B...B..B.BB.....BBB.B.", "B.....B....BBBBBB......B..B....B.B.....BB.BB", "..BBB.BB.B.B.B.B..B....B.B..BBB...B.B..B....", "B.....B..B....BB..B...B.B...B.......B.....B.", ".BB..BB.BB..BB....B....BB.B.B.BB.....B....B.", "B............B.B.....B..BBBB.B....BBB..BBB.B", ".....B.....B...B..BB...BBBBBB..B.BB.........", ".....B..BBBB.B..BBB.B.B...B......B...BBB..BB", ".BB.B.BB.........B..B.B.BB.BB.....BBB..BB...", "...B.B.B....B.....B..B.........B.B..........", "..B.B..BBB...B...B..BBBB..B..BBB..B..B....B.", "B....B........BB.......B.B.....B..BB...B....", "B....B...B.B..B....BB.BB.B....BB...BBB...BB.", ".........BB....BB....B.BB....BBB.B.B..B..B..", ".B....BB...B....BB.BBBBB.B.....BB...B.B.B...", ".....BBB....B............B..B.......BB.B..B."]), 9.578544061302683))
    print(math.isclose(f([".BBBBB.B.BBBBBBBBB.B.BB.BBBB.B..BBBBB....B.BB.BBB", "BB.B.BB.BBB.BBBBBBB.BB.BBBBBBB.BBBBBBBBBBBBBB.BBB", "BBBBBB.BBBBB.BB.BBB.BBBBBBBBBBBBB.BB.B.BBB.BBB.BB", "BBBBB.BBBBBBBBBBBBBBBBBBBBBBB.BBBBBBB.BBBBB.B...B", "B.BBBBBBBBBB.BBBBBB.BBBBBBB.BBBBBBBBB..BBBBBBBBBB", "BBBBBBBBBBBBBBBBB.BBB..BB..BBBBBBBBBBBB.BBBB.BBBB", "BB.BBBBBBBBBBBBBBB.BBBBBBBBBBBBBBBBBBBB.BBBBBBBBB", "BB...BBBBBB.BBB.BBBBBBBBBBBBBBBBBBB.BBBBBB.BBB.BB", ".BBBBBBBBB.BBBBBB.BBBBBB.BBBBBBBBBBBBBBBB...BBBBB", "BBBBBBB.BBB.BBBBBB.BBB.BB.BBBBBBB.B.BB.BBBBBBBBBB", "BBBB.B.BBBB.BBB..BBBB..BBBBBBBBBBBBBBBBBBBBBBBBBB", "BBBBBBB.BB.BBBB.BBBBBB..BBBBBBBBBBBBB.BBBBBBB.BBB", "BBB.BBBBBBBBBBBBB.BBBBB.BB..BBBBBBBB.BBBBB.BBBBBB", ".B.BBBBBBB..BBBBBB.B.BBBBB..BBBBBBBBBBBBBBBBBBBB.", "BBBBBBB.B.BBBBBBBBBBBBBBBBBBBB.BB.BBBBB.BBBBBBBBB", ".BBBBBBBBBBBBB..BBBBBBBBB.BB.BBBBBBBBBBBBBBBBBB.B", "BBBBBBBBBBBB.B.BBB.B.BBBB.BBBB.BBBBBBBBBBBBB.BBBB", "BBBBBB...B.BBBBBBBBBBBBBBB.BBBBB.BBBBBBBBBB.B.BBB", "B.BBBBBBBBB.BB.BB.BBBBBBBB.BBBBBBB.BBB.BBBBBBBBBB", "BBBBBBBB.B.BBBBBBBBBBBB..B.BBBBBBBBBBB.BBBBB.B..B", "BBBBBB.BBBB.BB.BBBBBBB.BBBB.BBBBBBBB.BBBBBBBBBBBB", "BB.BB.BBBBBBBBBBB.BBBBBBB.BBBBBBBBBBBBBBBBBBBB.BB", "..BBB.B.BBBBBBBBBBBBBBBBBBBBBBBB..BBBB.B.B.BBB.B.", "BBBBBBBBBBBBB.BBBB.BBB.B.BBBBBBBBBBBBBB..BBBBBBBB", ".BBB.BBBBBBBBBBBBB.BBBBB.BBB.BBBB.BBB.BBBBBBBB..B", "BB.BB.BBB.BBBBBB.BBBBBBB..BBBBBB..B.BBBBBBBBBBBB.", "BB.BBB.BBBBBBB.BBBB.BBBBBBBBBBBBBBBBBBBBBBBBB.BBB", "B.BBBB.BBBBBB..BBBBBBB.BBBBBBBBBBBB..B.B.BB.B.BBB", ".B.BBBBBBBBBBBBBBBBBBBB.BBBBBBBBBBBBBBBBBBBBBB.BB", "BB.BBBBBBBBBBBBBBBB.BBBBBBBB.BBBBBBBB..BBBBBBBBBB", "BB.BBBBBBBBBBBBBB.BBBBBBBBBBBBBBBB.B.B.BBB..B.BB.", ".BB.BB..B.BBBBBBBBBBBB.BBBBB.B..BB..BBBBBB.BB.BBB", "BBBB..BBBBBBBBB..BBBBBBBBBBBBBB.BB.BBBBBBBBBBBBBB", "BBBBB.BBBBBBBBBBBBBBBBBBBBBB..BBBBB.BBB.B.BBBBBBB", "BB.BBBBBBB.B..BBBBBBBBBBBBBBB.BBB.BBBBBBBBB.BBBBB", "BBBBBBB.BB.BBBB..BBBBBBBBBBBBBBBBBBBBBB.BBBBBBB.B", "BBB.BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB..BBB.B.", "BBBBBBBB.BBBBBBBBBBBBB..BBBBBBBB.BBBBBBBBBBBBBBBB", "BBBBBBBBBBBB.BB.BB..BBBBBBBBBBBBBBB.BBBBB.BBB.BBB", "BBBBBBBBBBBBBBBBBBBBB.BBBBBBBBBBBBBBBBBBBBB.BBB..", "..B.BBBBBBBBB.BB.BBBBB..BBBBBB.BBBBBBBBBBBBBBB.BB", "BBBBB..BBB.BBB.BB.BBBBBBBB.B.BBBBBBB.BBBBBB.BBBBB", "BBBBBBBB..BBBBBBBBBBBBBBBB.BBBB..BBBBBBBBBB.BBBB.", "BBBBBBBBB.BBBBBBBBBBBBBBBB.BBBBB.BBBBB.BBB..B.BBB", "BBBBBBBBBBBBBBBBB.BBB.BBBBBBBBBBB.BBB.BBBBBBBBBBB"]), 0.0))
    print(math.isclose(f(["B..", "...", "..B"]), 50.0))
    print(math.isclose(f([".....", ".....", "..B..", ".....", "....."]), 94.11764705882354))
    print(math.isclose(f([".........", ".B..B..B.", ".........", ".........", ".B..B..B.", ".........", ".........", ".B..B..B.", "........."]), 0.0))
    print(math.isclose(f(["......B.......B..B...........................B....", "..............B..................BB..B............", "B.B.B.............B.....B..............B..........", "...................B...B....................BB....", "...B.....B.........................B.......B.....B", "B.B.........B.....B.......B..B......B.B...B.BB....", "..B...................BB...............B..........", ".........B...B................B..B................", ".......BB.......B....B................B.....BBB...", ".......BB..........B..............B......BB.......", "...................BB.....................B.......", "...B.B.B......B..............B...B......B.........", "B................B................................", "....B..........B.....B..BB....B...............BB..", "..B....B.....B.............B.....B............B...", "...................B.B........B..B.........B.B....", ".....B.....B......................................", "...........BB......BB...B.B........B...B..........", ".....BBB..........................................", ".B...........B....B...BB......B......B...B.B......", "..................B........BB................B....", "...............................B..B....BBB.B....B.", "..........B.......................................", ".....B..........B....BB......B.B......B......B....", ".....B..................B........B................", "............B.....B..B....BB...B....BB........B...", "..B.................B.........B...................", ".BB..............B................................", "...B....B..................B.................B....", "......B...B......B......................B.B.......", "..............B..................B.......B........", ".....B........BB...B.....B........................", ".......B......B.B..B..........B...........B....B..", "B...B...........B...B...B......B.B...B..B......B..", "....B..B.....B.B.......BB..B............B.B....B..", "B.......B..........B.........B...B.BB......B......", "....B...............................B.............", ".....B.B..........................................", "..........B............B......B.B..B....B.........", "....B...B.......................B.................", "B.................B...........B..B....B...........", "...B.....B........................................", "...B..B......................................BBB..", ".B...B....................................B....B..", "...B...B..........B...B.B.........................", ".....B.............B...BB..........B..BBB.BB......", "....................B.....B.......................", "........B..BB..........B.B....B...........B......B", ".........B.....BB..B.............B....BB..........", "....B..B..............B...B..B..........B........."]), 77.67558528428094))
    print(math.isclose(f([".BBBBBBBBBBBBB"]), 0.0))
    print(math.isclose(f([".......BB"]), 75.0))

#### redsquares: https://community.topcoder.com/stat?c=problem_statement&pm=2891&rd=5073&rm=&cr=7498599

Please use this Google doc to code during your interview. To free your hands for coding, we recommend that you use a headset or a phone with speaker option.

Problem Statement
    	
Checkerboards are used for many things besides playing checkers. In fact, the word "checkerboard" is in much wider use, and has more meanings, than the word "checker". In most cases, including here, a checkerboard is a pattern consisting of a rectangular array of squares where adjacent squares (horizontally and vertically) are different colors. There are two colors of squares on our checkerboards, red and black. The squares in our checkerboard are numbered vertically, from 1 to maxRank inclusive, with 1 being at the bottom. The coordinate in this vertical direction is called the rank. The squares in our checkerboard are also numbered horizontally, from 1 to maxFile inclusive, with 1 on the left. The coordinate in this horizontal direction is called the file. The location of a square is specified by the pair of coordinates: (rank, file)

On all of the checkerboards in this problem, the RIGHT-most square in the BOTTOM-most rank, (1, maxFile), IS ALWAYS BLACK.

Given the height and width (maxRank and maxFile) of our checkerboard, and the coordinates of all of the game pieces on the checkerboard, return the count of all the red squares which are empty. A square is empty if no game piece is on that square. The locations of the game pieces will be specified with two int[]s, rank and file. (rank[0], file[0]) is the location of piece 0, (rank[1], file[1]) is the location of piece 1, etc.

For example: 

maxRank = 3, maxFile = 5, rank = {1, 2, 2}, file = {4, 1, 2} 

Our checkerboard would look like this:

b R b R b   where "b" represents an empty black square,
x x R b R         "R" represents an empty red square and
b R b x b         "x" represents the location of a game piece
There are 5 empty red squares, so you return 5.

 
Definition
    	
Class:	RedSquare
Method:	countTheEmptyReds
Parameters:	int, int, int[], int[]
Returns:	int
Method signature:	int countTheEmptyReds(int maxRank, int maxFile, int[] rank, int[] file)
(be sure your method is public)
    
 
Constraints
-	maxRank will be between 1 and 50 inclusive.
-	maxFile will be between 1 and 50 inclusive.
-	rank will contain between 0 and 50 elements, inclusive.
-	rank will contain between 0 and maxRank*maxFile elements, inclusive.
-	file will contain exactly the same number of elements as rank
-	each element of rank will be between 1 and maxRank inclusive.
-	each element of file will be between 1 and maxFile inclusive.
-	All game piece locations (rank[i], file[i]) will be distinct.
 
Examples
0)	
    	
3
5
{1, 2, 2}
{4, 1, 2}
Returns: 5
The example from above.
1)	
    	
3
3
{1,2,3,1,2,3,1,2,3}
{1,1,1,2,2,2,3,3,3}
Returns: 0
A full board, no empty squares of either color.
2)	
    	
5
5
{1,1,2,2,2,3,3,4,4,4,5,5}
{2,4,1,3,5,2,4,1,3,5,2,4}
Returns: 0
All 12 red squares are occupied, none are empty.
3)	
    	
5
6
{1,1,2,2,2,3,3,4,4,4,5,5}
{2,4,1,3,5,2,4,1,3,5,2,4}
Returns: 15
12 black squares are occupied, but all 15 red squares are empty.
4)	
    	
1
1
{}
{}
Returns: 0
One black square, no red squares.
5)	
    	
50
50
{1}
{1}
Returns: 1249
One piece, which is on a red square.
6)	
    	
50
50
{1,2,3,4,5,6,7,8,9,10,
 1,2,3,4,5,6,7,8,9,10,
 1,2,3,4,5,6,7,8,9,10,
 1,2,3,4,5,6,7,8,9,10,
 1,2,3,4,5,6,7,8,9,10}
{1,1,1,1,1,1,1,1,1,1,
 2,2,2,2,2,2,2,2,2,2,
 3,3,3,3,3,3,3,3,3,3,
 4,4,4,4,4,4,4,4,4,4,
 5,5,5,5,5,5,5,5,5,5}
Returns: 1225
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 198 Round 1 - Division I, Level One 
       Single Round Match 198 Round 1 - Division II, Level Two


In [ ]:
class RedSquare(object):

    def CountEmptyRed(self, maxr, maxf, r, f):
        """
        Return count of empty red squares in checkerboard.

        Parameters:
            maxr(int): Total rows.
            maxf(int): Total columns.
            r(list): rows coordinates of pieces.
            f(list): cols coordinates of pieces.
        Return:
            int: number of empty red squares.
        """
        board = {}
        total = 0
        used = 0
        for col in range(maxf -1, -1, -1):
            precol = "red" if (maxf - 1 - col) % 2 == 0 else "black"
            for row in range(maxr):
                color = "black" if precol == "red" else "red"
                board[(row, col)] = color
                precol = color
                if color == "red": total += 1
        for i in range(len(r)):
            col = f[i] - 1
            row = r[i] - 1
            if board[(row, col)] == "red": used += 1
        return total - used

if True:
    f = RedSquare().CountEmptyRed
    print(f(3, 5, [1, 2, 2], [4, 1, 2]) == 5)
    print(f(3, 3, [1, 2, 3, 1, 2, 3, 1, 2, 3], [1, 1, 1, 2, 2, 2, 3, 3, 3]) == 0)
    print(f(5, 5, [1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5], [2, 4, 1, 3, 5, 2, 4, 1, 3, 5, 2, 4]) == 0)
    print(f(5, 6, [1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5], [2, 4, 1, 3, 5, 2, 4, 1, 3, 5, 2, 4]) == 15)
    print(f(1, 1, [], []) == 0)
    print(f(50, 50, [1], [1]) == 1249)
    print(f(49, 50, [1], [1]) == 1224)
    print(f(50, 49, [1], [1]) == 1225)
    print(f(49, 49, [1], [1]) == 1200)
    print(f(50, 50, [], []) == 1250)
    print(f(50, 49, [], []) == 1225)
    print(f(49, 50, [], []) == 1225)
    print(f(49, 49, [], []) == 1200)
    print(f(1, 50, [1], [1]) == 24)
    print(f(50, 1, [1], [1]) == 25)
    print(f(50, 50, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]) == 1225)
    print(f(50, 50, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]) == 1225)
    print(f(2, 2, [1, 1], [1, 2]) == 1)
    print(f(4, 5, [1, 1, 2, 2, 2], [2, 4, 1, 3, 5]) == 5)

#### oldestone: https://community.topcoder.com/stat?c=problem_statement&pm=1913




 Problem Statement for OldestOne


Problem Statement
    	Our student data is a mess! Each student is described with one String. At least a student's data always starts with her name, then her age, and then her address, and none of these is ever missing. The name and age are separated by one or more spaces and the age is separated from the address by one or more spaces. There may be leading and trailing spaces in a student's data.
A student's name always consists only of uppercase letters 'A'-'Z' and embedded (not leading or trailing) spaces. A student's age always consists of exactly 1, 2, or 3 digits, and its leading digit is not a zero. A student's address always contains at least one non-space character.

We need to find the name of the oldest student. If there are several, we want the one that appears earliest in the data. The name should include its embedded spaces (even if there are several consecutive embedded spaces), but no leading spaces and no trailing spaces.

Create a class OldestOne that contains a method oldest that is given a String[] data where each element is the data for a student, and that returns the name of the oldest student.

 
Definition
    	
Class:	OldestOne
Method:	oldest
Parameters:	String[]
Returns:	String
Method signature:	String oldest(String[] data)
(be sure your method is public)
    
 
Constraints
-	data will contain between 1 and 50 elements inclusive
-	Each element of data will contain between 5 and 50 characters inclusive
-	Each character in each element of data will be an uppercase letter 'A'-'Z', a digit '0'-'9' or a space (' ').
-	Each element of data will contain a name followed by an age and address, formatted as explained above, and may contain leading and trailing spaces.
-	The name in each element will contain only uppercase letters 'A'-'Z' and embedded spaces.
-	The age in each element will have no leading zeroes and will consist of between 1 and 3 digits inclusive.
 
Examples
0)	
    	
{"DOUG JONES 22 213 ALDEN LANE","   BOB     A SMITH  102 CLARK ST"}
Returns: "BOB     A SMITH"
Bob is clearly the oldest of these students. Note that the leading spaces and trailing spaces in the name are not included in the return, but the spaces embedded in the name must be preserved.
1)	
    	
{"DOUG JONES 102 213 ALDEN LANE","   BOB     A SMITH  102 CLARK ST",
 "A 1 999ELM"}
Returns: "DOUG JONES"
Now Doug Jones is 102 and is tied for oldest. A has an age of 1 (the 999 is part of his address). Bob is also 102, but comes later in the data than Doug.
2)	
    	
{"DOUG JONES 122 213 ALDEN LANE","   BOB     A SMITH  102 CLARK ST",
 "A 199 ELM"}
Returns: "A"
A has a one letter name, but is the oldest.
3)	
    	
{"   DOUG                 JONES   122 213 ALDEN LANE",
"   BOB     A SMITH                       102  3",
 " J O H N N Y           199 ELM"}
Returns: "J O H N N Y"
I don't know why Johnny likes to put spaces in his name, but we have to respect his wishes.
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 177 Round 1 - Division II, Level Two


In [ ]:
class OldestOne(object):

    def oldest(self, data):
        """Return oldest and earliest record in data"""
        names = []
        ages = []
        maxi = 0

        for i, d in enumerate(data):
            name = ""
            age = ""
            count = 0

            while d[count].isalpha() or d[count] == " ":
                name +=d[count]
                count += 1

            name = name.rstrip()
            name = name.lstrip()

            while d[count].isdigit():
                age += d[count]
                count += 1

            age = int(age)
            names.append(name)
            ages.append(age)
            if age > ages[maxi]: maxi = i

        return names[maxi]
if True:
    f = OldestOne().oldest
    print(f(["DOUG JONES 22 213 ALDEN LANE", " BOB A SMITH 102 CLARK ST"]) == "BOB A SMITH")
    print(f(["DOUG JONES 102 213 ALDEN LANE", " BOB A SMITH 102 CLARK ST", "A 1 999ELM"]) == "DOUG JONES")
    print(f(["DOUG JONES 122 213 ALDEN LANE", " BOB A SMITH 102 CLARK ST", "A 199 ELM"]) == "A")
    print(f(["DOUG JONES 122 213 ALDEN LANE", " BOB A SMITH 102 CLARK ST", "AL JOHNSON CARVER 199 ELM"]) == "AL JOHNSON CARVER")
    print(f([" DOUG JONES 122 213 ALDEN LANE", " BOB A SMITH 102 3", " J O H N N Y 199 ELM"]) == "J O H N N Y")
    print(f(["A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "AB 15 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 9 9", "A 10 9", "A 11 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9", "A 10 9"]) == "AB")
    print(f([" AL ALAL AL AL AL AL AL AL AL AL AL AL 921 AA"]) == "AL ALAL AL AL AL AL AL AL AL AL AL AL")
    print(f([" AL ALAL AL AL AL AL AL AL AL AL AL AL 921 AA", "B 99 9"]) == "AL ALAL AL AL AL AL AL AL AL AL AL AL")
    print(f([" AL ALAL AL AL AL AL AL AL AL AL AL AL 921 AA", "B 990 9"]) == "B")
    print(f(["A 10 9", "B 9 9", "C C 8 9", "D 7 9", "E 6 9", "F 5 9", "GG 4 9", "H H H 3 9"]) == "A")
    print(f(["A 10 9", "B 90 9", "C C 80 9", "D 709 9", "E 699 9", "F 5 9", "GG 54 9", "H H H 30 9"]) == "D")
    print(f(["A A 10 9", "B 90 9", "C C 80 9", " D D A 709 9", "E 699 9", "F 5 9", "GG 54 9", "H H H 30 9"]) == "D D A")
    print(f(["A A 10 9", "B 90 9", "C C 80 9", " D D A 709 9", "E 699 9", "F 5 9", "GG 54 9", "H H H 730 9"]) == "H H H")
    print(f(["A 1 A 7 A 1", "A 2 A 7 A 1", "A 3 A 7 A 1", "A A 4 A 7 A 1"]) == "A A")
    print(f(["A 1 A 7 A 1", "B 2 A 7 A 1", "C 3 A 7 A 1", "A A 3 A 7 A 1"]) == "C")
    print(f(["A 1 A 7 A 1", "ABCDEEEEEEEEEEEEEEEEEEEEEEEEEEEE E 20 A 7 A 1", "A 3 A 7 A 1", "A A 4 A 7 A 1"]) == "ABCDEEEEEEEEEEEEEEEEEEEEEEEEEEEE E")
    print(f([" EAM MEA 123 321 RD FFF FGF F", " EAS SDSSD 124 123 43 DESES", "EEEE EEEE 123 232 ESDEDS", " EEEEE WWWWW 125 128REREEER"]) == "EEEEE WWWWW")
    print(f(["DOUG JONES 22 213 ALDEN LANE", " BOB A SMITH 102 CLARK ST"]) == "BOB A SMITH")
    print(f([" HELLO THERE 29 AKAK", " NO W H II TE 33 AAA444"]) == "NO W H II TE")
    print(f(["A A A 111 111"]) == "A A A")
    print(f(["DOUG JONES 999 213 ALDEN LANE", " BOB A SMITH 999 CLARK ST"]) == "DOUG JONES")

#### linearsalesman: https://community.topcoder.com/stat?c=problem_statement&pm=10917




 Problem Statement for LinearTravellingSalesman


Problem Statement
    	A traveling salesman is going to sell his wares in Linear Kingdom. Linear Kingdom has N cities on a coordinate plane. Each city is a point, and the i-th city is located at coordinates (x[i], y[i]). All the cities in Linear Kingdom are collinear, meaning that there exists a straight line which passes through all of them. 



The traveling salesman is going to visit all of these cities. The distance he needs to travel between two cities located at (X1, Y1) and (X2, Y2) is equal to the Manhattan Distance between them, which is defined as |X1 - X2| + |Y1 - Y2|. He may start at any city and end at any city. Return the minimum total distance he will need to travel.
 
Definition
    	
Class:	LinearTravellingSalesman
Method:	findMinimumDistance
Parameters:	int[], int[]
Returns:	int
Method signature:	int findMinimumDistance(int[] x, int[] y)
(be sure your method is public)
    
 
Constraints
-	x and y will contain between 2 and 50 elements, inclusive.
-	x and y will have the same number of elements.
-	Each element of x and y will be between 0 and 10000, inclusive.
-	No two cities will have the same location.
-	All cities will be collinear, meaning that there exists a straight line which passes through all of them.
 
Examples
0)	
    	
{1,3,5}
{2,2,2}
Returns: 4
One optimal journey is (1,2) -> (3,2) -> (5,2). The Manhattan Distance between each consecutive pair of cities is 2. The total distance is 4.
1)	
    	
{3,2,1}
{3,2,1}
Returns: 4
One optimal journey is (1,1) -> (2,2) -> (3,3).
2)	
    	
{0,100,1000,10000}
{0,10,100,1000}
Returns: 11000
3)	
    	
{80,60,70,50}
{50,70,60,80}
Returns: 60
4)	
    	
{7,7,7,7,7,7,7}
{105,1231,5663,295,3062,380,7777}
Returns: 7672
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 470 Round 1 - Division II, Level One

In [ ]:
class LinearSalesman(object):
    
    def mindistance(self, x, y):
        """Return minimum travelling distance."""
        minx, maxx = min(x), max(x)
        miny, maxy = min(y), max(y)
        
        if minx == maxx: return maxy - miny
        if miny == maxy: return maxx - minx
        
        for i, e in enumerate(x):
            if e == minx: LEFT = i
            if e == maxx: RIGHT = i
                
        D = abs(x[RIGHT] - x[LEFT])
        D += abs(y[RIGHT] - y[LEFT])
        
        return D
    
if True:
    f = LinearSalesman().mindistance
    print(f([1, 3, 5], [2, 2, 2]) == 4)
    print(f([3, 2, 1], [3, 2, 1]) == 4)
    print(f([0, 100, 1000, 10000], [0, 10, 100, 1000]) == 11000)
    print(f([80, 60, 70, 50], [50, 70, 60, 80]) == 60)
    print(f([7, 7, 7, 7, 7, 7, 7], [105, 1231, 5663, 295, 3062, 380, 7777]) == 7672)
    print(f([3, 9987], [6843, 6843]) == 9984)
    print(f([6843, 6843], [3, 9987]) == 9984)
    print(f([0, 10000], [0, 10000]) == 20000)
    print(f([0, 1], [0, 0]) == 1)
    print(f([5000, 4900, 4800, 4700, 4600, 4500, 4400, 4300, 4200, 4100, 4000, 3900, 3800, 3700, 3600, 3500, 3400, 3300, 3200, 3100, 3000, 2900, 2800, 2700, 2600, 2500, 2400, 2300, 2200, 2100, 2000, 1900, 1800, 1700, 1600, 1500, 1400, 1300, 1200, 1100, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 100], [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]) == 4900)
    print(f([5000, 4881, 4762, 4643, 4524, 4405, 4286, 4167, 4048, 3929, 3810, 3691, 3572, 3453, 3334, 3215, 3096, 2977, 2858, 2739, 2620, 2501, 2382, 2263, 2144, 2025, 1906, 1787, 1668, 1549, 1430, 1311, 1192, 1073, 954, 835, 716, 597, 478, 359, 240, 121], [5000, 5119, 5238, 5357, 5476, 5595, 5714, 5833, 5952, 6071, 6190, 6309, 6428, 6547, 6666, 6785, 6904, 7023, 7142, 7261, 7380, 7499, 7618, 7737, 7856, 7975, 8094, 8213, 8332, 8451, 8570, 8689, 8808, 8927, 9046, 9165, 9284, 9403, 9522, 9641, 9760, 9879]) == 9758)
    print(f([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000], [5000, 4667, 4334, 4001, 3668, 3335, 3002, 2669, 2336, 2003, 1670, 1337, 1004, 671, 338]) == 4662)
    print(f([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000], [5000, 5185, 5370, 5555, 5740, 5925, 6110, 6295, 6480, 6665, 6850, 7035, 7220, 7405, 7590, 7775, 7960, 8145, 8330, 8515, 8700, 8885, 9070, 9255, 9440, 9625, 9810]) == 4810)
    print(f([5000, 6250, 7500, 8750], [5000, 3750, 2500, 1250]) == 7500)
    print(f([5000, 5125, 5250, 5375, 5500, 5625, 5750, 5875, 6000, 6125, 6250, 6375, 6500, 6625, 6750, 6875, 7000, 7125, 7250, 7375, 7500, 7625, 7750, 7875, 8000, 8125, 8250, 8375, 8500, 8625, 8750, 8875, 9000, 9125, 9250, 9375, 9500, 9625, 9750, 9875], [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]) == 4875)
    print(f([9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033], [5022, 5560, 4209, 3753, 4974, 2054, 9209, 3801, 207, 240, 5239, 2009, 1545, 2160, 637, 6632, 123, 4187, 671, 8526, 6142, 6762, 6333, 739, 9290, 1766, 6751, 3174, 702, 2704, 8424, 3009, 8900, 3300, 7162, 2967, 2812, 6873, 5038, 1556, 9673, 4280, 9080, 6407, 6056, 7760, 5730, 8376, 9839, 2086]) == 9716)
    print(f([5022, 5560, 4209, 3753, 4974, 2054, 9209, 3801, 207, 240, 5239, 2009, 1545, 2160, 637, 6632, 123, 4187, 671, 8526, 6142, 6762, 6333, 739, 9290, 1766, 6751, 3174, 702, 2704, 8424, 3009, 8900, 3300, 7162, 2967, 2812, 6873, 5038, 1556, 9673, 4280, 9080, 6407, 6056, 7760, 5730, 8376, 9839, 2086], [9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033, 9033]) == 9716)
    print(f([6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709], [7203, 4947, 7524, 9670, 8177, 5283, 2919, 7085, 1403, 848, 8443, 1070, 4236, 5734, 7824, 7, 3278, 3538, 2396, 2910, 639, 75, 4768, 2835, 7479, 6648, 9932, 6820, 2993, 7236, 7416, 1616, 7313, 2418, 8178, 2347, 3045, 5691, 2712, 6764, 5432, 3341, 4875, 4731, 1099, 2065, 6927, 4413, 7370, 3086]) == 9925)
    print(f([7203, 4947, 7524, 9670, 8177, 5283, 2919, 7085, 1403, 848, 8443, 1070, 4236, 5734, 7824, 7, 3278, 3538, 2396, 2910, 639, 75, 4768, 2835, 7479, 6648, 9932, 6820, 2993, 7236, 7416, 1616, 7313, 2418, 8178, 2347, 3045, 5691, 2712, 6764, 5432, 3341, 4875, 4731, 1099, 2065, 6927, 4413, 7370, 3086], [6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709, 6709]) == 9925)
    print(f([5919, 8443], [8269, 555]) == 10238)
    print(f([1188, 7588, 9988, 5988], [8891, 6851, 6086, 7361]) == 11605)
    print(f([8948, 8029, 4353, 1596, 7110, 9867], [7753, 6902, 3498, 945, 6051, 8604]) == 15930)
    print(f([1713, 7537, 4625, 621, 1349, 8629, 9721, 5353], [5979, 5851, 5915, 6003, 5987, 5827, 5803, 5899]) == 9300)
    print(f([5953, 7265, 6609, 6937, 6445, 5625, 7593, 5461, 5789, 6281], [2155, 7891, 5023, 6457, 4306, 721, 9325, 4, 1438, 3589]) == 11453)
    print(f([8189, 9279, 7971, 8843, 9715, 9061, 7535, 7753, 7317, 8625, 9933, 8407], [2600, 5815, 1957, 4529, 7101, 5172, 671, 1314, 28, 3886, 7744, 3243]) == 10332)
    print(f([3101, 7266, 9646, 6671, 4886, 126, 1911, 4291, 6076, 9051, 721, 2506, 8456, 5481], [1200, 1242, 1266, 1236, 1218, 1170, 1188, 1212, 1230, 1260, 1176, 1194, 1254, 1224]) == 9616)
    print(f([6388, 7088, 6738, 2888, 1138, 3063, 263, 6913, 613, 3763, 5513, 1838, 4463, 7263, 2188, 1663], [9421, 9865, 9643, 7201, 6091, 7312, 5536, 9754, 5758, 7756, 8866, 6535, 8200, 9976, 6757, 6424]) == 11440)
    print(f([8052, 4972, 8612, 8332, 5252, 6372, 5812, 6652, 4692, 7212, 7492, 6932, 5532, 8892, 6092, 4412, 9172, 7772], [7505, 1477, 8601, 8053, 2025, 4217, 3121, 4765, 929, 5861, 6409, 5313, 2573, 9149, 3669, 381, 9697, 6957]) == 14076)
    print(f([7367, 4357, 7152, 2422, 5217, 8227, 4787, 3497, 6077, 3712, 8657, 9947, 8012, 9087, 3282, 6937, 4572, 6507, 4142, 9732], [6616, 2878, 6349, 475, 3946, 7684, 3412, 1810, 5014, 2077, 8218, 9820, 7417, 8752, 1543, 6082, 3145, 5548, 2611, 9553]) == 16870)
    print(f([82, 3007, 5932, 6257, 5607, 3332, 4307, 1707, 4632, 732, 4957, 2682, 2357, 1057, 6907, 2032, 5282, 407, 3657, 6582, 1382, 3982], [4736, 2738, 740, 518, 962, 2516, 1850, 3626, 1628, 4292, 1406, 2960, 3182, 4070, 74, 3404, 1184, 4514, 2294, 296, 3848, 2072]) == 11487)
    print(f([2006, 4631, 3881, 5006, 8006, 9506, 6131, 5381, 9881, 7256, 1631, 506, 6881, 2381, 3131, 5756, 7631, 9131, 8381, 2756, 6506, 1256, 4256, 3506], [1939, 2800, 2554, 2923, 3907, 4399, 3292, 3046, 4522, 3661, 1816, 1447, 3538, 2062, 2308, 3169, 3784, 4276, 4030, 2185, 3415, 1693, 2677, 2431]) == 12450)
    print(f([3241, 5386, 1591, 6046, 6211, 4396, 3901, 3736, 4726, 5716, 3571, 4066, 5551, 2086, 4891, 3406, 2911, 2416, 1921, 5221, 4561, 3076, 2251, 4231, 2581, 1756], [3624, 8057, 214, 9421, 9762, 6011, 4988, 4647, 6693, 8739, 4306, 5329, 8398, 1237, 7034, 3965, 2942, 1919, 896, 7716, 6352, 3283, 1578, 5670, 2260, 555]) == 14168)
    print(f([6698, 4934, 7034, 8462, 3674, 2246, 8042, 8630, 4682, 2750, 7958, 734, 650, 6026, 5270, 62, 6866, 902, 5018, 7286, 5774, 7454, 1070, 4094, 5102, 7622, 8378, 1574], [8759, 7709, 8959, 9809, 6959, 6109, 9559, 9909, 7559, 6409, 9509, 5209, 5159, 8359, 7909, 4809, 8859, 5309, 7759, 9109, 8209, 9209, 5409, 7209, 7809, 9309, 9759, 5709]) == 13668)
    print(f([6110, 2667, 8614, 3919, 9866, 8301, 4545, 6736, 9240, 5171, 3606, 2980, 4858, 1415, 2354, 4232, 476, 7675, 163, 1728, 3293, 2041, 6423, 8927, 7362, 7988, 5484, 9553, 7049, 789], [5006, 6106, 4206, 5706, 3806, 4306, 5506, 4806, 4006, 5306, 5806, 6006, 5406, 6506, 6206, 5606, 6806, 4506, 6906, 6406, 5906, 6306, 4906, 4106, 4606, 4406, 5206, 3906, 4706, 6706]) == 12803)
    print(f([8726, 9734, 7286, 7478, 9014, 8534, 9830, 7574, 9158, 8294, 8054, 8630, 8246, 9590, 9782, 9638, 9926, 8486, 8918, 7718, 9494, 6710, 9446, 9542, 9686, 8678, 6326, 8966, 6950, 8342, 8102, 6422], [6598, 9328, 2698, 3218, 7378, 6078, 9588, 3478, 7768, 5428, 4778, 6338, 5298, 8938, 9458, 9068, 9848, 5948, 7118, 3868, 8678, 1138, 8548, 8808, 9198, 6468, 98, 7248, 1788, 5558, 4908, 358]) == 13350)
    print(f([5699, 7337, 2150, 3515, 4334, 8429, 6518, 4880, 4061, 6791, 1604, 8156, 2423, 1877, 785, 7610, 3788, 239, 7883, 7064, 5972, 8702, 2696, 9248, 2969, 512, 1058, 5426, 8975, 3242, 9521, 1331, 5153, 6245], [3981, 4947, 1888, 2693, 3176, 5591, 4464, 3498, 3015, 4625, 1566, 5430, 2049, 1727, 1083, 5108, 2854, 761, 5269, 4786, 4142, 5752, 2210, 6074, 2371, 922, 1244, 3820, 5913, 2532, 6235, 1405, 3659, 4303]) == 14756)
    print(f([7462, 1962, 9937, 3062, 6087, 6912, 6637, 8837, 5812, 8012, 4987, 8287, 2512, 9387, 1137, 312, 4437, 3612, 2237, 4712, 862, 5537, 37, 587, 8562, 5262, 2787, 1687, 1412, 4162, 7187, 9662, 3337, 6362, 3887, 9112], [7887, 4887, 9237, 5487, 7137, 7587, 7437, 8637, 6987, 8187, 6537, 8337, 5187, 8937, 4437, 3987, 6237, 5787, 5037, 6387, 4287, 6837, 3837, 4137, 8487, 6687, 5337, 4737, 4587, 6087, 7737, 9087, 5637, 7287, 5937, 8787]) == 15300)
    print(f([4093, 6373, 7273, 6793, 6313, 4933, 4573, 6253, 7153, 5293, 6553, 5953, 4213, 4753, 4993, 7213, 6913, 7033, 4513, 5773, 5833, 6613, 6013, 4033, 5173, 5353, 5653, 5593, 5053, 6073, 5113, 4633, 4873, 5713, 5473, 5893, 6193, 4813], [9717, 2839, 124, 1572, 3020, 7183, 8269, 3201, 486, 6097, 2296, 4106, 9355, 7726, 7002, 305, 1210, 848, 8450, 4649, 4468, 2115, 3925, 9898, 6459, 5916, 5011, 5192, 6821, 3744, 6640, 8088, 7364, 4830, 5554, 4287, 3382, 7545]) == 13014)
    print(f([3076, 2930, 4025, 6215, 6726, 5558, 6945, 8843, 8478, 9281, 9646, 8405, 4244, 6799, 8186, 4609, 3003, 6507, 8551, 9135, 9865, 8113, 4463, 8989, 7456, 9938, 8697, 4098, 7894, 8916, 8624, 5339, 8332, 6069, 6142, 9792, 9354, 3441, 6580, 5193], [126, 72, 477, 1287, 1476, 1044, 1557, 2259, 2124, 2421, 2556, 2097, 558, 1503, 2016, 693, 99, 1395, 2151, 2367, 2637, 1989, 639, 2313, 1746, 2664, 2205, 504, 1908, 2286, 2178, 963, 2070, 1233, 1260, 2610, 2448, 261, 1422, 909]) == 9600)
    print(f([5083, 4966, 637, 520, 403, 3328, 2509, 52, 4381, 3445, 286, 4264, 4849, 2977, 3094, 2743, 2392, 988, 871, 4498, 2626, 2275, 5317, 4030, 754, 1924, 3562, 5200, 2860, 169, 4732, 4147, 3913, 1807, 1690, 1339, 1456, 1222, 3796, 1573, 2041, 3211], [526, 746, 8886, 9106, 9326, 3826, 5366, 9986, 1846, 3606, 9546, 2066, 966, 4486, 4266, 4926, 5586, 8226, 8446, 1626, 5146, 5806, 86, 2506, 8666, 6466, 3386, 306, 4706, 9766, 1186, 2286, 2726, 6686, 6906, 7566, 7346, 7786, 2946, 7126, 6246, 4046]) == 15165)
    print(f([6938, 7814, 6719, 6135, 7960, 7741, 9712, 9420, 8325, 9055, 8982, 9493, 8544, 6500, 7011, 9128, 7449, 6354, 9931, 7157, 8690, 9566, 9858, 8909, 6573, 8252, 7595, 8617, 6208, 9347, 8471, 6792, 7303, 6062, 7230, 6281, 8398, 6646, 9201, 9639, 7084, 7376, 6427, 7887], [1849, 3685, 1390, 166, 3991, 3532, 7663, 7051, 4756, 6286, 6133, 7204, 5215, 931, 2002, 6439, 2920, 625, 8122, 2308, 5521, 7357, 7969, 5980, 1084, 4603, 3226, 5368, 319, 6898, 5062, 1543, 2614, 13, 2461, 472, 4909, 1237, 6592, 7510, 2155, 2767, 778, 3838]) == 11978)
    print(f([4256, 3266, 5411, 3596, 2606, 5246, 626, 2276, 1946, 9701, 1286, 5741, 791, 6401, 7886, 5906, 2111, 2441, 1451, 3926, 956, 8876, 8711, 1121, 3761, 1616, 2771, 6566, 4421, 9206, 5081, 6896, 9536, 9866, 4091, 3101, 7721, 6236, 9371, 7391, 5576, 7061, 8051, 1781, 4751, 8546], [7685, 8309, 6957, 8101, 8725, 7061, 9973, 8933, 9141, 4253, 9557, 6749, 9869, 6333, 5397, 6645, 9037, 8829, 9453, 7893, 9765, 4773, 4877, 9661, 7997, 9349, 8621, 6229, 7581, 4565, 7165, 6021, 4357, 4149, 7789, 8413, 5501, 6437, 4461, 5709, 6853, 5917, 5293, 9245, 7373, 4981]) == 15064)
    print(f([9286, 6066, 1726, 9706, 4386, 186, 8866, 9566, 6206, 7466, 7046, 7326, 8166, 606, 9846, 9426, 2846, 4526, 8726, 886, 3546, 5366, 8306, 8586, 5926, 3126, 1586, 2706, 3406, 8446, 7746, 4946, 3266, 9146, 9986, 5226, 1446, 1866, 2986, 9006, 466, 8026, 2006, 2426, 4106, 2146, 326, 3826], [1386, 2950, 5058, 1182, 3766, 5806, 1590, 1250, 2882, 2270, 2474, 2338, 1930, 5602, 1114, 1318, 4514, 3698, 1658, 5466, 4174, 3290, 1862, 1726, 3018, 4378, 5126, 4582, 4242, 1794, 2134, 3494, 4310, 1454, 1046, 3358, 5194, 4990, 4446, 1522, 5670, 1998, 4922, 4718, 3902, 4854, 5738, 4038]) == 14560)
    print(f([6280, 5790, 3585, 7946, 3389, 3193, 9367, 8583, 4565, 5447, 3046, 7407, 8485, 2213, 7603, 5692, 2507, 6770, 8240, 5251, 7848, 7750, 8730, 2654, 5937, 3683, 9808, 3095, 1625, 7554, 7897, 5349, 3242, 9171, 9024, 1772, 1478, 8338, 7456, 5398, 3977, 8044, 4663, 3144, 9955, 9416, 9661, 8877, 3781, 4320], [5047, 4547, 2297, 6747, 2097, 1897, 8197, 7397, 3297, 4197, 1747, 6197, 7297, 897, 6397, 4447, 1197, 5547, 7047, 3997, 6647, 6547, 7547, 1347, 4697, 2397, 8647, 1797, 297, 6347, 6697, 4097, 1947, 7997, 7847, 447, 147, 7147, 6247, 4147, 2697, 6847, 3397, 1847, 8797, 8247, 8497, 7697, 2497, 3047]) == 17127)
    print(f([2438, 6897, 4076, 7170, 5350, 6988, 5532, 7352, 3075, 7989, 4713, 3348, 3985, 8353, 6260, 6624, 2802, 4986, 5441, 5805, 8080, 6442, 2984, 7079, 6351, 6078, 7807, 7625, 6806, 3439, 5714, 3803, 4167, 5623, 6533, 5987, 6169, 4258, 4531, 6715, 7716, 3712, 5896, 2620, 3894, 2711, 7261, 8444, 8262, 3530], [9900, 2599, 7218, 2152, 5132, 2450, 4834, 1854, 8857, 811, 6175, 8410, 7367, 215, 3642, 3046, 9304, 5728, 4983, 4387, 662, 3344, 9006, 2301, 3493, 3940, 1109, 1407, 2748, 8261, 4536, 7665, 7069, 4685, 3195, 4089, 3791, 6920, 6473, 2897, 1258, 7814, 4238, 9602, 7516, 9453, 2003, 66, 364, 8112]) == 15840)
    print(f([2234, 994, 3754, 8634, 3674, 834, 4114, 1914, 8354, 9834, 634, 6874, 4594, 2634, 5354, 8474, 9874, 8514, 7194, 8914, 4274, 1354, 5754, 5594, 1314, 2554, 6234, 5434, 4554, 3554, 7834, 9794, 5234, 4434, 7794, 9354, 4514, 9914, 8114, 5474, 5514, 6274, 1794, 5114, 2354, 3954, 2434, 3394, 4194, 2994], [7484, 8104, 6724, 4284, 6764, 8184, 6544, 7644, 4424, 3684, 8284, 5164, 6304, 7284, 5924, 4364, 3664, 4344, 5004, 4144, 6464, 7924, 5724, 5804, 7944, 7324, 5484, 5884, 6324, 6824, 4684, 3704, 5984, 6384, 4704, 3924, 6344, 3644, 4544, 5864, 5844, 5464, 7704, 6044, 7424, 6624, 7384, 6904, 6504, 7104]) == 13920)
    print(f([6047, 6333, 4903, 5163, 6021, 4591, 7399, 6385, 7217, 6749, 5553, 6515, 6255, 5761, 7009, 4981, 6697, 7685, 7139, 7243, 5527, 5449, 5033, 4877, 5111, 6931, 5241, 7633, 5917, 4565, 5189, 7113, 6827, 5085, 6125, 5579, 7503, 6359, 6879, 6541, 5735, 7295, 5423, 5345, 5683, 6437, 7581, 5839, 5943, 6957], [5267, 4365, 8875, 8055, 5349, 9859, 1003, 4201, 1577, 3053, 6825, 3791, 4611, 6169, 2233, 8629, 3217, 101, 1823, 1495, 6907, 7153, 8465, 8957, 8219, 2479, 7809, 265, 5677, 9941, 7973, 1905, 2807, 8301, 5021, 6743, 675, 4283, 2643, 3709, 6251, 1331, 7235, 7481, 6415, 4037, 429, 5923, 5595, 2397]) == 12960)
    print(f([6965, 6197, 5429, 9461, 6389, 6053, 6437, 6341, 7205, 5525, 8453, 9941, 6581, 5909, 7349, 9269, 5573, 9173, 8165, 8357, 7877, 6917, 5669, 6725, 9029, 9845, 9797, 7733, 5621, 5381, 9749, 8981, 9989, 9077, 5141, 8549, 9557, 6773, 7541, 8117, 9125, 8741, 7157, 8501, 7109, 9893, 8837, 5237, 5189, 9653], [1028, 628, 228, 2328, 728, 553, 753, 703, 1153, 278, 1803, 2578, 828, 478, 1228, 2228, 303, 2178, 1653, 1753, 1503, 1003, 353, 903, 2103, 2528, 2503, 1428, 328, 203, 2478, 2078, 2603, 2128, 78, 1853, 2378, 928, 1328, 1628, 2153, 1953, 1128, 1828, 1103, 2553, 2003, 128, 103, 2428]) == 7373)
    print(f([5967, 1699, 8683, 7325, 1893, 923, 2863, 9459, 7713, 8877, 3833, 1117, 5773, 2087, 7131, 1505, 2669, 9071, 5385, 6549, 4609, 341, 8489, 8101, 8295, 6937, 3057, 9265, 9847, 6355, 3639, 1311, 7519, 4027, 6743, 9653, 5579, 2475, 7907, 5191, 4803, 147, 6161, 4221, 2281, 535, 3251, 4997, 3445, 729], [3447, 3139, 3643, 3545, 3153, 3083, 3223, 3699, 3573, 3657, 3293, 3097, 3433, 3167, 3531, 3125, 3209, 3671, 3405, 3489, 3349, 3041, 3629, 3601, 3615, 3517, 3237, 3685, 3727, 3475, 3279, 3111, 3559, 3307, 3503, 3713, 3419, 3195, 3587, 3391, 3363, 3027, 3461, 3321, 3181, 3055, 3251, 3377, 3265, 3069]) == 10400)
    print(f([1063, 288], [660, 1629]) == 1744)
    print(f([851, 953, 137, 647], [598, 679, 31, 436]) == 1464)
    print(f([1370, 1725, 2080, 305, 1015, 660], [1479, 900, 321, 3216, 2058, 2637]) == 4670)
    print(f([4201, 421, 3121, 2581, 4741, 961, 2041, 1501], [4469, 3671, 4241, 4127, 4583, 3785, 4013, 3899]) == 5232)
    print(f([4936, 4832, 4754, 4468, 4364, 4338, 4676, 4910, 4572, 4650], [253, 1829, 3011, 7345, 8921, 9315, 4193, 647, 5769, 4587]) == 9660)
    print(f([403, 327, 323, 319, 411, 115, 211, 427, 199, 179, 147, 451], [8299, 6247, 6139, 6031, 8515, 523, 3115, 8947, 2791, 2251, 1387, 9595]) == 9408)
    print(f([3811, 3934, 490, 1228, 2089, 2950, 859, 2458, 1966, 3565, 5041, 736, 2581, 3196], [3409, 3315, 5947, 5383, 4725, 4067, 5665, 4443, 4819, 3597, 2469, 5759, 4349, 3879]) == 8029)
    print(f([2508, 2211, 2379, 2371, 2546, 2190, 2415, 2463, 2485, 2361, 2401, 2543, 2290, 2489, 2397, 2419], [996, 8124, 4092, 4284, 84, 8628, 3228, 2076, 1548, 4524, 3564, 156, 6228, 1452, 3660, 3132]) == 8900)
    print(f([7408, 118, 604, 6922, 928, 2386, 2872, 5140, 3358, 6598, 5788, 1414, 4816, 3844, 1900, 8380, 1576, 4006], [810, 5940, 5598, 1152, 5370, 4344, 4002, 2406, 3660, 1380, 1950, 5028, 2634, 3318, 4686, 126, 4914, 3204]) == 14076)
    print(f([7192, 5182, 3574, 3306, 9604, 6924, 5048, 3440, 8934, 1162, 4780, 2100, 5852, 1564, 4646, 1832, 4914, 6120, 5450, 8800], [7316, 5936, 4832, 4648, 8972, 7132, 5844, 4740, 8512, 3176, 5660, 3820, 6396, 3452, 5568, 3636, 5752, 6580, 6120, 8420]) == 14238)
    print(f([963, 2279, 3924, 2232, 2843, 1245, 4676, 4488, 1903, 2326, 164, 775, 3031, 1997, 1950, 1527, 2796, 3783, 2044, 5005, 1339, 1104], [2911, 3275, 3730, 3262, 3431, 2989, 3938, 3886, 3171, 3288, 2690, 2859, 3483, 3197, 3184, 3067, 3418, 3691, 3210, 4029, 3015, 2950]) == 6180)
    print(f([856, 356, 1906, 756, 2156, 1356, 556, 2706, 2456, 2406, 906, 2206, 2906, 2756, 2006, 506, 406, 1306, 956, 1656, 1456, 1256, 156, 2256], [2217, 2417, 1797, 2257, 1697, 2017, 2337, 1477, 1577, 1597, 2197, 1677, 1397, 1457, 1757, 2357, 2397, 2037, 2177, 1897, 1977, 2057, 2497, 1657]) == 3850)
    print(f([7733, 3413, 3503, 2963, 713, 1883, 7463, 7103, 6203, 3953, 2243, 4043, 5303, 2423, 2693, 1523, 2063, 2603, 1343, 6023, 5663, 6743, 6113, 3053, 7283, 6833], [3599, 1391, 1437, 1161, 11, 609, 3461, 3277, 2817, 1667, 793, 1713, 2357, 885, 1023, 425, 701, 977, 333, 2725, 2541, 3093, 2771, 1207, 3369, 3139]) == 10608)
    print(f([46, 2750, 358, 1502, 5610, 6962, 2542, 2854, 3270, 2438, 4414, 5454, 3426, 5766, 7222, 1658, 6806, 4622, 2698, 3582, 5558, 4830, 6702, 6442, 7898, 7846, 6338, 1034], [298, 2586, 562, 1530, 5006, 6150, 2410, 2674, 3026, 2322, 3994, 4874, 3158, 5138, 6370, 1662, 6018, 4170, 2542, 3290, 4962, 4346, 5930, 5710, 6942, 6898, 5622, 1134]) == 14496)
    print(f([1831, 3259, 1999, 2755, 1411, 1327, 2335, 2167, 2251, 3511, 3007, 2923, 991, 1495, 3175, 1243, 3091, 3763, 1663, 3679, 2671, 1579, 1075, 2503, 2083, 1915, 823, 907, 2419, 2839], [1149, 2764, 1339, 2194, 674, 579, 1719, 1529, 1624, 3049, 2479, 2384, 199, 769, 2669, 484, 2574, 3334, 959, 3239, 2099, 864, 294, 1909, 1434, 1244, 9, 104, 1814, 2289]) == 6265)
    print(f([1873, 1509, 2321, 1985, 921, 2629, 1929, 1005, 1901, 837, 2349, 2377, 949, 1845, 1957, 1061, 2209, 2601, 977, 2713, 2769, 809, 2265, 1481, 2461, 1201, 1173, 1565, 1425, 2573, 1817, 1453], [576, 381, 816, 636, 66, 981, 606, 111, 591, 21, 831, 846, 81, 561, 621, 141, 756, 966, 96, 1026, 1056, 6, 786, 366, 891, 216, 201, 411, 336, 951, 546, 351]) == 3010)
    print(f([1238, 1230, 1038, 1326, 30, 1094, 1054, 1390, 286, 310, 198, 1102, 246, 622, 374, 590, 182, 862, 46, 262, 838, 1278, 1366, 1270, 918, 654, 894, 1014, 1166, 702, 438, 1070, 486, 470], [999, 1017, 1449, 801, 3717, 1323, 1413, 657, 3141, 3087, 3339, 1305, 3231, 2385, 2943, 2457, 3375, 1845, 3681, 3195, 1899, 909, 711, 927, 1719, 2313, 1773, 1503, 1161, 2205, 2799, 1377, 2691, 2727]) == 4420)
    print(f([5645, 5576, 6266, 5852, 6404, 4472, 7094, 6128, 7646, 6059, 7163, 4886, 7508, 6680, 6473, 4334, 7784, 7439, 6956, 7232, 4817, 6197, 7025, 5369, 4610, 6611, 7301, 5921, 6542, 5300, 4679, 4955, 6749, 6818, 5231, 5093], [1197, 1139, 1719, 1371, 1835, 211, 2415, 1603, 2879, 1545, 2473, 559, 2763, 2067, 1893, 95, 2995, 2705, 2299, 2531, 501, 1661, 2357, 965, 327, 2009, 2589, 1429, 1951, 907, 385, 617, 2125, 2183, 849, 733]) == 6350)
    print(f([5218, 4856, 592, 3396, 2232, 8086, 7656, 1504, 8514, 7190, 2604, 876, 8806, 140, 9616, 7772, 7894, 4776, 6912, 4410, 1672, 2782, 3906, 8772, 5310, 9148, 6046, 6344, 9038, 7018, 8954, 3428, 8280, 5122, 1168, 886, 132, 3228], [4521, 4702, 6834, 5432, 6014, 3087, 3302, 6378, 2873, 3535, 5828, 6692, 2727, 7060, 2322, 3244, 3183, 4742, 3674, 4925, 6294, 5739, 5177, 2744, 4475, 2556, 4107, 3958, 2611, 3621, 2653, 5416, 2990, 4569, 6546, 6687, 7064, 5516]) == 14226)
    print(f([942, 6042, 3407, 4002, 857, 3152, 2302, 1962, 262, 2557, 3662, 5192, 2217, 3747, 3917, 2812, 6552, 177, 3492, 5022, 2047, 5532, 4767, 5617, 1707, 6212, 6297, 6467, 6637, 5787, 5957, 1197, 4172, 2132, 4087, 7, 517, 1282, 6382, 1622], [6057, 6897, 6463, 6561, 6043, 6421, 6281, 6225, 5945, 6323, 6505, 6757, 6267, 6519, 6547, 6365, 6981, 5931, 6477, 6729, 6239, 6813, 6687, 6827, 6183, 6925, 6939, 6967, 6995, 6855, 6883, 6099, 6589, 6253, 6575, 5903, 5987, 6113, 6953, 6169]) == 7722)
    print(f([1571, 979, 899, 595, 19, 1267, 1123, 99, 611, 403, 1379, 515, 323, 1155, 1475, 1635, 1459, 1539, 1619, 83, 995, 707, 259, 963, 1299, 563, 739, 435, 419, 691, 627, 3, 947, 659, 1075, 179, 227, 547, 643, 1283, 787, 1427], [244, 2427, 2722, 3843, 5967, 1365, 1896, 5672, 3784, 4551, 952, 4138, 4846, 1778, 598, 8, 657, 362, 67, 5731, 2368, 3430, 5082, 2486, 1247, 3961, 3312, 4433, 4492, 3489, 3725, 6026, 2545, 3607, 2073, 5377, 5200, 4020, 3666, 1306, 3135, 775]) == 7650)
    print(f([5605, 7573, 6805, 7477, 6949, 6661, 6325, 3877, 4981, 4789, 4453, 7525, 4309, 6421, 5989, 6757, 5941, 4501, 4117, 4069, 6277, 4645, 5509, 5845, 6373, 6133, 4405, 4357, 5653, 5221, 3829, 3637, 5125, 5413, 4165, 5173, 5797, 5317, 3541, 3973, 6517, 3781, 6229, 7333], [4798, 1, 1873, 235, 1522, 2224, 3043, 9010, 6319, 6787, 7606, 118, 7957, 2809, 3862, 1990, 3979, 7489, 8425, 8542, 3160, 7138, 5032, 4213, 2926, 3511, 7723, 7840, 4681, 5734, 9127, 9595, 5968, 5266, 8308, 5851, 4330, 5500, 9829, 8776, 2575, 9244, 3277, 586]) == 13860)
    print(f([2683, 4507, 1144, 1942, 4393, 6046, 3823, 1828, 5761, 61, 4450, 5134, 5077, 7471, 1315, 1543, 6673, 6160, 5419, 4906, 5989, 1771, 745, 973, 2740, 7756, 4621, 2911, 5191, 4, 6274, 2170, 6616, 118, 4222, 4279, 2968, 4108, 7528, 4165, 4564, 7585, 6445, 7870, 4963, 3082], [6523, 7739, 5497, 6029, 7663, 8765, 7283, 5953, 8575, 4775, 7701, 8157, 8119, 9715, 5611, 5763, 9183, 8841, 8347, 8005, 8727, 5915, 5231, 5383, 6561, 9905, 7815, 6675, 8195, 4737, 8917, 6181, 9145, 4813, 7549, 7587, 6713, 7473, 9753, 7511, 7777, 9791, 9031, 9981, 8043, 6789]) == 13110)
    print(f([3765, 638, 1700, 4532, 4355, 3116, 3057, 2113, 107, 2349, 3824, 815, 4886, 2939, 4473, 1936, 1995, 2821, 3647, 3529, 166, 874, 2762, 3293, 2290, 4296, 3352, 2585, 3470, 4945, 520, 5417, 1346, 2703, 3942, 1759, 5476, 5122, 1405, 1582, 3234, 4237, 343, 2644, 5299, 1641, 1051, 402], [6767, 1255, 3127, 8119, 7807, 5623, 5519, 3855, 319, 4271, 6871, 1567, 8743, 5311, 8015, 3543, 3647, 5103, 6559, 6351, 423, 1671, 4999, 5935, 4167, 7703, 6039, 4687, 6247, 8847, 1047, 9679, 2503, 4895, 7079, 3231, 9783, 9159, 2607, 2919, 5831, 7599, 735, 4791, 9471, 3023, 1983, 839]) == 14833)
    print(f([3781, 4087, 5345, 4257, 5787, 6331, 5583, 4835, 6297, 4359, 6535, 5753, 4597, 4053, 5651, 3645, 5821, 6195, 4121, 6433, 5073, 3747, 3543, 5481, 6059, 3815, 6365, 4563, 3679, 5311, 4461, 3509, 3849, 5141, 4291, 4801, 4937, 4155, 4529, 5957, 5277, 5243, 6501, 5005, 5413, 4223, 4495, 3883, 4427, 6229], [932, 1913, 5946, 2458, 7363, 9107, 6709, 4311, 8998, 2785, 9761, 7254, 3548, 1804, 6927, 496, 7472, 8671, 2022, 9434, 5074, 823, 169, 6382, 8235, 1041, 9216, 3439, 605, 5837, 3112, 60, 1150, 5292, 2567, 4202, 4638, 2131, 3330, 7908, 5728, 5619, 9652, 4856, 6164, 2349, 3221, 1259, 3003, 8780]) == 12727)
    print(f([2407, 5180, 5697, 3394, 1561, 2125, 5791, 245, 2501, 2736, 5368, 715, 4804, 1467, 6120, 3629, 5509, 6167, 1749, 151, 3864, 3065, 5932, 527, 1655, 621, 5274, 1608, 433, 4569, 1138, 5650, 1514, 5415, 903, 2313, 4710, 4663, 809, 3817, 4099, 4945, 5556, 1843, 3958, 10, 1232, 3676, 3159, 4052], [1926, 746, 526, 1506, 2286, 2046, 486, 2846, 1886, 1786, 666, 2646, 906, 2326, 346, 1406, 606, 326, 2206, 2886, 1306, 1646, 426, 2726, 2246, 2686, 706, 2266, 2766, 1006, 2466, 546, 2306, 646, 2566, 1966, 946, 966, 2606, 1326, 1206, 846, 586, 2166, 1266, 2946, 2426, 1386, 1606, 1226]) == 8777)
    print(f([7032, 6828, 7916, 4856, 8392, 7168, 5808, 7372, 5264, 7304, 8664, 8324, 6420, 8460, 8256, 5060, 5400, 6692, 8052, 5536, 8528, 5196, 6760, 4992, 8732, 8188, 4788, 8936, 8120, 6284, 7236, 7984, 7440, 6624, 6556, 4924, 8868, 5944, 7712, 8596, 5468, 5876, 6488, 7508, 5604, 7644, 5128, 6080, 7848, 6148], [4549, 4153, 6265, 325, 7189, 4813, 2173, 5209, 1117, 5077, 7717, 7057, 3361, 7321, 6925, 721, 1381, 3889, 6529, 1645, 7453, 985, 4021, 589, 7849, 6793, 193, 8245, 6661, 3097, 4945, 6397, 5341, 3757, 3625, 457, 8113, 2437, 5869, 7585, 1513, 2305, 3493, 5473, 1777, 5737, 853, 2701, 6133, 2833]) == 12200)
    print(f([1313, 1047, 1117, 1327, 963, 235, 725, 263, 431, 1215, 1033, 655, 1439, 165, 305, 179, 837, 753, 669, 1397, 221, 1243, 1075, 991, 851, 39, 53, 529, 865, 1411, 949, 739, 515, 627, 81, 1103, 1061, 375, 879, 1355, 711, 1201, 1369, 683, 389, 95, 151, 1257, 193, 1271], [427, 1301, 1071, 381, 1577, 3969, 2359, 3877, 3325, 749, 1347, 2589, 13, 4199, 3739, 4153, 1991, 2267, 2543, 151, 4015, 657, 1209, 1485, 1945, 4613, 4567, 3003, 1899, 105, 1623, 2313, 3049, 2681, 4475, 1117, 1255, 3509, 1853, 289, 2405, 795, 243, 2497, 3463, 4429, 4245, 611, 4107, 565]) == 6000)
    print(f([3474, 3648, 284, 574, 545, 1879, 1966, 3938, 603, 2865, 2488, 719, 4025, 1908, 2082, 1270, 1647, 139, 3184, 429, 4315, 1328, 1299, 2720, 3677, 3735, 2372, 1212, 3126, 3880, 1995, 3155, 1937, 1792, 3822, 3561, 3619, 661, 3097, 1821, 1241, 2227, 3010, 3068, 2343, 2111, 2923, 1067, 3445, 2517], [532, 430, 2402, 2232, 2249, 1467, 1416, 260, 2215, 889, 1110, 2147, 209, 1450, 1348, 1824, 1603, 2487, 702, 2317, 39, 1790, 1807, 974, 413, 379, 1178, 1858, 736, 294, 1399, 719, 1433, 1518, 328, 481, 447, 2181, 753, 1501, 1841, 1263, 804, 770, 1195, 1331, 855, 1943, 549, 1093]) == 6624)
    print(f([3051, 4826, 1844, 5039, 2270, 7027, 779, 7098, 7169, 3974, 6317, 7737, 6459, 6104, 5607, 6814, 8092, 3335, 2128, 1489, 4471, 211, 6246, 3264, 1560, 5962, 6388, 6601, 1418, 7240, 850, 5536, 3761, 4968, 708, 3619, 6672, 5181, 2696, 1773, 637, 2199, 2483, 69, 4542, 5252, 4613, 2980, 3548, 4400], [5183, 6158, 4520, 6275, 4754, 7367, 3935, 7406, 7445, 5690, 6977, 7757, 7055, 6860, 6587, 7250, 7952, 5339, 4676, 4325, 5963, 3623, 6938, 5300, 4364, 6782, 7016, 7133, 4286, 7484, 3974, 6548, 5573, 6236, 3896, 5495, 7172, 6353, 4988, 4481, 3857, 4715, 4871, 3545, 6002, 6392, 6041, 5144, 5456, 5924]) == 12430)
    print(f([210, 220, 230, 240, 250, 260, 270, 280], [5062, 5061, 5060, 5059, 5058, 5057, 5056, 5055]) == 77)
    print(f([80, 60, 70, 50], [50, 70, 60, 80]) == 60)
    print(f([1, 3, 2], [1, 3, 2]) == 4)
    print(f([7, 7, 7, 7, 7, 7, 7], [105, 1231, 5663, 295, 3062, 380, 7777]) == 7672)
    print(f([1, 2, 3], [1, 2, 3]) == 4)
    print(f([1, 3, 2], [1, 1, 1]) == 2)
    print(f([1, 5, 4, 3, 2, 7, 8], [1, 5, 4, 3, 2, 7, 8]) == 14)
    print(f([1, 2, 3], [10000, 10000, 10000]) == 2)
    print(f([1, 100, 50], [0, 0, 0]) == 99)
    print(f([1, 1, 1], [2, 1, 0]) == 2)
    print(f([1, 7, 5], [2, 2, 2]) == 6)
    print(f([7, 7, 7, 7], [5396, 290, 490, 380]) == 5106)
    print(f([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], [105, 1231, 5663, 295, 3062, 380, 7777, 7778, 7779, 7780, 7781, 7785, 7783, 7782, 7784]) == 7680)
    print(f([1, 1], [3, 1]) == 2)
    print(f([1, 1, 1], [2, 5, 3]) == 3)
    print(f([7, 7, 7], [777, 77, 7]) == 770)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]) == 26)
    print(f([1, 3, 2], [3, 3, 3]) == 2)
    print(f([5, 1, 3], [2, 2, 2]) == 4)
    print(f([2, 1], [1, 2]) == 2)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]) == 98)
    print(f([1, 0, 2], [0, 0, 0]) == 2)
    print(f([7, 7, 7, 7, 7, 7, 7], [105, 1231, 5663, 295, 3062, 380, 1]) == 5662)
    print(f([1, 1, 1], [1, 3, 2]) == 2)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 39)
    print(f([2, 2, 2, 2], [3, 1, 5, 7]) == 6)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]) == 72)
    print(f([0, 0], [0, 80]) == 80)
    print(f([100, 1, 1000], [100, 1, 1000]) == 1998)
    print(f([0, 0, 0], [3, 1, 2]) == 2)
    print(f([1, 10, 5], [1, 10, 5]) == 18)
    print(f([2, 3, 1, 4], [3, 2, 4, 1]) == 6)
    print(f([0, 10000, 1000, 100], [0, 1000, 100, 10]) == 11000)
    print(f([1, 1, 1], [4, 7, 5]) == 3)
    print(f([105, 1231, 5663, 295, 3062, 380, 7777, 1], [7, 7, 7, 7, 7, 7, 7, 7]) == 7776)
    print(f([5, 10], [5, 3]) == 7)
    print(f([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]) == 25)

#### aquarium: https://community.topcoder.com/stat?c=problem_statement&pm=6124




 Problem Statement for Aquarium


Problem Statement
    	You have a fish aquarium. You are given a int[] fishSizes, denoting the size of each fish in the aquarium. They've been getting along pretty well so far, but you would now like to add a new fish (called Bob). You know that fish sometimes eat each other. You estimate that a fish might eat another fish if and only if it is at least two times larger, but no more than ten times larger, than that other fish. 

Considering this, you would like to choose Bob's size such that: 

- Bob is not in danger of being eaten by any other fish (i.e., its size is not between 1/10 and 1/2, inclusive, the size of any other fish). 

- Bob is not tempted to eat any other fish (i.e., no other fish in the aquarium has a size between 1/10 and 1/2, inclusive, of Bob's size). 



Your task is to return the number of different integer sizes for Bob between minSize and maxSize, inclusive, that won't cause any eating "conflicts" with other fish.
 
Definition
    	
Class:	Aquarium
Method:	peaceful
Parameters:	int, int, int[]
Returns:	int
Method signature:	int peaceful(int minSize, int maxSize, int[] fishSizes)
(be sure your method is public)
    
 
Notes
-	Mulitple fish may have the same size. Also, we know that the existing fish get along, so we do not need to worry about eating conflicts between existing fish, regardless of their relative sizes.
 
Constraints
-	minSize is between 1 and 1000, inclusive.
-	maxSize is between minSize and 1000, inclusive.
-	fishSizes has between 1 and 50 elements, inclusive.
-	Each element of fishSizes is between 1 and 1000, inclusive.
 
Examples
0)	
    	
1
12
{1}
Returns: 3
We want to check which sizes between 1 and 12 will satisfy our constraints. Since we have a fish of size 1 in the aquarium, a new fish with a size between 2 and 10 will put it in danger. The remaining available sizes for Bob are 1, 11 and 12. Thus, the answer is 3.
1)	
    	
1
36
{3}
Returns: 10
There are 10 valid sizes: 2, 3, 4, 5, 31, 32, 33, 34, 35 and 36. Note that Bob cannot have a size of 1 since it may be eaten by the size 3 fish in the aquarium.
2)	
    	
1
1000
{10, 100, 500}
Returns: 4
3)	
    	
5
880
{1, 3, 9, 27, 243, 729}
Returns: 121
4)	
    	
3
997
{10, 11, 12, 13, 14, 16, 82, 83, 84, 85, 720, 730, 740, 750, 760, 770}
Returns: 147
5)	
    	
2
999
{941, 797, 120, 45, 7, 120}
Returns: 10
6)	
    	
1
791
{1, 4, 12, 25, 79, 556, 625, 800, 801, 950, 952, 954, 956, 958, 980, 1000}
Returns: 1
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 293 Round 1 - Division II, Level One

In [ ]:
class Aquarium(object):
    
    def peaceful(self, mn, mx, fishes):
        """return total number of sizes bob can be."""
        ret = 0
        for i in range(mn, mx + 1):
            ok = True
            for f in fishes:
                if i >= f / 10 and i <= f / 2: ok = False
                if f >= i / 10 and f <= i / 2: ok = False
            ret += ok    
        return ret
    
if True:
    f = Aquarium().peaceful
    print(f(3, 997, [10, 11, 12, 13, 14, 16, 82, 83, 84, 85, 720, 730, 740, 750, 760, 770]) == 147)
    print(f(2, 999, [941, 797, 120, 45, 7, 120]) == 10)

In [ ]:
1 * False

#### bearsong: https://community.topcoder.com/stat?c=problem_statement&pm=14082

Problem Statement for BearSong


Problem Statement
    	
Bear Limak has recently learned about musical notes. He then listened to a song and noticed that some notes appeared less often than others. In fact, some notes were so rare that they appeared in the song only once!



Limak now wants to look for such notes in other songs. Write a program that will look for the rare notes.



You are given a int[] notes that describes a song. Each number in notes represents one note of the song. Different numbers represent different notes, equal numbers represent equal notes.



Compute and return the number of notes that occur exactly once in the given song.

 
Definition
    	
Class:	BearSong
Method:	countRareNotes
Parameters:	int[]
Returns:	int
Method signature:	int countRareNotes(int[] notes)
(be sure your method is public)
    
 
Constraints
-	notes will contain between 1 and 50 elements, inclusive.
-	Each element in notes will be between 1 and 1000, inclusive.
 
Examples
0)	
    	
{9,10,7,8,9}
Returns: 3
The rare notes are the notes 7, 8, and 10. Each of these notes occurs exactly once in the whole song. The note 9 occurs twice, so it is not a rare note
1)	
    	
{8,8,7,6,7,3,5,10,9,3}
Returns: 4
Each of the notes 5, 6, 9, and 10 occurs exactly once in this song.
2)	
    	
{234,462,715,596,906}
Returns: 5
3)	
    	
{17}
Returns: 1
4)	
    	
{1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,
1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,
1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000}
Returns: 0
This problem statement is the exclusive and proprietary property of TopCoder, Inc. Any unauthorized use or reproduction of this information without the prior written consent of TopCoder, Inc. is strictly prohibited. (c)2010, TopCoder, Inc. All rights reserved.




This problem was used for: 
       Single Round Match 673 Round 1 - Division II, Level One

In [ ]:
class BearSong(object):
    
    def countRareNotes(self, notes):
        """return count of notes that happen only once."""
        count = {}
        for n in notes:
            if n not in count:
                count[n] = True
            else:
                count[n] = False
        return len([k for k in count if count[k]])
    
if True:
    f = BearSong().countRareNotes
    print(f([234,462,715,596,906]) == 5)
    print(f([9, 10, 7, 8, 9]) == 3)
    print(f([8, 8, 7, 6, 7, 3, 5, 10, 9, 3]) == 4)
    print(f([234, 462, 715, 596, 906]) == 5)
    print(f([17]) == 1)
    print(f([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]) == 0)
    print(f([999, 640, 652, 28, 794]) == 5)
    print(f([101, 109, 106, 107, 104, 109, 102, 109, 101, 108]) == 5)
    print(f([109, 109, 102, 103, 101, 102, 105, 110, 106, 100]) == 6)
    print(f([108, 105, 108, 108, 100, 103, 106, 109, 104, 105]) == 5)
    print(f([26, 632, 564, 762, 349, 2, 767, 763, 834, 995, 918, 67, 457, 632, 662]) == 13)
    print(f([518, 508, 510, 519, 517, 514, 501, 501, 517, 512, 514, 506, 503, 514, 511]) == 8)
    print(f([900, 905, 900, 903, 903, 905, 909, 910, 900, 902, 905, 903, 907, 909, 903, 905, 908, 904, 909, 908]) == 4)
    print(f([831, 17, 35, 688, 951, 353, 785, 591, 987, 114, 234, 668, 819, 509, 265, 286, 483, 863, 871, 220, 646, 451, 226, 143, 190, 864, 654, 291, 715, 456, 30, 545, 824, 65, 585, 126, 417, 721, 716, 755, 834, 302, 774, 5, 810, 38, 642, 292, 900, 513]) == 50)
    print(f([992, 991, 993, 991, 1000, 998, 1000, 987, 990, 996, 992, 988, 993, 999, 988, 993, 993, 996, 987, 996, 992, 991, 993, 991, 995, 992, 999, 988, 996, 996, 992, 987, 987, 996, 989, 998, 994, 989, 996, 995, 996, 1000, 994, 988, 998, 995, 992, 990, 988, 991]) == 0)
    print(f([14, 75, 41, 86, 66, 1, 68, 94, 50, 41, 73, 16, 29, 49, 95, 57, 64, 30, 89, 28, 11, 39, 43, 18, 54, 84, 69, 68, 30, 78, 4, 95, 5, 44, 80, 22, 44, 47, 15, 45, 39, 87, 60, 67, 36, 7, 76, 51, 36, 64]) == 34)
    print(f([212, 150, 384, 144, 338, 218, 130, 278, 227, 169, 417, 182, 223, 431, 305, 224, 182, 200, 251, 298, 264, 301, 187, 236, 380, 334, 314, 395, 406, 249, 228, 186, 399, 302, 329, 304, 209, 336, 271, 313, 194, 255, 372, 294, 376, 366, 208, 125, 133, 336]) == 46)
    print(f([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]) == 50)
    print(f([1000, 5, 5]) == 1)
    print(f([999, 1000]) == 2)
    print(f([9, 9, 9]) == 0)
    print(f([1, 3, 3, 3, 4]) == 2)
    print(f([9, 10, 7, 8, 9]) == 3)
    print(f([2, 2, 3, 3, 4, 4, 5, 5, 6, 6]) == 0)
    print(f([1, 1]) == 0)
    print(f([234, 462, 715, 596, 906]) == 5)
    print(f([2, 2, 3, 3, 4, 4]) == 0)
    print(f([1000, 1000]) == 0)
    print(f([3, 3, 3, 1]) == 1)
    print(f([1, 2, 2, 2]) == 1)
    print(f([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]) == 0)
    print(f([1, 1, 1]) == 0)
    print(f([1000, 50, 250]) == 3)
    print(f([99, 99, 99, 99]) == 0)
    print(f([1, 1000, 1, 1000]) == 0)
    print(f([2, 2, 2]) == 0)

#### byteland: https://community.topcoder.com/stat?c=problem_statement&pm=13215

review: https://www.topcoder.com/community/data-science/data-science-tutorials/computational-complexity-section-1/

Byteland is a city with many skyscrapers, so it's a perfect venue for BASE jumping. Danilo is an enthusiastic BASE jumper. He plans to come to Byteland and to jump off some of its buildings.

Danilo wants to make M jumps in Byteland. However, he has some rules. First, he never jumps off the same building twice. Second, all buildings he selects for his jumps must have the same number of floors. (This is for safety reasons: It is hard to get the timing right if each jump starts at a different height.)

Philipe is the mayor of Byteland. He welcomes Danilo's visit as he would like to use it as a publicity stunt. However, Philipe knows that Danilo will only come to Byteland if there are at least M buildings that each have the same number of floors. To ensure that, the mayor is willing to build additional floors on some of the skyscrapers in Byteland.

You are given the int M and a int[] heights. Each element of heights is the number of floors in one of Byteland's skyscrapers. Compute and return the smallest number of additional floors the mayor has to build so that there will be at least M buildings with the same number of floors.

 
Definition
    	
Class:	BuildingHeightsEasy
Method:	minimum
Parameters:	int, int[]
Returns:	int
Method signature:	int minimum(int M, int[] heights)
(be sure your method is public)
    
 
Constraints
-	heights will contain between 1 and 50 elements, inclusive.
-	M will be between 1 and the number of elements in heights, inclusive.
-	Each element in heights will be between 1 and 50, inclusive.
 
Examples
0)	
    	
2
{1, 2, 1, 4, 3}
Returns: 0
Note that we already have two buildings that have the same number of floors. Hence, no additional floors need to be built.

1)	
    	
3
{1, 3, 5, 2, 1}
Returns: 2
We want to have at least three buildings with the same number of floors. The best way to reach this goal is to build one floor on building #0 and one floor on building #4 (0-based indices). After these changes, buildings #0, #3, and #4 will have two floors each.

2)	
    	
1
{43, 19, 15}
Returns: 0

3)	
    	
3
{19, 23, 9, 12}
Returns: 15

4)	
    	
12
{25, 18, 38, 1, 42, 41, 14, 16, 19, 46, 42, 39, 38, 31, 43, 37, 26, 41, 33, 37, 45, 27, 19, 24, 33, 11, 22, 20, 36, 4, 4}
Returns: 47


In [ ]:
import math
class BuildingHeightsEasy(object):
    
    """Class to find required floors."""
    
    def minimum(self, m, heights):
        """Find the minimum number of floors required to have ``m`` equal buildings."""
        if m == 1: return 0
        heights.sort()
        counts = {}
        reviewed = []
        minimum = math.inf
        for i, current in enumerate(heights):
            try:
                counts[current] += 1
            except KeyError:
                counts[current] = 1
                reviewed.append(current)

            required = 0
            missing = m - counts[current]
            if missing == 0: return 0
            for j in reviewed[::-1][1:]:
                delta = current - j
                if missing > counts[j]:
                    missing -= counts[j]
                    required += counts[j] * delta
                else:
                    required += missing * delta
                    if required < minimum: minimum = required
                    break
            
        return minimum
if True:
    m = BuildingHeightsEasy().minimum
    print(m(2, [1, 2, 1, 4, 3]) == 0)
    print(m(3, [1, 3, 5, 2, 1]) == 2)
    print(m(1, [43, 19, 15]) == 0)
    print(m(3, [19, 23, 9, 12]) == 15)
    print(m(12, [25, 18, 38, 1, 42, 41, 14, 16, 19, 46, 42, 39, 38, 31, 43, 37, 26, 41, 33, 37, 45, 27, 19, 24, 33, 11, 22, 20, 36, 4, 4]) == 47)
    print(m(4, [41, 12, 15, 33, 25, 32, 2, 23, 23, 40]) == 14)
    print(m(10, [34, 48, 29, 26, 30, 17, 23, 8, 37, 48, 40, 47]) == 118)
    print(m(9, [32, 25, 15, 43, 48, 5, 5, 14, 40, 31, 46, 41, 3, 20, 32, 16, 23, 16, 14]) == 72)
    print(m(1, [13]) == 0)
    print(m(39, [33, 3, 36, 7, 17, 31, 6, 24, 35, 20, 15, 17, 15, 5, 22, 34, 36, 37, 9, 3, 50, 13, 46, 43, 34, 12, 45, 40, 24, 36, 45, 9, 41, 30, 15, 9, 10, 23, 32, 46, 44, 48]) == 795)
    print(m(1, [2, 34, 43, 40, 20, 3, 44, 22, 18, 41, 14, 1, 3]) == 0)
    print(m(2, [28, 45, 36, 36, 37, 18, 3, 46, 29, 27]) == 0)
    print(m(1, [20, 25, 39, 29, 28, 22, 23, 19, 43, 26, 15, 14, 45, 5, 29, 47, 9, 38, 38, 37, 35, 26, 24, 21, 45, 28, 16]) == 0)
    print(m(7, [45, 49, 23, 21, 37, 1, 50, 10, 24, 19, 5, 1, 35, 20, 45, 39, 49, 41, 50, 38, 30, 38, 22]) == 21)
    print(m(5, [45, 1, 39, 12, 26, 42, 6]) == 61)
    print(m(13, [28, 13, 17, 28, 24, 40, 48, 30, 42, 32, 50, 36, 22, 48, 28, 21, 37, 10, 8, 11, 18, 21, 5, 19, 9, 16]) == 96)
    print(m(36, [24, 21, 18, 1, 35, 37, 30, 11, 28, 29, 40, 20, 12, 39, 7, 36, 38, 37, 6, 27, 48, 16, 37, 15, 36, 41, 35, 46, 8, 30, 49, 31, 3, 19, 34, 37, 5, 15, 49, 34, 46, 39, 3, 7, 29, 12]) == 510)
    print(m(27, [50, 50, 45, 47, 17, 31, 11, 2, 23, 46, 49, 30, 25, 49, 13, 29, 17, 46, 16, 23, 12, 14, 7, 7, 4, 11, 14, 33, 22, 7, 3, 23, 8, 47, 19, 24, 27, 30, 27, 1, 25, 25]) == 303)
    print(m(21, [24, 42, 32, 42, 39, 47, 15, 1, 12, 23, 7, 18, 33, 22, 2, 7, 31, 4, 29, 38, 2, 48, 12, 30, 29, 40, 30, 5, 15, 9]) == 304)
    print(m(5, [3, 38, 31, 41, 36]) == 56)
    print(m(10, [48, 19, 50, 17, 2, 23, 18, 10, 3, 23, 38, 43, 24, 37, 6, 5, 15, 45, 34, 19, 11, 44, 26, 50, 46, 13, 33, 37, 49, 29, 29, 48, 50, 28, 14, 3, 1, 33, 12, 5, 7, 1, 47, 30, 38, 2, 36]) == 23)
    print(m(15, [8, 12, 43, 46, 10, 23, 24, 9, 22, 4, 22, 26, 6, 6, 37, 12, 12, 40, 9, 43, 29, 12, 28, 32, 10, 48, 4, 19, 10, 3, 29, 19, 17, 21, 16, 26, 43, 39, 35, 17, 44, 8, 42, 1, 16, 31, 13, 27]) == 48)
    print(m(3, [22, 50, 36, 1, 31, 46, 49, 37, 16, 10, 39, 44, 28, 5, 16, 43, 33, 10, 33, 19]) == 2)
    print(m(13, [28, 20, 31, 43, 2, 43, 22, 23, 17, 43, 22, 3, 45, 2, 50, 43, 40, 15, 2, 31, 10, 31, 35, 27, 25, 19, 36, 10]) == 92)
    print(m(4, [40, 17, 35, 20, 9, 38, 15, 32, 10, 31, 26, 33, 35, 21, 36, 34, 15, 26, 48, 19, 6, 9, 1, 42, 35, 26, 11, 23, 37, 49, 38, 28, 17, 22, 47, 28, 9, 13, 9]) == 0)
    print(m(4, [35, 2, 28, 7, 37, 14, 21, 14, 13, 39, 19, 22, 40, 11, 8, 17, 23, 30, 5, 23]) == 3)
    print(m(5, [42, 40, 28, 19, 50, 43, 27, 19, 36, 13, 22, 15, 19, 11, 30, 40, 24]) == 10)
    print(m(6, [45, 14, 19, 7, 21, 37, 29, 1, 41, 3, 19, 22, 44, 9, 2, 12, 8, 44, 41, 29, 31, 3, 50, 47, 24, 12, 27, 13, 36, 19, 44, 32, 34, 15, 38, 6, 1, 18, 8, 42, 22, 27, 15]) == 8)
    print(m(7, [16, 29, 44, 11, 21, 22, 41, 24, 22, 40, 47, 33, 16, 11, 20, 36]) == 26)
    print(m(3, [21, 18, 40, 26]) == 13)
    print(m(5, [34, 14, 29, 12, 28, 46, 46, 46, 26, 39, 8, 47, 13, 49, 20, 34, 40, 18, 18, 5, 30]) == 10)
    print(m(2, [33, 42, 12, 3, 34, 37, 25, 40, 22, 38, 20, 33, 17, 17, 28, 14, 43, 19, 22, 39, 33, 22, 10, 18, 11, 27, 35, 17, 6, 26, 8, 40, 20, 19, 44, 3, 7, 18, 42, 29]) == 0)
    print(m(1, [9]) == 0)
    print(m(2, [5, 2, 1]) == 1)
    print(m(3, [2, 3, 3, 3, 3]) == 0)
    print(m(3, [2, 5, 6, 7, 11, 17]) == 3)
    print(m(3, [1, 1, 50]) == 98)
    print(m(50, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 50]) == 2401)
    print(m(3, [1, 2, 19, 19, 19]) == 0)
    print(m(12, [25, 18, 38, 1, 42, 41, 14, 16, 19, 46, 42, 39, 38, 31, 43, 37, 26, 41, 33, 37, 45, 27, 19, 24, 33, 11, 22, 20, 36, 4, 4]) == 47)
    print(m(4, [3, 3, 4, 4, 4]) == 1)
    print(m(2, [1, 2, 5, 10, 20, 30, 50]) == 1)
    print(m(2, [3, 1, 4, 4]) == 0)
    print(m(25, [34, 37, 28, 16, 44, 36, 37, 43, 50, 22, 13, 28, 41, 10, 14, 27, 41, 27, 23, 37, 12, 19, 18, 30, 33, 31, 13, 24, 18, 36, 30, 3, 23, 9, 20, 18, 44, 7, 12, 43, 30, 24, 22, 20, 35, 38, 49, 25, 16, 21]) == 194)
    print(m(3, [10, 20, 30, 30]) == 10)
    print(m(3, [1, 1, 2, 2, 3, 3]) == 1)
    print(m(1, [1, 2, 3]) == 0)
    print(m(5, [1, 1, 1, 1, 50, 50, 50, 50]) == 49)

#### badgers: https://community.topcoder.com/stat?c=problem_statement&pm=10797

solution: http://apps.topcoder.com/wiki/display/tc/SRM+476

Problem Statement
    	Badgers are lovely furry animals, and Manao has just decided to start keeping a few. The pet shop has offered him N badgers, and they are all so cute that Manao wants to take as many as he can feed. Normally, a badger needs some amount of food per day to be satisfied. However, if he sees other badgers eating, his greed awakens and he wants to eat more. A badger will want a fixed additional amount of food for each co-eater.



You're given int[]s hunger and greed, both containing N elements. The i-th element of hunger is the number of units of food that the i-th badger needs per day if he's alone. The i-th element of greed is the amount of additional units of food the i-th badger will need for each co-eater. Return the maximum number of badgers Manao can take while keeping them all satisfied if he can supply no more than totalFood units of food per day.
 
Definition
    	
Class:	Badgers
Method:	feedMost
Parameters:	int[], int[], int
Returns:	int
Method signature:	int feedMost(int[] hunger, int[] greed, int totalFood)
(be sure your method is public)
    
 
Constraints
-	hunger will contain between 1 and 50 elements, inclusive.
-	greed will contain the same number of elements as hunger.
-	Each element of hunger will be between 1 and 1000, inclusive.
-	Each element of greed will be between 0 and 1000, inclusive.
-	totalFood will be between 1 and 1000000, inclusive.
 
Examples

0)	
    	
{1,2,3}
{2,2,1}
7
Returns: 2
Manao can take badger 0 and one of the other two badgers.

1)	
    	
{5,2,1,5}
{0,2,4,1}
19
Returns: 3
Badger 2 is too greedy, but the rest can be fed together and will only need (5 + 2 * 0) + (2 + 2 * 2) + (5 + 2 * 1) = 18 units of food.

2)	
    	
{1,1,1,1,1}
{1000,1000,1000,1000,1000}
10
Returns: 1
They are too greedy to eat together.

3)	
    	
{1,2,3,4,5,6,7,8,9,10}
{10,9,8,7,6,5,4,3,2,1}
100
Returns: 5

In [ ]:
class Badgers(object):
    
    def feed_most(self, hunger, greed, total):
        """Return total badgers Manao can feed."""
        n = len(hunger)
        cost = []
        for i in range(1, n + 1):
            cost = sorted([hunger[j] + (i - 1) * greed[j] for j in range(n)])
            if sum(cost[:i]) > total: return i - 1
if True:
    f = Badgers().feed_most
    print(f([1,2,3], [2,2,1], 7))
    print(f([5,2,1,5], [0,2,4,1], 19 ))
    print(f([1,1,1,1,1], [1000,1000,1000,1000,1000], 100))
    print(f([1,2,3,4,5,6,7,8,9,10], [10,9,8,7,6,5,4,3,2,1], 100))

#### rowsofnodes: https://community.topcoder.com/stat?c=problem_statement&pm=13546

solution: **will do later**

Problem Statement
    	
You have two rows of nodes. Each row contains N nodes, numbered 0 through N-1 from the left to the right.

Within each row, adjacent nodes are already connected by edges. You are given the lengths of these edges as int[]s a and b, each containing N-1 elements. For each valid i, a[i] is the length of the edge between nodes i and (i+1) in the top row, and b[i] is the length of the edge between nodes i and (i+1) in the bottom row.

You want to add exactly K new edges to this graph. Each of the new edges must be vertical -- i.e., it must connect some vertex i in the top row to the vertex i in the bottom row. All new edges will have length 0.

By adding the K new edges we will produce a connected graph. The diameter of this graph is the maximum of all shortest distances among pairs of its nodes. In other words, the diameter is the smallest number D such that it is possible to travel from any node to any other node using a path of length D or less.

Given a, b, and the int K, compute and return the smallest possible diameter of the resulting graph.

 
Definition
    	
Class:	BridgeBuildingDiv2
Method:	minDiameter
Parameters:	int[], int[], int
Returns:	int
Method signature:	int minDiameter(int[] a, int[] b, int K)
(be sure your method is public)
    
 
Constraints
-	N will be between 2 and 11, inclusive.
-	a,b will contain exactly N-1 elements each.
-	K will be between 1 and N, inclusive.
-	Each element of a,b will be between 1 and 50, inclusive.
 
Examples

0)	
    	
{2,1,1,1,2}
{1,9,1,9,1}
4
Returns: 6
One example of an optimal solution is to draw the bridges as follows:


1)	
    	
{1,50,1,50,1,50,1,50}
{50,1,50,1,50,1,50,1}
9
Returns: 8

2)	
    	
{50,10,15,31,20,23,7,48,5,50}
{2,5,1,8,3,2,16,11,9,1}
3
Returns: 124

3)	
    	
{2,4,10,2,2,22,30,7,28}
{5,26,1,2,6,2,16,3,15}
5
Returns: 54

#### littlepolar: https://community.topcoder.com/stat?c=problem_statement&pm=14083

Problem Statement
    	
Limak is a little polar bear. He has a int[] w containing a sequence of N distinct numbers. He wants to sort this sequence into ascending order.



Limak knows some fast sorting algorithms but in the real world such knowledge sometimes isn't enough. In order to sort the sequence w Limak must physically move the numbers into their correct places. Such a thing can be hard for a little bear.



In a single move Limak can take all elements he can reach and sort them into ascending order. The problem is that Limak's arms are too short. Regardless of where he stands, he can only reach N-1 consecutive elements of w. Hence, in each move he can either sort all elements except for the last one, or all elements except for the first one.



Limak can make the moves in any order he likes. Compute and return the smallest number of moves necessary to sort the given sequence w.

 
Definition
    	
Class:	BearSlowlySorts
Method:	minMoves
Parameters:	int[]
Returns:	int
Method signature:	int minMoves(int[] w)
(be sure your method is public)
    
 
Constraints
-	w will contain between 3 and 50 elements, inclusive.
-	Each element in w will be between 1 and 1000, inclusive.
-	w will contain distinct elements.
 
Examples
0)	
    	
{2,6,8,5}
Returns: 1
Limak can sort this sequence in a single move. All he needs to do is to sort all elements except for the first one. In this single move Limak will pick up the elements {6,8,5} and reorder them into {5,6,8}. This will change the given w into {2,5,6,8}, and that is a sorted sequence.
1)	
    	
{4,3,1,6,2,5}
Returns: 2
One of the shortest ways is to sort first N-1 numbers first to get {1,2,3,4,6,5} and then to sort the last N-1 numbers.
2)	
    	
{93,155,178,205,213,242,299,307,455,470,514,549,581,617,677}
Returns: 0
This sequence is already sorted, no moves are necessary.
3)	
    	
{50,20,30,40,10}
Returns: 3
4)	
    	
{234,462,715,596,906,231,278,223,767,925,9,526,369,319,241,354,317,880,5,696}
Returns: 2

In [ ]:
class BearSlowlySorts(object):
    
    def moves(self, w):
        """Return min moves needed."""
        sw = sorted(w)
        if sw == w: return 0
    
        moves = [0, 0]
        for i in range(2):
            a, direction = w, i
            while True:
                moves[i] += 1
                direction += 1
                if direction % 2 == 0:
                    l, ls = a[0], sorted(a[1:])
                    a = [l] + ls
                    if l <= ls[0]: break
                else:
                    r, rs = a[-1], sorted(a[:-1])
                    a = rs + [r]
                    if r >= rs[-1]: break
            
        return min(moves)
    
if True:
    f = BearSlowlySorts().moves
    print(f([2,6,8,5]) == 1)
    print(f([4,3,1,6,2,5]) == 2)
    print(f([93,155,178,205,213,242,299,307,455,470,514,549,581,617,677]) == 0)
    print(f([50,20,30,40,10]) == 3)
    print(f([234,462,715,596,906,231,278,223,767,925,9,526,369,319,241,354,317,880,5,696]) == 2)
    print(f([5,4,3,2,1]) == 3)
    print(f([5,1,2,3,4]) == 2)
    print(f([5,2,3,4,1]) == 3)
    print(f([1,3,2,4]) == 1)
    print(f([9,10,2]) == 2)
    print(f([8,4,7,6]) == 2)
    print(f([7,8,10,9,6]) == 2)
    print(f([6,1,8,3,7,5]) == 2)
    print(f([9,526,369,319,241,354,317,880,5,696]) == 2)
    print(f([714,26,67,349,457,564,632,662,2,762,763,767,834,918,995]) == 1)
    print(f([98,215,234,245,275,288,292,296,329,330,432,980,468,599,623,635,738,858,935,936,939,966,354]) == 1)
    print(f([987,35,101,114,220,234,265,286,353,388,450,483,498,509,512,591,646,668,688,785,819,831,852,863,871,951,17]) == 3)
    print(f([5,30,38,715,456,143,190,226,291,292,302,315,417,451,126,513,545,585,642,654,65,824,721,755,774,810,716,834,864,897,900]) == 1)
    print(f([18,2,26,15,27,42,49,3,10,19,20,16,5,21,39,9,30,50,24,28,14,37,25,34,31,12,8,44,35,41,36,1,23,29,45,7,11,43,4,22,47,17,48,6,40,32,33,38,46,13]) == 2)
    print(f([12,68,76,80,83,85,126,128,135,145,157,177,182,223,264,275,277,286,359,365,413,421,422,433,461,474,484,500,510,517,548,592,639,644,650,654,666,700,701,779,802,803,810,819,832,901,912,936,937,960]) == 0)
    print(f([816,27,90,99,109,114,132,137,144,149,152,156,188,252,254,273,275,282,283,289,292,384,389,414,442,483,499,501,527,564,672,679,705,715,718,810,20,828,852,857,863,865,884,889,917,958,971,986,991,996]) == 1)
    print(f([927,58,63,72,103,104,143,150,154,169,200,201,232,238,332,337,357,390,420,423,461,472,521,525,534,538,568,575,585,618,643,653,675,683,693,717,736,811,823,851,873,878,910,914,34,934,947,954,960,971]) == 1)
    print(f([26,28,48,76,104,150,164,213,214,222,272,277,301,348,366,378,397,433,447,449,454,592,471,487,519,521,524,581,992,627,635,673,695,727,742,748,751,760,774,787,819,863,877,927,942,949,957,964,986,459]) == 1)
    print(f([5,50,53,891,176,205,247,260,266,277,299,321,372,377,382,415,454,529,548,549,585,616,617,621,631,653,660,669,675,677,705,725,735,744,763,779,786,819,820,825,867,878,880,881,882,1000,898,906,971,85]) == 1)
    print(f([975,49,80,131,133,156,183,185,207,220,244,281,386,438,439,450,455,463,466,468,469,493,496,524,568,590,597,610,619,660,684,700,701,713,717,749,752,757,773,785,787,804,824,837,893,895,896,905,950,29]) == 3)
    print(f([957,8,30,51,59,62,83,95,100,157,158,174,195,218,270,283,401,413,429,443,465,466,490,495,498,548,556,580,596,602,627,683,689,694,710,718,740,786,809,827,842,848,858,864,911,920,921,926,927,1]) == 3)
    print(f([3,22,39,54,65,179,81,97,121,123,132,161,73,191,604,222,223,224,225,255,284,813,372,438,448,450,460,482,505,510,517,525,551,574,576,587,215,611,625,663,673,692,700,732,743,756,327,877,922,987]) == 1)
    print(f([28,40,291,82,87,149,158,167,186,197,209,224,245,250,265,289,52,297,303,342,363,386,442,453,510,523,525,537,557,563,849,586,567,614,658,659,667,765,774,811,822,832,600,851,859,867,885,897,913,926]) == 1)
    print(f([1,2,3]) == 0)
    print(f([2,1,3]) == 1)
    print(f([3,1,2]) == 2)
    print(f([1,3,2]) == 1)
    print(f([2,3,1]) == 2)
    print(f([3,2,1]) == 3)
    print(f([50,20,30,40,10]) == 3)
    print(f([1,3,2,4]) == 1)
    print(f([93,155,178,205,213,242,299,307,455,470,514,549,581,617,677]) == 0)
    print(f([5,1,2,3,4]) == 2)
    print(f([3,5,4,2,1]) == 2)
    print(f([2,4,3,5]) == 1)
    print(f([5,3,7,8,9,10]) == 1)
    print(f([2,1,3]) == 1)
    print(f([5,1,2,3]) == 2)
    print(f([3,5,2,4,1]) == 2)
    print(f([50,2,6,8]) == 2)
    print(f([2,3,1]) == 2)
    print(f([1,2,3,4,5]) == 0)
    print(f([5,1,2]) == 2)
    print(f([1,3,2,4,5]) == 1)
    print(f([2,6,8,5]) == 1)
    print(f([80,60,10,20]) == 2)
    print(f([7,3,1,6,2,5]) == 2)
    print(f([210,2,8,5,3]) == 2)
    print(f([7,8,6,9]) == 1)
    print(f([2,3,4,1]) == 2)
    print(f([1,2,5,4,6]) == 1)
    print(f([4,1,2,3]) == 2)
    print(f([2,5,6,8]) == 0)
    print(f([6,3,2]) == 3)
    print(f([1,4,3,2,5]) == 1)
    print(f([1,4,3,2]) == 1)
    print(f([2,3,1,5,4]) == 2)
    print(f([4,2,3,1]) == 3)
    print(f([3,2,4,1]) == 2)
    print(f([3,5,1]) == 2)
    print(f([1,3,2]) == 1)
    print(f([3,2,1,4]) == 1)
    print(f([234,462,715,596,906,231,278,223,767,925,9,526,369,319,241,354,317,880,5,696]) == 2)
    print(f([3,4,2,1]) == 2)